
# build predictive model of genes -> STP


1) Use ABA VISp,Hipp gene expression and STP data to fit RF for STP. 

2) test predictions - crossvalidation, test on hippocampal data, test on exc. neurons data?

3) feature selection - compare datasets? - compare with Paul analysis

4) stp factors genes - find better factors using interactions networks?

## 1) fit RF ABA ge -> STP : Hippocampus, VISp

a. select gene sets

b. load gene expression data Hipp, Ent 

c. hippocampus STP data 

d. combine STP and gene expression data for Hippocampus

e.load gene expression data VISp

f.  coretex STP data

g. combine STP and gene expression data for VISp cortex

h. fit RF from sklit-learn

f. test predictions :  cross validation - part of genes, part of STP labels, genes from different cortical areas, genes from hippocampus (Harris 2018)

g. compare gene sets


In [ ]:
import pandas as pd
import numpy as np


do_laptop=1   # Laptop or desctop version


do_add_scvi_latent_factors = 1
do_add_scvi_imputed_genes = 1

# a. select gene sets - take Paul 2017 best pre-synaptic + CAMs
# Paul 2017


In [ ]:
def get_prot_list(df_stp_ds, name):
    pre_main=df_stp_ds.loc[:,['Protein','Gene rat']]
    pre_main_nm = pre_main.loc[pre_main.loc[:,'Protein'].str.contains(':',na=False),'Protein']


    pre_main_nm=pre_main_nm.reset_index()
    i = pre_main_nm.loc[pre_main_nm.loc[:,'Protein']==name,:]
    i1 = i.loc[:,'index']
    i2 = pre_main_nm.loc[i.index[0]+1,'index']

    Cas = df_stp_ds.loc[np.arange(i1.values[0]+1,i2),'Gene rat']
    Cas=Cas.loc[Cas.loc[:].isna()==False]
    Cas = Cas.str.split('\W+',expand=True).iloc[:,0].values.tolist()
    return Cas

In [ ]:
d8 = '~/Documents/references/transcriptomes to STP/'
if do_laptop:
    d8 = '/Users/stepaniu/Documents/jan_2020/'

df_stp_ds = pd.read_excel(d8+'Datasets_STP_hippocampus.xlsx', sheet_name='Sheet8')

Cas = get_prot_list(df_stp_ds, 'Calcium sensors:')
AZ2 = get_prot_list(df_stp_ds, 'active zone:')

print(Cas)
print(AZ2)


d6=''
if do_laptop:
    d6 = '/Users/stepaniu/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/'

fn6 = 'Paul_2017.xlsx'
df_Paul_gs=pd.read_excel(d6 + fn6,header=2)

d7 = '~/Documents/references/transcriptomes to STP/transcriptomes/Linnerson_2018/'
fn7 = 'GO: synaptic vesicle cycle, mouse.xlsx'
if do_laptop:
    d7 = '/Users/stepaniu/Documents/jan_2020/'
    fn7 = 'GO_ synaptic vesicle cycle, mouse.xlsx'
df_go_priming = pd.read_excel(d7+fn7)
SV_cycle=df_go_priming.loc[:,'Gene/product label'].value_counts().index.tolist()


#print(df_Paul_gs.head())

#select presynaptic :


pre = ['Netrin-Unc5-Slit-Robo',
'Nrxn+Nlgn+Nphx+Dystroglycan+Cbln',
'neuronal_IgCAMs',
'Contactins',
'Semaphorin+Plexin',
'LRR+Slitrk+Elfn+Lphn',
'Eph+EphR',
'Calcium channel regulatory subunits',
'Calcium channel poreforming subunits',
'Calcium channel gamma/TARP subunits',
'Orphan GPCRs (based off HGNC)',
'Metabotropic neurotransmitter receptors',
'Neuropeptide receptors',
'Trimeric G_protein_alpha',
'Trimeric G_protein_beta',
'Trimeric G_protein_gamma',
'Rho-GEF',
'Dock-GEF',
'Dock + Rho-GEF',
'Complexin-Syt',
'SNAP-complex',
'Syntaxin',
'RIM-proteins/Active zone',
'bHLH',
'Nuclear receptors with C4 zinc fingers',
'Homeo domain factors',
'High-mobility group (HMG) domain factors',
'C2H2 zinc finger factors',
'bZIP',
'ARID domain factors',
'Runt domain factors',
'Other C4 zinc finger-type',
'C2HC zinc finger factors',
'SMAD/NF-1 DNA-binding domain factors',
'TATA-binding proteins',
'Rel homology region (RHR) factors',
'Tryptophan cluster factors',
'MADS box factors',
'Cold-shock domain factors',
'CXXC zinc finger factors',
'Leucine-rich repeat flightless-interacting proteins',
'STAT domain factors',
'Grainyhead domain factors'
]

#pre = ['Calcium channel regulatory subunits', 'Netrin-Unc5-Slit-Robo',
#       'Nrxn+Nlgn+Nphx+Dystroglycan+Cbln', 'neuronal_IgCAMs', 'Contactins',
#       'Metabotropic neurotransmitter receptors',      'Dock + Rho-GEF', 'Complexin-Syt']

#pre = ['Syntaxin', 'RIM-proteins/Active zone', 'Dock-GEF','SNAP-complex' ]

#pre = ['RIM-proteins/Active zone'] 

#pre = []


Elfn = ['Elfn1', 'Elfn2']

pregs = []
for i in pre:
    in_i = (df_Paul_gs.loc[:,'Gene Set']==i)
    n_in_i = df_Paul_gs.loc[in_i,'Number of genes'].astype(int).values[0]
    pri = df_Paul_gs.loc[in_i,df_Paul_gs.columns[4:4+n_in_i]]
    pregs = pregs + pri.values.tolist()[0]
    
#pregs = pregs +  Elfn  
#pregs = pregs + SV_cycle
pregs = pregs +Cas + AZ2 
#pregs = pregs +Cas 

## best gs
#pregs = best_pre

post = ['Netrin-Unc5-Slit-Robo',
'Nrxn+Nlgn+Nphx+Dystroglycan+Cbln',
'neuronal_IgCAMs',
'Contactins',
'Semaphorin+Plexin',
'LRR+Slitrk+Elfn+Lphn',
'Eph+EphR',
'Calcium channel regulatory subunits',
'Calcium channel poreforming subunits',
'Calcium channel gamma/TARP subunits',
'AMPA+TARP',
'Orphan GPCRs (based off HGNC)',
'Metabotropic neurotransmitter receptors',
'Neuropeptide receptors']


# post = ['Netrin-Unc5-Slit-Robo',
# 'Nrxn+Nlgn+Nphx+Dystroglycan+Cbln',
# 'neuronal_IgCAMs',
# 'Contactins',
# 'Semaphorin+Plexin',
# 'LRR+Slitrk+Elfn+Lphn',
# 'Eph+EphR']

#post = ['Netrin-Unc5-Slit-Robo',
#        'Nrxn+Nlgn+Nphx+Dystroglycan+Cbln', 'neuronal_IgCAMs', 'Contactins']


#post = [ 'Nrxn+Nlgn+Nphx+Dystroglycan+Cbln']

#post=[]


postgs = []
for i in post:
    in_i = (df_Paul_gs.loc[:,'Gene Set']==i)
    n_in_i = df_Paul_gs.loc[in_i,'Number of genes'].astype(int).values[0]
    posti = df_Paul_gs.loc[in_i,df_Paul_gs.columns[4:4+n_in_i]]
    postgs = postgs + posti.values.tolist()[0]
    
postgs = postgs +  Elfn    



In [ ]:
pregs = pregs[0:14]
postgs = postgs[0:9]

In [ ]:
print(len(pregs))
print(len(postgs))
postgs7='post__' + pd.DataFrame(postgs)
pregs7='pre__' + pd.DataFrame(pregs)
gs7 = pd.concat([pregs7, postgs7],axis=0).reset_index(drop=True)
gs7.to_excel('gene_set_names.xlsx')
gs7

In [ ]:
## SET FLAG : do_select_best_genes_from_previous_RF_run

In [ ]:
### fi2 = results of previous search
#fi2.to_excel('best_genes_1691_RF.xlsx')
#fi2.to_excel('best_genes_280_RF_2.xlsx')
#fi2.to_excel('best_genes_280_RF_hip.xlsx')

In [ ]:
do_select_best_genes_from_previous_RF_run=0
if do_select_best_genes_from_previous_RF_run==1:
    # fi2 = results of previous search
    fi2  = pd.read_excel('best_genes_1691_RF.xlsx')
    fi2  = pd.read_excel('best_genes_280_RF.xlsx')
    fi2  = pd.read_excel('best_genes_280_RF.xlsx')
    fi2  = pd.read_excel('best_genes_280_RF_hip.xlsx')

    fi2 = pd.read_excel('best_genes_hipp.xlsx')
    fi3 = pd.read_excel('best_genes_visp.xlsx')
    #
    nbest = 10
    bestn = fi2.iloc[0:nbest,:]
    #best_pre = bestn.index[bestn.loc[:,'compartment']=='pre'].values.tolist()
    #best_post = bestn.index[bestn.loc[:,'compartment']=='post'].values.tolist()
    
    best_pre = bestn.loc[bestn.loc[:,'compartment']=='pre','Unnamed: 0'].values.tolist()
    best_post = bestn.loc[bestn.loc[:,'compartment']=='post','Unnamed: 0'].values.tolist()
    
    print('best presynaptic genes')
    print(best_pre)
    print('best postsynaptic genes')
    print(best_post)
    
    ## best gs
    ## best gs
    pregs = best_pre
    postgs = best_post

print('postsynaptic')
print(postgs)
print(len(postgs))

print('presynaptic')
print(pregs)
print(len(pregs))
print(len(postgs))

pregs =np.unique(pregs).tolist()
postgs =np.unique(postgs).tolist()

import pickle
RF_file = open('RF_290_hipp_gs.pickle', mode='wb')
pickle.dump([pregs, postgs],RF_file)

do_visp_features=0
if do_visp_features==1:
    #pickle_in = open('RF_290_hipp.pickle',"rb")
    RF_file = open('RF_290_visp_gs.pickle', mode='rb')
    
    gs_visp = pickle.load(RF_file)
    pregs =gs_visp[0] 
    postgs=gs_visp[1]
    
    
##from mutual_info import entropy
##e = entropy(X, k=1)
#np.finfo(float).eps
#print(len(pregs))
#print(np.unique(pregs).shape)
#print(len(postgs))
#print(np.unique(postgs).shape)
#len(pregs)+len(postgs)    

# STP modeling functions: 

In [ ]:
# PV->AAC ???
Dt=50
tF=10
tD=410
p0=0.23
p=p0
n = (1-p)
n=1-(1-n)*np.exp(-Dt/tD)
print(n)
p=p+(1-p)*p0
p=p0 +(p -p0)*np.exp((-Dt/tF))
print(p)
n*p/p0


In [ ]:
# CCK->CCK ???
Dt=50
tF=1542
tD=115
p0=0.11
p=p0
n = (1-p)
n=1-(1-n)*np.exp(-Dt/tD)
print(n)
p=p+(1-p)*p0
p=p0 +(p -p0)*np.exp((-Dt/tF))
print(p)
n*p/p0

In [ ]:
# PV->AAC ???
Dt=50
tF=1.6
tD=930
p0=0.26
p=p0
n = (1-p)
n=1-(1-n)*np.exp(-Dt/tD)
print(n)
p=p+(1-p)*p0
p=p0 +(p -p0)*np.exp((-Dt/tF))
print(p)
n*p/p0

In [ ]:
# Ivy->PC ???
Dt=50
tF=62
tD=144
p0=0.32
p=p0
n = (1-p)
n=1-(1-n)*np.exp(-Dt/tD)
print(n)
p=p+(1-p)*p0
p=p0 +(p -p0)*np.exp((-Dt/tF))
print(p)
n*p/p0

In [ ]:
# CCK_DTI->PC ???
Dt=50
tF=14
tD=185
p0=0.15
p=p0
n = (1-p)
n=1-(1-n)*np.exp(-Dt/tD)
print(n)
p=p+(1-p)*p0
p=p0 +(p -p0)*np.exp((-Dt/tF))
print(p)
n*p/p0

In [ ]:
def  STP_sim(ge_data, T, init_state=None ):
    # transform labels from TM to An:A1
    #f = 20 # Hz
    #N = 3
    #T = np.arange(N)*1000/f

    N    = len(T)
    nc   = len(ge_data.index)

    x_lower = np.array([1,       0.01,    1,      0.1,     1])
    x_upper = np.array([10000,   1,      10000,    10,     1])
    stp_ns =          ['tF',    'p0',    'tD',   'dp/p0', 'A']
    for jj in range(len(stp_ns)):
        nsj = stp_ns[jj]
        ge_data.loc[:,nsj] = np.maximum(ge_data.loc[:,nsj].values,x_lower[jj])
        ge_data.loc[:,nsj] = np.minimum(ge_data.loc[:,nsj].values,x_upper[jj])
        if nsj=='dp/p0':
            p0   = ge_data.loc[:,'p0'].values
            dpp0 = ge_data.loc[:,'dp/p0'].values
            dp = p0*dpp0
            dp = np.minimum(dp,1)
            ge_data.loc[:,nsj] = dp/(p0 +np.finfo(float).eps)

    dpp0 = ge_data.loc[:,'dp/p0'].values
    p0   = ge_data.loc[:,'p0'].values
    tF   = ge_data.loc[:,'tF'].values
    tD   = ge_data.loc[:,'tD'].values
    A    = 1 + 0*ge_data.loc[:,'A'].values # simplify A

    As = np.zeros((nc,N))
    n = np.zeros((nc,))
    p = np.zeros((nc,))
    ns2 = np.zeros((nc,N))
    ps2 = np.zeros((nc,N))

    i=0
    
    if init_state is None :
        n[:] = 1
        p[:] = p0
    else:
        n = init_state[0]
        p = init_state[1]


    As[:,i] = A*n*p
    
    n = n*(1-p)
    p = p + dp*(1-p)
    
    ns2[:,i]=n
    ps2[:,i]=p

    for i in range(1,N):
        Dt=T[i]-T[i-1]
        #n = 1 - (1 - (n -p*n))*np.exp((-Dt/tD).astype(float))
        #p=p0 -(p0-(p + dpp0*p0*(1-p)))*np.exp((-Dt/tF).astype(float))
        #As[:,i]=A*n*p
        #ns2[:,i]=n
        #ps2[:,i]=p

        
        n = 1 - (1 - n )*np.exp((-Dt/tD ).astype(float))
        p=p0 +(p -p0)*np.exp((-Dt/tF).astype(float))
            

        As[:,i]=A*n*p
       
        n = n*(1-p)
        p = p + dp*(1-p)
        
        ns2[:,i]=n
        ps2[:,i]=p

    

    #aa = [As, ns2, ps2]
    
    return As, ns2, ps2, dpp0, p0, tF, tD, A

In [ ]:
def  STP_sim2(x, T, init_state=None, model_type='tm5' ):

    N    = len(T)
    tF      = x[0] #.astype(float)
    p00     = x[1]
    tD      = x[2] #.astype(float)
    dp      = x[3]
    A       = 1 #x[4] # simplify A
    
    #breakpoint()
    mod_fdr2=False
    if model_type=='tm5_fdr2':  # should be :check freq. dependent recovery
        tDmin     = x[5]
        dd        = x[6]
        t_FDR     = x[7]
        mod_fdr2=True
        tDmax  = tD
        itDmin = 1/tDmin
        itDmax = 1/tDmax
        #breakpoint()
        
    mod_smr=False
    if model_type=='tm5_smr':  # should be :check freq. dependent recovery
        t_SMR   = x[8]
        dp0     = x[9]
        mod_smr=True
        #p00  = p00

    As = np.zeros((N,))
    state = np.zeros((N,4))

   
    if init_state is None :
        n = 1
        p0=p00
        p = p0
        d = 0
    else:
        n = init_state[0]
        p = init_state[1]
        d = init_state[2]
        p0= init_state[3]

    
    for i in range(0,N):
        if i==0:
            Dt = T[i]
        else:
            Dt = T[i]-T[i-1]
        
        if mod_fdr2:
            d0=d
            d = d*np.exp(-Dt/t_FDR) 
            n = 1 - (1 - n )*np.exp(-Dt*itDmax -(itDmin -itDmax)*t_FDR*(d0-d))
        else:
            #print(x,tD)
            n = 1 - (1 - n )*np.exp(-Dt/tD )
            
        if mod_smr:
            p01=p0
            p0=p00 + (p0 -p00)*np.exp(-Dt/t_SMR)
            p=p0 +(p -p01)*np.exp(-Dt/tF)
        else:
            p=p0 +(p -p0)*np.exp(-Dt/tF)
            

        As[i]=A*n*p
       
        n = n*(1-p)
        p = p + dp*(1-p)
        if mod_fdr2:
            d  = d + dd*(1-d) 
        if mod_smr:
            p0  = p0 - dp0*p0    
 
        state[i] = [n,p,d,p0]

    #return As, ns2, ps2, dpp0, p0, tF, tD, A
    return As, state

In [ ]:
def STP_sim_complex(ge_data,l_pre_post2,stp_aba_names):
    # transform labels from TM to An:A1
    #fs = [20, 50, 10] # Hz
    fs = [20, 50, 10]
    N = 5

    #Trec = [250, 500, 1000]
    Trec =[250, 1000]
    DT0 = 25000
    if l_pre_post2<ge_data.shape[0]:
        xs  =ge_data.iloc[l_pre_post2:,:].loc[:,stp_aba_names].values
        xs  =np.delete(xs, [5,6],axis=1)
        As2=np.zeros((xs.shape[0],0))
    
    
    As=np.zeros((l_pre_post2,0))
    if 1:
        raz=0
        
        for f in fs:
            raz=raz+1
            T = np.arange(N)*1000/f

            if l_pre_post2>0:
                if raz==1:
                    print('model type = 4 parameters TM')
                Asf, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data.iloc[0:l_pre_post2,:],T)
                As = np.concatenate([As,Asf],axis=1)

                for ri in range(len(Trec)):
                    Asr, nsr, psr, dpp0, p0, tF, tD, A = STP_sim(ge_data.iloc[0:l_pre_post2,:],[Trec[ri]],init_state=[ns[:,-1],ps[:,-1]])
                    As = np.concatenate([As,Asr],axis=1)

            if l_pre_post2<ge_data.shape[0]:
                if raz==1:
                    print('model type =  TM 5 parameters , smr')
                    
                As2f = np.zeros((xs.shape[0],N+len(Trec)))
                for i2 in range(xs.shape[0]):
                    #as2, sts2 = STP_sim2(xs[i2,:],np.arange(8)*50,model_type = 'tm5') # preconditioning series???
                    #as2, sts2 = STP_sim2(xs[i2,:], T+DT0, init_state=sts2[-1], model_type='tm5' ) 

                    as2, sts2 = STP_sim2(xs[i2,:],T,model_type = 'tm5_smr')

                    As2f[i2,0:N] = as2
                    for ri in range(len(Trec)):
                        as2r, sts2r = STP_sim2(xs[i2,:], [Trec[ri]], init_state=sts2[-1], model_type='tm5_smr' ) 
                        As2f[i2,N+ri] = as2r

                As2 = np.concatenate([As2,As2f],axis=1)

        if l_pre_post2<ge_data.shape[0]:
            As2 = As2/As2[:,0].reshape((-1,1))
        else:
            As2  = np.zeros((0,As.shape[1]))
        
        
        if l_pre_post2>0:
            As = As/As[:,0].reshape((-1,1))
            As = np.concatenate([As,As2],axis=0)
        else:
            As =As2
    else:
        As, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data,T)


    #import matplotlib.pyplot as plt    
    #plt.plot(As2[400:410,:].transpose(),'o-')
    
    return As

## 
##
## Hippocampus gene expression
##
## 

In [ ]:
import scvi

In [ ]:
import os
print(os.getcwd())
#os.listdir()

if do_laptop:
    from scvi.dataset import LoomDataset, CsvDataset, Dataset10X, AnnDataset
else:
    from scvi.dataset import DownloadableAnnDataset

d_aba  ='/home/stepaniu/Documents/references/transcriptomes to STP/transcriptomes/ABA_2019/'
if do_laptop:
    d_aba  ='/Users/stepaniu/Documents/jan_2020/ABA_2019/'

df_aba_g = pd.read_csv(d_aba + 'medians.csv')
df_aba_c = pd.read_csv(d_aba + 'sample_annotations.csv')


d7 = '/home/stepaniu/Documents/references/transcriptomes to STP/transcriptomes/ABA_2019/'
if do_laptop:
    d7  ='/Users/stepaniu/Documents/jan_2020/ABA_2019/'

if do_laptop:    
    aba_vis_in = AnnDataset("ABA_2019_transcriptome.h5ad", new_n_genes = 45768,
                             save_path = d7) 
else:     
    aba_vis_in = DownloadableAnnDataset(d7 +"ABA_2019_transcriptome.h5ad") 

obs = pd.read_excel(d7 +'ABA_2019_obs.xlsx') # samples
var = pd.read_excel(d7 + 'ABA_2019_var.xlsx') # gene names
#obs = aba_vis_in._obs # samples
#var = aba_vis_in._var # gene names

in_gaba = df_aba_c.loc[df_aba_c.loc[:,'sample_name'].isin(obs.loc[:,'samples']),['sample_name', 'class_label']]
in_gaba = in_gaba.reset_index().set_index('sample_name',drop=True)
in_gaba = in_gaba.loc[obs.loc[:,'samples'],:]
in_gaba = in_gaba.reset_index().reset_index().set_index('sample_name',drop=True)
in_gaba = in_gaba.loc[in_gaba.loc[:,'class_label']=='GABAergic',:].reset_index().set_index('level_0',drop=True)

#aba_vis_in.gene_names = var.iloc[:,0].values
aba_vis_in.gene_names = var.loc[:,'genes'].values
vis_dat = aba_vis_in
aba_vis_in = []

in_ds = df_aba_c.loc[df_aba_c.loc[:,'sample_name'].isin(obs.loc[:,'samples']),['sample_name', 'class_label']]
in_ds = in_ds.reset_index().set_index('sample_name',drop=True)
in_ds = in_ds.loc[obs.loc[:,'samples'],:]
in_ds = in_ds.reset_index().reset_index().set_index('sample_name',drop=True)
#in_ds

df_aba_vis_in_c = df_aba_c.loc[in_ds.loc[:,'index'],:]

### add scVI latent factors and imputed genes

In [ ]:
do_replace_to_scVI_imputed=1      # replace raw gene expression data with their scvi imputation
do_add_scvi_latent_factors=1      # add scvi latent factors data
do_use_all_scvi_genes=0           # use all scvi genes expression in addition to pregs, postgs

gs_name = '_gs5188' #'_gs1512' #'_gs5188' #'_gs1512' #'_gs15656' #'_gs5188' #'_gs1512' #'_gs219'
if (gs_name == '_gs1512')|(gs_name == '_gs5188')|(gs_name == '_gs15656'):
    nlf = 20
elif gs_name == '_gs219':
    nlf = 10
    
if (do_replace_to_scVI_imputed==1)|(do_add_scvi_latent_factors==1):
    scVI_all = pd.read_hdf('scVI_latent_factors_and_imputation_aba2019'+gs_name+'.hdf',key='data') # this is 290 genes dataset
    
    scVI_all = scVI_all.set_index('sample_name') # scvi imputad genes and latent factors for hipp., enth. and vis. cortex

    
    gncolumns = scVI_all.columns[nlf+1:] # gene names columns in scvi data
    lfcolumns = scVI_all.columns[:nlf]     # latent factors columns in scvi data

    gnh = vis_dat.gene_names               # hippocampal vis_dat gene names
    smh = vis_dat.obs                      # hippocampal vis_dat samples names

    smh2 = smh.reset_index().set_index('samples')  # hippocampal samples  and their indexes in vis_dat X

    gnscvi = pd.DataFrame(gncolumns)   # df from scvi genenames

    gnh2 = pd.DataFrame(gnh).reset_index().set_index(0)   
    ignh2 = gnh2.loc[gnscvi.loc[:,0],:].loc[:,'index'].values  # indeces of scvi gene names in vis_dat gene names

    if (do_replace_to_scVI_imputed==1):
        vis_dat._X[:,ignh2] = scVI_all.loc[smh2.index,gncolumns].values  # modified vis_dat matrix (imputed part of genes )
    if (do_add_scvi_latent_factors==1):
        scVI_lf_h = scVI_all.loc[smh2.index,lfcolumns]                   # latent factors for hippocampal samples
        vis_dat._X = np.concatenate([vis_dat._X, scVI_lf_h.values],axis=1)
        
        lf_scvi_columns = lfcolumns.astype(str)
        vis_dat.gene_names = np.concatenate([vis_dat.gene_names, lf_scvi_columns])
        pregs = pregs + lf_scvi_columns.tolist()
        postgs = postgs + lf_scvi_columns.tolist()
    if (do_use_all_scvi_genes==1): 
        #pregs = list(set(pregs + gncolumns.tolist()))
        #postgs = list(set(postgs + gncolumns.tolist()))
        pregs = gncolumns.tolist()+ lf_scvi_columns.tolist()
        postgs = gncolumns.tolist()+ lf_scvi_columns.tolist()
        

In [ ]:
vis_dat.gene_names.shape

In [ ]:
print(len(pregs))
len(postgs)


##
##
##  HIPPOCAMPUS STP
##
##

In [ ]:
#do_laptop=1

In [ ]:
#
#
#  HIPPOCAMPUS STP
#
#
import re
import time
t1 = time.time()


# names of gene expression based synapses types

glu_l23=[] #list(set(df_aba_vis_l23glu_c.loc[:,'cluster']))

dsg = [['PC']+glu_l23,
       ['Pvalb','Pvalb Tpbg','Pvalb Reln Itm2a','Pvalb Reln Tac1','Pvalb Sema3e Kank4'], 
       ['Sst','Sst Calb2 Pdlim5','Sst Calb2 Necab1', 'Sst Hpse Cbln4','Sst Hpse Sema3c','Sst Nr2f2 Necab1'],
       ['Vip','Vip Lmo1 Myl1', 'Vip Rspo1 Itga4','Vip Ptprt Pkp2','Vip Rspo4 Rxfp1 Chat'],
       ['Lamp5','Lamp5 Plch2 Dock5'],
       ['MC','L23MC','Sst Calb2 Pdlim5'],
        'L5MC']
      

#set(df_stp.loc[:,'synapse_type_2'])

# names of electrophysiology based synapses types
dse = [['PC', 'L23P', 'L23PC', 'L5P'],
       ['Pvalb', 'PV', 'L23BC', 'L5BC'], 
       ['Sst', 'L23MC', 'L5MC', 'Sst', 'MC'], 
       ['Vip', 'VIP', 'BTC'],
       ['NGC', 'L23NGC'],
       ['L23MC' ],
       ['L5MC']]


# load stp data table
#do_laptop=0
if do_laptop:
    #d5 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/additional cortical STP data/'
    d5 = '/Users/stepaniu/Documents/jan_2020/'
else:
    d5 = '/home/stepaniu/Documents/references/transcriptomes to STP/scVI_data/additional cortical STP data/'
#fn5='STP_cortex_interneurons_Cre_labeled.xlsx'
#fn5='STP_cortex_interneurons_Cre_labeled_UDF2.xlsx'
#fn5='STP_cortex_interneurons_Cre_labeled_UDF3.xlsx'

do_3pars_TM=0 # 3 or 4 parametric TM model?
if do_3pars_TM:
    fn5='STP_cortex_interneurons_Cre_labeled_UDF5_3pars.xlsx'
else:
    #fn5='STP_cortex_interneurons_Cre_labeled_UDF5_4pars.xlsx'
    fn5='STP_hippocampus_all_UDF3.xlsx'

df_stp=pd.read_excel(d5+fn5)
#df_stp.loc[df_stp.loc[:,'stp_comment'].isna() ,'stp_comment']=''

############   
do_averagig_stp =0
stp_TM_names=['tF', 'p0','tD','dp/p0','A']
stp_TM_start = np.nonzero(df_stp.columns=='references')[0][0]+1 # 187
ncol = len(df_stp.columns)
df_tm_name = pd.DataFrame(np.zeros(ncol).reshape((1,ncol)), columns =df_stp.columns )
df_stp = pd.concat([df_tm_name, df_stp],axis=0).reset_index(drop=True)

for i,nm in enumerate(stp_TM_names):
    ii = stp_TM_start + np.arange(0,ncol-i-stp_TM_start,len(stp_TM_names))+i
    df_stp.iloc[0,ii] = nm


#print(df_stp.head())
#############
##Remove pure PPR data (Jaing_2015_PC)
#df_stp = df_stp.loc[df_stp.loc[:,'name'].str.contains('Jaing_2015_PC')!=True,:].reset_index(drop=True) 

############


#stp_TM_names_2=['tF', 'p0','tD','dp','A','A1','A2','tDmin','dd','t_FDR','t_SMR','dp0'] # tm5+smr
##################
#stp_2_default = np.ones(7)
#for i in range(1,df_stp.shape[0]):
#    stp_par = df_stp.iloc[i,stp_TM_start:].values.reshape((len(stp_TM_names),-1))
    

##################

df_stp

## Hippocampus: allign gene expression and stp data (for selected gene sets)

In [ ]:
# Parameters:
do_bootstrap=1
do_median=0

nsample_hipp = 200

#for cell types diagrams : do_bootstrap=1
#do_median=0
#nsample_hipp = 400

do_log='none'

In [ ]:
# rules for cell types to aba clusters mapping:
#1 dorsal : 'subregion_label'== 'anterior' , ventral: 'subregion_label'== posterior' ,  or by genes : see Cembrowski 2016
#2 TA = ECIII (PCP4+) ->CA1 
#3 pp = ECII (Reelin+) ->CA3, DG, CA2
#4 TA = ECII (Wlf1+, Calb2+) ->CCK ppa in CA1 mostly
#5 no cre lines annotations - use only old cck datatypes->clusters or corresponding cortical cre lines?
#6 Ivy 1, 2 ?? - 2 clusters (Lhx6 1,2) and may be Lhx6 3 (no reelin), but also could be some Lamp5_5? - less reelin, more Nos1 
#  use only only PPF Ivy cells? or try 2 subtypes = 2 clusters?? - dep=lhx6_4
#7 NGF types 1,2? Price 2005 - correlate with age - select only facilitating (type 2)!
#8 HIPP - Sst, pp-associated ~OLM??
#9 HICAP - CCk - comissural assoc - ~ CCK Schffer comm : Sncg6 (Sncg/Ndnf HPF1)
#10 Kohus: CCK DTI vs BC? - use Cbln4, Lgi2 ratio? (Favuzzi 2019) - Sncg2?, (Sncg6) - DTI vs Sncg/Ndnf HPF2,3,6, Serpinf1, Sncg7, Vip_14
#11 CA1 PC: use subregions: anterior, mid-anterior  for dorsal



# batches and gene sets
pre_post = df_stp.loc[:,'synapse_type_2'].str.split(pat='->',expand=True)

vbi = vis_dat.batch_indices.reshape((vis_dat.batch_indices.shape[0],))
vbi=vbi>0

df_gs  = pd.DataFrame( vis_dat.gene_names ) 
in_pregs = df_gs.loc[:,0].isin(pregs).values
in_postgs = df_gs.loc[:,0].isin(postgs).values

pregs2 = df_gs.loc[in_pregs,0].values
postgs2 = df_gs.loc[in_postgs,0].values

do_hippocampus = 1
if do_hippocampus ==0:

    do_2=1;
    # select stp data for appropriate frequency and ages 
    stp_type = {'name':['Walker_2016_PV->MC L23',
                   'Walker_2016_VIP->MC L23',
                   'Yuste_2016_taus_PV->Sst L23 recalc',
                   'Yuste_2016_taus_PV->VIP L23 recalc',
                   'Yuste_2016_taus_Sst->VIP L23 recalc',
                   'Yuste_2016_taus_VIP->Sst L23 recalc',
                   'Yuste_2016_taus_VIP->VIP L23 recalc'],
            'stp_freq': [20, 40, 50], 'stp_comment':  ['', ' P20-21', ' P20-22', ' P20-23', ' P25-31'],
            'stp_data_type': df_stp.loc[0, :]!=0 }

    if do_2==1:
        stp_type = {'name':['Walker_2016_PV->MC L23',
                    'Walker_2016_VIP->MC L23',
                    'Yuste_2016_taus_PV->Sst L23 recalc',
                    'Yuste_2016_taus_VIP->Sst L23 recalc'],
            'stp_freq': [20, 40, 50], 'stp_comment':  ['', ' P20-21', ' P20-22', ' P20-23', ' P25-31'],
            'stp_data_type': df_stp.loc[0, :]!=0 }




    in_stp_data = df_stp.loc[:,'stp_freq'].isin([20])
    in_stp_data= in_stp_data|(df_stp.loc[:,'stp_freq'].isin([40,50])&df_stp.loc[:,'name'].isin(stp_type['name']))
    in_stp_data = in_stp_data&df_stp.loc[:,'stp_comment'].isin(pd.Series(stp_type['stp_comment']).str.strip())

else:
    
    in_stp_data = pd.Series(np.ones(df_stp.shape[0])==1)
    # filter stp dataset 
    in_stp_data.loc[df_stp.loc[:,'name'].isin(['Price_2005EC->NGF type1',
                                               'Qin_2017EC->Ivy cell type1',
                                               'Qin_2017CA3->Ivy cell type1'])]=False
    #rem = df_stp.loc[:,'name'].str.contains('HIPP').fillna(value=True)
    #in_stp_data.loc[rem]=False
    
    # 17–20 Sprague-Dawley rats?? - too high depression? - 0.39
    rem = df_stp.loc[:,'name'].str.contains('Fuentealba_2008CA1->Ivy cell').fillna(value=True)
    in_stp_data.loc[rem]=False    
    
# show selected stp dataset
ddd=df_stp.loc[in_stp_data,['name','synapse_type_2', 'stp_freq','area']]
print(ddd)
print(in_stp_data.sum())

synij = np.repeat(False,in_stp_data.shape[0])

cols2 = ['name','perc']
cl2post =pd.DataFrame([],columns= cols2)
cl2pre =pd.DataFrame([],columns= cols2)


# strip cluster names
#list(set(df_aba_vis_in_c.loc[df_aba_vis_in_c.loc[:,'subclass'].str.contains(clns6),'cluster']))
df_aba_vis_in_c.loc[:,'cluster_label']=df_aba_vis_in_c.loc[:,'cluster_label'].str.strip()
#df_aba_vis_l23glu_c.loc[:,'cluster']=df_aba_vis_l23glu_c.loc[:,'cluster'].str.strip()

do_simplfy_clusters = 1 # do not take into account minor subclasses
Sst_NonMC ='Sst, Sst Calb2 Pdlim5-, Sst Chrna2 Glra3-, Sst Chrna2 Ptgdr-, Sst Myh8 Etv1-, Sst Tac2 Myh4-, Sst Nr2f2 Necab1-, Sst Chodl-,  Sst Calb2 Necab1-, Sst Tac1 Htr1d-, Sst Myh8 Fibin-';

pregs3 = ['pre__'+s for s in pregs2.tolist()]
postgs3 = ['post__'+s for s in postgs2.tolist()]
ge_columns = ['cell_type2_pre', 'cell_type2_post',
                                             'layer_pre','layer_post',
                                             'cre_line_pre', 'cre_line_post'] +pregs3 +postgs3 + ['samples_pre', 'samples_post']

stp_columns = stp_TM_names #list(set(df_stp.loc[0,:]).difference(set([0])))

# output datasets
stp_data = pd.DataFrame([], columns =['presynaptic', 'postsynaptic', 'stp_data'] )
ge_data = pd.DataFrame([], columns =ge_columns+stp_columns+['index_ds'])


for i in range(1,len(pre_post.index)): #range(1,len(pre_post.index)): # [1]: 
    
    # syne3 - all available pre- and post- synaptic cells annotations : cre-lines, layers, morphologies
    l = str.strip(df_stp.loc[i,'area'])
    synel = re.split(';',l)
    if len(synel)==1:
        l1=l
    else:
        l  = str.strip(synel[1])
        l1  = str.strip(synel[0])
            
    
    if type(df_stp.loc[i,'synapse_type'])!=float:
        #syne1 = re.split('\;',df_stp.loc[i,'synapse_type']) no cre-line annotations for hippocampal data
        syne1 = ''
    else:
        syne1 =''    
    #syne2 = df_stp.loc[i,'synapse_type2'].str.split(pat='->',expand=True)
    syne2 = re.split('->',df_stp.loc[i,'synapse_type_2'])
    if len(syne2)==1:
        pat ='→'
        syne2 = re.split(pat,df_stp.loc[i,'synapse_type_2'])

    syne3 ={'layer_pre':l1,  'cre_line_pre':'none', 'cell_type2_pre':str.strip(syne2[0]),'cluster_pre':[],
            'layer_post':l, 'cre_line_post':'none','cell_type2_post':str.strip(syne2[1]),'cluster_post':[],  } 

    # add cre-lines when available
    if len(syne1)>1:
        syne3['cre_line_pre']  =str.strip(syne1[0])
        syne3['cre_line_post'] =str.strip(syne1[1])
    
    # PV BC
    if (syne3['cell_type2_post']=='PV')|(syne3['cell_type2_post']=='PVBC'):  
        syne3['cluster_post']=[['Pvalb_3, Pvalb_10, Pvalb_4, Pvalb_6',100]] # modify!!! 
    if (syne3['cell_type2_pre']=='PV')|(syne3['cell_type2_pre']=='PVBC'):  
        syne3['cluster_pre']=[['Pvalb_3, Pvalb_10, Pvalb_4, Pvalb_6',100]] # modify!!!   
                     
  
    # HIPP -> HIPP Bartos
    if syne3['cell_type2_post']=='HIPP':  
        if syne3['layer_post'] == 'DG':
            syne3['cluster_post']=[['Sst_15, Sst_24, Sst_13, Sst_6',100]] # modify!!!        
    if syne3['cell_type2_pre']=='HIPP':  
        if syne3['layer_pre'] == 'DG':
            syne3['cluster_pre']=[['Sst_15, Sst_24, Sst_13, Sst_6',100]] # modify!!! 
            
            
    # HICAP -> HICAP Bartos
    if syne3['cell_type2_post']=='HICAP':  
        if syne3['layer_post'] == 'DG':
            syne3['cluster_post']=[['Sncg6, Sncg/Ndnf HPF_1',100]] # modify!!!        
    if syne3['cell_type2_pre']=='HICAP':  
        if syne3['layer_pre'] == 'DG':
            syne3['cluster_pre']=[['Sncg6, Sncg/Ndnf HPF_1',100]] # modify!!!      
            
            
    # O-LM
    if syne3['cell_type2_post']=='O-LM':  
        if syne3['layer_post'] == 'CA1':
            syne3['cluster_post']=[['Sst_15, Sst_24, Sst_13, Sst_6',100]] # modify!!!     
            
    # CCK BC
    if syne3['cell_type2_post']=='CCKBC':  
        syne3['cluster_post']=[['Sncg/Ndnf HPF_2, Sncg/Ndnf HPF_3, Sncg/Ndnf HPF_6, Serpinf1_1, Sncg_7, Vip_14',100]] # modify!!!    
    if syne3['cell_type2_pre']=='CCKBC':  
        syne3['cluster_pre']=[['Sncg/Ndnf HPF_2, Sncg/Ndnf HPF_3, Sncg/Ndnf HPF_6, Serpinf1_1, Sncg_7, Vip_14',100]] # modify!!!    

    
    # CCK DTI
    if syne3['cell_type2_post']=='CCK_DTI':  
        syne3['cluster_post']=[['Sncg/Ndnf HPF_1, Sncg6, Sncg2',100]] # modify!!!    
    if syne3['cell_type2_pre']=='CCK_DTI':  
        syne3['cluster_pre']=[['Sncg/Ndnf HPF_1, Sncg6, Sncg2',100]] # modify!!!    

        
        
    # O-Bi
    if syne3['cell_type2_post']=='O-Bi':  
        if syne3['layer_post'] == 'CA1':
            syne3['cluster_post']=[['Sst_27, Pvalb_1, Sst_25',100]] # modify!!!   
     
    # AAC
    if syne3['cell_type2_pre']=='AAC':  
        if syne3['layer_pre'] == 'CA3':
            syne3['cluster_pre']=[['Pvalb_12, Pvalb_11',100]] # modify!!!   
    if syne3['cell_type2_post']=='AAC':  
        if syne3['layer_post'] == 'CA3':
            syne3['cluster_post']=[['Pvalb_12, Pvalb_11',100]] # modify!!!         

    # NGF    ? type2 vs type1? - remove type 1?  
    if syne3['cell_type2_pre']=='NGF':  
        syne3['cluster_pre']=[['Lamp5_2, Lamp5_3, Lamp5_4, Lamp5_5, Lamp5 Lhx6_2, Lamp5 Lhx6_3',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    if (syne3['cell_type2_post']=='NGF')|(syne3['cell_type2_post']=='NGF type1')|(syne3['cell_type2_post']=='NGF type2'):  
        syne3['cluster_post']=[['Lamp5_2, Lamp5_3, Lamp5_4, Lamp5_5, Lamp5 Lhx6_2, Lamp5 Lhx6_3',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   
     
    # Ivy cells   ? type2 vs type1? : CGE vs MGE? - no CGE is seen? or Lhs1 vs Lhs4? 
    if syne3['cell_type2_pre']=='Ivy cell':  
        syne3['cluster_pre']=[['Lamp5 Lhx6_1, Lamp5 Lhx6_4',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    if (syne3['cell_type2_post']=='Ivy cell')|(syne3['cell_type2_post']=='Ivy cell type1')|(syne3['cell_type2_post']=='Ivy cell type2'):  
        syne3['cluster_post']=[['Lamp5 Lhx6_1, Lamp5 Lhx6_4',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   
       
    #IS3
    if syne3['cell_type2_pre']=='IS3':  
        syne3['cluster_pre']=[['Vip_6, Vip_12, Vip_15, Vip_17',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    
    ###### EXCitatory
    # CA1 PC        
    if (syne3['cell_type2_pre']=='CA1'): 
        syne3['cell_type2_pre']=='PC'
        if syne3['layer_pre'] == 'CA1':
            syne3['cluster_pre']=[['CA1sp_7, CA1sp_8',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    if (syne3['cell_type2_post']=='CA1'):  
        syne3['cell_type2_post']=='PC'
        if syne3['layer_post'][0:3] == 'CA1':
            syne3['cluster_post']=[['CA1sp_7, CA1sp_8',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   
 
            
    # CA3 PC        
    if (syne3['cell_type2_pre']=='CA3'): 
        syne3['cell_type2_pre']=='PC'
        if syne3['layer_pre'] == 'CA3':
            syne3['cluster_pre']=[['CA3sp_6, CA3sp_5, CA3sp_7, CA3sp_1',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    if (syne3['cell_type2_post']=='CA3'):  
        syne3['cell_type2_post']=='PC'
        if syne3['layer_post'] == 'CA3':
            syne3['cluster_post']=[['CA3sp_6, CA3sp_5, CA3sp_7, CA3sp_1',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   
    
    # CA3 PC ventral
    if (syne3['cell_type2_pre']=='CA3')|(syne3['cell_type2_pre']=='PC'): 
        syne3['cell_type2_pre']=='PC'
        if (syne3['layer_pre'] == 'CA3 ventral')&(syne3['layer_post'] == 'CA1 ventral'):
            syne3['cluster_pre']=[['CA3sp_2, CA3sp_4',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   

    
    
 
    # ECII SS        
    if (syne3['cell_type2_pre']=='EC'): 
        syne3['cell_type2_pre']=='PC'
        if (syne3['layer_pre'] == 'EC')&((syne3['layer_post'] == 'DG')|(syne3['layer_post'] == 'CA3')):
            syne3['cluster_pre']=[['L2/3 IT Ndst4 Endou_2',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       


    # ECIII PC        
    if (syne3['cell_type2_pre']=='EC'):
        syne3['cell_type2_pre']=='PC'
        if (syne3['layer_pre'] == 'EC')&((syne3['layer_post'][0:3] == 'CA1')):
            syne3['cluster_pre']=[['L2/3 IT Plch1_1, L2/3 IT Plch1_2',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       

            
    # CA1 PC ventral       
    if (syne3['cell_type2_pre']=='CA1 ventral'):  
        syne3['cell_type2_pre']=='PC'
        if syne3['layer_pre'] == 'CA1 ventral':
            syne3['cluster_pre']=[['CA1sp_1, CA1sp_2, CA1sp_3',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    if (syne3['cell_type2_post']=='CA1 ventral'):  
        syne3['cell_type2_post']=='PC'
        if syne3['layer_post'] == 'CA1 ventral':
            syne3['cluster_post']=[['CA1sp_1, CA1sp_2, CA1sp_3',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   
    
    

    # DG PC      
    if (syne3['cell_type2_pre']=='DG'):
        syne3['cell_type2_pre']=='PC'
        #if syne3['layer_pre'] == 'DG':
        syne3['cluster_pre']=[['DG_1, DG_2, DG_3',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    if (syne3['cell_type2_post']=='DG'): 
        syne3['cell_type2_post']=='PC'
        #if syne3['layer_post'] == 'DG':
        syne3['cluster_post']=[['DG_1, DG_2, DG_3',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   
   
    
    
    ###########  Select data:
    ##synij = (pre_post.loc[:,0].isin(pree))&(pre_post.loc[:,1].isin(poste))
    

    synij[:] = False
    synij[i] = in_stp_data[i]
    
    #print(syne3)
    
    if sum(synij)>0:
        print('\n\n')
        print('i = ',str(i))
        print(syne3)
        #if sum(in_stp_data&synij)>0:
        #    in_stp_data = in_stp_data&synij

        #add ge data
        #'layer_pre':l,  'cre_line_pre':'', 'cell_type2_pre':syne2[0].str.strip(),'cluster_pre':[]
        
        # PC or GABAergic dataset?
        batch = [ syne3['cell_type2_pre']=='PC',syne3['cell_type2_post']=='PC']
        
        # add ge data depending on available stp cell types labels:
        #  select presynaptic cells
        if 0: #batch[0]:
            #in_preg  = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']]) #modify!!!
            in_preg  = df_aba_vis_l23glu_c.loc[:,'brain_subregion']==syne3['layer_pre'] #modify!!!
        else:
            if len(syne3['cluster_pre'])==0:
                # if len(syne3['cre_line_pre'])>0:
                in_preg = df_aba_vis_in_c.loc[:,'full_genotype_label'].str.contains(syne3['cre_line_pre'])
                ##in_preg = in_preg&df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']])
                #in_preg = in_preg&df_aba_vis_in_c.loc[:,'brain_subregion'].str.contains(syne3['layer_pre'])
            else:                  
                ##in_preg = df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']])
                in_preg = df_aba_vis_in_c.loc[:,'region_label'].str.contains(syne3['layer_pre'])
                in_preg[:] = True
                
                cl = syne3['cluster_pre']
                #cols2 = ['name','perc']
                #cl2pre =pd.DataFrame([],columns= cols2)
                in_preg0 = in_preg.copy()
                in_preg[:] = False 
                for icl in range(len(cl)):
                    clp = cl[icl][1]
                    clns = re.split(',',cl[icl][0])
                    cls3 = []
                    cls3m=[]
                    for icls in range(len(clns)):
                        clns4 = str.strip(clns[icls])
                        clns5 = re.split('\_',clns4)
                        if len(clns5)<2:
                            clns6=clns5[0]
                            cls3=cls3 + list(set(df_aba_vis_in_c.loc[df_aba_vis_in_c.loc[:,'subclass_label'].str.contains(clns6),'cluster_label']))
                        else:
                            clns4=str.strip(clns4)
                            clns7=re.split('\-',clns4)
                            if len(clns7)>1:
                                clns4=clns7[0]
                                cls3m=cls3m+[str.strip(clns4)]
                            else:
                                cls3=cls3+[str.strip(clns4)]
                    cls3 = list(set(cls3).difference(set(cls3m)))
                    cl22=pd.DataFrame([cls3,[clp]],index=cols2).T
                    cl2pre = cl2pre.append(cl22)
                    
                    in_preg = in_preg|(in_preg0&df_aba_vis_in_c.loc[:,'cluster_label'].isin(cls3))
                    
                if (len(syne3['cre_line_pre'])>0)&(syne3['cre_line_pre']!='none'):
                    #in_preg = in_preg&df_aba_vis_in_c.loc[:,'driver_lines'].str.contains(syne3['cre_line_pre'])
                    in_line=df_aba_vis_in_c.loc[:,'full_genotype_label'].str.contains(syne3['cre_line_pre'])
                    print('is pre cre line')
                    if in_line.sum()>0:
                        in_preg = in_preg&in_line                        
        
        #  select postsynaptic cells
        if 0: #batch[1]:
            #in_postg = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin([syne3['layer_post']]) #modify!!!
            in_postg = df_aba_vis_l23glu_c.loc[:,'brain_subregion']==syne3['layer_post'] #modify!!!
        else:    
            if len(syne3['cluster_post'])==0:
                # if len(syne3['cre_line_post'])>0:
                in_postg = df_aba_vis_in_c.loc[:,'full_genotype_label'].str.contains(syne3['cre_line_post'])
                #in_postg = in_postg&df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_post']])
                #in_postg = in_postg&df_aba_vis_in_c.loc[:,'brain_subregion_label'].str.contains(syne3['layer_post'])
            else:    
                ## under construction ...
                ##in_postg = df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_post']])
                in_postg = df_aba_vis_in_c.loc[:,'region_label'].str.contains(syne3['layer_post'])
                in_postg[:] = True
                
                cl = syne3['cluster_post']
                #cols2 = ['name','perc']
                #cl2post =pd.DataFrame([],columns= cols2)
                in_postg0 = in_postg.copy()
                in_postg[:] = False                 
                for icl in range(len(cl)):
                    clp = cl[icl][1]
                    clns = re.split(',',cl[icl][0])
                    cls3 = []
                    cls3m=[]
                    for icls in range(len(clns)):
                        clns4 = str.strip(clns[icls])
                        clns5 = re.split('\_',clns4)
                        if len(clns5)<2:
                            clns6=clns5[0]
                            cls3=cls3 + list(set(df_aba_vis_in_c.loc[df_aba_vis_in_c.loc[:,'subclass_label'].str.contains(clns6),'cluster_label']))
                        else:
                            clns4=str.strip(clns4)
                            clns7=re.split('\-',clns4)
                            if len(clns7)>1:
                                clns4=clns7[0]
                                cls3m=cls3m+[str.strip(clns4)]
                            else:
                                cls3=cls3+[str.strip(clns4)]
                    cls3 = list(set(cls3).difference(set(cls3m)))
                    cl22=pd.DataFrame([cls3,[clp]],index=cols2).T
                    cl2post = cl2post.append(cl22)
                    
                    #in_postg = in_postg&df_aba_vis_in_c.loc[:,'cluster'].isin(cls3)
                    in_postg = in_postg|(in_postg0&df_aba_vis_in_c.loc[:,'cluster_label'].isin(cls3))
                    
                if (len(syne3['cre_line_post'])>0)&(syne3['cre_line_post']!='none'):
                    in_line=df_aba_vis_in_c.loc[:,'full_genotype_label'].str.contains(syne3['cre_line_post'])
                    print('is post cre line')
                    if in_line.sum()>0:
                        in_postg = in_postg&in_line
            
           
            
        # make equal size pre- and post- datasets
        in_preg = in_preg&(in_preg.isna()==False)
        in_postg = in_postg&(in_postg.isna()==False)
        npre = in_preg.sum()
        npost = in_postg.sum()
        nsample = max([npost, npre] )
        #nsample = min([npost, npre] ) # number of cells
        
        if (nsample<60)&(npre>0)&(npost>0):
            nsample=60
            
        if (nsample>180)&(npre>0)&(npost>0):  # restrict nsample to avoid overrepresentation?
            nsample=180    
            
        print('npre = '+ str(npre)+' , npost = ' + str(npost) + ' , nsample = '+str(nsample))    
        
        
        ## select cell indices for gene expression sampling
        #do_bootstrap=1
        #if  do_bootstrap==0:
        #    idx2 =   np.mod(np.arange(nsample),npre) 
        #    in_preg = np.nonzero(in_preg)[0][idx2]
        #    idx2 =   np.mod(np.arange(nsample),npost) 
        #    in_postg = np.nonzero(in_postg)[0][idx2]
        #else :
        #    #random.uniform(low=0.0, high=1.0, size=None)
        #    idx2 =  np.floor( np.random.uniform(0,npre, nsample) ).astype('int')
        #    in_preg = np.nonzero(in_preg)[0][idx2]
        #    idx2 =  np.floor( np.random.uniform(0,npost, nsample) ).astype('int')
        #    in_postg = np.nonzero(in_postg)[0][idx2]
            
            
        # select cell indices for gene expression sampling
        
                
        ##
        ##
        ##  PARAMETERS OF GS AVERAGING
        ##
        ##
        do_bootstrap=1
        if  do_bootstrap==0:
            
            if (nsample<60)&(npre>0)&(npost>0):
                nsample=60
            
            if (nsample>180)&(npre>0)&(npost>0):  # restrict nsample to avoid overrepresentation?
                nsample=180 
            
            idx2 =   np.mod(np.arange(nsample),npre) 
            in_preg = np.nonzero(in_preg)[0][idx2]
            idx2 =   np.mod(np.arange(nsample),npost) 
            in_postg = np.nonzero(in_postg)[0][idx2]
            
            in_preg_ = in_preg
            in_postg_ = in_postg
        else :
            if (npre>0)&(npost>0):
                nsample= nsample_hipp #= 200
                
            #random.uniform(low=0.0, high=1.0, size=None)
            idx2pre =  np.floor( np.random.uniform(0,npre, (nsample,1)) ).astype('int')
            in_preg_bs = np.nonzero(in_preg)[0][idx2pre]
            idx2post =  np.floor( np.random.uniform(0,npost, (nsample,1)) ).astype('int')
            in_postg_bs = np.nonzero(in_postg)[0][idx2post]
            in_preg_ = in_preg_bs[:,0]
            in_postg_ = in_postg_bs[:,0]    
            


        
        sampl_post =df_aba_vis_in_c.iloc[in_postg_,:].loc[:,'sample_name']
        sampl_pre =df_aba_vis_in_c.iloc[in_preg_,:].loc[:,'sample_name']
        
        if batch[0]:
            #in_preg_cl = list(set(df_aba_vis_l23glu_c.iloc[in_preg,:].loc[:,'cluster']))
            cl_pre = df_aba_vis_in_c.iloc[in_preg_,:].loc[:,'cluster_label']
            in_preg_cl = cl_pre.value_counts()
            
        else:
            #in_preg_cl = list(set(df_aba_vis_in_c.iloc[in_preg,:].loc[:,'cluster']))
            cl_pre = df_aba_vis_in_c.iloc[in_preg_,:].loc[:,'cluster_label']
            in_preg_cl = cl_pre.value_counts()
            #in_preg_cl = df_aba_vis_in_c.iloc[in_preg_,:].loc[:,'cluster_label'].value_counts()
        
        if batch[1]:
            #in_postg_cl = list(set(df_aba_vis_l23glu_c.iloc[in_postg,:].loc[:,'cluster']))
            cl_post =df_aba_vis_in_c.iloc[in_postg_,:].loc[:,'cluster_label']
            in_postg_cl = cl_post.value_counts()
            #in_postg_cl = df_aba_vis_in_c.iloc[in_postg_,:].loc[:,'cluster_label'].value_counts()
        else:        
            #in_postg_cl = list(set(df_aba_vis_in_c.iloc[in_postg,:].loc[:,'cluster']))
            
            cl_post =df_aba_vis_in_c.iloc[in_postg_,:].loc[:,'cluster_label']
            in_postg_cl = cl_post.value_counts()
            #in_postg_cl = df_aba_vis_in_c.iloc[in_postg_,:].loc[:,'cluster_label'].value_counts()
        in_preg_cl = in_preg_cl/in_preg_cl.sum()*100    
        in_postg_cl = in_postg_cl/in_postg_cl.sum()*100    
        print('presynaptic clusters')
        print(in_preg_cl)
        print('postsynaptic clusters')
        print(in_postg_cl)

        #if npost>npre:
        #    idx = np.mod(np.arange(npost),npre)
        #    #prege = prege[idx,:]
        #    #postge = postge
        #    prege = prege
        #    postge = postge[0:npre,:]            
        #elif npost<npre :
        #    idx = np.mod(np.arange(npre),npost)
        #    #prege = prege
        #    #postge = postge[idx,:]
        #    prege = prege[0:npost,:]
        #    postge = postge         
        
        # take ge data    
        #prege = vis_dat.X[:, in_pregs]  # presynaptic gene set expression
        ##prege = prege[(vbi==batch[0]), :]
        #prege = prege[in_preg,:]

        #postge = vis_dat.X[:,  in_postgs] # postsynaptic gene set expression
        ##postge = postge[(vbi==batch[1]),  :]
        #postge = postge[in_postg,:]
        
        
        
        prege = vis_dat.X[:, in_pregs]  # presynaptic gene set expression
        prege = prege[(vbi==batch[0]), :]
        
        postge = vis_dat.X[:,  in_postgs] # postsynaptic gene set expression
        postge = postge[(vbi==batch[1]),  :]
        
        ##
        ##
        ##  PARAMETERS OF GS AVERAGING
        ##
        ##
        #do_median=0      # do median of gene expression data
        do_log='none'     # do logarithm transform of ge counts
        
        if  (do_bootstrap==0)|(do_median==0):
            prege = prege[in_preg_,:]
            postge = postge[in_postg_,:]
            do_samples_columns=1
        else:
            do_samples_columns=0
            nsample = nsample_hipp #200 
            Nbs = nsample
            ##random.uniform(low=0.0, high=1.0, size=None)
            #nmax=5
            nmax = npre #np.min([1000,npre])
            idx2pre =  np.floor( np.random.uniform(0,npre, (nmax,Nbs)) ).astype('int')
            in_preg_ = np.nonzero(in_preg)[0][np.arange(npre)]
            nmax = npost #np.min([1000,npost])
            idx2post =  np.floor( np.random.uniform(0,npost, (nmax,Nbs)) ).astype('int')
            in_postg_ = np.nonzero(in_postg)[0][np.arange(npost)]

                        # 
            # ??? transform ge to median bootstraped log ???
            prege_ = prege[in_preg_,:]
            postge_ = postge[in_postg_,:]

            if do_log=='none':
                 prege_ =  prege_
            elif do_log=='log2':       
                y = np.ma.masked_where(prege_ == 0, prege_)
                prege_=np.ma.log2(y).filled(0)
                y = np.ma.masked_where(postge_ == 0, postge_)
                postge_=np.ma.log2(y).filled(0)
            elif do_log=='log1p': 
                prege_ = np.log1p(prege_)
                postge_ =  np.log1p(postge_)



            npregs=prege_.shape[1]
            ge_bs = np.zeros((Nbs,npregs+postge_.shape[1]))
            for ibs in range(Nbs):
                #ans=np.apply_along_axis(lambda v: np.median(v[v!=0]), 0, x)
                #ans[np.isnan(ans)]=0.
                xbs = prege_[idx2pre[:,ibs],:]
                pregbs  = np.apply_along_axis(lambda v: np.median(v[v!=0]), 0, xbs)
                pregbs[np.isnan(pregbs)]=0.
                xbs = postge_[idx2post[:,ibs],:]
                postgbs = np.apply_along_axis(lambda v: np.median(v[v!=0]), 0, xbs)
                postgbs[np.isnan(postgbs)]=0.

                ge_bs[ibs,0:npregs] =pregbs
                ge_bs[ibs,npregs:]  =postgbs


            #df_hr2 = np.log2(df_hr2d)
            #df_hr2 = pd.concat([df_hr2,df_hr2s],axis=1) # log2 normalization
            #df_hr2 = df_hr2.replace({0:np.nan})
            #df_hr2 = np.log2(df_hr2)

            #df_hr2 = pd.concat([df_hr.iloc[1:,0],df_hr2],axis=1)

            #df_hr2.dtypes
            #df_hr2.head()

            #df_hr2 = df_hr2.replace({0:np.nan})
            #df_hrmg=df_hr2.groupby(['Class']) # logarithm_2 of UMI counts
            #df_hrmg=df_hrmg.median(numeric_only=True) # median of log2(UMI)

            #df_hrmg0 = df_hrmg.iloc[:,0:-1]

            #df_hrmsg = pd.concat([df_hrmg.iloc[:,-1]]*df_hrmg0.shape[1],axis=1)
            #df_hrmsg.columns = df_hrmg.iloc[:,0:-1].columns

            #df_hrmg = df_hrmg.iloc[:,0:-1] - df_hrmsg +6*np.log2(10)  # transform from log2( UMI counts) to log2(CPM)

            # ??? transform ge to median bootstraped log ???


        
        

              

        ## add data to the final dataset    
        #gnij = np.repeat(np.array([syne3['cell_type2_pre'], syne3['cell_type2_post'],
        #                           syne3['layer_pre'], syne3['layer_post'],
        #                           syne3['cre_line_pre'], syne3['cre_line_post']]).reshape((1,6)),nsample,axis=0 )
        #gnij = pd.DataFrame(gnij, columns = ['cell_type2_pre', 'cell_type2_post',
        #                                     'layer_pre','layer_post',
        #                                     'cre_line_pre', 'cre_line_post'] )
        #prege  = pd.DataFrame(prege,columns  = pregs3 )
        #postge = pd.DataFrame(postge,columns = postgs3 )


        #ge_dataij = pd.concat([gnij, prege, postge ], axis = 1)
        
        
        # add data to the final dataset    
        gnij = np.repeat(np.array([syne3['cell_type2_pre'], syne3['cell_type2_post'],
                                   syne3['layer_pre'], syne3['layer_post'],
                                   syne3['cre_line_pre'], syne3['cre_line_post']]).reshape((1,6)),nsample,axis=0 )
        gnij = pd.DataFrame(gnij, columns = ['cell_type2_pre', 'cell_type2_post',
                                             'layer_pre','layer_post',
                                             'cre_line_pre', 'cre_line_post'] )
        
        if  (do_bootstrap==0)|(do_median==0):
            prege  = pd.DataFrame(prege,columns  = pregs3 )
            postge = pd.DataFrame(postge,columns = postgs3 )

            samples_pre = pd.DataFrame(sampl_pre.values[:gnij.shape[0]],columns = ['samples_pre'] )
            samples_post = pd.DataFrame(sampl_post.values[:gnij.shape[0]],columns = ['samples_post'] )
            
            ge_dataij = pd.concat([gnij, prege, postge, samples_pre, samples_post ], axis = 1)
            
            
        else:
            ge_bs = pd.DataFrame(ge_bs,columns = pregs3 + postgs3 )


            ge_dataij = pd.concat([gnij, ge_bs ], axis = 1)

       
        
        
        # add stp data
        # stp_dataij =  df_stp.loc[[0,i], stp_type['stp_data_type']!=0]
        stp_dataij =  df_stp.loc[[0,i], df_stp.loc[0, :]!=0]
        

        i2 = np.trunc(np.arange(len(stp_dataij.columns))/len(stp_columns)).astype(int)
        stp_dataij  = stp_dataij.T.set_index(i2).pivot(columns=0,values=i).loc[:,stp_columns] #ss.set_index('par',append=True).unstack('par')
        #stp_dataij =  stp_dataij.T.pivot(index=0, values=synij).T
        
        idx = np.mod(np.arange(nsample),len(stp_dataij.index))
        stp_dataij=stp_dataij.iloc[idx,:].reset_index(drop=True)  
        
        i_data_set = pd.DataFrame(i+np.zeros(len(stp_dataij.index)), index = stp_dataij.index, columns = ['index_ds'])
        stp_dataij=pd.concat([stp_dataij, i_data_set],axis=1)
        
        #if do_averagig_stp:
        #    #stp_dataij.loc[:,:] = stp_dataij.mean(axis=0).values
        #    stp_dataij.loc[:,:] = stp_dataij.median(axis=0).values

        
        #stp_dataij = pd.DataFrame({'presynaptic':pree[0], 'postsynaptic': poste[0], 
        #                           'stp_data':stp_dataij.iloc[0,:].values})
        #stp_data  = stp_data.append(stp_dataij) 
        
        # names of ge and stp columns

        
        ge_dataij  = pd.concat([ge_dataij,stp_dataij] , axis=1)
        
        # add to all ge and stp dataframe
        ge_data  = pd.concat([ge_data, ge_dataij] , axis=0)
        
#stp_data =  stp_data.reset_index(drop=True)  



#stp_data.loc[stp_data.loc[:,'postsynaptic']=='NGC','postsynaptic']='Lamp5'
#stp_data2 = ge_data.iloc[:,0:3].copy(deep=True)                                 # modify!
#stp_data2.columns=['presynaptic', 'postsynaptic', 'ppr']
#print(stp_data2.head())
#for i in stp_data.index: #stp_data.index:
#    pre=stp_data.loc[i,'presynaptic']
#    post=stp_data.loc[i,'postsynaptic']
#    stp_data2.loc[stp_data2.loc[:,'presynaptic'].isin([pre])&stp_data2.loc[:,'postsynaptic'].isin([post]),'ppr']=stp_data.loc[i,'stp_data'] 
    
ge_data = ge_data.reset_index()    




# transform labels from TM to An:A1
#f = 20 # Hz
#N = 5
#T = np.arange(N)*1000/f


#As, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data,T)
As = STP_sim_complex(ge_data,ge_data.shape[0],[])
N  = As.shape[1]

#As =  As[:,1:]/np.tile(As[:,0],(N-1,1)).T
As =  As[:,1:]/np.tile(As[:,0],(N-1,1)).T

#As =  As[:,1:]/np.tile(As[:,0],(N-1,1)).T

# add new stp parameters to ge_data dataset
stp_columns2 = ['A'+str(s)+'/A1' for s in range(2,N+1)]
stp_data2 = pd.DataFrame(As,columns = stp_columns2, index = ge_data.index)

ge_data = pd.concat([ge_data, stp_data2 ],axis=1)

if do_averagig_stp:
     #stp_dataij.loc[:,:] = stp_dataij.mean(axis=0).values
     #stp_dataij.loc[:,:] = stp_dataij.median(axis=0).values
    all_i_ds =  ge_data.loc[:,'index_ds']                             
    for i_ds in list(set(all_i_ds)):
              ids2 =  ge_data.loc[:,'index_ds'].isin([i_ds])  
              ge_data.loc[ids2,stp_columns2] = ge_data.loc[ids2,stp_columns2].median(axis=0).values                   
                                   
                                   
## filter TM parameters
do_filter_pars = 0

if do_filter_pars!=0:
    print((ge_data.loc[:,stp_columns ]<0).sum().sum())
    print(len(ge_data.index))

    nannot=6
    annot_columns = ge_columns[0:nannot]
    ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()
    f,ax = plt.subplots(figsize=(5, 5))
    ge_data2.T.plot(ax=ax)
    plt.gca().legend('')
    #plt.gca().title('An:A1 before filtering data')
    ax.title.set_text('An:A1 before filtering data')
    #ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()
    print('mean of stp before filtering:')
    print(ge_data2)

    #i_filter_out = (ge_data.loc[:,'dp/p0']*ge_data.loc[:,'p0']+)|(ge_data.loc[:,'tD'])
    #i_filter_out = i_filter_out&((ge_data.loc[:,'tF'])|(ge_data.loc[:,'tF']))
    i_filter_out = np.sum(ps2>1,axis=1)|np.sum(ps2<0,axis=1)
    i_filter_out = i_filter_out|(np.sum(ns2>1,axis=1)|np.sum(ns2<0,axis=1))
    ge_data = ge_data.loc[i_filter_out==False ,:]

    ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()
    f1,ax1 = plt.subplots(figsize=(5, 5))
    ge_data2.T.plot(ax=ax1)
    plt.gca().legend('')
    #plt.gca().title('An:A1 after filtering data')
    ax1.title.set_text('An:A1 after filtering data')
    #ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()

    print('mean of stp after filtering:')
    print(ge_data2)
    
#print('mean of stp after filtering:')
#print(ge_data2)


print(len(ge_data.index))
#ge_data.loc[:,stp_columns ] =ge_data.loc[:,stp_columns ].abs()  


#
## normalize stp parameters - for TM parameters
do_normalize_stp_pars = 0;
if do_normalize_stp_pars!=0:
    ge_data.loc[:,stp_columns  ] =np.log(ge_data.loc[:,stp_columns  ].values.astype(float))

#  assign training and labels 
stp_columns1 = stp_columns 
stp_columns = stp_columns2 # select training labels names

nannot = 7
nstp =len(stp_columns)

annot_columns = ge_columns[0:6]+['index_ds']
if do_samples_columns==1:
    nannot = 9
    annot_columns = annot_columns + ['samples_pre', 'samples_post']

## averaging of stp for each synapse type 


#X = ge_data.loc[:,ge_data.columns[0:]].values
X = ge_data.loc[:,annot_columns + ge_columns[6:]].values
y = ge_data.loc[:,annot_columns + stp_columns].values 
#y = stp_data2.loc[:,:].values

## normalize stp parameters - for TM parameters
#y[:,nannot: ] =np.log(y[:,nannot: ].astype(float))


    

# print elapsed time
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))

print(y.shape)
X.shape

In [ ]:
ge_data_h = ge_data
X_h = X
y_h = y
annot_columns_h = annot_columns
ge_columns_h = ge_columns
stp_columns_h = stp_columns

In [ ]:
ge_data.loc[0:10*0+5,['tF', 'p0','tD','dp']]

In [ ]:
#ge_data_h.loc[:,annot_columns_h[-1]]


In [ ]:
#df_aba_vis_in_c.columns
len(ge_columns_h)

In [ ]:
print(ge_data_h.shape)
print(X_h.shape)
print(y_h.shape)
print(len(annot_columns_h))
print(len(ge_columns_h))
print(len(stp_columns_h))
print(ge_columns_h[300:])
print(ge_columns_h[0:10])

In [ ]:
#ge_data_h.to_excel('ge_data_h.xlsx')

##
##
##  CORTEX gene expression
##
##

In [ ]:
save_path = '' #'data/'

#d4='~/Documents/references/transcriptomes to STP/transcriptomes/ABA/mouse_VISp_gene_expression_matrices_2018-06-14/'
if do_laptop:
    d4 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/'
else:
    d4='~/Documents/references/transcriptomes to STP/transcriptomes/ABA/mouse_VISp_gene_expression_matrices_2018-06-14/'

fn='mouse_VISp_2018-06-14_samples-columns.csv'
df_aba_vis_c=pd.read_csv(d4+fn)
print(df_aba_vis_c.columns)
print(df_aba_vis_c.head())

fn = 'mouse_VISp_2018-06-14_genes-rows.csv'
df_aba_g=pd.read_csv(d4+fn)
print(df_aba_g.head())
print(len(df_aba_vis_c.index))

in_glut_l23 = (df_aba_vis_c.loc[:,'class']=='Glutamatergic') #&(df_aba_vis_c.loc[:,'brain_subregion'].isin(['L2/3','L4','L5']))

in_gaba = df_aba_vis_c.loc[:,'class']=='GABAergic'
print(sum(in_gaba))
df_aba_vis_in_c=df_aba_vis_c.loc[in_gaba,:]
df_aba_vis_l23glu_c=df_aba_vis_c.loc[in_glut_l23,:]

###
print('loading gene expression...')

d14 = '/home/stepaniu/Documents/references/transcriptomes to STP/scVI_data/'
if do_laptop:
    d14 = '/Users/stepaniu/Documents/jan_2020/'
    from scvi.dataset import LoomDataset, CsvDataset, Dataset10X, AnnDataset
else:
    from scvi.dataset import DownloadableAnnDataset

if do_laptop:    
##df_aba_ad.write('mouse_VISp_2018-06-14_exon-matrix_GABAergic.h5ad')
    aba_vis_in = AnnDataset("mouse_VISp_2018-06-14_exon-matrix_GABAergic.h5ad", new_n_genes = 45768,
                                   save_path = save_path) 
else:
    aba_vis_in = DownloadableAnnDataset(d14+"data/mouse_VISp_2018-06-14_exon-matrix_GABAergic.h5ad") 

if do_laptop:
    #aba_vis_l23glu = AnnDataset("mouse_VISp_2018-06-14_exon-matrix_L23_Glutamatergic_hda.h5ad", new_n_genes = 45768,
    #                               save_path = save_path) 
    #aba_vis_l23glu = AnnDataset("mouse_VISp_2018-06-14_exon-matrix_L2345_Glutamatergic_hda.h5ad", new_n_genes = 45768,
    #                               save_path = save_path) 

    aba_vis_l23glu = AnnDataset("mouse_VISp_2018-06-14_exon-matrix_L23456_Glutamatergic_hda.h5ad", new_n_genes = 45768,
                                   save_path = '') 
else:
    aba_vis_l23glu = DownloadableAnnDataset(d14+"mouse_VISp_2018-06-14_exon-matrix_L23456_Glutamatergic_hda.h5ad") 

diff_order = sum(aba_vis_in.gene_names[0:].astype(int) - np.arange(len(aba_vis_in.gene_names)))
print(diff_order)

aba_vis_in.gene_names = df_aba_g.loc[:,'gene_symbol'].values

aba_vis_l23glu.gene_names = df_aba_g.loc[:,'gene_symbol'].values



print(len(aba_vis_in.gene_names))
print(len(aba_vis_l23glu.gene_names))
len(list(set(aba_vis_l23glu.gene_names).difference(set(aba_vis_in.gene_names))))
print(len(df_aba_vis_l23glu_c.index))
print(len(df_aba_vis_l23glu_c.index))
print(len(df_aba_vis_in_c.index))

from scvi.dataset.dataset import GeneExpressionDataset
if do_laptop:
    vis_dat = GeneExpressionDataset.concat_datasets(aba_vis_in,aba_vis_l23glu)    
else:    
    vis_dat  = GeneExpressionDataset()
    vis_dat.populate_from_datasets([aba_vis_in,aba_vis_l23glu]) 
   

print((vis_dat.batch_indices==0).sum())
(vis_dat.batch_indices==1).sum()

### replace some genes expression with scvi imputed ge, add scvi latent factors to data

In [ ]:
# flags were defined for hippocampus:
#do_replace_to_scVI_imputed=1      # replace raw gene expression data with their scvi imputation
#do_add_scvi_latent_factors=1      # add scvi latent factors data
#gs_name = '_gs219'
if (do_replace_to_scVI_imputed==1)|(do_add_scvi_latent_factors==1):
    scVI_all = pd.read_hdf('scVI_latent_factors_and_imputation_aba2019'+gs_name+'.hdf',key='data') # this is 290 genes dataset
    
    scVI_all = scVI_all.set_index('sample_name') # scvi imputed genes and latent factors for hipp., enth. and vis. cortex

    # were defined before
    #nlf = 10
    #gncolumns = scVI_all.columns[nlf+1:-1] # gene names columns in scvi data
    #lfcolumns = scVI_all.columns[:nlf]     # latent factors columns in scvi data

    
    smhi = aba_vis_in.obs
    smhi = pd.DataFrame(np.arange(smhi.shape[0]).transpose().reshape((-1,1)),index = smhi.index.values, columns=['index'])
    #smhi
    smhi = smhi.reset_index().set_index('index')  # cortex interneurons samples and their indexes in ge-data
    smhi.columns=['samples']
    smh2i = smhi.reset_index().set_index('samples')  # cortex interneurons samples and their indexes in ge-data
    #smh2i
    
    smhe = aba_vis_l23glu.obs

    smhe = pd.DataFrame(smhe.reset_index(drop=True).loc[:,'sample_name'])
    smhe.columns = ['samples']
    #smhe

    smh2e = smhe.reset_index().set_index('samples')  # cortex interneurons samples and their indexes in ge-data
    #smh2e
    
    #smh = aba_vis_in.obs
    smh2 = pd.concat([smh2i, smh2e+smh2i.shape[0]])
    #smh2
    
    gnscvi = pd.DataFrame(gncolumns)   # df from scvi genenames

    # assumption ??: the same gene names are in vis_dat, aba_vis_l23glu and aba_vis_in
    gnh = vis_dat.gene_names               # cortex vis_dat gene names (the same names are in inh and exc data)
    #smh = vis_dat.obs                     # cortex vis_dat samples names
    gnh2 = pd.DataFrame(gnh).reset_index().set_index(0)   
    ignh2 = gnh2.loc[gnscvi.loc[:,0],:].loc[:,'index'].values  # indices of scvi gene names in vis_dat gene names

    gnhi = aba_vis_in.gene_names           # cortex aba_vis_in gene names (the same names are in inh and exc data)
    #smh = vis_dat.obs                     # cortex aba_vis_in samples names
    gnh2i = pd.DataFrame(gnhi).reset_index().set_index(0)   
    ignh2i = gnh2i.loc[gnscvi.loc[:,0],:].loc[:,'index'].values  # indices of scvi gene names in aba_vis_in gene names

    gnhe = aba_vis_l23glu.gene_names              # cortex aba_vis_l23glu gene names (the same names are in inh and exc data)
    #smh = aba_vis_l23glu.obs                     # cortex aba_vis_l23glu samples names
    gnh2e = pd.DataFrame(gnhe).reset_index().set_index(0)   
    ignh2e = gnh2e.loc[gnscvi.loc[:,0],:].loc[:,'index'].values  # indices of scvi gene names in aba_vis_l23glu gene names
    
    
    if (do_replace_to_scVI_imputed==1):
        # correct this!
        vis_dat._X[:,ignh2] = scVI_all.loc[smh2.index,gncolumns].values  # modified vis_dat matrix (imputed part of genes replaced)
        aba_vis_l23glu._X[:,ignh2e] = scVI_all.loc[smh2e.index,gncolumns].values  # modified vis_dat matrix (imputed part of genes replaced)
        aba_vis_in._X[:,ignh2i] = scVI_all.loc[smh2i.index,gncolumns].values  # modified vis_dat matrix (imputed part of genes replaced)
    
    
    if (do_add_scvi_latent_factors==1):
        scVI_lf_ci = scVI_all.loc[smh2i.index,lfcolumns]    # latent factors for hippocampal samples
        scVI_lf_ce = scVI_all.loc[smh2e.index,lfcolumns]
        scVI_lf_c = pd.concat([scVI_lf_ci,scVI_lf_ce])
        vis_dat._X = np.concatenate([vis_dat._X, scVI_lf_c.values],axis=1)
        # correct this!
        
        aba_vis_in._X = np.concatenate([aba_vis_in._X, scVI_lf_ci.values],axis=1)
        aba_vis_l23glu._X = np.concatenate([aba_vis_l23glu._X, scVI_lf_ce.values],axis=1)
        
        #lf_scvi_columns = lfcolumns.astype(str)
        vis_dat.gene_names = np.concatenate([vis_dat.gene_names, lf_scvi_columns])
        # correct this!
        
        aba_vis_in.gene_names = np.concatenate([aba_vis_in.gene_names, lf_scvi_columns])
        aba_vis_l23glu.gene_names = np.concatenate([aba_vis_l23glu.gene_names, lf_scvi_columns])
        
        #pregs = pregs + lf_scvi_columns.tolist()
        #postgs = postgs + lf_scvi_columns.tolist()
        

In [ ]:
print(len(pregs))
print(len(postgs))

In [ ]:
import matplotlib.pyplot as plt
i='Sst-IRES-Cre'
li = df_aba_vis_in_c.loc[(df_aba_vis_in_c.loc[:,'driver_lines']==i),
                             ['subclass','cluster','brain_subregion']]
set(li.loc[:,'brain_subregion'])
for reg in list(set(li.loc[:,'brain_subregion'])):
    #reg = 'L4'
    li = df_aba_vis_in_c.loc[(df_aba_vis_in_c.loc[:,'driver_lines']==i)&(df_aba_vis_in_c.loc[:,'brain_subregion']==reg),
                             ['subclass','cluster','brain_subregion']]

    f = plt.figure()
    ax = f.add_axes()
    plt.title(str(i)+' '+reg) 
    vc = li.loc[:,'cluster'].value_counts() 

    vc2 = vc/vc.sum()*100

    if len(vc.index)>0: 
        vc2.plot(kind='bar',ax=ax)

In [ ]:
i='Pvalb-IRES-Cre'
li = df_aba_vis_in_c.loc[(df_aba_vis_in_c.loc[:,'driver_lines']==i),
                             ['subclass','cluster','brain_subregion']]
set(li.loc[:,'brain_subregion'])
for reg in list(set(li.loc[:,'brain_subregion'])):
    #reg = 'L4'
    li = df_aba_vis_in_c.loc[(df_aba_vis_in_c.loc[:,'driver_lines']==i)&(df_aba_vis_in_c.loc[:,'brain_subregion']==reg),
                             ['subclass','cluster','brain_subregion']]

    f = plt.figure()
    ax = f.add_axes()
    plt.title(str(i)+' '+reg) 
    vc = li.loc[:,'cluster'].value_counts() 

    vc2 = vc/vc.sum()*100

    if len(vc.index)>0: 
        vc2.plot(kind='bar',ax=ax)

##
##
##  CORTEX STP
##
##

In [ ]:


if do_laptop:
    d5 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/additional cortical STP data/'
else:
    d5 = '/home/stepaniu/Documents/references/transcriptomes to STP/scVI_data/additional cortical STP data/'
#fn5='STP_cortex_interneurons_Cre_labeled.xlsx'
#fn5='STP_cortex_interneurons_Cre_labeled_UDF2.xlsx'
fn5='STP_cortex_interneurons_Cre_labeled_UDF3.xlsx'
fn5='STP_cortex_interneurons_Cre_labeled_UDF5_4pars.xlsx'

df_stp=pd.read_excel(d5+fn5)
df_stp.loc[df_stp.loc[:,'stp_comment'].isna() ,'stp_comment']=''

############   
do_averagig_stp =0
stp_TM_names=['tF', 'p0','tD','dp/p0','A']
stp_TM_start = 29
ncol = len(df_stp.columns)
df_tm_name = pd.DataFrame(np.zeros(ncol).reshape((1,ncol)), columns =df_stp.columns )
df_stp = pd.concat([df_tm_name, df_stp],axis=0).reset_index(drop=True)

for i,nm in enumerate(stp_TM_names):
    ii = stp_TM_start + np.arange(0,ncol-i-stp_TM_start,len(stp_TM_names))+i
    df_stp.iloc[0,ii] = nm

#df_stp.loc[df_stp.loc[:,'name'].str.contains('Jaing_2015_PC')!=True,:].reset_index(drop=True)

################# ADD ABA STP data
do_add_aba_synphys=True
do_add_aba_synphys_new=True

#QTX.to_hdf('fit_aba_2019_A1_8_fited_pulses_syntypes_results','data')
d_aba = '/home/stepaniu/Documents/references/transcriptomes to STP/'
if do_laptop:
    d_aba = '/Users/stepaniu/Documents/jan_2020/'

# ACHTUNG!!! ACHTUNG!!!ACHTUNG!!!ACHTUNG!!!
do_simple_TM_model = False  # simple tm vs tm+smr  
# ACHTUNG!!! ACHTUNG!!!ACHTUNG!!!ACHTUNG!!!

if do_simple_TM_model:    
    STP_aba = pd.read_hdf(d_aba + 'fit_aba_2019_A1_8_fited_pulses_syntypes_results') # tm5
    QTX2 = STP_aba.iloc[0:43,0:14]
    QTX3 = STP_aba.iloc[43:,14:]
    QTX3.index = QTX2.index
    STP_aba = pd.concat([QTX2, QTX3],axis=1)
else:
    #STP_aba = pd.read_hdf(d_aba + 'fit_aba_2019_A1_8_fited_pulses_TM_8_syntypes_results') # tm5 + FDR
    STP_aba = pd.read_hdf(d_aba + 'fit_aba_2019_mean_fited_pulses_TM_SMP0_100bs_syntypes_results') # tm5 + SMR
    #STP_aba = pd.read_hdf(d_aba + 'fit_aba_2019_mean_fited_pulses_TM_SMP0_50bs_syntypes_results') 

STP_aba

In [ ]:
df_stp

In [ ]:
STP_aba.index

In [ ]:
n_stp_cells = STP_aba.loc[:,'comments'].str.split(pat='=', n=-1, expand=True)[1].astype(int)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
f, ax =plt.subplots(figsize=(12, 5))
#    plt.title(str(i0)+ '  '+str_syntype + '  ' +stp_columns2[3])
#    plt.xlim((0.25,1.25))

n_stp_cells.plot(kind = 'bar',ax=ax)
n_stp_cells.sum()

In [ ]:
n_stp_cells2 = n_stp_cells.reset_index()
n_stp_cells3 = n_stp_cells2.loc[:,'index'].str.split(pat=';', n=-1, expand=True)
n_stp_cells3[0] = n_stp_cells3[0].str.strip()
n_stp_cells3[1] = n_stp_cells3[1].str.strip()

In [ ]:
i2 = ['nr5a1 L2/3; nr5a1 L2/3', 'unknown L2/3; unknown L2/3',
      'rorb L4; rorb L4', 'rbp4 L4; rbp4 L4',
      'sim1 L5; sim1 L5', 'tlx3 L5; tlx3 L5', 'tlx3 L5; fam84b L5', 
      'ntsr1 L6; ntsr1 L6', 
       'unknown L2/3; pvalb L2/3','unknown L2/3; vip L2/3',
       'nr5a1 L4; pvalb L4', 'nr5a1 L4; sst L4','nr5a1 L4; vip L4',
       'sim1 L5; pvalb L5','sim1 L5; sst L5', 'tlx3 L5; sst L5', 'tlx3 L5; pvalb L2/3',
       'ntsr1 L6; pvalb L6','ntsr1 L6; sst L6',
        'pvalb L4; nr5a1 L4', 'pvalb L4; nr5a1 L5','pvalb L5; sim1 L5','pvalb L5; tlx3 L5',
       'pvalb L6; ntsr1 L6',
       'sst L2/3; sim1 L5','sst L4; nr5a1 L4','sst L5; tlx3 L5',
       'pvalb L2/3; pvalb L2/3','pvalb L4; pvalb L2/3','pvalb L5; pvalb L5','pvalb,sst L5; pvalb L5',
      'pvalb L6; pvalb L6','pvalb L2/3; sst L2/3','pvalb L2/3; vip L2/3', 
       'sst L6; pvalb L5','sst L2/3; sst L2/3','sst L5; sst L5','sst L6; sst L6','sst L2/3; vip L2/3','sst L4; vip L4',
       'vip L2/3; sst L2/3','vip L4; sst L5','vip L2/3; vip L4',
     ]

n_stp_cells4 = n_stp_cells.loc[i2]
n_stp_cells4 = n_stp_cells4.reset_index()
is_uu = n_stp_cells4.loc[:,'index']=='unknown L2/3; unknown L2/3'
n_stp_cells4.loc[is_uu,'index']='unknown L2/3; nr5a1 L2/3'
n_stp_cells4 = n_stp_cells4.set_index('index')
#%matplotlib inline
#import matplotlib.pyplot as plt
f, ax =plt.subplots(figsize=(15, 5))
#    plt.title(str(i0)+ '  '+str_syntype + '  ' +stp_columns2[3])
#    plt.xlim((0.25,1.25))

n_stp_cells4.plot(kind = 'bar',ax=ax)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylabel('Number of cells',fontsize=20)
plt.xlabel('Synapse type',fontsize=20)
plt.legend('')
print(set(n_stp_cells.index).difference(set(i2)))
n_stp_cells4.sum()


In [ ]:
do_samples_columns

## CORTEX : allign gene expression and stp data (for selected gene sets)

In [ ]:
# allign gene expression and stp data (for selected gene sets)
import re
import time
t1 = time.time()



# names of gene expression based synapses types

glu_l23=list(set(df_aba_vis_l23glu_c.loc[:,'cluster']))

dsg = [['PC']+glu_l23,
       ['Pvalb','Pvalb Tpbg','Pvalb Reln Itm2a','Pvalb Reln Tac1','Pvalb Sema3e Kank4'], 
       ['Sst','Sst Calb2 Pdlim5','Sst Calb2 Necab1', 'Sst Hpse Cbln4','Sst Hpse Sema3c','Sst Nr2f2 Necab1'],
       ['Vip','Vip Lmo1 Myl1', 'Vip Rspo1 Itga4','Vip Ptprt Pkp2','Vip Rspo4 Rxfp1 Chat'],
       ['Lamp5','Lamp5 Plch2 Dock5'],
       ['MC','L23MC','Sst Calb2 Pdlim5'],
        'L5MC']
      

set(df_stp.loc[:,'synapse_type_2'])

# names of electrophysiology based synapses types
dse = [['PC', 'L23P', 'L23PC', 'L5P'],
       ['Pvalb', 'PV', 'L23BC', 'L5BC'], 
       ['Sst', 'L23MC', 'L5MC', 'Sst', 'MC'], 
       ['Vip', 'VIP', 'BTC'],
       ['NGC', 'L23NGC'],
       ['L23MC' ],
       ['L5MC']]



# load stp data table
if do_laptop:
    d5 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/additional cortical STP data/'
else:
    d5 = '/home/stepaniu/Documents/references/transcriptomes to STP/scVI_data/additional cortical STP data/'
#fn5='STP_cortex_interneurons_Cre_labeled.xlsx'
#fn5='STP_cortex_interneurons_Cre_labeled_UDF2.xlsx'
#fn5='STP_cortex_interneurons_Cre_labeled_UDF3.xlsx'

do_3pars_TM=0 # 3 or 4 parametric TM model?
if do_3pars_TM:
    fn5='STP_cortex_interneurons_Cre_labeled_UDF5_3pars.xlsx'
else:
    fn5='STP_cortex_interneurons_Cre_labeled_UDF5_4pars.xlsx'

df_stp=pd.read_excel(d5+fn5)
df_stp.loc[df_stp.loc[:,'stp_comment'].isna() ,'stp_comment']=''

# drop TC->
TC_names = ['Miao_2016_fig1_TC->PC','Miao_2016_fig1_TC->PV']
df_stp = df_stp.drop(index = df_stp.index[df_stp.loc[:,'name'].isin(TC_names)] )

############   
do_averagig_stp =0
stp_TM_names=['tF', 'p0','tD','dp/p0','A']
stp_TM_start = 29
ncol = len(df_stp.columns)
df_tm_name = pd.DataFrame(np.zeros(ncol).reshape((1,ncol)), columns =df_stp.columns )
df_stp = pd.concat([df_tm_name, df_stp],axis=0).reset_index(drop=True)

for i,nm in enumerate(stp_TM_names):
    ii = stp_TM_start + np.arange(0,ncol-i-stp_TM_start,len(stp_TM_names))+i
    df_stp.iloc[0,ii] = nm



print(df_stp.head())
#############
##Remove pure PPR data (Jaing_2015_PC)
#df_stp = df_stp.loc[df_stp.loc[:,'name'].str.contains('Jaing_2015_PC')!=True,:].reset_index(drop=True) 

############

# batches and gene sets
pre_post = df_stp.loc[:,'synapse_type_2'].str.split(pat='->',expand=True)

if do_add_aba_synphys:
    pre_post_aba = STP_aba.loc[:,'synapse_type_2'].str.split(pat=';',expand=True)

vbi = vis_dat.batch_indices.reshape((vis_dat.batch_indices.shape[0],))
vbi=vbi>0

df_gs  = pd.DataFrame( vis_dat.gene_names ) 
in_pregs = df_gs.loc[:,0].isin(pregs).values
in_postgs = df_gs.loc[:,0].isin(postgs).values

pregs2 = df_gs.loc[in_pregs,0].values
postgs2 = df_gs.loc[in_postgs,0].values


do_2=1;
# select stp data for appropriate frequency and ages 
stp_type = {'name':['Walker_2016_PV->MC L23',
                    'Walker_2016_VIP->MC L23',
                    'Yuste_2016_taus_PV->Sst L23 recalc',
                    'Yuste_2016_taus_PV->VIP L23 recalc',
                    'Yuste_2016_taus_Sst->VIP L23 recalc',
                    'Yuste_2016_taus_VIP->Sst L23 recalc',
                    'Yuste_2016_taus_VIP->VIP L23 recalc'],
            'name_remove': [], 
            'stp_freq': [20, 40, 50], 'stp_comment':  ['', ' P20-21', ' P20-22', ' P20-23', ' P25-31'],
            'stp_data_type': df_stp.loc[0, :]!=0 }

if do_add_aba_synphys:
    STP_aba_type = {'name':[],
            'stp_freq': [], 'stp_comment':  [],
            'stp_data_type': 'differential_evolution best x' }  # 'random_starts best x'
    if do_simple_TM_model:
        stp_aba_names = ['tF', 'p0','tD','dp','A','A1','A2'] # tm5
    else:
        stp_aba_names = ['tF', 'p0','tD','dp','A','A1','A2','tDmin','dd','t_FDR','t_SMR','dp0'] # tm5+smr
    
    # ACHTUNG!!! ACHTUNG!!! ACHTUNG!!!
    do_shrink_stp_distribution = False # # shrink bootstrap distribution of stp parameters towords mean values - to test stp variability reduction
    if do_shrink_stp_distribution==True:
        shrink_stp_distribution_coef = 0*0.1
    
if do_2==1:
    #stp_type = {'name':['Walker_2016_PV->MC L23',
    #               'Walker_2016_VIP->MC L23',
    #                'Yuste_2016_taus_PV->Sst L23 recalc',
    #                'Yuste_2016_taus_VIP->Sst L23 recalc'],
    #        'stp_freq': [20, 40, 50], 'stp_comment':  ['', ' P20-21', ' P20-22', ' P20-23', ' P25-31'],
    #        'stp_data_type': df_stp.loc[0, :]!=0 }
    
    # without Yuste and
    stp_type = {'name':['Walker_2016_PV->MC L23',
                    'Walker_2016_VIP->MC L23'],
             'name_remove': ['Jaing','Yuste'],  
            'stp_freq': [20, 40, 50], 'stp_comment':  ['', ' P20-21', ' P20-22', ' P20-23', ' P25-31'],
            'stp_data_type': df_stp.loc[0, :]!=0 }




in_stp_data = df_stp.loc[:,'stp_freq'].isin([20])
in_stp_data= in_stp_data|(df_stp.loc[:,'stp_freq'].isin([40,50])&df_stp.loc[:,'name'].isin(stp_type['name']))
in_stp_data = in_stp_data&df_stp.loc[:,'stp_comment'].isin(pd.Series(stp_type['stp_comment']).str.strip())
if len(stp_type['name_remove'])>0:
        remove = df_stp.loc[:,'name'].str.contains(stp_type['name_remove'][0])
        for rmn in stp_type['name_remove']:
             remove= remove|df_stp.loc[:,'name'].str.contains(rmn)
        in_stp_data = in_stp_data&( remove!=True)
        
#show selected stp dataset
ddd=df_stp.loc[in_stp_data,['name','synapse_type_2', 'stp_freq','area']]
print(ddd)
print(in_stp_data.sum())

synij = np.repeat(False,in_stp_data.shape[0])
if do_add_aba_synphys:
    synij = np.repeat(False,in_stp_data.shape[0]+STP_aba.shape[0])

cols2 = ['name','perc']
cl2post =pd.DataFrame([],columns= cols2)
cl2pre =pd.DataFrame([],columns= cols2)


# strip cluster names
#list(set(df_aba_vis_in_c.loc[df_aba_vis_in_c.loc[:,'subclass'].str.contains(clns6),'cluster']))
df_aba_vis_in_c.loc[:,'cluster']=df_aba_vis_in_c.loc[:,'cluster'].str.strip()
df_aba_vis_l23glu_c.loc[:,'cluster']=df_aba_vis_l23glu_c.loc[:,'cluster'].str.strip()

do_simplfy_clusters = 1 # do not take into account minor subclasses
Sst_NonMC ='Sst, Sst Calb2 Pdlim5-, Sst Chrna2 Glra3-, Sst Chrna2 Ptgdr-, Sst Myh8 Etv1-, Sst Tac2 Myh4-, Sst Nr2f2 Necab1-, Sst Chodl-,  Sst Calb2 Necab1-, Sst Tac1 Htr1d-, Sst Myh8 Fibin-';

pregs3 = ['pre__'+s for s in pregs2.tolist()]
postgs3 = ['post__'+s for s in postgs2.tolist()]
ge_columns = ['cell_type2_pre', 'cell_type2_post',
                                             'layer_pre','layer_post',
                                             'cre_line_pre', 'cre_line_post'] +pregs3 +postgs3 + ['samples_pre', 'samples_post']

stp_columns = stp_TM_names #list(set(df_stp.loc[0,:]).difference(set([0])))

# output datasets
stp_data = pd.DataFrame([], columns =['presynaptic', 'postsynaptic', 'stp_data'] )
ge_data = pd.DataFrame([], columns =ge_columns+stp_columns+['index_ds'])

if do_add_aba_synphys:
    l_pre_post=len(pre_post.index)+len(pre_post_aba.index)
else:
    l_pre_post=len(pre_post.index)
l_pre_post1 = len(pre_post.index)

for i in range(1,l_pre_post): #range(1,len(pre_post.index)):# range(len(pre_post.index)): # [1]: #range(len(pre_post.index)):
    if i==l_pre_post1:
        l_pre_post2 = ge_data.shape[0]
        
    # syne3 - all available pre- and post- synaptic cells annotations : cre-lines, layers, morphologies
    if i<l_pre_post1:
        l = str.strip(df_stp.loc[i,'area'])
        if type(df_stp.loc[i,'synapse_type'])!=float:
            syne1 = re.split('\;',df_stp.loc[i,'synapse_type'])
        else:
            syne1 =''    
        #syne2 = df_stp.loc[i,'synapse_type2'].str.split(pat='->',expand=True)
        syne2 = re.split('->',df_stp.loc[i,'synapse_type_2'])
        if len(syne2)==1:
            pat ='→'
            syne2 = re.split(pat,df_stp.loc[i,'synapse_type_2'])

        syne3 ={'layer_pre':l,  'cre_line_pre':'none', 'cell_type2_pre':str.strip(syne2[0]),'cluster_pre':[],
                'layer_post':l, 'cre_line_post':'none','cell_type2_post':str.strip(syne2[1]),'cluster_post':[],  } 

        # add cre-lines when available
        if len(syne1)>1:
            syne3['cre_line_pre']  =str.strip(syne1[0])
            syne3['cre_line_post'] =str.strip(syne1[1])

        # introduce study-specific modifications to syne3 (using morphology+layer->clusters map)
        if  df_stp.loc[i,'name'][0:5]=='Jaing':

            # modify layer_post and reduce cell_type_post to mtype name
            if syne3['cell_type2_post'][0:3]=='L23':
                syne3['layer_post'] = 'L2/3'
                syne3['cell_type2_post']=syne3['cell_type2_post'][3:]
            elif syne3['cell_type2_post'][0:2]=='L5':
                     syne3['layer_post'] = 'L5'
                     syne3['cell_type2_post']=syne3['cell_type2_post'][2:]  

            #  reduce cell_type_pre to mtype name
            if syne3['cell_type2_pre'][0:3]=='L23':
                syne3['layer_pre'] = 'L2/3'
                syne3['cell_type2_pre']=syne3['cell_type2_pre'][3:]
            elif syne3['cell_type2_pre'][0:2]=='L5':
                     syne3['layer_pre'] = 'L5'
                     syne3['cell_type2_pre']=syne3['cell_type2_pre'][2:]    

            # for each mtype specify expected ABI gene custers and their expected % (based on Jiang 2015 cre-lines to morhology table)
            if syne3['cell_type2_post']=='MC':
                if syne3['layer_post'] == 'L2/3':
                    syne3['cluster_post']=[['Sst Calb2 Pdlim5',100]]
                if syne3['layer_post'] == 'L5':
                    syne3['cre_line_post']='Chrna2'  
                    syne3['cluster_post']=[['Sst',100]]

            if syne3['cell_type2_post']=='BC':  
                if syne3['layer_post'] == 'L2/3':
                    if do_simplfy_clusters==0:
                        syne3['cluster_post']=[['Pvalb, Pvalb Vipr2-',75],[Sst_NonMC,25]] # modify!!!    
                    else:
                        syne3['cluster_post']=[['Pvalb, Pvalb Vipr2-',100]] # modify!!!  

                if syne3['layer_post'] == 'L5':
                    if do_simplfy_clusters==0:
                        syne3['cluster_post']=[['Pvalb, Pvalb Vipr2-',75],[Sst_NonMC,25]] # modify!!!    
                    else:
                        syne3['cluster_post']=[['Pvalb, Pvalb Vipr2-',100]] # modify!!!    
            if syne3['cell_type2_post']=='BTC':
                if do_simplfy_clusters==0:
                    syne3['cluster_post']=[['Vip Lmo1 Myl1, Vip Ptprt Pkp2, Vip Rspo4 Rxfp1 Chat',90], # modify!!! 
                                         [Sst_NonMC,10]]
                else:
                    syne3['cluster_post']=[['Vip Lmo1 Myl1, Vip Ptprt Pkp2, Vip Rspo4 Rxfp1 Chat',100]]
            if syne3['cell_type2_post']=='HEC':  # modify!!! 
                if do_simplfy_clusters==0:
                    syne3['cluster_post']=[['Pvalb, Pvalb Vipr2-,Pvalb Tpbg-,Pvalb Reln Itm2a-,Pvalb Reln Tac1-',75],
                                           [Sst_NonMC,25]]
                else:
                    syne3['cluster_post']=[['Pvalb, Pvalb Vipr2-,Pvalb Tpbg-,Pvalb Reln Itm2a-,Pvalb Reln Tac1-',100]]            
            if syne3['cell_type2_post']=='NGC':            
                  syne3['cluster_post']=[['Lamp5 Ntn1 Npy2r, Lamp5 Plch2 Dock5',100]] 

        if  df_stp.loc[i,'name'][0:5]=='Walke':    
            if syne3['cell_type2_post']=='MC':
                 syne3['cluster_post']=[['Sst Calb2 Pdlim5',100]]            

        #if  df_stp.loc[i,'name'][0:5]=='Miao_':     

        #if  df_stp.loc[i,'name'][0:5]=='Yuste':     
    
    # ABA synphys dataset
    if i>=l_pre_post1:
        #l = str.strip(STP_aba.loc[i1,'area'])

            
        i1 = STP_aba.index[i - l_pre_post1]
          
        syne0 = re.split('\,',STP_aba.loc[i1,'synapse_type_2'])
        syne1 = re.split('\;',syne0[0])
        
        syne0 = re.split('\,',STP_aba.loc[i1,'comments'])
        syne1 = re.split('\;',syne0[0])
        syne12 = re.split('\'',syne1[0])[1]
        syne13 = re.split('\'',syne1[1])[0]

        syne_pre = re.split('\s+',str.strip(syne12))
        syne_post = re.split('\s+',str.strip(syne13))
        
   
        l1 = str.strip(syne_pre[1])
        l2 = str.strip(syne_post[1])
        
        syne3 ={'layer_pre':[l1],  'cre_line_pre':'none', 'cell_type2_pre':'PC','cluster_pre':[],
                'layer_post':[l2], 'cre_line_post':'none','cell_type2_post':'PC','cluster_post':[]  } 

        # add cre-lines when available
        in_lines = pd.DataFrame(['vip','pvalb','sst'])
        if str.strip(syne_pre[0])!='unknown':
            syne3['cre_line_pre']   =str.strip(syne_pre[0])
            if in_lines.isin([syne3['cre_line_pre']]).any()[0]:
                syne3['cell_type2_pre'] = syne3['cre_line_pre'] 
        else:
            syne3['cell_type2_pre']   = 'PC'
            
        if str.strip(syne_post[0])!='unknown':
            syne3['cre_line_post']  =str.strip(syne_post[0])  
            if in_lines.isin([syne3['cre_line_post']]).any()[0]:
                syne3['cell_type2_post'] = syne3['cre_line_post'] 
        else:
            syne3['cell_type2_post']   = 'PC'    
            
            
        if syne3['cre_line_pre']=='tlx3':
            syne3['layer_pre']=['L4-L6']
                    
        if syne3['cre_line_post']=='tlx3':
            syne3['layer_post']=['L4-L6']
        
                    
        if syne3['cre_line_pre']=='sim1':
            syne3['layer_pre']=['L4-L5','L4-L6']
        
                    
        if syne3['cre_line_post']=='sim1':
            syne3['layer_post']=['L4-L5','L4-L6']
            
        if syne3['cre_line_post']=='vip':
            if syne3['layer_post'][0]=='L2/3':
                syne3['layer_post']=['L1-L2/3']
        if syne3['cre_line_pre']=='vip':
            if syne3['layer_pre'][0]=='L2/3':
                syne3['layer_pre']=['L1-L2/3']
                
        if syne3['cre_line_post']=='sst':
            if syne3['layer_post'][0]=='L2/3':
                syne3['layer_post']=['L1-L2/3']
        if syne3['cre_line_pre']=='sst':
            if syne3['layer_pre'][0]=='L2/3':
                syne3['layer_pre']=['L1-L2/3']        
 
    
    # # find sets of gene clusters corresponding to each pre- and post- neuron types pair from stp-table
    # for ii in range(len(dse)): 
    #    if sum(np.array(dse[ii])==pre_post.loc[i,0])>0:
    #        i1=ii
    #    if sum(np.array(dse[ii])==pre_post.loc[i,1])>0:
    #        j1=ii    
    #
    ## select names of stp-types
    #pree = dse[i1]
    #poste = dse[j1]
    
    ## select names of genes-types (ABA cluster)
    #preg = dsg[i1]
    #postg = dsg[j1]

    ##synij = (pre_post.loc[:,0].isin(pree))&(pre_post.loc[:,1].isin(poste))
    

    synij[:] = False
    
    if i<l_pre_post1:
        synij[i] = in_stp_data[i]
    else:
        synij[i] = True
    
    #print(syne3)
    
    if sum(synij)>0:
        print('\n\n')
        print('i = ',str(i))
        print(syne3)
        #if sum(in_stp_data&synij)>0:
        #    in_stp_data = in_stp_data&synij

        #add ge data
        #'layer_pre':l,  'cre_line_pre':'', 'cell_type2_pre':syne2[0].str.strip(),'cluster_pre':[]
        
        # PC or GABAergic dataset?
        batch = [ syne3['cell_type2_pre']=='PC',syne3['cell_type2_post']=='PC']
        
        # add ge data depending on available stp cell types labels:
        #  select presynaptic cells
        if batch[0]:

            if i>=l_pre_post1:
                in_preg  = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin(syne3['layer_pre']) #modify!!!
                #in_preg  = in_preg&(df_aba_vis_l23glu_c.loc[:,'driver_lines']==syne3['cre_line_pre']) #modify!!!
                if syne3['cre_line_pre']!='none':
                    in_preg  = in_preg&(df_aba_vis_l23glu_c.loc[:,'driver_lines'].str.lower().str.contains(syne3['cre_line_pre'])) #modify!!!
                #set(df_aba_vis_l23glu_c.loc[:,'driver_lines'].str.lower().str.contains('nr5a1'))
            else:
                #in_preg  = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']]) #modify!!!
                in_preg  = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']]) #modify!!!
        else:
            if len(syne3['cluster_pre'])==0:
                # if len(syne3['cre_line_pre'])>0:
                in_preg = df_aba_vis_in_c.loc[:,'driver_lines'].str.lower().str.contains(str.lower(syne3['cre_line_pre']))
                #in_preg = in_preg&df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']])
                if i>=l_pre_post1:
                    in_preg = in_preg&df_aba_vis_in_c.loc[:,'brain_subregion'].isin(syne3['layer_pre'])
                else:
                    in_preg = in_preg&(df_aba_vis_in_c.loc[:,'brain_subregion'].str.contains(syne3['layer_pre']))
                    
                #if i==72:
                #    breakpoint()
                    
                    
            else:                  
                #in_preg = df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']])
                in_preg = df_aba_vis_in_c.loc[:,'brain_subregion'].str.contains(syne3['layer_pre'])
                cl = syne3['cluster_pre']
                #cols2 = ['name','perc']
                #cl2pre =pd.DataFrame([],columns= cols2)
                in_preg0 = in_preg.copy()
                in_preg[:] = False 
                for icl in range(len(cl)):
                    clp = cl[icl][1]
                    clns = re.split(',',cl[icl][0])
                    cls3 = []
                    cls3m=[]
                    for icls in range(len(clns)):
                        clns4 = clns[icls]
                        clns5 = re.split('\s+',clns4)
                        if len(clns5)<3:
                            clns6=clns5[0]
                            cls3=cls3 + list(set(df_aba_vis_in_c.loc[df_aba_vis_in_c.loc[:,'subclass'].str.contains(clns6),'cluster']))
                        else:
                            clns4=str.strip(clns4)
                            clns7=re.split('\-',clns4)
                            if len(clns7)>1:
                                clns4=clns7[0]
                                cls3m=cls3m+[str.strip(clns4)]
                            else:
                                cls3=cls3+[str.strip(clns4)]
                    cls3 = list(set(cls3).difference(set(cls3m)))
                    cl22=pd.DataFrame([cls3,[clp]],index=cols2).T
                    cl2pre = cl2pre.append(cl22)
                    
                    in_preg = in_preg|(in_preg0&df_aba_vis_in_c.loc[:,'cluster'].isin(cls3))
                    
                if len(syne3['cre_line_pre'])>0:
                    #in_preg = in_preg&df_aba_vis_in_c.loc[:,'driver_lines'].str.contains(syne3['cre_line_pre'])
                    in_line=df_aba_vis_in_c.loc[:,'driver_lines'].str.contains(syne3['cre_line_pre'])
                    if in_line.sum()>0:
                        in_preg = in_preg&in_line                        
        
        #  select postsynaptic cells
        if batch[1]:

            if i>=l_pre_post1:
                in_postg  = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin(syne3['layer_post']) #modify!!!
                if syne3['cre_line_post']!='none':
                    in_postg  = in_postg&(df_aba_vis_l23glu_c.loc[:,'driver_lines'].str.lower().str.contains(syne3['cre_line_post'])) #modify!!!
            else:
                #in_postg = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin([syne3['layer_post']]) #modify!!!
                in_postg = df_aba_vis_l23glu_c.loc[:,'brain_subregion']==syne3['layer_post'] #modify!!!
        else:    
            if len(syne3['cluster_post'])==0:
                # if len(syne3['cre_line_post'])>0:
                in_postg = df_aba_vis_in_c.loc[:,'driver_lines'].str.lower().str.contains(str.lower(syne3['cre_line_post']))
                #in_postg = in_postg&df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_post']])
                if i>=l_pre_post1:
                    in_postg = in_postg&df_aba_vis_in_c.loc[:,'brain_subregion'].isin(syne3['layer_post'])
                else:
                    in_postg = in_postg&df_aba_vis_in_c.loc[:,'brain_subregion'].str.contains(syne3['layer_post'])
            else:    
                ## under construction ...
                #in_postg = df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_post']])
                in_postg = df_aba_vis_in_c.loc[:,'brain_subregion'].str.contains(syne3['layer_post'])
                cl = syne3['cluster_post']
                #cols2 = ['name','perc']
                #cl2post =pd.DataFrame([],columns= cols2)
                in_postg0 = in_postg.copy()
                in_postg[:] = False                 
                for icl in range(len(cl)):
                    clp = cl[icl][1]
                    clns = re.split(',',cl[icl][0])
                    cls3 = []
                    cls3m=[]
                    for icls in range(len(clns)):
                        clns4 = clns[icls]
                        clns5 = re.split('\s+',clns4)
                        if len(clns5)<3:
                            clns6=clns5[0]
                            cls3=cls3 + list(set(df_aba_vis_in_c.loc[df_aba_vis_in_c.loc[:,'subclass'].str.contains(clns6),'cluster']))
                        else:
                            clns4=str.strip(clns4)
                            clns7=re.split('\-',clns4)
                            if len(clns7)>1:
                                clns4=clns7[0]
                                cls3m=cls3m+[str.strip(clns4)]
                            else:
                                cls3=cls3+[str.strip(clns4)]
                    cls3 = list(set(cls3).difference(set(cls3m)))
                    cl22=pd.DataFrame([cls3,[clp]],index=cols2).T
                    cl2post = cl2post.append(cl22)
                    
                    #in_postg = in_postg&df_aba_vis_in_c.loc[:,'cluster'].isin(cls3)
                    in_postg = in_postg|(in_postg0&df_aba_vis_in_c.loc[:,'cluster'].isin(cls3))
                    
                if len(syne3['cre_line_post'])>0:
                    in_line=df_aba_vis_in_c.loc[:,'driver_lines'].str.contains(syne3['cre_line_post'])
                    if in_line.sum()>0:
                        in_postg = in_postg&in_line
            
            
        ## make equal size pre- and post- datasets
        #in_preg = in_preg&(in_preg.isna()==False)
        #in_postg = in_postg&(in_postg.isna()==False)
        #npre = in_preg.sum()
        #npost = in_postg.sum()
        ##nsample = max([npost, npre] )
        #nsample = min([npost, npre] ) # number of cells
        #print('npre = '+ str(npre)+' , npost = ',str(npost))
        #if (nsample<150)&(npre>0)&(npost>0):
        #    nsample=150
        
        #idx2 =  np.mod(np.arange(nsample),npre)
        #in_preg = np.nonzero(in_preg)[0][idx2]
        #idx2 =  np.mod(np.arange(nsample),npost)
        #in_postg = np.nonzero(in_postg)[0][idx2]
        
        
        # make equal size pre- and post- datasets
        in_preg = in_preg&(in_preg.isna()==False)
        in_postg = in_postg&(in_postg.isna()==False)
        npre = in_preg.sum()
        npost = in_postg.sum()
        nsample = max([npost, npre] )
        #nsample = min([npost, npre] ) # number of cells
        

        
   
            
        print('npre = '+ str(npre)+' , npost = ' + str(npost) + ' , nsample = '+str(nsample))    
        
        
        # select cell indices for gene expression sampling
        #do_bootstrap=1 # defined in hippocampus
        if  do_bootstrap==0:
            
            if (nsample<60)&(npre>0)&(npost>0):
                nsample=60
            
            if (nsample>180)&(npre>0)&(npost>0):  # restrict nsample to avoid overrepresentation?
                nsample=180 
            
            idx2 =   np.mod(np.arange(nsample),npre) 
            in_preg = np.nonzero(in_preg)[0][idx2]
            idx2 =   np.mod(np.arange(nsample),npost) 
            in_postg = np.nonzero(in_postg)[0][idx2]
            
            in_preg_ = in_preg
            in_postg_ = in_postg
        else :
            if (npre>0)&(npost>0):
                nsample=nsample_hipp #200 #defined in hippocampus
            #random.uniform(low=0.0, high=1.0, size=None)
            idx2pre =  np.floor( np.random.uniform(0,npre, (nsample,1)) ).astype('int')
            in_preg_bs = np.nonzero(in_preg)[0][idx2pre]
            idx2post =  np.floor( np.random.uniform(0,npost, (nsample,1)) ).astype('int')
            in_postg_bs = np.nonzero(in_postg)[0][idx2post]
            in_preg_ = in_preg_bs[:,0]
            in_postg_ = in_postg_bs[:,0]
        
        

        if batch[0]:
            #in_preg_cl = list(set(df_aba_vis_l23glu_c.iloc[in_preg,:].loc[:,'cluster']))
            in_preg_cl = df_aba_vis_l23glu_c.iloc[in_preg_,:].loc[:,'cluster'].value_counts()       
            sampl_pre = df_aba_vis_l23glu_c.iloc[in_preg_,:].loc[:,'sample_name']
        else:
            #in_preg_cl = list(set(df_aba_vis_in_c.iloc[in_preg,:].loc[:,'cluster']))
            in_preg_cl = df_aba_vis_in_c.iloc[in_preg_,:].loc[:,'cluster'].value_counts()
            sampl_pre =df_aba_vis_in_c.iloc[in_preg_,:].loc[:,'sample_name']
        
        if batch[1]:
            #in_postg_cl = list(set(df_aba_vis_l23glu_c.iloc[in_postg,:].loc[:,'cluster']))
            in_postg_cl = df_aba_vis_l23glu_c.iloc[in_postg_,:].loc[:,'cluster'].value_counts()
            sampl_post =df_aba_vis_l23glu_c.iloc[in_postg_,:].loc[:,'sample_name']
        else:        
            #in_postg_cl = list(set(df_aba_vis_in_c.iloc[in_postg,:].loc[:,'cluster']))
            in_postg_cl = df_aba_vis_in_c.iloc[in_postg_,:].loc[:,'cluster'].value_counts()
            sampl_post =df_aba_vis_in_c.iloc[in_postg_,:].loc[:,'sample_name']
        in_preg_cl = in_preg_cl/in_preg_cl.sum()*100    
        in_postg_cl = in_postg_cl/in_postg_cl.sum()*100    
        print('presynaptic clusters')
        print(in_preg_cl)
        print('postsynaptic clusters')
        print(in_postg_cl)
        
        

        #if npost>npre:
        #    idx = np.mod(np.arange(npost),npre)
        #    #prege = prege[idx,:]
        #    #postge = postge
        #    prege = prege
        #    postge = postge[0:npre,:]            
        #elif npost<npre :
        #    idx = np.mod(np.arange(npre),npost)
        #    #prege = prege
        #    #postge = postge[idx,:]
        #    prege = prege[0:npost,:]
        #    postge = postge         
        
        # take ge data  
        
        prege = vis_dat.X[:, in_pregs]  # presynaptic gene set expression
        prege = prege[(vbi==batch[0]), :]
        
        postge = vis_dat.X[:,  in_postgs] # postsynaptic gene set expression
        postge = postge[(vbi==batch[1]),  :]
        
        # use hippocampal settings
        #do_median=1        # median over synapse types
        #do_log = 'none'    # logarithmic transform of ge counts
        
        if  (do_bootstrap==0)|(do_median==0):
            prege = prege[in_preg_,:]
            postge = postge[in_postg_,:]
            # do_samples_columns=1
        else: 
            # do_samples_columns=0
            #nsample = 100 # defined in hippocampus
            Nbs = nsample_hipp
            ##random.uniform(low=0.0, high=1.0, size=None)
            #nmax=5
            nmax = npre #np.min([1000,npre])
            idx2pre =  np.floor( np.random.uniform(0,npre, (nmax,Nbs)) ).astype('int')
            in_preg_ = np.nonzero(in_preg)[0][np.arange(npre)]
            nmax = npost #np.min([1000,npost])
            idx2post =  np.floor( np.random.uniform(0,npost, (nmax,Nbs)) ).astype('int')
            in_postg_ = np.nonzero(in_postg)[0][np.arange(npost)]

            # 
            # ??? transform ge to median bootstraped log ???
            prege_ = prege[in_preg_,:]
            postge_ = postge[in_postg_,:]
            
            if do_log=='none':
                 prege_ =  prege_
            elif do_log=='log2':       
                y = np.ma.masked_where(prege_ == 0, prege_)
                prege_=np.ma.log2(y).filled(0)
                y = np.ma.masked_where(postge_ == 0, postge_)
                postge_=np.ma.log2(y).filled(0)
            elif do_log=='log1p': 
                prege_ = np.log1p(prege_)
                postge_ =  np.log1p(postge_)




            npregs=prege_.shape[1]
            ge_bs = np.zeros((Nbs,npregs+postge_.shape[1]))
            for ibs in range(Nbs):
                #ans=np.apply_along_axis(lambda v: np.median(v[v!=0]), 0, x)
                #ans[np.isnan(ans)]=0.
                xbs = prege_[idx2pre[:,ibs],:]
                pregbs  = np.apply_along_axis(lambda v: np.median(v[v!=0]), 0, xbs)
                pregbs[np.isnan(pregbs)]=0.
                xbs = postge_[idx2post[:,ibs],:]
                postgbs = np.apply_along_axis(lambda v: np.median(v[v!=0]), 0, xbs)
                postgbs[np.isnan(postgbs)]=0.

                ge_bs[ibs,0:npregs] =pregbs
                ge_bs[ibs,npregs:]  =postgbs


            #df_hr2 = np.log2(df_hr2d)
            #df_hr2 = pd.concat([df_hr2,df_hr2s],axis=1) # log2 normalization
            #df_hr2 = df_hr2.replace({0:np.nan})
            #df_hr2 = np.log2(df_hr2)

            #df_hr2 = pd.concat([df_hr.iloc[1:,0],df_hr2],axis=1)

            #df_hr2.dtypes
            #df_hr2.head()

            #df_hr2 = df_hr2.replace({0:np.nan})
            #df_hrmg=df_hr2.groupby(['Class']) # logarithm_2 of UMI counts
            #df_hrmg=df_hrmg.median(numeric_only=True) # median of log2(UMI)

            #df_hrmg0 = df_hrmg.iloc[:,0:-1]

            #df_hrmsg = pd.concat([df_hrmg.iloc[:,-1]]*df_hrmg0.shape[1],axis=1)
            #df_hrmsg.columns = df_hrmg.iloc[:,0:-1].columns

            #df_hrmg = df_hrmg.iloc[:,0:-1] - df_hrmsg +6*np.log2(10)  # transform from log2( UMI counts) to log2(CPM)

            # ??? transform ge to median bootstraped log ???


              

        # add data to the final dataset    
        gnij = np.repeat(np.array([syne3['cell_type2_pre'], syne3['cell_type2_post'],
                                   syne3['layer_pre'], syne3['layer_post'],
                                   syne3['cre_line_pre'], syne3['cre_line_post']]).reshape((1,6)),nsample,axis=0 )
        gnij = pd.DataFrame(gnij, columns = ['cell_type2_pre', 'cell_type2_post',
                                             'layer_pre','layer_post',
                                             'cre_line_pre', 'cre_line_post'] )
        
        if  (do_bootstrap==0)|(do_median==0):
            prege  = pd.DataFrame(prege,columns  = pregs3 )
            postge = pd.DataFrame(postge,columns = postgs3 )


            #ge_dataij = pd.concat([gnij, prege, postge ], axis = 1)
            samples_pre = pd.DataFrame(sampl_pre.values[:gnij.shape[0]],columns = ['samples_pre'] )
            samples_post = pd.DataFrame(sampl_post.values[:gnij.shape[0]],columns = ['samples_post'] )
            
            
            ge_dataij = pd.concat([gnij, prege, postge, samples_pre, samples_post ], axis = 1)
            
            
            
        else:
            ge_bs = pd.DataFrame(ge_bs,columns = pregs3 + postgs3 )


            ge_dataij = pd.concat([gnij, ge_bs ], axis = 1)
       

        
        
        
        # add stp data
        if i<l_pre_post1:
            stp_dataij =  df_stp.loc[[0,i], stp_type['stp_data_type']].T
            stp_columns = stp_TM_names
        else:
            #stp_dataij  = STP_aba.loc[i1, STP_aba_type['stp_data_type']]
            #stp_columns = np.array(stp_aba_names)
            #nbs = int(len(stp_dataij)/len(stp_columns))
            #stp_dataij  = pd.DataFrame(stp_dataij, index = np.repeat(stp_columns,nbs)).T
            ##i2 = np.trunc(np.arange(len(stp_dataij.columns))/len(stp_columns)).astype(int)
            #stp_dataij = stp_dataij.T.reset_index(drop=False)
            #stp_dataij.columns =[0,i]
            
            stp_dataij  = STP_aba.loc[i1, STP_aba_type['stp_data_type']]
            stp_columns = np.array(stp_aba_names)
            #nbs = int(len(stp_dataij)/len(stp_columns))
            nbs = int(stp_dataij.shape[0]) #/len(stp_columns))
            stp_dataij  = pd.DataFrame(stp_dataij.reshape((-1,)),
                                       index = np.tile(np.array(stp_aba_names),(nbs,))).T
            
            
            if do_shrink_stp_distribution==True: # shrink bootstrap distribution of stp parameters towards mean values - to test stp variability reduction
                print('shrink bootstrap distribution of stp parameters : ',shrink_stp_distribution_coef)
                stp_dataij0 = stp_dataij.iloc[:,0:len(stp_aba_names)].values
                #nbs00 = int(stp_dataij.shape[1]/len(stp_aba_names))
                stp_dataij0  = pd.DataFrame(np.tile(stp_dataij0,[1,nbs]), columns = stp_dataij.columns)
                stp_dataij = stp_dataij0 + (stp_dataij - stp_dataij0)*shrink_stp_distribution_coef
            
            
            
            #i2 = np.trunc(np.arange(len(stp_dataij.columns))/len(stp_columns)).astype(int)
            stp_dataij = stp_dataij.T.reset_index(drop=False)
            stp_dataij.columns =[0,i]
            
            
        

        i2 = np.trunc(np.arange(len(stp_dataij.index))/len(stp_columns)).astype(int)
        stp_dataij  = stp_dataij.set_index(i2).pivot(columns=0,values=i).loc[:,stp_columns] #ss.set_index('par',append=True).unstack('par')
        #stp_dataij =  stp_dataij.T.pivot(index=0, values=synij).T
        
        idx = np.mod(np.arange(nsample),len(stp_dataij.index))
        stp_dataij=stp_dataij.iloc[idx,:].reset_index(drop=True)  
        
        i_data_set = pd.DataFrame(i+np.zeros(len(stp_dataij.index)), index = stp_dataij.index, columns = ['index_ds'])
        stp_dataij=pd.concat([stp_dataij, i_data_set],axis=1)
        
        #if do_averagig_stp:
        #    #stp_dataij.loc[:,:] = stp_dataij.mean(axis=0).values
        #    stp_dataij.loc[:,:] = stp_dataij.median(axis=0).values

        
        #stp_dataij = pd.DataFrame({'presynaptic':pree[0], 'postsynaptic': poste[0], 
        #                           'stp_data':stp_dataij.iloc[0,:].values})
        #stp_data  = stp_data.append(stp_dataij) 
        
        # names of ge and stp columns

        
        ge_dataij  = pd.concat([ge_dataij,stp_dataij] , axis=1)
        
        # add to all ge and stp dataframe
        ge_data  = pd.concat([ge_data, ge_dataij] , axis=0)
        

        
#stp_data =  stp_data.reset_index(drop=True)  



#stp_data.loc[stp_data.loc[:,'postsynaptic']=='NGC','postsynaptic']='Lamp5'
#stp_data2 = ge_data.iloc[:,0:3].copy(deep=True)                                 # modify!
#stp_data2.columns=['presynaptic', 'postsynaptic', 'ppr']
#print(stp_data2.head())
#for i in stp_data.index: #stp_data.index:
#    pre=stp_data.loc[i,'presynaptic']
#    post=stp_data.loc[i,'postsynaptic']
#    stp_data2.loc[stp_data2.loc[:,'presynaptic'].isin([pre])&stp_data2.loc[:,'postsynaptic'].isin([post]),'ppr']=stp_data.loc[i,'stp_data'] 
    
ge_data = ge_data.reset_index()    

 
## transform labels from TM to An:A1
As = STP_sim_complex(ge_data,l_pre_post2,stp_aba_names)

N = As.shape[1]

As =  As[:,1:]/As[:,0].reshape((-1,1))

#As =  As[:,1:]/np.tile(As[:,0],(N-1,1)).T

## add new stp parameters to ge_data dataset
#stp_columns2 = ['A'+str(s)+'/A1' for s in range(1,N+1)]
stp_columns2 = ['A'+str(s)+'/A1' for s in range(2,N+1)]
stp_data2 = pd.DataFrame(As,columns = stp_columns2, index = ge_data.index)

ge_data = pd.concat([ge_data, stp_data2 ],axis=1)

if do_averagig_stp:
     #stp_dataij.loc[:,:] = stp_dataij.mean(axis=0).values
     #stp_dataij.loc[:,:] = stp_dataij.median(axis=0).values
    all_i_ds =  ge_data.loc[:,'index_ds']                             
    for i_ds in list(set(all_i_ds)):
              ids2 =  ge_data.loc[:,'index_ds'].isin([i_ds])  
              ge_data.loc[ids2,stp_columns2] = ge_data.loc[ids2,stp_columns2].median(axis=0).values                   
                                   
                                   
## filter TM parameters
do_filter_pars = 0

if do_filter_pars!=0:
    print((ge_data.loc[:,stp_columns ]<0).sum().sum())
    print(len(ge_data.index))

    nannot=6
    annot_columns = ge_columns[0:nannot]
    ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()
    f,ax = plt.subplots(figsize=(5, 5))
    ge_data2.T.plot(ax=ax)
    plt.gca().legend('')
    #plt.gca().title('An:A1 before filtering data')
    ax.title.set_text('An:A1 before filtering data')
    #ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()
    print('mean of stp before filtering:')
    print(ge_data2)

    #i_filter_out = (ge_data.loc[:,'dp/p0']*ge_data.loc[:,'p0']+)|(ge_data.loc[:,'tD'])
    #i_filter_out = i_filter_out&((ge_data.loc[:,'tF'])|(ge_data.loc[:,'tF']))
    i_filter_out = np.sum(ps2>1,axis=1)|np.sum(ps2<0,axis=1)
    i_filter_out = i_filter_out|(np.sum(ns2>1,axis=1)|np.sum(ns2<0,axis=1))
    ge_data = ge_data.loc[i_filter_out==False ,:]

    ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()
    f1,ax1 = plt.subplots(figsize=(5, 5))
    ge_data2.T.plot(ax=ax1)
    plt.gca().legend('')
    #plt.gca().title('An:A1 after filtering data')
    ax1.title.set_text('An:A1 after filtering data')
    #ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()

    print('mean of stp after filtering:')
    print(ge_data2)
    
#print('mean of stp after filtering:')
#print(ge_data2)


print(len(ge_data.index))
#ge_data.loc[:,stp_columns ] =ge_data.loc[:,stp_columns ].abs()  


#
## normalize stp parameters - for TM parameters
do_normalize_stp_pars = 0;
if do_normalize_stp_pars!=0:
    ge_data.loc[:,stp_columns  ] =np.log(ge_data.loc[:,stp_columns  ].values.astype(float))

#  assign training and labels 
stp_columns1 = stp_columns 
stp_columns = stp_columns2 # select training labels names

nannot = 7
nstp =len(stp_columns)

annot_columns = ge_columns[0:6]+['index_ds']
if do_samples_columns==1:
    nannot = 9
    annot_columns = annot_columns + ['samples_pre', 'samples_post']    

## averaging of stp for each synapse type 


#X = ge_data.loc[:,ge_data.columns[0:]].values
X_c = ge_data.loc[:,annot_columns + ge_columns[6:]].values
y_c = ge_data.loc[:,annot_columns + stp_columns].values 
#y = stp_data2.loc[:,:].values

## normalize stp parameters - for TM parameters
#y[:,nannot: ] =np.log(y[:,nannot: ].astype(float))


    

# print elapsed time
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))

print(y_c.shape)
X_c.shape

In [ ]:
(ge_data.columns[:]=='samples_pre').sum()

# CORRECTED : 
there were errors in STP_cortex_interneurons_Cre_labeled_UDF5_4pars.xlsx table cre lines for Sst neurons assignment



# ACHTUNG!!! - ERROR IN CLUSTER ASSIGNMENT!!!
i =  22
{'layer_pre': 'L4', 'cre_line_pre': 'PC', 'cell_type2_pre': 'PC', 'cluster_pre': [], 'layer_post': 'L4', 'cre_line_post': 'Pvalb-IRES-Cre', 'cell_type2_post': 'Sst', 'cluster_post': []}
npre = 1034 , npost = 53 , nsample = 1034
presynaptic clusters
L4 IT VISp Rspo1            89.5
L5 IT VISp Hsd11b1 Endou     8.5
L5 IT VISp Whrn Tox2         0.5
L5 NP VISp Trhr Met          0.5
L2/3 IT VISp Agmat           0.5
L5 IT VISp Batf3             0.5
Name: cluster, dtype: float64
postsynaptic clusters
Pvalb Reln Itm2a     36.5
Pvalb Reln Tac1      34.5
Pvalb Tpbg           25.0
Pvalb Calb1 Sst       3.0
Pvalb Gpr149 Islr     1.0
Name: cluster, dtype: float64


In [ ]:
i =  30
{'layer_pre': 'L4', 'cre_line_pre': 'Pvalb-IRES-Cre', 'cell_type2_pre': 'PV', 'cluster_pre': [], 'layer_post': 'L4', 'cre_line_post': 'Sst-IRES-Cre', 'cell_type2_post': 'Sst', 'cluster_post': []}
npre = 53 , npost = 16 , nsample = 53
presynaptic clusters
Pvalb Reln Itm2a     42.5
Pvalb Reln Tac1      33.0
Pvalb Tpbg           19.5
Pvalb Gpr149 Islr     3.0
Pvalb Calb1 Sst       2.0
Name: cluster, dtype: float64
postsynaptic clusters
Sst Hpse Sema3c      26.5
Sst Tac1 Htr1d       21.5
Sst Calb2 Pdlim5     20.5
Sst Chrna2 Ptgdr     18.0
Sst Hpse Cbln4        7.5
Pvalb Gpr149 Islr     6.0
Name: cluster, dtype: float64


In [ ]:
print(ge_data_h.shape)
print(X_h.shape)
print(y_h.shape)
print(len(annot_columns_h))
print(len(ge_columns_h))
print(len(stp_columns_h))
print(ge_columns_h[300:])
print(ge_columns_h[0:10])

In [ ]:
print(ge_data.shape)
print(X_c.shape)
print(y_c.shape)
print(len(annot_columns))
print(len(ge_columns))
print(len(stp_columns))
print(ge_columns[300:])
print(ge_columns[0:10])

In [ ]:
ge_data.loc[11*200:11*200+5,['tF', 'p0','tD','dp']]

In [ ]:
import time
do_save_temporary=1
if do_save_temporary==1:
    t1=time.time()
    print('save ge_data_h...')
    #ge_data_h.to_excel('ge_data_h'+gs_name+'.xlsx')
    ge_data_h.to_hdf('ge_data_h'+gs_name+'.hdf',key='data')
    t2=time.time()
    print('Elapsed time ',t2-t1, ' s')
    print('save X_h...')
    t1=time.time()
    (pd.DataFrame(X_h)).to_hdf('X_h'+gs_name+'.hdf',key='data')
    t2=time.time()
    print('Elapsed time ',t2-t1, ' s')
    print('save y_h...')
    (pd.DataFrame(y_h)).to_hdf('y_h'+gs_name+'.hdf',key='data')
    print('save annot_columns_h...')
    (pd.DataFrame(annot_columns_h)).to_excel('annot_columns_h'+gs_name+'.xlsx')
    print('save ge_columns_h...')
    (pd.DataFrame(ge_columns_h)).to_excel('ge_columns_h'+gs_name+'.xlsx')
    print('save stp_columns_h...')
    (pd.DataFrame(stp_columns_h)).to_excel('stp_columns_h'+gs_name+'.xlsx')


In [ ]:
do_save_temporary=1
if do_save_temporary==1:
    print('save ge_data_c...')
    t1=time.time()
    #ge_data_h.to_excel('ge_data_h'+gs_name+'.xlsx')
    ge_data.to_hdf('ge_data_c'+gs_name+'.hdf',key='data')
    t2=time.time()
    print('Elapsed time ',t2-t1, ' s')
    print('save X_c...')
    t1=time.time()
    (pd.DataFrame(X_c)).to_hdf('X_c'+gs_name+'.hdf',key='data')
    t2=time.time()
    print('Elapsed time ',t2-t1, ' s')
    print('save y_c...')
    (pd.DataFrame(y_c)).to_hdf('y_c'+gs_name+'.hdf',key='data')
    print('save annot_columns_c...')
    (pd.DataFrame(annot_columns)).to_excel('annot_columns_c'+gs_name+'.xlsx')
    print('save ge_columns_c...')
    (pd.DataFrame(ge_columns)).to_excel('ge_columns_c'+gs_name+'.xlsx')
    print('save stp_columns_c...')
    (pd.DataFrame(stp_columns)).to_excel('stp_columns_c'+gs_name+'.xlsx')


In [ ]:
STP_aba_names = pd.DataFrame(STP_aba.index)
STP_aba_names.to_excel('stp_aba_names.xlsx')
ge_data.loc[np.arange(0,ge_data.index[-1],200),:].to_excel('ge_data_c_names'+gs_name+'.xlsx')

## SKIP NEXT SECTION : GRAPHS

In [ ]:
import matplotlib.cm as cm

# plot gene expression
print(ge_data.loc[[0,200,400,600,800,1000,1200],annot_columns])
print(postgs)
print(pregs)

apostgs = ['post__'+p for p in postgs]
apostgs = np.array(apostgs)[0:30]
apostgs2 = np.array(postgs)[0:30]
imda = ge_data.loc[1200,apostgs].map(float).values.reshape([1,-1])

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))

c_orange = np.array([241,132,62])/255
#c_blue = np.array([52,122,175])/255
c_blue = np.array([65,43,93])/255
#cols = np.array( [[1, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]])
cols = np.array( [[65,43,93], [241,132,62], [0, 255, 0]])/255


im = ax.imshow(imda,cmap=cm.RdYlGn)
Dn22=1
# We want to show all ticks...
xD = np.arange(0,apostgs.shape[0])
yD = np.arange(0)
ax.set_xticks(xD)
ax.set_yticks(yD)
# ... and label them with the respective list entries
#ax.set_yticklabels(nms4)
ax.set_xticklabels(apostgs2)
#ax.set_yticklabels(vse_pre2[yD])
#ax.tick_params(axis='x', colors='none')

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor", fontsize=16)

#axes.set_ylim(-1, nmm+1)

ax.grid( b=False)

apostgs = ['pre__'+p for p in pregs]
apostgs = np.array(apostgs)[np.arange(0,140,5)]
apostgs2 = np.array(pregs)[np.arange(0,140,5)]
imda = ge_data.loc[1200,apostgs].map(float).values.reshape([1,-1])

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))

c_orange = np.array([241,132,62])/255
#c_blue = np.array([52,122,175])/255
c_blue = np.array([65,43,93])/255
#cols = np.array( [[1, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]])
cols = np.array( [[65,43,93], [241,132,62], [0, 255, 0]])/255


im = ax.imshow(imda,cmap=cm.RdYlGn)
Dn22=1
# We want to show all ticks...
xD = np.arange(0,apostgs.shape[0])
yD = np.arange(0)
ax.set_xticks(xD)
ax.set_yticks(yD)
# ... and label them with the respective list entries
#ax.set_yticklabels(nms4)
ax.set_xticklabels(apostgs2)
#ax.set_yticklabels(vse_pre2[yD])
#ax.tick_params(axis='x', colors='none')

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor", fontsize=16)

#axes.set_ylim(-1, nmm+1)

ax.grid( b=False)



In [ ]:
import scipy as sci
def epsp_sim(A, T_A,dt,tplus,tminus,Tmax):
    
    l, v = sci.linalg.eig(np.array([[-1/tplus,0],[1/tplus,-1/tminus]])) 
    l = np.real(l).reshape([-1,1])
    vm = sci.linalg.inv(v)
    #x = v*np.exp(l*t)*np.array([1,0]).reshape([-1,1])                  
    X = np.array([0,0]).reshape([-1,1])

    t=np.array([0])
    for i in np.arange(1,T_A.size):
        if T_A[i]-T_A[i-1]>Tmax+10*dt:
            t2  =  np.arange(int(Tmax/dt))[np.newaxis]*dt  
            t3  =  - 10*dt + np.arange(10)[np.newaxis]*dt   
            
            t5 = np.append(t2,t3+Tmax+10*dt ,axis=1)
            t2 = np.append(t2,t3+T_A[i] - T_A[i-1],axis=1)

        else: 
            t2  =  np.arange(int((T_A[i]-T_A[i-1])/dt))[np.newaxis]*dt 
            t5 = t2

        #print(i)    
        x0 = X[:,[-1]]   
        x0[0,:] = x0[0,:]+A[i-1] 
        vvmx=(v@np.diag((vm@x0).ravel()))
        x = vvmx@np.exp(l@t2)
        X = np.append(X,x,axis=1)
        #t = np.append(t,T_A[i-1] +t5)   
        t = np.append(t,t[-1] +t5)   
    
    return X, t

In [ ]:

dt=0.1
tplus=0.3
tminus=10
Tmax = 100
A = np.array([0,2,3,4,5,0])
T_A = np.array([0,200,250,300,550,650])

X, t = epsp_sim(A, T_A,dt,tplus,tminus,Tmax)
    
f, ax =plt.subplots(figsize=(sx, sy))
plt.plot(t,X.T)

f, ax =plt.subplots(figsize=(sx, sy))
plt.plot(X.T)


In [ ]:
l_pre_post23 = 2 #200 #int(200/10)
ge_data_temp = ge_data.iloc[::l_pre_post23,:] #.loc[:,stp_aba_names]

In [ ]:
ge_data_temp.shape

In [ ]:
#annot_columns2 =np.array(annot_columns)
annot_columns
ge_data_temp.loc[:,annot_columns]

In [ ]:

dt=0.1
import matplotlib.pyplot as plt 
%matplotlib inline
# transform labels from TM to An:A1
fs = [20 , 10, 40] # Hz
N = 8

Trec = [250, 1000]
Trec=[]
DT0 = 25000
xs  =ge_data_temp.iloc[:,:].loc[:,stp_aba_names].values
xs  =np.delete(xs, [5,6],axis=1)
#As=np.zeros((l_pre_post2,0))
As=np.zeros((xs.shape[0],0))
As2=np.zeros((xs.shape[0],0))
Tall_A = np.arange(1)
if do_add_aba_synphys:
    
    for f in fs:
        T = np.arange(N)*1000/f
        Tall_A = np.append(Tall_A, T+DT0+Tall_A[-1])
        
        Asf, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data_temp.iloc[0:,:],T)
        As = np.concatenate([As,Asf],axis=1)
        
        for ri in range(len(Trec)):
            Asr, nsr, psr, dpp0, p0, tF, tD, A = STP_sim(ge_data_temp.iloc[0:,:],[Trec[ri]],init_state=[ns[:,-1],ps[:,-1]])
            As = np.concatenate([As,Asr],axis=1)
        Tall_A = np.append(Tall_A, np.array(Trec)+Tall_A[-1])
        

        As2f = np.zeros((xs.shape[0],N+len(Trec)))
        for i2 in range(xs.shape[0]):
            #as2, sts2 = STP_sim2(xs[i2,:],np.arange(8)*50,model_type = 'tm5') # preconditioning series???
            #as2, sts2 = STP_sim2(xs[i2,:], T+DT0, init_state=sts2[-1], model_type='tm5' ) 
            
            as2, sts2 = STP_sim2(xs[i2,:],T,model_type = 'tm5_smr')
            
            As2f[i2,0:N] = as2
            for ri in range(len(Trec)):
                as2r, sts2r = STP_sim2(xs[i2,:], [Trec[ri]], init_state=sts2[-1], model_type='tm5_smr' ) 
                As2f[i2,N+ri] = as2r
                
        As2 = np.concatenate([As2,As2f],axis=1)
            
    
    As2 = As2/As2[:,0].reshape((-1,1))
    As = As/As[:,0].reshape((-1,1))
    Na1 = As.shape[1]
    As = np.concatenate([As,As2],axis=0)
else:
    As, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data,T)
    

sx=10
sy=5
do_this=0
if do_this==1:
    for ii in range(As2.shape[0]):
        f, ax =plt.subplots(figsize=(sx, sy))
        plt.plot(As2[ii,:],'o-',linewidth=4, markersize=12)
        #plt.ylim([0,1.2])
        plt.xticks(fontsize=18)
        plt.xticks(ticks=[0,5,10,15,20],fontsize=18)
        plt.yticks(ticks=[0.5,1.0,1.5],fontsize=18)
        plt.ylabel('An:A1',fontsize=20)
        plt.xlabel('Stimulus number',fontsize=20)
        # cell_type2_pre	cell_type2_post
        str_syn_type = ge_data_temp.iloc[ii,:].loc['cell_type2_pre']+'->'+ge_data_temp.iloc[ii,:].loc['cell_type2_post']
        plt.title(str(ii)+' '+str_syn_type)
        #plt.legend(fontsize=20)
        #print(set(n_stp_cells.index).difference(set(i2)))

do_this=0
if do_this==1:
    iis = [19,23,11,28,25,29,44,46,10]    
    for ii in range(len(iis)): #(As.shape[0]): #(len(iis)):    
        if ii<4:
            cc=0  # sst
            col = np.array([255, 0, 0])/255
        if (ii>=4) and (ii<7):
            cc=1  # pvalb
            col = np.array([127, 96, 0])/255
        if ii>=7:
            cc=2  # vip 
            col = np.array([112, 48, 160])/255

        f, ax =plt.subplots(figsize=(sx, sy))
        dt=0.1
        tplus=1
        tminus=20
        Tmax = 150
        ##A = As[0,:Na1] #np.mean(As[:200,:Na1],axis=0)
        #A = As[iis[ii],Na1-9:Na1]
        #A = np.append([0],A)
        #T_A = Tall_A[-9:-1]
        #T_A = np.append(T_A,[T_A[-1]+2*Tmax])

        A = As[iis[ii],0:Na1]
        A = np.append([0],A)
        T_A = Tall_A[0:-1]
        T_A = np.append(T_A,[T_A[-1]+2*Tmax])

        epsp, t = epsp_sim(A, T_A,dt,tplus,tminus,Tmax)
        plt.plot(t[14800:], epsp[1,14800:].T,linewidth=4,color=col)

        str_syn_type = ge_data_temp.iloc[iis[ii],:].loc['cell_type2_pre']+'->'+ge_data_temp.iloc[iis[ii],:].loc['cell_type2_post']
        #plt.title(str(iis[ii])+' '+str_syn_type)







    iis = [19,23,11,28,25,29,44,46,10]    
    for ii in range(len(iis)):    
        if ii<4:
            cc=0  # sst
            col = np.array([255, 248, 82])/255
        if (ii>=4) and (ii<7):
            cc=1  # pvalb
            col = np.array([0, 69, 245])/255
        if ii>=7:
            cc=2  # vip 
            col = np.array([115, 244, 64])/255

        f, ax =plt.subplots(figsize=(sx, sy))
        dt=0.1
        tplus=1
        tminus=20
        Tmax = 150
        ##A = As[0,:Na1] #np.mean(As[:200,:Na1],axis=0)
        #A = As[iis[ii],Na1-9:Na1]
        #A = np.append([0],A)
        #T_A = Tall_A[-9:-1]
        #T_A = np.append(T_A,[T_A[-1]+2*Tmax])

        A = As[iis[ii],0:Na1]
        A = np.append([0],A)
        T_A = Tall_A[0:-1]
        T_A = np.append(T_A,[T_A[-1]+2*Tmax])

        epsp, t = epsp_sim(A, T_A,dt,tplus,tminus,Tmax)
        plt.plot(t[14800:], epsp[1,14800:].T,linewidth=4,color=col)

        str_syn_type = ge_data_temp.iloc[iis[ii],:].loc['cell_type2_pre']+'->'+ge_data_temp.iloc[iis[ii],:].loc['cell_type2_post']
        #plt.title(str(iis[ii])+' '+str_syn_type)    





In [ ]:
iis = [11,13] #, 14]
dn2=100
for ii in range(len(iis)):    
    col = np.array([200, 200, 245])/255
    
    f, ax =plt.subplots(figsize=(sx, sy))
    dt=0.1
    tplus=1
    tminus=20
    Tmax = 150
    ##A = As[0,:Na1] #np.mean(As[:200,:Na1],axis=0)
    #A = As[iis[ii],Na1-9:Na1]
    #A = np.append([0],A)
    #T_A = Tall_A[-9:-1]
    #T_A = np.append(T_A,[T_A[-1]+2*Tmax])
    Epsp =[]
    for iii in range(dn2):
        A = As[iis[ii]*dn2+iii,0:Na1]
        A = np.append([0],A)
        T_A = Tall_A[0:-1]
        T_A = np.append(T_A,[T_A[-1]+2*Tmax])

        epsp, t = epsp_sim(A, T_A,dt,tplus,tminus,Tmax)
        Epsp = Epsp + [epsp[1,:].reshape([-1,1])]
     
    
    amps3 = As[iis[ii]*dn2+np.arange(dn2),0:Na1][:,-8:-1]
    ism1 = amps3[:,-1]>0.8
    amps3 = amps3[:,:]
    amps3 = amps3*(1 + np.random.normal(size=amps3.shape)*0.15)
    #positions = t[14800] +3.7+25 + int(25)*np.arange(amps3.shape[1])
    
    Epsp = np.concatenate(Epsp, axis=1)
    
    Epsp = Epsp[:,ism1]
    Epsp = (np.random.normal(size=Epsp.shape)*0.1+1)*Epsp
    plt.plot(t[14800:], Epsp[14800:,:],linewidth=1,color=np.array([0, 69, 245])/255) 
    
    fig,ax = plt.subplots(figsize=(sx, sy))
    plt.plot(np.arange(amps3.shape[1])+1,amps3.transpose(),'-o',linewidth=1,color=col)
    plt.boxplot(amps3, widths = 0.7)

    ax.set_ylim([0,1.7])
    
#     pyplot.boxplot(x, notch=None, sym=None, vert=None, whis=None, positions=None, widths=None, 
#                    patch_artist=None, bootstrap=None, usermedians=None, conf_intervals=None, meanline=None, showmeans=None, 
#                    showcaps=None, showbox=None, showfliers=None, boxprops=None, labels=None, flierprops=None, medianprops=None,
#                    meanprops=None, 
#                    capprops=None, whiskerprops=None, manage_ticks=True, autorange=False, zorder=None, *, data=None)
    
    str_syn_type = ge_data_temp.iloc[iis[ii]*dn2,:].loc['cell_type2_pre']+'->'+ge_data_temp.iloc[iis[ii],:].loc['cell_type2_post']
    #plt.title(str(iis[ii])+' '+str_syn_type) 
    
    fig,ax = plt.subplots(figsize=(sx, sy))
    ge_data_temp.iloc[iis[ii]*dn2,:].loc['cell_type2_pre']
    
    plt.plot(np.arange(amps3.shape[1])+1,amps3.transpose(),'-o',linewidth=1,color=col)


    

In [ ]:
amps3.shape

In [ ]:
positions.shape

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline
# transform labels from TM to An:A1
fs = [20 , 10, 40] # Hz
N = 8

Trec = [250, 1000]
Trec=[]
DT0 = 25000
xs  =ge_data.iloc[l_pre_post2:,:].loc[:,stp_aba_names].values
xs  =np.delete(xs, [5,6],axis=1)
As=np.zeros((l_pre_post2,0))
As2=np.zeros((xs.shape[0],0))
Tall_A = np.arange(1)
if do_add_aba_synphys:
    
    for f in fs:
        T = np.arange(N)*1000/f
        Tall_A = np.append(Tall_A, T+DT0+Tall_A[-1])
        
        Asf, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data.iloc[0:l_pre_post2,:],T)
        As = np.concatenate([As,Asf],axis=1)
        
        for ri in range(len(Trec)):
            Asr, nsr, psr, dpp0, p0, tF, tD, A = STP_sim(ge_data.iloc[0:l_pre_post2,:],[Trec[ri]],init_state=[ns[:,-1],ps[:,-1]])
            As = np.concatenate([As,Asr],axis=1)
        Tall_A = np.append(Tall_A, np.array(Trec)+Tall_A[-1])
        

        As2f = np.zeros((xs.shape[0],N+len(Trec)))
        for i2 in range(xs.shape[0]):
            #as2, sts2 = STP_sim2(xs[i2,:],np.arange(8)*50,model_type = 'tm5') # preconditioning series???
            #as2, sts2 = STP_sim2(xs[i2,:], T+DT0, init_state=sts2[-1], model_type='tm5' ) 
            
            as2, sts2 = STP_sim2(xs[i2,:],T,model_type = 'tm5_smr')
            
            As2f[i2,0:N] = as2
            for ri in range(len(Trec)):
                as2r, sts2r = STP_sim2(xs[i2,:], [Trec[ri]], init_state=sts2[-1], model_type='tm5_smr' ) 
                As2f[i2,N+ri] = as2r
                
        As2 = np.concatenate([As2,As2f],axis=1)
            
    
    As2 = As2/As2[:,0].reshape((-1,1))
    As = As/As[:,0].reshape((-1,1))
    Na1 = As.shape[1]
    As = np.concatenate([As,As2],axis=0)
else:
    As, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data,T)
    

sx=12
sy=3
f, ax =plt.subplots(figsize=(sx, sy))
plt.plot(As2[0:1,:].transpose(),'o-',linewidth=4, markersize=12)
plt.ylim([0,1.2])
plt.xticks(fontsize=18)
plt.xticks(ticks=[0,5,10,15,20],fontsize=18)
plt.yticks(ticks=[0.5,1.0,1.5],fontsize=18)
plt.ylabel('An:A1',fontsize=20)
plt.xlabel('Stimulus number',fontsize=20)
#plt.legend(fontsize=20)
#print(set(n_stp_cells.index).difference(set(i2)))


f, ax =plt.subplots(figsize=(sx, sy))
dt=0.1
tplus=1
tminus=10
Tmax = 300
#A = As[0,:Na1] #np.mean(As[:200,:Na1],axis=0)
A = As[0,:Na1]
A = np.append([0],A)
T_A = Tall_A
T_A = np.append(T_A,[T_A[-1]+Tmax])

epsp, t = epsp_sim(A, T_A,dt,tplus,tminus,Tmax)
plt.plot(t, epsp[1,:].T,linewidth=4)

In [ ]:
f, ax =plt.subplots(figsize=(sx, sy))
dt=0.1
tplus=5
tminus=50
Tmax = 300

DT = 200/6

T_A = np.array([0]+(100+np.arange(9)*200/6).tolist()+[100+200/6*8+1000])

i0=23 #42 #[18,23,42]
ii = i0*200 + np.arange(200)
A = np.mean(As[ii,:Na1],axis=0) #As[0,:Na1]#np.mean(As[:100,:Na1],axis=0)
A = np.append([0],A)
#T_A = Tall_A
T_A = np.append(T_A,[T_A[-1]+400])

epsp, t = epsp_sim(A, T_A,dt,tplus,tminus,Tmax)
plt.plot(t, epsp[1,:].T)

## START FROM NEXT SECTION : Classify STP

### Classify STP for cortex : 3 types by D/F ratio

In [ ]:
%matplotlib inline


#
#do_medians, do_log should be off!!!
# Nsamples = 200 !!!


## some distributions
#stp_columns

# f, ax =plt.subplots(figsize=(16, 7))
# plt.plot(ge_data.iloc[3600:3800,:].loc[:,stp_columns].transpose(),'o-')
# plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
#          rotation_mode="anchor")

#f, ax =plt.subplots(figsize=(16, 7))
stp_columns2=np.array(stp_columns)
annot_columns2 =np.array(annot_columns)
sx=5
sy=4


i0=0
Nbootstraps = 100

skip_this=1

Stp_type_p = []

for i0 in range(0,53):#[42]: # [18,23,42] #range(0,53):
    str_syntype = ge_data.loc[i0*Nbootstraps,annot_columns2[[0,2,1,3]]].astype(str).str.cat(others=['_','->','_','']).str.cat()

    #print(ge_data.loc[i0*200,annot_columns[:-3]])

    ii = i0*Nbootstraps + np.arange(Nbootstraps)
    
    f, ax =plt.subplots(figsize=(sx, sy))
    dat=ge_data.iloc[ii,:].loc[:,'tD'].map(float)
    #dat=dat.loc[dat.loc[:]<800]
    dat.loc[dat.loc[:]>500]=500
    dat2=ge_data.iloc[ii,:].loc[:,'tF'].map(float)
    #dat2=dat2.loc[dat2.loc[:]<5500]
    dat2.loc[dat2.loc[:]>500]=500
    plt.scatter(dat,dat2)
    
    #stp_ns =          ['tF',    'p0',    'tD',   'dp/p0', 'A']
    dat_p0=ge_data.iloc[ii,:].loc[:,'p0'].map(float)
    dat_dp=ge_data.iloc[ii,:].loc[:,'dp/p0'].map(float)
    
    #dat.hist(bins=25)
    plt.title(str(i0)+ '  '+str_syntype + '  ' +'D->F')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel('D, ms',fontsize=20)
    plt.ylabel('F, ms',fontsize=20)

    f, ax =plt.subplots(figsize=(sx, sy))
    #ge_data.iloc[ii,:].loc[:,'tF'].map(float).hist(bins=25)
    
    dat3 = dat2/dat
    dat3.hist(bins=25)
    plt.title(str(i0)+ '  '+str_syntype + '  ' +'F/D')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel('F/D',fontsize=20)
    plt.ylabel('Frequency',fontsize=20)
    nsam = dat3.shape[0]
    #print(str(i0)+ '  '+str_syntype + '  ' +'F/D  :','<0.2 - ',(dat3<0.2).sum()/nsam,
    #      '; >0.2, <2 - ',((dat3>0.2)&(dat3<2)).sum()/nsam, '; >2 - ',(dat3>2).sum()/nsam  )
#     stp_type_p = [str(i0)+ '  '+str_syntype,
#                   (dat3<0.4).sum()/nsam, ((dat3>0.4)&(dat3<2)).sum()/nsam, (dat3>2).sum()/nsam ,
#                   (dat3.iloc[0]<0.4), (dat3.iloc[0]>0.4)&(dat3.iloc[0]<2), (dat3.iloc[0]>2),
#                   np.median(dat3),dat3.iloc[0], dat3.values,
#                   np.array([dat.iloc[0],dat2.iloc[0],dat_p0.iloc[0], dat_dp.iloc[0]])]
    if i0<10:
        stp_names = ['tD', 'tF','p0','dp/p0']
    else:
        # 'tF', 'p0','tD','dp','A','A1','A2','tDmin','dd','t_FDR','t_SMR','dp0'
        stp_names = ['tD', 'tF','p0','dp','t_SMR','dp0']
    stp_type_p = [str(i0)+ '  '+str_syntype,
                  (dat3<0.4).sum()/nsam, ((dat3>0.4)&(dat3<2)).sum()/nsam, (dat3>2).sum()/nsam ,
                  (dat3.iloc[0]<0.4), (dat3.iloc[0]>0.4)&(dat3.iloc[0]<2), (dat3.iloc[0]>2),
                  np.median(dat3),dat3.iloc[0], dat3.values,
                  np.array([dat.iloc[0],dat2.iloc[0],dat_p0.iloc[0], dat_dp.iloc[0]]),
                  ge_data.iloc[ii[0],:].loc[stp_names].values]
    Stp_type_p = Stp_type_p + [stp_type_p]
    
    
    
    print(str(i0)+ '  '+str_syntype + '  ' +'F/D  :',stp_type_p   )

### STP classes in hippocampus based on D/F ratio

In [ ]:
%matplotlib inline


#
#do_medians, do_log should be off!!!
# Nsamples = 200 !!!


## some distributions
#stp_columns

# f, ax =plt.subplots(figsize=(16, 7))
# plt.plot(ge_data.iloc[3600:3800,:].loc[:,stp_columns].transpose(),'o-')
# plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
#          rotation_mode="anchor")

#f, ax =plt.subplots(figsize=(16, 7))
stp_columns2=np.array(stp_columns)
annot_columns2 =np.array(annot_columns)
sx=5
sy=4
i0=0
skip_this=1

Stp_type_p_h = []

for i0 in range(0,40):#[42]: # [18,23,42] #range(0,53):
    str_syntype = ge_data_h.loc[i0*Nbootstraps,annot_columns2[[0,2,1,3]]].astype(str).str.cat(others=['_','->','_','']).str.cat()

    #print(ge_data.loc[i0*200,annot_columns[:-3]])

    ii = i0*Nbootstraps + np.arange(Nbootstraps)
    
    f, ax =plt.subplots(figsize=(sx, sy))
    dat=ge_data_h.iloc[ii,:].loc[:,'tD'].map(float)
    #dat=dat.loc[dat.loc[:]<800]
    dat.loc[dat.loc[:]>500]=500
    dat2=ge_data_h.iloc[ii,:].loc[:,'tF'].map(float)
    #dat2=dat2.loc[dat2.loc[:]<5500]
    dat2.loc[dat2.loc[:]>500]=500
    plt.scatter(dat,dat2)
    
    #stp_ns =          ['tF',    'p0',    'tD',   'dp/p0', 'A']
    dat_p0=ge_data.iloc[ii,:].loc[:,'p0'].map(float)
    dat_dp=ge_data.iloc[ii,:].loc[:,'dp/p0'].map(float)
    
    #dat.hist(bins=25)
    plt.title(str(i0)+ '  '+str_syntype + '  ' +'D->F')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel('D, ms',fontsize=20)
    plt.ylabel('F, ms',fontsize=20)

    f, ax =plt.subplots(figsize=(sx, sy))
    #ge_data.iloc[ii,:].loc[:,'tF'].map(float).hist(bins=25)
    
    dat3 = dat2/dat
    dat3.hist(bins=25)
    plt.title(str(i0)+ '  '+str_syntype + '  ' +'F/D')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel('F/D',fontsize=20)
    plt.ylabel('Frequency',fontsize=20)
    nsam = dat3.shape[0]
    #print(str(i0)+ '  '+str_syntype + '  ' +'F/D  :','<0.2 - ',(dat3<0.2).sum()/nsam,
    #      '; >0.2, <2 - ',((dat3>0.2)&(dat3<2)).sum()/nsam, '; >2 - ',(dat3>2).sum()/nsam  )
    #stp_type_p = [str(i0)+ '  '+str_syntype, (dat3<0.4).sum()/nsam, ((dat3>0.4)&(dat3<2)).sum()/nsam, (dat3>2).sum()/nsam, np.median(dat3), dat3.values ]
    
    stp_names = ['tD', 'tF','p0','dp/p0']
    stp_type_p = [str(i0)+ '  '+str_syntype,
                  (dat3<0.4).sum()/nsam, ((dat3>0.4)&(dat3<2)).sum()/nsam, (dat3>2).sum()/nsam ,
                  (dat3.iloc[0]<0.4), (dat3.iloc[0]>0.4)&(dat3.iloc[0]<2), (dat3.iloc[0]>2),
                  np.median(dat3),dat3.iloc[0], dat3.values,
                  np.array([dat.iloc[0],dat2.iloc[0],dat_p0.iloc[0], dat_dp.iloc[0]]),
                 ge_data.iloc[ii[0],:].loc[stp_names].values]
    Stp_type_p_h = Stp_type_p_h + [stp_type_p]
    
    print(str(i0)+ '  '+str_syntype + '  ' +'F/D  :',stp_type_p   )

In [ ]:
df_Stp_type_p = pd.DataFrame(Stp_type_p + Stp_type_p_h, columns=['name','type_2','type_3','type_1',
                                                                 'dtype_2','dtype_3','dtype_1',
                                                                 'median','F_D_0','F_D','tm_DFUdU','tm_all'])
df_Stp_type_p.head()

In [ ]:
df_Stp_type_p.to_excel('stp types probability cortex and hippocampus.xlsx')

In [ ]:
print(dat.iloc[0])
print(dat2.iloc[0])
print(dat_p0.iloc[0]) 
print(dat_dp.iloc[0])


### Plot EPSPs for different synaptic types (aka fig9b)

In [ ]:
df_Stp_type_p = pd.read_excel('stp types probability cortex and hippocampus.xlsx')
df_Stp_type_p['polarity'] = 'inh'

is_PC_presynaptic = ge_data.loc[np.arange(df_Stp_type_p.shape[0])*Nbootstraps,annot_columns2[0]].astype(str).str.contains('PC').values
df_Stp_type_p.loc[is_PC_presynaptic,'polarity']='ex'

ex_types = ['CA1', 'CA3', 'EC','DG']
for ex_type in ex_types:
    is_PC_presynaptic = ge_data_h.loc[np.arange(df_Stp_type_p.shape[0])*Nbootstraps,annot_columns2[0]].astype(str).str.contains(ex_type).values
    is_PC_presynaptic = np.nonzero(is_PC_presynaptic )[0]+53 
    df_Stp_type_p.loc[is_PC_presynaptic,'polarity']='ex'    

df_Stp_type_p.loc[53:73,:]

In [ ]:
def Q_TM_aba_synphys(x, amps,sig,Ts,DT0,model_type='tm5'):
    #Q, A, A3 =  QA_TM_aba_synphys(x, amps,sig,Ts)
    Q, A =  QA_TM_aba_synphys(x, amps,sig,Ts,DT0,model_type)
    return Q    

In [ ]:
def QA_TM_aba_synphys(x, amps,sig,Ts,DT0,model_type='tm5'):
    #amps=args[0] 
    #sig=args[1] 
    #Ts=args[2]
    
    
    fl=0
    if fl==0:
        ams = x[4:7]
        #breakpoint()
        if len(x)>7:
            if model_type=='tm5_fdr2':
                x2=np.copy(np.delete(x,[5,6]))
            if model_type=='tm5_smr':
                x2=np.copy(np.delete(x,[5,6]))    
            
            #x2[5:] = x[7:]
        else:
            x2=np.copy(x[0:5])
    
    n  = [0, 32, 44]
    t2 = [6, 1,  1]
    #T2  =[125, 250, 500, 1000, 2000, 4000]
    eps = np.finfo(float).eps
    z4 = np.arange(4)
    z8 = np.arange(8)
    A = np.zeros((amps.shape[0],))
    #A3 = np.ones((amps.shape[0],))
    #S3 = np.zeros((amps.shape[0],))
    Q=0
    for ii in range(3):
        vv = z8 + n[ii]
        A1e = amps[vv]  
        S1e = sig[vv]
        if fl==0:
            x2[4] = ams[ii] # independent amplitudes for each stimulation frequency - to account for rundown etc.
        
        #vv_nonz = np.nonzero(np.abs(A1e[:])>eps*1e3)[0]
        vv_z = np.abs(A1e[:])<eps*1e3
        #print(vv_nonz)
        #if len(vv_nonz)>0:
        if sum(vv_z)!=len(A1e):
            # preconditioning:
            A1, states = STP_sim(x2, Ts[vv],model_type=model_type ) 
            np0 = states[-1]
            
            # first 8 stimuli responces:
            #A1, states = STP_sim(x2, Ts[vv]+DT0, init_state=np0, model_type=model_type ) 
            #breakpoint()
            A1[vv_z] = 0
            A[vv] = A1
            
            #Q = Q + np.sum((A1e[vv_nonz]-A1[vv_nonz])**2/(S1e[vv_nonz]**2 + eps))
            Q = Q + np.sum((A1e-A1)**2/(S1e**2 + eps))
            # recovery responces
            #np0 = [n12[-1], p12[-1], d12[-1]]
            np0 = states[-1]
            for i3 in range(t2[ii]):
                vv2 = vv[-1]+1+z4+4*i3
                A2e = amps[vv2] 
                S2e = sig[vv2] 
                ##A3[vv2] = np.median(A1e)
                #vv_nonz2 = np.nonzero(np.abs(A2e[:])>eps*1e3)[0]
                #print(vv_nonz2)
                vv_z2 = np.abs(A2e[:])<eps*1e3
                #if len(vv_nonz2)>0:
                if sum(vv_z2)!=len(A2e):
                    A2, states2 = STP_sim(x2, Ts[vv2], init_state=np0, model_type=model_type )
                    
                    A2[vv_z2] = 0
                    
                    A[vv2] = A2
                    
                    #breakpoint()
                    #A2 = A2/np.median(A1)
                    #A[vv2] = A2
                    #breakpoint()
                    
                    #Q = Q + np.sum((A2e[vv_nonz2]-A2[vv_nonz2])**2/(S2e[vv_nonz2]**2 + eps))
                    Q = Q + np.sum((A2e-A2)**2/(S2e**2 + eps))
    
    return Q, A #, A3

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
x = [1.00000000e+01, 4.00000000e-01, 1.00000000e+02, 4.00000000e-01,
 4.59968078e+00, 4.59968078e+00,4.59968078e+00, 1.00000000e+01,
 5.00000000e-04, 1.00000000e+02]

Ts = df.loc[1,'comments1']
Ts = np.array(Ts[0][:].strip().split()).astype(float)
print(Ts)

vv=np.arange(len(As))
npar=56
amps = np.ones(npar)
amps[vv]=As*1.05
sig = amps/5
DT0=2500000
model_type = 'tm5'
Q2,As2 = QA_TM_aba_synphys(x[0:7], amps,sig,Ts,DT0,model_type )
plt.plot(As2)
As2

In [ ]:
#i=1
def fit_TM_cell(i,df,par):
    #breakpoint()
    aba_case=False
    if 'aba_case' in par:
        aba_case = par['aba_case']
        
    model_type='tm5'
    if 'model_type' in par:
        model_type = par['model_type']   
        
    dont_plot=0
    if 'dont_plot' in par:
        dont_plot = par['dont_plot'] 
    
    if par['fited_pulses']==True:
        ampsa = df.loc[i,'stp_mean_fit_bootstrap']
        siga = df.loc[i,'stp_sigma_fit_bootstrap']  
        #if abs(ampsa[0])<1e-14:
            #ampsa = df.loc[i,'stp_mean_bootstrap']
            #siga = df.loc[i,'stp_sigma_bootstrap']
    else:
        ampsa = df.loc[i,'stp_mean_bootstrap']
        siga = df.loc[i,'stp_sigma_bootstrap']
        
    Ts = df.loc[i,'comments1']
    Ts = np.array(Ts[0][:].strip().split()).astype(float)
    
    #breakpoint()

    #A3 = df.loc[i,'comments']
    #A3i = np.array(A3[0][:].strip().split()).astype(float)
    #A4 = np.ones((len(A3),len(A3i)))
    #for ia3 in range(len(A3)):
    #    A3i = np.array(A3[ia3][:].strip().split()).astype(float)
    #    A4[ia3,:] = A3i
    ##A4.shape 
    
    
    ibs  = par['ibs']
    npar = 56
    #amps = ampsa[ibs*npar + np.arange(npar)]
    ampsa1 = ampsa.reshape((-1,npar))
    amps = ampsa1[ibs,:].ravel()
    
    if aba_case:
        # sig  = siga #[(ibs-1)*npar + np.arange(npar)]
        #A3e  = A4[:,ibs ].ravel()

        siga1 = siga.reshape((-1,npar))
        sig = siga1[ibs,:].ravel()
    else:
        #sig = np.std(ampsa1,axis=0).ravel() # reestimate sigma
        sig=pd.DataFrame(ampsa1).std(axis=0).values
        
    DT0 = 25000
    #breakpoint()
    # restrict set of fited data
    vv = par['amplitudes_selected_for_TM_training']
    #vv = np.arange(8)    # A 1:8
    #vv = np.arange(npar) # all
    #vv = np.concatenate([vv,vv+32, vv+44]) # without recovery
    
    amps0 = amps
    amps = np.zeros(amps.shape)
    amps[vv] = amps0[vv]

         #  tF   p0     tD  dp         A            A1           A2      tDmin  dd   t_FDR t_SMR dp0 
    x0 = [ 100,  0.1,  100, 0.1, amps[0]/0.1, amps[0]/0.1,  amps[0]/0.1, 10,   0.05, 100,  100,  0.02 ]
    
    x0 = np.array(x0)
    
    #dxx = np.array([1e3, 20, 1e3, 1e2,  20, 20, 20,  1e1, 20, 1e2 ])
    dxx = np.array([1e3,  10, 1e3, 10,  20, 20, 20,  1e3, 20, 1e3, 1e3, 50 ])
    
    #breakpoint()
    if model_type=='tm5':
        x0 = x0[0:7]
        dxx = dxx[0:7]
        
    if model_type=='tm4':
        x0 = x0[0:7]
        dxx = dxx[0:7]
        
    if model_type=='tm5_fdr2':
        x0 = x0[0:10]
        dxx = dxx[0:10]
        
    if model_type=='tm5_smr':
        x0 = x0    
        dxx = dxx
    #print(x0)
    Q0, A0 = QA_TM_aba_synphys(x0, amps, sig, Ts,DT0,model_type)
    #print(x0)
    print('initial Q: ',str(Q0))

    #           tF      p0    tD        dp           A                A1                  A2              tDmin   dd   tD2
    #xlower = [  0.1,   0.01,   0.1,     0.0001,     amps[0]/1*1e-1,   amps[0]/1*1e-1,     amps[0]/1*1e-1,  1,     1e-3, 1]
    xlower = x0/dxx
    xlower = np.array(xlower)
    #xupper = [  1e5,   1,      1e5,     1,          amps[0]/0.01*1e1, amps[0]/0.01*1e1,   amps[0]/0.01*1e1, 100,  1,    1e4]
    #xupper = np.array(xupper)
    xupper = x0*dxx
    bounds = [] #np.zeros(len(xlower))
    for ibo in range(len(xlower)):
        bounds = bounds + [(xlower[ibo],xupper[ibo])]
    tt=[]
    #import time
    #import scipy.optimize as opt
    t1 = time.time()
    
    #print(x0)
    #print(A0)
    #breakpoint()
    
    do_this=0
    if do_this==1:
        res1 = opt.shgo(Q_TM_aba_synphys, bounds , args=((amps, sig, Ts,DT0,model_type)), constraints=None, n=10,
                            iters=1, callback=None, minimizer_kwargs=None, options=None, sampling_method='simplicial')
        x_shgo = res1.x
    else:
        x_shgo = x0
    
    t2 = time.time()
    tt = tt + [t2-t1]
    print('shgo : elapsed time '+str(t2-t1)+'s ')

    t1 = time.time()
    do_good=True
    if do_good:
        popsize = 65 # best
        maxiter=1000
        tol=0.01
    else:
        popsize = 5  # fast
        maxiter=10
        tol=0.1
        
    #breakpoint()    
    do_this=0
    if do_this==1:
        res2 = opt.differential_evolution(Q_TM_aba_synphys, bounds , args=((amps, sig, Ts,DT0,model_type)), strategy='best1bin',
                           maxiter=maxiter, popsize=popsize, tol=tol, mutation=(0.5, 1), recombination=0.7, seed=None,
                           callback=None, disp=False, polish=True, init='latinhypercube', atol=0,
                           updating='immediate', workers=1)
        x_de = res2.x
    else:
        x_de = x0  
        
    t2 = time.time()
    tt = tt + [t2-t1]
    print('differential_evolution: elapsed time '+str(t2-t1)+'s ')

    t1 = time.time()
    do_good=False
    if do_good:
        maxiter=1000 # good
        accept=-5.0 
        maxfun=10000000.0
    else:
        maxiter=10 # fast
        maxfun=10.0
        accept=-1.0
    fl=0  
    do_this=0
    if do_this==1:
        res3 = opt.dual_annealing(Q_TM_aba_synphys, bounds , args=((amps, sig, Ts,DT0,model_type)), maxiter=1000, 
                              local_search_options={},
                              initial_temp=5230.0,restart_temp_ratio=2e-05, visit=2.62, 
                              accept=accept, maxfun=maxfun, seed=None,
                              no_local_search=False, callback=None, x0=None)
        x_da = res3.x
    else:
        x_da = x0 
        
    t2 = time.time()
    Q, A = QA_TM_aba_synphys(x_da, amps, sig, Ts,DT0,model_type)
    A_da=A
    tt = tt + [t2-t1]
    print('dual_annealing: elapsed time '+str(t2-t1)+'s ')

    t1 = time.time()
    do_this=0
    if do_this==1:
        res4 = opt.basinhopping(Q_TM_aba_synphys, x0 , niter=1, T=1.0, stepsize=1.0,
                            minimizer_kwargs={'method':"L-BFGS-B", 'args':(amps, sig, Ts,DT0,model_type)}, 
                            take_step=None, accept_test=None, callback=None, interval=50, disp=False,
                           niter_success=None, seed=None)
        x_bh = res4.x
    else:
        x_bh = x0 
        
    t2 = time.time()
    tt = tt + [t2-t1]
    print('basinhopping: elapsed time '+str(t2-t1)+'s ')
    #t1 = time.time()
    #res4 = opt.basinhopping(Q_TM_aba_synphys, x0 , niter=100, T=1.0, stepsize=1.0,
    #                        minimizer_kwargs={'method':"L-BFGS-B", 'args':(amps, sig, Ts,DT0)}, 
    #                        take_step=None, accept_test=None, callback=None, interval=50, disp=False,
    #                        niter_success=None, seed=None)
    t1 = time.time()
    do_this=1
    if do_this:
        prob = pg.problem(jit_rosenbrock2(amps.reshape((-1,1)),sig.reshape((-1,1)),Ts,model_type,
                                          np.array(xlower).reshape((-1,)),np.array(xupper).reshape((-1,))))
        #pop = pg.population(prob=prob, size = 1000)
        uda = pg.de1220(gen = 100, allowed_variants = [2,3,7,10,13,14,15,16], variant_adptv = 1, ftol = 1e-6, xtol = 1e-6, memory = False)
        algo = pg.algorithm(uda)
        archi = pg.archipelago(n=32,algo=algo, prob=prob, pop_size=150)
        
        #pop2 = algo.evolve(pop)
        archi.evolve() 
        archi.wait()
        
        qq=np.array(archi.get_champions_f())
        xx=np.array(archi.get_champions_x())
        
        x_sade = np.exp(xx[qq.argmin(),:])

    else:
        x_sade  = x0 
    
    t2 = time.time()
    Q, A = QA_TM_aba_synphys(x_sade, amps, sig, Ts,DT0,model_type)
    A_sade=A
    print('self adaptive differential evolution: time ' + str(t2 - t1), '; Q ',str(Q)) 
    
    
    
    t1 = time.time()
    do_this=0
    if do_this==1:
        #res4 = opt.basinhopping(Q_TM_aba_synphys, x0 , niter=100, T=1.0, stepsize=1.0,
        #                        minimizer_kwargs={'method':"L-BFGS-B", 'args':(amps, sig, Ts,DT0)}, 
        #                        take_step=None, accept_test=None, callback=None, interval=50, disp=False,
        #                        niter_success=None, seed=None)
        #nst=50 # good
        nst = 10 #150 # better fit
        Qs =  np.zeros(nst)
        As =  np.zeros((nst,npar))
        #vx = np.arange(5)  # fit 1 amplitude
        vx = np.arange(len(x0)) # fit separate amplitude for each frequency
        nx = len(x0)
        xs =  np.zeros((nst,len(x0[vx]) ))
        for ist in range(nst):

            #x0i = np.exp(np.log(xlower) + np.random.rand(len(xlower))*(np.log(xupper)-np.log(xlower)))
            x0i = x0*np.exp( 2*(np.random.rand(len(xlower))-0.5)*np.log(dxx)  )
            x0i = x0i[vx]
            #x0i[4:7] = x0i[4]
            fl=1
            #breakpoint()
            resi=opt.minimize(Q_TM_aba_synphys, x0i, args=((amps, sig, Ts,DT0,model_type)), method=None, jac=None,
                      hess=None, hessp=None, bounds=bounds[0:nx],
                      constraints=(), tol=None, callback=None, 
                      options=None)
            x=resi.x
            Q,A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
            As[ist,:] = A
            xs[ist,:] = x
            Qs[ist] = Q

            
            Q_rs = np.min(Qs)
            ia = np.nonzero(Q_rs==Qs)[0]
            x_rs=xs[ia,:].ravel()
            A_rs = As[ia,:].ravel()
    else: 
        x_rs=x0
        Q_rs = 1e5
        A_rs = np.zeros((npar,))
            
    t2=time.time()
    tt = tt + [t2-t1]
    print('random_start: elapsed time '+str(t2-t1)+'s ')
    Q2 = []
    algs = ['shgo', 'differential_evolution','dual_annealing','basinhopping', 'self ajusting differential evolution',  'random_starts' ]
    colors = ['y',   'm',                           'c',          'y',          'r',                                 'g' ]
        
    if dont_plot==0:
        #%matplotlib
        #import matplotlib.pyplot as plt
        A3e=1
        
        plt.errorbar(np.arange(len(amps)), amps*A3e, yerr=sig*A3e, fmt =  'bo')
        #plt.hold(True)


        ii=0
        x = x_shgo #res1.x
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        Q2 = Q2 + [Q]
        #plt.plot(np.arange(len(amps)), A, 'o-' ,color=colors[ii])
        print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])



        ii=1
        x = x_de #res2.x
        x2=x
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        Q2 = Q2 + [Q]
        #plt.plot(np.arange(len(amps)), A, 'd-' ,color=colors[ii])
        print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])
        ii=2
        x = x_da #res3.x
        #x3=x
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        Q2 = Q2 + [Q]
        #plt.plot(np.arange(len(amps)), A, 'x--',color=colors[ii] )
        print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])
        ii=3
        x = x_bh #res4.x
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        Q2 = Q2 + [Q]
        
        #plt.plot(np.arange(len(amps)), A, '.--',color=colors[ii] )
        print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])

        ii=4

        x = x_sade #res4.x
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        Q2 = Q2 + [Q]
        plt.plot(np.arange(len(amps)), A, '.-',color='r')
        print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])

        ii=5


        #Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0)
        Q=Q_rs 
        Q2 = Q2 + [Q]
        #plt.plot(np.arange(len(amps)), A_rs, '.--',color=colors[ii] )
        print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])

        #algs_short = ['differential_evolution',  'random_starts' ]
        algs_short = ['random_starts' ]
        #plt.legend(algs)
        plt.legend(algs_short)

        #fig = plt.gcf()
        #set_position(plt.gca, [10, 10, 100, 500], which='both')
        plt.xlim((min(vv)-1,max(vv)+2)) 
        #thismanager = plt.get_current_fig_manager()
        #thismanager.window.SetPosition((500, 0))
        #plt.hold(False)
        plt.pause(0.5) 
    
    #fig = plt.gcf()
    AA = [A_sade,A_da,A_rs]
    xx = [x_sade,x_da,x_rs]
    return Q2, tt, xx, algs, colors, AA

In [ ]:
import scipy as sci
def epsp_sim(A, T_A,dt,tplus,tminus,Tmax):
    
    l, v = sci.linalg.eig(np.array([[-1/tplus,0],[1/tplus,-1/tminus]])) 
    l = np.real(l).reshape([-1,1])
    vm = sci.linalg.inv(v)
    #x = v*np.exp(l*t)*np.array([1,0]).reshape([-1,1])                  
    X = np.array([0,0]).reshape([-1,1])

    t=np.array([0])
    for i in np.arange(1,T_A.size):
        if T_A[i]-T_A[i-1]>Tmax+10*dt:
            t2  =  np.arange(int(Tmax/dt))[np.newaxis]*dt  
            t3  =  - 10*dt + np.arange(10)[np.newaxis]*dt   
            
            t5 = np.append(t2,t3+Tmax+10*dt ,axis=1)
            t2 = np.append(t2,t3+T_A[i] - T_A[i-1],axis=1)

        else: 
            t2  =  np.arange(int((T_A[i]-T_A[i-1])/dt))[np.newaxis]*dt 
            t5 = t2

        #print(i)    
        x0 = X[:,[-1]]   
        x0[0,:] = x0[0,:]+A[i-1] 
        vvmx=(v@np.diag((vm@x0).ravel()))
        x = vvmx@np.exp(l@t2)
        X = np.append(X,x,axis=1)
        #t = np.append(t,T_A[i-1] +t5)   
        t = np.append(t,t[-1] +t5)   
    
    return X, t

In [ ]:
def plot_fig9b(i0,ge_data, tplus=5, tminus=50, Nbootstraps=200, Dn=200, model_type='tm'):
    dt=0.1
    #tplus=5
    #tminus=50
    Tmax = 1000
    DT = 200/6

    #i0=23 #42 #[18,23,42]
    ii = i0*Nbootstraps + np.arange(0,Nbootstraps,Dn)
    #T_A = np.array((100+np.arange(8)*200/6).tolist()+[100+200/6*8+500]) #  from BBP cell paper figure
    T_A = np.array((100+np.arange(9)*200/6).tolist()+[100+200/6*9+400]) # for classification task
    
    
    if model_type=='tm':
        stp_aba_names = ['tF', 'p0','tD','dp','A','A1','A2'] # tm5
    if model_type=='tm_smr':
        stp_aba_names = ['tF', 'p0','tD','dp','A','A1','A2','tDmin','dd','t_FDR','t_SMR','dp0'] # tm5+smr
    
    
    
    if model_type=='tm':
        T_A0 = T_A-T_A[0]
        As, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data.iloc[ii,:],T_A0)
    if model_type=='tm_smr':
        xs  =ge_data.iloc[ii,:].loc[:,stp_aba_names].values
        xs  =np.delete(xs, [5,6],axis=1)

        T_A0 = T_A-T_A[0]
        As = np.zeros((xs.shape[0],T_A.shape[0]))
        for i2 in range(xs.shape[0]):
        
            as2, sts2 = STP_sim2(xs[i2,:],T_A0,model_type='tm5_smr') 
            As[i2,0:as2.shape[0]] = as2


    #A = np.mean(As,axis=0) #As[0,:Na1]#np.mean(As[:100,:Na1],axis=0)
    #A = np.append([0],A)
    ##T_A = Tall_A
    
    T_A =np.append([0],T_A)
    T_A = np.append(T_A,[T_A[-1]+400])
    
    
    Epsp =[]
    for iii in range(As.shape[0]):
        #A = As[iii,:]
        A = As[iii,:]/As[iii,0]
        A = np.append([0],A)
        A = np.append(A,[0])

        #print(T_A, A)
        epsp, t = epsp_sim(A, T_A,dt,tplus,tminus,Tmax)
        Epsp = Epsp + [epsp[1,:].reshape([-1,1])]
     
    
    Epsp = np.concatenate(Epsp, axis=1)
    
    
    
    #plt.plot(t, epsp[1,:].T)
    
    return np.mean(Epsp,axis=1), t, As, T_A0, Epsp

In [ ]:
Nbootstraps

In [ ]:
## Generate responses for each synapse type : 20Hz + 500ms recovery

In [ ]:
# simulate epsps for fig9a
stp_columns2=np.array(stp_columns)
annot_columns2 =np.array(annot_columns)
sx=5
sy=4
i0=0
skip_this=1
eps = np.finfo(float).eps
#Stp_type_p = []
EPSP=[]
Ams=[]
Ams_s=[]
Ams_m = []


### ACHTUNG!!! ACHTUNG!!! ACHTUNG!!! ACHTUNG!!!
#Nbootstraps = 100
# ACHTUNG!!! ACHTUNG!!! ACHTUNG!!! ACHTUNG!!!


for i0 in range(0,df_Stp_type_p.shape[0]):#[42]: # [18,23,42] #range(0,53):
    if i0<53:
        str_syntype = ge_data.loc[i0*Nbootstraps,
                              annot_columns2[[0,2,1,3]]].astype(str).str.cat(others=['_','->','_','']).str.cat()
    else:
        str_syntype = ge_data_h.loc[(i0-53)*Nbootstraps,
                              annot_columns2[[0,2,1,3]]].astype(str).str.cat(others=['_','->','_','']).str.cat()

    #print(ge_data.loc[i0*200,annot_columns[:-3]])

    #ii = i0*200 + np.arange(200)
    
    #epsp,t_epsp = plot_fig9b(i0,ge_data, tplus=2, tminus=20, Nbootstraps=200, Dn=200) # averaged all synapses
    if i0<10:
        model_type='tm'
    elif i0<53:
        model_type='tm_smr'
    else:
        model_type='tm'
        
    if i0<53:    
        epsp,t_epsp,ams, T_A,epsp0 = plot_fig9b(i0,ge_data, tplus=3, tminus=25, Nbootstraps=Nbootstraps, Dn=5, model_type=model_type)  # only 1st response
    else:
        epsp,t_epsp,ams, T_A,epsp0 = plot_fig9b(i0-53,ge_data_h, tplus=3, tminus=25, Nbootstraps=Nbootstraps, Dn=5, model_type=model_type)  # only 1st response
    
    
    EPSP = EPSP + [epsp0[:,0]]
    Ams = Ams + [ams/ams[:,[0]]]
    Ams_m = Ams_m + [np.mean(ams/ams[:,[0]],axis=0)]
    
    Ams_s = Ams_s + [np.std(ams/ams[:,[0]],axis=0)+10000*eps]
    
    print(str(i0)+ '  '+str_syntype + '  ' +'epsp')
    
    skip_this=1
    if i0<53:
        skip_this=0
    if skip_this==0:
        f, ax =plt.subplots(figsize=(sx, sy))

        plt.plot(t_epsp,epsp0,color=[0.7,0.7,0.7])
        plt.plot(t_epsp,epsp)
        
        plt.plot(t_epsp,epsp0[:,0],'g')

        #dat.hist(bins=25)
        stitle = str(i0)+ '  '+str_syntype + '  ' +'epsp '+np.array2string(df_Stp_type_p.loc[i0,
                                ['polarity','tm_all','tm_DFUdU','dtype_1','dtype_2','dtype_3',
                                 'type_1','type_2','type_3']].values)
        
        plt.title(stitle)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.xlabel('time, ms',fontsize=20)
        plt.ylabel('EPSP, mV',fontsize=20)
        
        f, ax =plt.subplots(figsize=(sx, sy))
        #plt.plot(T_A[1:-1],ams[0,:].transpose(),'og')
        plt.plot(T_A,ams[0,:].transpose(),'og')


In [ ]:
EPSP2 = np.array(EPSP)

Ams_m = np.array(Ams_m)
Ams_s = np.array(Ams_s)
print(Ams_m.shape)
EPSP2.shape

In [ ]:
def  STP_sim2(x, T, init_state=None, model_type='tm5' ):

    N    = len(T)
    tF      = x[0] #.astype(float)
    p00     = x[1]
    tD      = x[2] #.astype(float)
    dp      = x[3]
    A       = 1 #x[4] # simplify A
    
    #breakpoint()
    mod_fdr2=False
    if model_type=='tm5_fdr2':  # should be :check freq. dependent recovery
        tDmin     = x[5]
        dd        = x[6]
        t_FDR     = x[7]
        mod_fdr2=True
        tDmax  = tD
        itDmin = 1/tDmin
        itDmax = 1/tDmax
        #breakpoint()
        
    mod_smr=False
    if model_type=='tm5_smr':  # should be :check freq. dependent recovery
        t_SMR   = x[8]
        dp0     = x[9]
        mod_smr=True
        #p00  = p00

    As = np.zeros((N,))
    state = np.zeros((N,4))

   
    if init_state is None :
        n = 1
        p0=p00
        p = p0
        d = 0
    else:
        n = init_state[0]
        p = init_state[1]
        d = init_state[2]
        p0= init_state[3]

    
    for i in range(0,N):
        if i==0:
            Dt = T[i]
        else:
            Dt = T[i]-T[i-1]
        
        if mod_fdr2:
            d0=d
            d = d*np.exp(-Dt/t_FDR) 
            n = 1 - (1 - n )*np.exp(-Dt*itDmax -(itDmin -itDmax)*t_FDR*(d0-d))
        else:
            #print(x,tD)
            n = 1 - (1 - n )*np.exp(-Dt/tD )
            
        if mod_smr:
            p01=p0
            p0=p00 + (p0 -p00)*np.exp(-Dt/t_SMR)
            p=p0 +(p -p01)*np.exp(-Dt/tF)
        else:
            p=p0 +(p -p0)*np.exp(-Dt/tF)
            

        As[i]=A*n*p
       
        n = n*(1-p)
        p = p + dp*(1-p)
        if mod_fdr2:
            d  = d + dd*(1-d) 
        if mod_smr:
            p0  = p0 - dp0*p0    
 
        state[i] = [n,p,d,p0]

    #return As, ns2, ps2, dpp0, p0, tF, tD, A
    return As, state

In [ ]:
def Q_TM_simple(x, amps,sig,Ts,model_type='tm4'):
    #Q, A, A3 =  QA_TM_aba_synphys(x, amps,sig,Ts)
    Q, _ =  QA_TM_simple(x, amps,sig,Ts,model_type=model_type)
    return Q  

In [ ]:
def QA_TM_simple(x, amps,sig,T_A,model_type='tm4'):
    eps = np.finfo(float).eps
    
    if model_type=='tm4':
        #x = [tF, p0,tD,  dp]
        x[3] = x[1] 
        amps2, sts2 = STP_sim2(x,T_A,model_type='tm5')
    else:  
        amps2, sts2 = STP_sim2(x,T_A,model_type=model_type)
    
    amps2 = amps2/amps2[0]
    Q = np.sum((amps2-amps)**2/(sig**2 + eps))
    return Q, amps2    

In [ ]:
def fit_STP_model(amps,sig,Ts,par,model_type = 'tm4'):

    npar = amps.shape[1]


         #  tF   p0     tD  dp         A            A1           A2      tDmin  dd   t_FDR t_SMR dp0 
    x0 = [ 100,  0.1,  100, 0.1, amps[0]/0.1, amps[0]/0.1,  amps[0]/0.1, 10,   0.05, 100,  100,  0.02 ]
    
    x0 = np.array(x0)
    
    #dxx = np.array([1e3, 20, 1e3, 1e2,  20, 20, 20,  1e1, 20, 1e2 ])
    dxx = np.array([1e3,  10, 1e3, 10,  20, 20, 20,  1e3, 20, 1e3, 1e3, 50 ])
    
    #breakpoint()
    if model_type=='tm5':
        x0 = x0[0:4]
        dxx = dxx[0:4]

        
    if model_type=='tm4':
        x0 = x0[0:4]
        dxx = dxx[0:4]
        

    #print(x0)
    Q0, A0 = QA_TM_simple(x0, amps, sig, Ts,model_type=model_type)
    #print(x0)
    print('initial Q: ',str(Q0))

    #           tF      p0    tD        dp           A                A1                  A2              tDmin   dd   tD2
    #xlower = [  0.1,   0.01,   0.1,     0.0001,     amps[0]/1*1e-1,   amps[0]/1*1e-1,     amps[0]/1*1e-1,  1,     1e-3, 1]
    xlower = x0/dxx
    xlower = np.array(xlower)
    #xupper = [  1e5,   1,      1e5,     1,          amps[0]/0.01*1e1, amps[0]/0.01*1e1,   amps[0]/0.01*1e1, 100,  1,    1e4]
    #xupper = np.array(xupper)
    xupper = x0*dxx
    bounds = [] #np.zeros(len(xlower))
    for ibo in range(len(xlower)):
        bounds = bounds + [(xlower[ibo],xupper[ibo])]
    tt=[]
    #import time
    #import scipy.optimize as opt
    t1 = time.time()

    do_this=1
    if do_this==1:
        #res4 = opt.basinhopping(Q_TM_aba_synphys, x0 , niter=100, T=1.0, stepsize=1.0,
        #                        minimizer_kwargs={'method':"L-BFGS-B", 'args':(amps, sig, Ts,DT0)}, 
        #                        take_step=None, accept_test=None, callback=None, interval=50, disp=False,
        #                        niter_success=None, seed=None)
        #nst=50 # good
        nst = 100 #150 # better fit
        Qs =  np.zeros(nst)
        As =  np.zeros((nst,npar))
        #vx = np.arange(5)  # fit 1 amplitude
        vx = np.arange(len(x0)) # fit separate amplitude for each frequency
        nx = len(x0)
        xs =  np.zeros((nst,len(x0[vx]) ))
        for ist in range(nst):

            #x0i = np.exp(np.log(xlower) + np.random.rand(len(xlower))*(np.log(xupper)-np.log(xlower)))
            x0i = x0*np.exp( 2*(np.random.rand(len(xlower))-0.5)*np.log(dxx)  )
            x0i = x0i[vx]
            #x0i[4:7] = x0i[4]
            fl=1
            #breakpoint()
            resi=opt.minimize(Q_TM_simple, x0i, args=((amps, sig, Ts,model_type)), method=None, jac=None,
                      hess=None, hessp=None, bounds=bounds[0:nx],
                      constraints=(), tol=None, callback=None, 
                      options=None)
            x=resi.x
            if model_type=='tm4':
                #x = [tF, p0,tD,  dp]
                x[3] = x[1] 
            Q,A = QA_TM_simple(x, amps, sig, Ts,model_type=model_type)
            As[ist,:] = A
            xs[ist,:] = x
            Qs[ist] = Q

            
            Q_rs = np.min(Qs)
            ia = np.nonzero(Q_rs==Qs)[0]
            x_rs=xs[ia,:].ravel()
            A_rs = As[ia,:].ravel()
    else: 
        x_rs=x0
        Q_rs = 1e5
        A_rs = np.zeros((npar,))
            
    t2=time.time()
    tt = tt + [t2-t1]
    print('random_start: elapsed time '+str(t2-t1)+'s ')
    
    return x_rs, Q_rs, A_rs

In [ ]:
## refit from 4-parameters TM model (and SMR model) to 3 parameters TM, save new STP type to 'stp types probability cortex and hippocampus correct 33Hz.xlsx'

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import scipy.optimize as opt
#from pygmo import 
#from pylab import *


#  ACHTUNG!!! ACHTUNG!!! ACHTUNG!!! ACHTUNG!!!
Ts = T_A
#  ACHTUNG!!! ACHTUNG!!! ACHTUNG!!! ACHTUNG!!!

t00 = time.time()
Q3  = []
TT  = []
X   = []
Amps2 = []
Amps1 = []

nalg = 1

npar=EPSP2.shape[1]

par = {'fited_pulses':True, 'amplitudes_selected_for_TM_training':[], 'aba_case':True, 'figure':0,
       'model_type' : 'tm4' , 'ibs':0, 'x0': np.array([]) }

par['model_type'] = 'tm4'  # extended TM model

for i in range(EPSP2.shape[0]):# range(EPSP2.shape[0]): #range(0,len(df9.index)): #range(22,23): #range(len(df9.index)):
    idx = i #df9.index[i]
    print('\n\n'+str(i)+'  ')
    #print('\n\n'+str(i)+'  '+df9.loc[idx,'comments']+'\n')
    #Ts  = df9.loc[idx,'comments1']
    ##ampsa = df.loc[idx,'stp_mean_fit_bootstrap']
    ##is_fited_pulses = (abs(ampsa[0])>1e-14)
    
    is_fited_pulses = True
    if Ts.size!=0: #(len(Ts)!=0): #&((par['fited_pulses']==False)|is_fited_pulses):
        fig, ax = plt.subplots(figsize=(10, 4))
        par['figure']=fig
    #try:
        #par['fited_pulses']=True
        #Q2, tt, x, algs, colors = fit_TM_cell(idx,df9,par)
        amps = Ams_m[[i],:]
        sig = Ams_s[[i],:] #*0 + Ams_m[[i],:]*0.1

        i0=i
        #stp_aba_names = ['tF', 'p0','tD','dp/p0']
        #x0 = ge_data.iloc[i0*Nbootstraps,:].loc[stp_aba_names].values
        #x0[3] = x0[3]*x0[1]
        #Q0,amps0 = QA_TM_simple(x0, amps, sig, Ts,model_type='tm5') # ACHTUNG!!! - amps0 do not coincide with amps2???
        #print('Q0 ',Q0,'amps0 ', amps0,'x0 ', x0)
        
        vv = np.arange(amps.shape[1]-1)
        x,Q2,amps2 = fit_STP_model(amps[:,vv],sig[:,vv],Ts[vv],par,model_type = 'tm4')
        
        plt.plot(Ts,amps.ravel(),'-xb',markersize=12)
        plt.plot(Ts[vv],amps2.ravel(),'-og')
        #plt.plot(Ts,amps0.ravel(),'-dr')
        
        Amps2 = Amps2 + [amps2]
        Amps1 = Amps1 + [amps]
        
        if i0<53:
            str_syntype = ge_data.loc[i0*Nbootstraps,
                              annot_columns2[[0,2,1,3]]].astype(str).str.cat(others=['_','->','_','']).str.cat()
        else:
            str_syntype = ge_data_h.loc[(i0-53)*Nbootstraps,
                              annot_columns2[[0,2,1,3]]].astype(str).str.cat(others=['_','->','_','']).str.cat()
       
            
        stitle = ' '+np.array2string(df_Stp_type_p.loc[i0,
                                ['polarity','tm_all','tm_DFUdU','dtype_1','dtype_2','dtype_3',
                                 'type_1','type_2','type_3']].values)
        
        df_Stp_type_p.loc[i0, 'tm_DFUdU'] = np.array2string(x[[2,0,1,3]])
        tF_2_tD=x[0]/x[2]
        df_Stp_type_p.loc[i0, ['dtype_2','dtype_3','dtype_1']]=np.array([(tF_2_tD<0.4),(tF_2_tD>0.4)&(tF_2_tD<2),(tF_2_tD>2)])
    
        stitle = str(i0)+ '  '+str_syntype + '  '+ np.array2string(x[0:4])+np.array2string(df_Stp_type_p.loc[i0,
                                ['dtype_1','dtype_2','dtype_3']].values)
        
        plt.title(stitle)
        
        #plt.xticks(fontsize=16)
        #plt.yticks(fontsize=16)
        #plt.xlabel('time, ms',fontsize=20)
        #plt.ylabel('EPSP, mV',fontsize=20)
        plt.ylim([0, max([np.max(amps), np.max(amps2)])*1.2])
        
        nalg = 1 #len(tt)
        tt=[]
    #except:
    else:
        print(Ts)
        Q2 = (10*np.zeros(nalg)).tolist()
        tt = (0*np.zeros(nalg)).tolist()
        x = (0*np.zeros(nalg)).tolist()
        
    Q3 = Q3 + [Q2]
    TT = TT + [tt]
    X = X + [x]
    #df_Stp_type_p.loc[i0, 'tm_DFUdU'] = np.array2string(x[0,2,1,3])
    #tF_2_tD=x[2]/x[0]
    #df_Stp_type_p.loc[i0, ['dtype_2','dtype_3','dtype_1']]=np.array([(tF_2_tD<0.4),(tF_2_tD>0.4)&(tF_2_tD<2),(tF_2_tD>2)])
    print(df_Stp_type_p.loc[i0, ['dtype_1','dtype_2','dtype_3']])
    print('x tm4 ',np.array2string(x))
    print(stitle)

    
Q3 = pd.DataFrame(Q3)  
TT = pd.DataFrame(TT) 
X3 = pd.DataFrame(X)
t10 = time.time()
print('time : '+ str(t10-t00))




# # Save results

df_Stp_type_p.to_excel('stp types probability cortex and hippocampus correct 33Hz.xlsx')

# Q3.columns = ['random_starts'] #algs
# algs2=algs
# for ialg, alg in enumerate(algs2): 
#     algs2[ialg] = alg + ' , time'
# TT.columns = algs2
# QT = pd.concat([Q3,TT],axis=1)

# X3 = pd.DataFrame(X)
# #X3.columns = ['differential_evolution best x', 'dual_annealing best x', 'random_starts best x', 'shgo best x', 'basinhopping best x']
# X3.columns = ['differential_evolution best x', 'dual_annealing best x', 'random_starts best x']


# QTX_extended = pd.concat([df9,QT,X3],axis=1)

# #QTX_extended.to_hdf('refit_aba_2019_A1_8_fited_pulses_TM_8_syntypes_results','data')

In [ ]:
amps2

In [ ]:
np.array2string(df_Stp_type_p.loc[i0,['polarity','type_1', 'type_2', 'type_3']].values)

In [ ]:
Amps1 = np.array(Amps1)
Amps2 = np.array(Amps2)
Amps2.shape

In [ ]:
Amps1 = np.squeeze(Amps1)
Amps1.shape

In [ ]:
## Reestimate STP type for each synapse type using STP

In [ ]:
sx = 12
sy=4

do_draw_cortex = False # draw cortical or hippocampal synapses
if do_draw_cortex:
    remove = np.array( (53+np.arange(40)).tolist()+[4] )
else:
    remove = np.array( (0+np.arange(53)).tolist()+[] )
    
take = np.zeros(df_Stp_type_p.shape[0])==0
take[remove]=False
df_Stp_type_p2 = df_Stp_type_p.iloc[take,:]

dt=0.1

for pol in ['ex', 'inh']:
    for stype in ['dtype_1', 'dtype_2', 'dtype_3']:
        #df_pol_stype=df_Stp_type_p.loc[(df_Stp_type_p.loc[:,'polarity']==pol)&(df_Stp_type_p.loc[:,stype]>=0.7),:]
        df_pol_stype=df_Stp_type_p2.loc[(df_Stp_type_p2.loc[:,'polarity']==pol)&(df_Stp_type_p2.loc[:,stype]==True),:]
        if df_pol_stype.shape[0]>0:
#             if do_draw_cortex:
#                 idx = df_pol_stype.index[df_pol_stype.index[:]<53]
#             else:
#                 idx = df_pol_stype.index[df_pol_stype.index[:]>=53]
            
            idx = df_pol_stype.index
        
            amps2 = Amps2[idx,:].T # amplitudes 3_TM
            amps1 = Amps1[idx,:].T # amplitudes 4-TM
            
            epsp = EPSP2[idx,:].T
            am=np.max(epsp[0:int((100+15)/dt),:], axis=0)
            epsp =epsp/np.tile(am,[epsp.shape[0],1])
            
            if (pol=='inh')&(stype=='dtype_2'):
                take = np.zeros(epsp.shape[1])==0
                min_amp=np.min(-epsp[np.arange(int(50/dt))+int((100+200/6*7)/dt),:],axis=0)
                print(min_amp)
                take[min_amp<-0.8] = False
                epsp = epsp[:,take]
                
            if (pol=='ex')&(stype=='dtype_2'):
                
                take = np.zeros(epsp.shape[1])==0
                min_amp=np.max(epsp[np.arange(int(50/dt))+int((100+200/6*7)/dt),:],axis=0)
                print(min_amp)
                take[min_amp>1.1]= False
                epsp = epsp[:,take]    
            
            f, ax =plt.subplots(figsize=(sx, sy))

            if pol=='ex':
                print('ex', stype)
                plt.plot(t_epsp,epsp,color='b')
                plt.plot(t_epsp,np.median(epsp, axis=1),color='r')
            else: 
                print('inh', stype)
                plt.plot(t_epsp,-epsp,color='b')
                plt.plot(t_epsp,-np.median(epsp, axis=1),color='r')
            
            #dat.hist(bins=25)
            all_syn_names=df_pol_stype.loc[idx,'name'].str.cat(sep='; ')
            
            if pol=='ex':
                npol='E'
            else:
                npol='I'
            nstype = str.split(stype, '_')[1]        
            plt.title(npol+nstype, fontsize=25)
            plt.xticks(fontsize=16)
            plt.yticks(fontsize=16)
            plt.xlabel('time, ms',fontsize=20)
            plt.ylabel('EPSP, mV',fontsize=20)
            
            print(npol+nstype + '  synaptic pair types: ' +all_syn_names+'\n')
            
            
            f, ax =plt.subplots(figsize=(sx, sy))
            plt.plot(np.arange(amps1.shape[0]),amps1,color='b')
            plt.plot(np.arange(amps1.shape[0]),np.median(amps1, axis=1),color='r')
            
            f, ax =plt.subplots(figsize=(sx, sy))
            plt.plot(np.arange(amps2.shape[0]),amps2,color='b')
            plt.plot(np.arange(amps2.shape[0]),np.median(amps2, axis=1),color='r')
            
        

### Examples of distributions of An/A1 and U,D,F parameters.  Cortex

In [ ]:

%matplotlib inline


#
#do_medians, do_log should be off!!!
# Nsamples = 200 !!!


## some distributions
#stp_columns

# f, ax =plt.subplots(figsize=(16, 7))
# plt.plot(ge_data.iloc[3600:3800,:].loc[:,stp_columns].transpose(),'o-')
# plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
#          rotation_mode="anchor")

#f, ax =plt.subplots(figsize=(16, 7))
stp_columns2=np.array(stp_columns)
annot_columns2 =np.array(annot_columns)
sx=5
sy=4
i0=0

for i0 in [42]: # [18,23,42] #range(0,53):
    str_syntype = ge_data.loc[i0*200,annot_columns2[[0,2,1,3]]].astype(str).str.cat(others=['_','->','_','']).str.cat()

    #print(ge_data.loc[i0*200,annot_columns[:-3]])

    ii = i0*200 + np.arange(200)
    f, ax =plt.subplots(figsize=(sx, sy))
    ge_data.iloc[ii,:].loc[:,stp_columns2[0]].hist(bins=20)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +stp_columns2[0])
    plt.xlim((0.25,1.25)) 
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel('A2/A1',fontsize=20)
    plt.ylabel('Frequency',fontsize=20)

    f, ax =plt.subplots(figsize=(sx, sy))
    ge_data.iloc[ii,:].loc[:,stp_columns2[3]].hist(bins=20)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +stp_columns2[3])
    plt.xlim((0.25,1.25))
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel('A5/A1',fontsize=20)
    plt.ylabel('Frequency',fontsize=20)

    f, ax =plt.subplots(figsize=(sx, sy))
    ge_data.iloc[ii,:].loc[:,stp_columns2[4]].hist(bins=20)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +stp_columns2[4])
    plt.xlim((0.25,1.25))
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel('Arecovery/A1, ',fontsize=20)
    plt.ylabel('Frequency',fontsize=20)


    f, ax =plt.subplots(figsize=(sx, sy))
    dat=ge_data.iloc[ii,:].loc[:,'tD'].map(float)
    dat=dat.loc[dat.loc[:]<800]
    dat.hist(bins=25)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +'D')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel('D, ms',fontsize=20)
    plt.ylabel('Frequency',fontsize=20)

    f, ax =plt.subplots(figsize=(sx, sy))
    #ge_data.iloc[ii,:].loc[:,'tF'].map(float).hist(bins=25)
    dat=ge_data.iloc[ii,:].loc[:,'tF'].map(float)
    dat=dat.loc[dat.loc[:]<5500]
    dat.hist(bins=25)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +'F')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel('F, ms',fontsize=20)
    plt.ylabel('Frequency',fontsize=20)

    f, ax =plt.subplots(figsize=(sx, sy))
    #ge_data.iloc[ii,:].loc[:,'p0'].map(float).hist(bins=25)
    dat=ge_data.iloc[ii,:].loc[:,'p0'].map(float)
    dat=dat.loc[dat.loc[:]<1]
    dat.hist(bins=25)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +'U')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel('U',fontsize=20)
    plt.ylabel('Frequency',fontsize=20)

In [ ]:
%matplotlib inline

#
#do_medians, do_log should be off!!!
#


## some distributions
#stp_columns

# f, ax =plt.subplots(figsize=(16, 7))
# plt.plot(ge_data.iloc[3600:3800,:].loc[:,stp_columns].transpose(),'o-')
# plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
#          rotation_mode="anchor")

#f, ax =plt.subplots(figsize=(16, 7))
stp_columns2=np.array(stp_columns)
annot_columns2 =np.array(annot_columns)
sx=5
sy=4
i0=0

for i0 in [42]: # [18,23,42] #range(0,53):
    str_syntype = ge_data.loc[i0*200,annot_columns2[[0,2,1,3]]].astype(str).str.cat(others=['_','->','_','']).str.cat()

    #print(ge_data.loc[i0*200,annot_columns[:-3]])

    ii = i0*200 + np.arange(200)
    f, ax =plt.subplots(figsize=(sx, sy))
    ge_data.iloc[ii,:].loc[:,stp_columns2[0]].hist(bins=20)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +stp_columns2[0])
    plt.xlim((0.25,1.25))
    plt.xlabel(stp_columns2[0],fontsize=16)
    plt.ylabel('Frequency',fontsize=16)
    plt.setp(ax.get_xticklabels(),fontsize=14)
    plt.setp(ax.get_yticklabels(),fontsize=14)
    ax.grid(b=False)

    f, ax =plt.subplots(figsize=(sx, sy))
    ge_data.iloc[ii,:].loc[:,stp_columns2[3]].hist(bins=20)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +stp_columns2[3])
    plt.xlim((0.25,1.25))
    plt.xlabel(stp_columns2[3],fontsize=16)
    plt.ylabel('Frequency',fontsize=16)
    plt.setp(ax.get_xticklabels(),fontsize=14)
    plt.setp(ax.get_yticklabels(),fontsize=14)
    ax.grid(b=False)

    f, ax =plt.subplots(figsize=(sx, sy))
    ge_data.iloc[ii,:].loc[:,stp_columns2[4]].hist(bins=20)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +stp_columns2[4])
    plt.xlim((0.25,1.25))
    plt.xlabel(stp_columns2[4],fontsize=16)
    plt.ylabel('Frequency',fontsize=16)
    plt.setp(ax.get_xticklabels(),fontsize=14)
    plt.setp(ax.get_yticklabels(),fontsize=14)
    ax.grid(b=False)
    


    f, ax =plt.subplots(figsize=(sx, sy))
    dat=ge_data.iloc[ii,:].loc[:,'tD'].map(float)
    dat=dat.loc[dat.loc[:]<800]
    dat.hist(bins=25)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +'tD')
    plt.xlabel('D',fontsize=16)
    plt.ylabel('Frequency',fontsize=16)
    plt.setp(ax.get_xticklabels(),fontsize=14)
    plt.setp(ax.get_yticklabels(),fontsize=14)
    ax.grid(b=False)

    f, ax =plt.subplots(figsize=(sx, sy))
    #ge_data.iloc[ii,:].loc[:,'tF'].map(float).hist(bins=25)
    dat=ge_data.iloc[ii,:].loc[:,'tF'].map(float)
    dat=dat.loc[dat.loc[:]<5500]
    dat.hist(bins=25)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +'tF')
    plt.xlabel('F',fontsize=16)
    plt.ylabel('Frequency',fontsize=16)
    plt.setp(ax.get_xticklabels(),fontsize=14)
    plt.setp(ax.get_yticklabels(),fontsize=14)
    ax.grid(b=False)

    f, ax =plt.subplots(figsize=(sx, sy))
    #ge_data.iloc[ii,:].loc[:,'p0'].map(float).hist(bins=25)
    dat=ge_data.iloc[ii,:].loc[:,'p0'].map(float)
    dat=dat.loc[dat.loc[:]<1]
    dat.hist(bins=25)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +'p0')
    plt.xlabel('U',fontsize=16)
    plt.ylabel('Frequency',fontsize=16)
    plt.setp(ax.get_xticklabels(),fontsize=14)
    plt.setp(ax.get_yticklabels(),fontsize=14)
    ax.grid(b=False)
    
    f, ax =plt.subplots(figsize=(sx, sy))
    ge_data.iloc[ii,:].loc[:,'p0'].map(float).hist(bins=25)
    dat=ge_data.iloc[ii,:].loc[:,'p0'].map(float)
    dat=dat.loc[dat.loc[:]<1]
    dat.hist(bins=25)
    #plt.title(str(i0)+ '  '+str_syntype + '  ' +'p0'
    plt.xlabel('U',fontsize=16)
    plt.ylabel('Frequency',fontsize=16)
    plt.setp(ax.get_xticklabels(),fontsize=14)
    plt.setp(ax.get_yticklabels(),fontsize=14)
    plt.grid(b=False)

In [ ]:
# library & dataset
import seaborn as sns
#df = sns.load_dataset('iris')
 
# Basic 2D density plot
sns.set_style("white")
#sns.kdeplot(df.sepal_width, df.sepal_length)
#sns.plt.show()
 
# Custom it with the same argument as 1D density plot
dat=ge_data.iloc[ii,:].loc[:,'tD'].map(float)
dat2=ge_data.iloc[ii,:].loc[:,'tF'].map(float)
dat3=ge_data.iloc[ii,:].loc[:,'p0'].map(float)
sns.kdeplot(np.log10(dat), np.log10(dat2), cmap="Reds", shade=True, bw=.15) #, log_scale=True) #, log_scale=True

plt.xlabel('Log(D)',fontsize=16)
plt.ylabel('Log(F)',fontsize=16)
plt.setp(plt.gca().get_xticklabels(),fontsize=14)
plt.setp(plt.gca().get_yticklabels(),fontsize=14)
 
# Some features are characteristic of 2D: color palette and wether or not color the lowest range
#sns.kdeplot(df.sepal_width, df.sepal_length, cmap="Blues", shade=True, shade_lowest=True, )
##sns.plt.show()


In [ ]:
# Basic 2D density plot
sns.set_style("white")
#sns.kdeplot(df.sepal_width, df.sepal_length)
#sns.plt.show()
 
# Custom it with the same argument as 1D density plot
dat=ge_data.iloc[ii,:].loc[:,stp_columns2[0]].map(float)
dat2=ge_data.iloc[ii,:].loc[:,stp_columns2[3]].map(float)
dat3=ge_data.iloc[ii,:].loc[:,stp_columns2[4]].map(float)
sns.kdeplot((dat), (dat2), cmap="Reds", shade=True, bw=.05)
plt.xlabel('A2:A1 20Hz',fontsize=16)
plt.ylabel('A5:A1 20Hz',fontsize=16)
plt.setp(plt.gca().get_xticklabels(),fontsize=14)
plt.setp(plt.gca().get_yticklabels(),fontsize=14)
#ge_data.iloc[ii,:].loc[:,stp_columns2[0]].hist(bins=20)

In [ ]:
print(__doc__)


# Code source: Gaël Varoquaux
#              Andreas Müller
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

h = .02  # step size in the mesh

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

classifiers = [
    GaussianProcessClassifier(1.0 * RBF(1.0)),
]

X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [make_moons(noise=0.3, random_state=0),
            make_circles(noise=0.2, factor=0.5, random_state=1),
            linearly_separable
            ]

figure = plt.figure(figsize=(9, 9))
i = 1
# iterate over datasets
for ds_cnt, ds in enumerate(datasets):
    # preprocess dataset, split into training and test part
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.4, random_state=42)

    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    if ds_cnt == 0:
        ax.set_title("Input data")
    # Plot the training points
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
               edgecolors='k')
    # Plot the testing points
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6,
               edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    i += 1

    # iterate over classifiers
    for name, clf in zip(names, classifiers):
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

        # Plot the training points
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
                   edgecolors='k')
        # Plot the testing points
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
                   edgecolors='k', alpha=0.6)

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        if ds_cnt == 0:
            ax.set_title(name)
        ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
                size=15, horizontalalignment='right')
        i += 1

plt.tight_layout()
plt.show()

In [ ]:
df.head()

## Combine Cortex and Hipp. datasets, assign synapse type annotations, save to .hdf file 

In [ ]:
ge_data_h.shape

In [ ]:
ge_data.shape

In [ ]:
X_c.shape

In [ ]:
X_h.shape

In [ ]:
set(ge_data.columns).difference(set(ge_data_h.columns))

##
##
##  combine cortex and hippocampus datasets 
##
##

In [ ]:
# ge_data_h = ge_data
# X_h = X
# y_h = y
# annot_columns_h = annot_columns
# ge_columns_h = ge_columns
# stp_columns_h = stp_colum

X = np.concatenate([X_c, X_h],axis=0)
y = np.concatenate([y_c, y_h],axis=0)

print(X_h.shape)
print(X_c.shape)
X.shape

In [ ]:
Xan = pd.DataFrame(X[0::200,0:6])
Xan.to_excel('all_annotations.xlsx') 

In [ ]:
Xan2 = pd.DataFrame([[16+3,8+4,8+1,11+2],[9,7,6,12]],columns = ['In->In','In->Ex','Ex->In','In->In'],
                    index=['VISp','Hippocampus']).T

#import matplotlib.pyplot as plt
f, ax =plt.subplots(figsize=(10, 5))
#    plt.title(str(i0)+ '  '+str_syntype + '  ' +stp_columns2[3])
#    plt.xlim((0.25,1.25))

Xan2.plot(kind = 'bar',ax=ax)
plt.ylim([0,20])
plt.xticks(fontsize=18)
plt.yticks(ticks=[0,5,10,15,20],fontsize=18)
plt.ylabel('Number of synapse types',fontsize=20)
#plt.xlabel('Synapse type',fontsize=20)
plt.legend(fontsize=20)
#print(set(n_stp_cells.index).difference(set(i2)))
#n_stp_cells4.sum()



Xan2.sum()

In [ ]:
#pd.DataFrame(X).to_excel('temp.xlsx')

In [ ]:
'cell_type2_post', 'layer_post'   , 'cell_type2_pre', 'layer_pre'

In [ ]:
import time
t1 = time.time()
subclasses2_hipp = ['e_dg','e_ca','e_e','Pvalb_h','Sst_h','Vip','Lamp5_h','Cck_h']
subclasses3_hipp = [['DG','CA3','CA3 ventral'],
                    ['CA1','CA1 ventral'],
                    ['EC'],
                    ['PV','Pvalb','PVBC','AAC'],
                    ['Sst','HIPP','O-LM','O-Bi'],
                    ['Vip','IS3'],
                    ['Lamp5','Ivy cell','Ivy cell type2','NGF','NGF type2'],
                    ['Cck','CCKBC','HICAP','CCK_DTI']]


ge_data_h_subcl = pd.DataFrame(np.zeros((ge_data_h.shape[0],4)),columns = ['name_long_pre','name_long_post',
                                                                           'subclass_pre','subclass_post'])
ge_data_h_subcl.index = ge_data_h.index
precol = ['cell_type2_pre', 'layer_pre'] #ge_data_h.columns[[1,5]]
postcol = ['cell_type2_post', 'layer_post' ] # ge_data_h.columns[[2,6]]


all_type_1 = ge_data_h.loc[:,'cell_type2_pre']+ ' ' + ge_data_h.loc[:,'cre_line_pre'] +' '+ge_data_h.loc[:,'layer_pre']
all_type_2 = ge_data_h.loc[:,'cell_type2_post']+ ' ' + ge_data_h.loc[:,'cre_line_post'] +' '+ge_data_h.loc[:,'layer_post']

ge_data_h_subcl.loc[:,'name_long_pre'] = all_type_1
ge_data_h_subcl.loc[:,'name_long_post'] = all_type_2

for j in range(len(subclasses3_hipp)): 
    is_in_pre = ge_data_h.loc[:,precol[0]].isin(subclasses3_hipp[j])
    is_in_post = ge_data_h.loc[:,postcol[0]].isin(subclasses3_hipp[j])
    ge_data_h_subcl.loc[is_in_pre,'subclass_pre'] =subclasses2_hipp[j]
    ge_data_h_subcl.loc[is_in_post,'subclass_post'] =subclasses2_hipp[j] 
    
t2=time.time()
print('elapsed time : '+str(t2-t1))
ge_data_h_subcl    

In [ ]:
all_types = []
all_types2 = []
columns = ['cell_type2_pre','cell_type2_post','layer_pre','layer_post','cre_line_pre','cre_line_post']
for i in ge_data.index:
    all_type=''
    all_type2=''
    for j in range(3):
        sj = str(ge_data.loc[i,columns[2*j]])
        if sj=='MC':
            sj='Sst'    
        all_type = all_type  +' '+sj
        
    for j in range(3):
        sj = str(ge_data.loc[i,columns[2*j+1]])
        if sj=='MC':
            sj='Sst'    
        all_type2 = all_type2 +' '+sj    
    all_types = all_types  +[str.lower(all_type)]  
    all_types2 = all_types2  +[str.lower(all_type2)]   

all_types =  pd.DataFrame(all_types, columns=['pre'])
all_types2 =  pd.DataFrame(all_types2, columns=['post'])
all_types = pd.concat([all_types,all_types2], axis=1)

subclasses = pd.Series([ 'Sst','Pvalb','Vip','ev_l23','ev_l4','ev_l56','ev_l56'])
subclasses2 = pd.Series(['sst','pvalb','vip','l2',    'l4',   'l5',    'l6'])
subclasses2_visp =subclasses2   

subclasses_visp_1 = pd.DataFrame(pd.unique(all_types.loc[:,'pre'].values))
subclasses_visp_2 = pd.DataFrame(pd.unique(all_types.loc[:,'post'].values))
# subclasses =  pd.DataFrame(subclasses,columns=['pre', 'post'] )
# subclasses_visp = pd.concat([subclasses_visp1, subclasses], axis=1)

ge_data_c_subcl = pd.DataFrame(np.tile(all_types,(1,2)))
ge_data_c_subcl.columns=['name_long_pre','name_long_post','subclass_pre','subclass_post']
for i in range(subclasses_visp_1.shape[0]):
    type_name = str.lower(subclasses_visp_1.loc[i,0])
#     is_in=[]
#     for si in subclasses2:
#         is_in = is_in +[ss.contains(si)]
#     is_in=np.nonzero(np.array(is_in))[0][0] 
#     subcl_name = subclasses[is_in]
    
    for ii,si in enumerate(subclasses2):
        if si in type_name:
            subcl_name = subclasses[ii]
            break
    ge_data_c_subcl.loc[ge_data_c_subcl.loc[:,'name_long_pre']==type_name,'subclass_pre'] =subcl_name
    
for i in range(subclasses_visp_2.shape[0]):
    type_name = str.lower(subclasses_visp_2.loc[i,0])
    for ii,si in enumerate(subclasses2):
        if si in type_name:
            subcl_name = subclasses[ii]
            break
    ge_data_c_subcl.loc[ge_data_c_subcl.loc[:,'name_long_post']==type_name,'subclass_post'] =subcl_name    

#     ge_data_c_subcl.loc[ge_data_c_subcl.loc[:,'name_long'].isin([subclasses_visp.loc[i,0]]),'subclass_pre'] =subclasses_visp.loc[i,'pre']
#     ge_data_c_subcl.loc[ge_data_c_subcl.loc[:,'name_long'].isin([subclasses_visp.loc[i,0]]),'subclass_post'] =subclasses_visp.loc[i,'post']

#ge_data_c_subcl    

In [ ]:
ge_data_subcl = pd.concat([ge_data_c_subcl, ge_data_h_subcl],sort=False)
subclasses2 =pd.concat([subclasses, pd.Series(subclasses2_hipp)])
ge_data_subcl



# TRAIN MODEL

##
##
##  train RF 
##
##

In [ ]:
subclasses2

In [ ]:
classes = ['all','ex', 'inh', 'ex_ctx', 'ex_hipp','ex_ec','mge','cge','pvalb','vip','sst','cck','lamp5','ex_he',
          'ex_l23','ex_l6','ex_l4','ca','dg','pvalb_h','pvalb_c','sst_h','sst_c']
y_features = pd.DataFrame([['A2/A1_20Hz', 'A5/A1_20Hz', 'A2/A1_50Hz','A5/A1_50Hz','A250/A1','A1000/A1'],[0,4,8,11,5,6]],index=['names','index_y'])
print("y features : \n", y_features,"\n\n")
subclasses2 = pd.unique(subclasses2 )
print("subclasses : \n",subclasses2,"\n\n") 

cl_in_subcl = np.array([[0,1,2,3,4,5,6,7,8,9,10,11,12],[3,4,5,6,7,8],
                        [0,1,2,9,10,11,12],[3,4,5],[6,7],[8],[1,9,0,10],
                        [2,11,12],[1,9],[2],[0,10],[12],[11],[6,7,8],[3],[4],[5],[7],[6],[9],[1],[10],[0]])
classes = pd.DataFrame([np.array(classes), cl_in_subcl],index=['classes','subclasses'])
classes = classes.T.set_index('classes').T
classes

In [ ]:
def make_data(X,y,data_subclasses,subclasses):
    #X_train = np.array([]).reshape((0,X.shape[1]))
    #y_train = np.array([]).reshape((0,y.shape[1]))

    in_subcl = (data_subclasses.loc[:,'subclass_post']!=False)
    for i0 in range(len(subclasses[0])):
        in_subcl = in_subcl|(data_subclasses.loc[:,'subclass_pre']==subclasses[0][i0])
        
    for i0 in range(len(subclasses[1])):        
        #in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_post']==subclasses2[sbi[i0]])
        in_subcl = in_subcl|(data_subclasses.loc[:,'subclass_post']==subclasses[1][i0])
        
    in_subcl = np.nonzero(in_subcl.values)[0]
    #X_train = np.concatenate([X_train,X[in_subcl]])   #np.delete(X, in_subcl,axis=0)
    #y_train = np.concatenate([y_train,y[in_subcl]])   #np.delete(y, in_subcl,axis=0)
    X_train = np.copy(X[in_subcl])   #np.delete(X, in_subcl,axis=0)
    y_train = np.copy(y[in_subcl])   #np.delete(y, in_subcl,axis=0)
    return X_train, y_train    

In [ ]:
modf = pd.read_excel('synapses_types_tree.xlsx')
modf

In [ ]:
stp_columns3 = []
stp_columns0 = ['A1',
                'A2',
                'A3',
                'A4',
                'A5']
stpn_fr=[s+'_20Hz' for s in stp_columns0[1:]]
stp_columns3 = stp_columns3 +stpn_fr+['A250_20Hz', 'A1000_20Hz']

stpn_fr=[s+'_50Hz' for s in stp_columns0[0:]]
stp_columns3 = stp_columns3 +stpn_fr+['A250_50Hz', 'A1000_50Hz']

stpn_fr=[s+'_10Hz' for s in stp_columns0[0:]]
stp_columns3 = stp_columns3 +stpn_fr+['A250_10Hz', 'A1000_10Hz']
stp_columns3

X_train = pd.DataFrame(np.copy(X),columns = annot_columns+ge_columns[6:])   #np.delete(X, in_subcl,axis=0)
y_train = pd.DataFrame(np.copy(y),columns = annot_columns+stp_columns3)     #np.delete(X, in_subcl,axis=0)

X_train = pd.concat([X_train,y_train.loc[:,stp_columns3]],axis=1)

#X_train 


#'cla_cell_type_l2_pre'
#'cla_cell_type_l2_post'
#'cla_cell_type_l3_pre'
#'cla_cell_type_l3_post'


#classes 	all 	ex 	inh 	ex_ctx 	ex_hipp 	ex_ec 	mge 	cge 	pvalb 	vip 	sst 	cck 	lamp5

modf = pd.read_excel('synapses_types_tree.xlsx')
modf2 = pd.read_excel('synapses_types_tree.xlsx',sheet_name='Sheet2')
modf3 = pd.read_excel('synapses_types_tree.xlsx',sheet_name='Sheet3')
mod =[ {'name': 'all',    'classes':['all'],                                   'classes':'pre'},
       {'name': 'ex_inh', 'classes':['ex','inh'],                              'side':'pre'},
       {'name': 'ex_inh_post', 'classes':['ex','inh'],                         'side':'post'},
       {'name': 'ex2_inh', 'classes':['ex_ctx','ex_ec','ex_hipp','inh'],       'side':'pre'},
       {'name': 'ex_inh2', 'classes':['ex','mge','cge'],                       'side':'pre'},
       {'name': 'ex_inh', 'classes':['ex','inh'],                              'side':'pre'},
       {'name': 'ex2_inh', 'classes':['ex_ctx','ex_ec','ex_hipp','inh'],       'side':'pre'},
       {'name': 'ex_inh3', 'classes':['ex','pvalb','sst','cge'],               'side':'pre'},
       {'name': 'ex_inh4', 'classes':['ex','pvalb','sst','vip','cck','lamp5'], 'side':'pre'},
     ]

subcl_pre=ge_data_subcl.loc[:,'subclass_pre'].copy()
subcl_pre.index=X_train.index

subcl_post=ge_data_subcl.loc[:,'subclass_post'].copy()
subcl_post.index=X_train.index

# classes_columns = []
# for md in mod:
#     in_subcl = subcl_pre.copy()
#     for icl, cl in enumerate(md['classes']):
#         #in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_pre'].isin(classes.loc['subclasses']))
#         subcl = subclasses2[classes.loc['subclasses',cl]]
#         in_subcl.loc[subcl_pre.isin(subcl)] = icl #icl
#     in_subcl.name =  md['name']  
#     classes_columns = classes_columns + [in_subcl.name]
#     X_train = pd.concat([X_train,in_subcl],axis=1)    
 
# #For modf case     
# classes_columns = []
# for i in modf.index:
#     md = modf.loc[i,:]
#     in_subcl = subcl_pre.copy()
#     mdcl = np.char.strip(np.array(str.split(md['classes'],',')))
#     for icl, cl in enumerate(mdcl):
#         #in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_pre'].isin(classes.loc['subclasses']))
#         subcl = subclasses2[classes.loc['subclasses',cl]]
#         in_subcl.loc[subcl_pre.isin(subcl)] = icl #icl
#     in_subcl.name =  md['name']+'__pre'  
#     classes_columns = classes_columns + [in_subcl.name]
#     X_train = pd.concat([X_train,in_subcl],axis=1)  
    
#     in_subcl = subcl_post.copy()
#     mdcl = np.char.strip(np.array(str.split(md['classes_post'],',')))
#     for icl, cl in enumerate(mdcl):
#         #in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_pre'].isin(classes.loc['subclasses']))
#         subcl = subclasses2[classes.loc['subclasses',cl]]
#         in_subcl.loc[subcl_post.isin(subcl)] = icl #icl
#     in_subcl.name =  md['name']+'__post'  
#     classes_columns = classes_columns + [in_subcl.name]
#     X_train = pd.concat([X_train,in_subcl],axis=1)

## modf3 case - independent columns for pre and post cell types
classes_columns = []
for i in modf3.index:
    md = modf3.loc[i,:]
    in_subcl = subcl_pre.copy()
    in_subcl.loc[:]=0 
    mdcl = np.char.strip(np.array(str.split(md['classes'],',')))
    for icl, cl in enumerate(mdcl):
        #in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_pre'].isin(classes.loc['subclasses']))
        subcl = subclasses2[classes.loc['subclasses',cl]]
        in_subcl.loc[subcl_pre.isin(subcl)] = cl #icl+1 #icl
    in_subcl.name =  md['name']+'__pre'  
    classes_columns = classes_columns + [in_subcl.name]
    X_train = pd.concat([X_train,in_subcl],axis=1)  
    
    in_subcl = subcl_post.copy()
    in_subcl.loc[:]=0
    mdcl = np.char.strip(np.array(str.split(md['classes'],',')))
    for icl, cl in enumerate(mdcl):
        #in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_pre'].isin(classes.loc['subclasses']))
        subcl = subclasses2[classes.loc['subclasses',cl]]
        #if cl!='all' 
        in_subcl.loc[subcl_post.isin(subcl)] = cl #icl+1 #icl
    in_subcl.name =  md['name']+'__post'  
    classes_columns = classes_columns + [in_subcl.name]
    X_train = pd.concat([X_train,in_subcl],axis=1)

# add all short subclasses  
in_subcl = subcl_pre.copy()    
in_subcl.name =  'vse'+'__pre'  
classes_columns = classes_columns + [in_subcl.name]
X_train = pd.concat([X_train,in_subcl],axis=1)
    
in_subcl = subcl_post.copy()    
in_subcl.name =  'vse'+'__post'  
classes_columns = classes_columns + [in_subcl.name]
X_train = pd.concat([X_train,in_subcl],axis=1)
         
#for i in
#mod1_ = mod1_ +  [ sm.OLS( y_train, X_train) ]
#res = mod.fit()
#print(res.summary())
ge_columns_train      = ge_columns[6:]
annot_columns_train   = annot_columns
stp_columns_train     = stp_columns3
classes_columns_train = classes_columns
print(X_train.columns)
X_train

In [ ]:
in_subcl = subcl_post.copy()    
in_subcl.name =  'vse'+'__post'  
classes_columns = classes_columns + [in_subcl.name]
X_train = pd.concat([X_train,in_subcl],axis=1)
         
#for i in
#mod1_ = mod1_ +  [ sm.OLS( y_train, X_train) ]
#res = mod.fit()
#print(res.summary())
ge_columns_train      = ge_columns[6:]
annot_columns_train   = annot_columns
stp_columns_train     = stp_columns3
classes_columns_train = classes_columns
print(X_train.columns)
X_train

In [ ]:
#X_train0.to_excel('X_train0.xlsx')

In [ ]:
classes

In [ ]:
columns  =[
[ 'ge_columns_train',
  'annot_columns_train',
  'stp_columns_train',
  'classes_columns_train',],
[ ge_columns_train,
  annot_columns_train,
  stp_columns_train,
  classes_columns_train,
]]
columns = pd.DataFrame(columns)
columns=columns.T.set_index([0]).T
columns

In [ ]:
X_train.columns[X_train.columns[:].duplicated()]


In [ ]:
print('Remove duplicated columns ...')
print(X_train.columns.shape)
is_dup = np.nonzero(X_train.columns[:].duplicated())[0]
is_dup0 = is_dup
for i in is_dup:
    X_train.columns = np.array(X_train.columns[:i].tolist()+ [X_train.columns[i]+'_2']+X_train.columns[i+1:].tolist())

In [ ]:
is_dup = np.nonzero(X_train.columns[:].duplicated())[0]
print(is_dup0)
is_dup

In [ ]:
gs_name

In [ ]:
columns.to_hdf('stp_to_ge_all_columns'+gs_name+'.hdf',key='data')
X_train.to_hdf('stp_to_ge_all_data'+gs_name+'.hdf',key='data')

In [ ]:
classes_columns_train

# START HERE

In [ ]:
gs_name = ''

In [ ]:
import pandas as pd
import numpy as np
columns  = pd.read_hdf('stp_to_ge_all_columns'+gs_name+'.hdf')
ge_columns_train      = columns.loc[:,'ge_columns_train'].values[0]
annot_columns_train   = columns.loc[:,'annot_columns_train'].values[0]
stp_columns_train     = columns.loc[:,'stp_columns_train'].values[0]
classes_columns_train = columns.loc[:,'classes_columns_train'].values[0]
#print(X_train.columns)
X_train0 = pd.read_hdf('stp_to_ge_all_data'+gs_name+'.hdf')

In [ ]:
#classes 	all 	ex 	inh 	ex_ctx 	ex_hipp 	ex_ec 	mge 	cge 	pvalb 	vip 	sst 	cck 	lamp5

modf = pd.read_excel('synapses_types_tree.xlsx')
modf2 = pd.read_excel('synapses_types_tree.xlsx',sheet_name='Sheet2')
modf3 = pd.read_excel('synapses_types_tree.xlsx',sheet_name='Sheet3')
mod =[ {'name': 'all',    'classes':['all'],                                   'classes':'pre'},
       {'name': 'ex_inh', 'classes':['ex','inh'],                              'side':'pre'},
       {'name': 'ex_inh_post', 'classes':['ex','inh'],                         'side':'post'},
       {'name': 'ex2_inh', 'classes':['ex_ctx','ex_ec','ex_hipp','inh'],       'side':'pre'},
       {'name': 'ex_inh2', 'classes':['ex','mge','cge'],                       'side':'pre'},
       {'name': 'ex_inh', 'classes':['ex','inh'],                              'side':'pre'},
       {'name': 'ex2_inh', 'classes':['ex_ctx','ex_ec','ex_hipp','inh'],       'side':'pre'},
       {'name': 'ex_inh3', 'classes':['ex','pvalb','sst','cge'],               'side':'pre'},
       {'name': 'ex_inh4', 'classes':['ex','pvalb','sst','vip','cck','lamp5'], 'side':'pre'},
     ]
modf

In [ ]:
modf3

In [ ]:
def decode_classes2(class2,modf3):
    cla=[]
    for cl2 in class2:
        pref = str.rfind(cl2,'_pre')
        if pref==-1:
            pref = str.rfind(cl2,'_post')
            if pref==-1:
                spref='__pre'
                
            else:
                cl2 = cl2[0:pref]
                spref='__post'
                
        else:
            spref='__pre'
            cl2 = cl2[0:pref]
        
        #print(cl2)
        md = modf3.loc[modf3.loc[:,'name_classes2']==cl2 ,'classes2'].iloc[0]  
        #print(md)
        mdcl = np.char.strip(np.array(str.split(md,',')))
        for icl, cl in enumerate(mdcl):
            #in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_pre'].isin(classes.loc['subclasses']))
            cla = cla + [cl + spref]
    
    return cla    

In [ ]:
decode_classes2(['ex2_in_pre','ex2_in3_post'],modf3)

In [ ]:
import sklearn as sk

In [ ]:
## Fit cell types hierarhy tree with linear models in leaves

In [ ]:
def fit_classes_tree(X2,y2,X2_cl,cli,model_type='lasso',nmin = 20, alpha=1, l1_ratio=0.5):
    model = []
    support =[]
    ncl1 = np.max(X2_cl[:,cli[0]])+1 
    ncl2 = np.max(X2_cl[:,cli[1]])+1 
    for i1 in range(ncl1):
        model1 = []
        support1 = []
        for i2 in range(ncl2):

            is_in_cl = (X2_cl[:,cli[0]]==i1)&(X2_cl[:,cli[1]]==i2)
            #           support2 = [np.sum(is_in_cl),
            #           np.nonzero((X2_cl[:,cli[0]]==i1))[0],
            #           np.nonzero((X2_cl[:,cli[1]]==i2))[0]]
            support2 = np.sum(is_in_cl)
            
            support1 = support1 + [support2]
            regr = 0
            if support2>nmin:
                y_train = y2[is_in_cl,:]
                X_train = X2[is_in_cl,:]
                if model_type=='ridge':
                    regr = sk.linear_model.Ridge(alpha=0.50, fit_intercept=True, normalize=False,
                                                 copy_X=True, max_iter=None, tol=0.001, solver='auto',
                                                 random_state=None)
                
                elif model_type=='lasso':
                    regr = sk.linear_model.Lasso(alpha=alpha, fit_intercept=True, normalize=False,
                                                 precompute=False, copy_X=True,
                                                 max_iter=1000, tol=0.0001, warm_start=False, positive=False, 
                                                 random_state=None, selection='cyclic')
                elif model_type=='elastic_net':   
                    regr = sk.linear_model.ElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=True,
                                    normalize=False,
                                    max_iter=1000, copy_X=True, tol=0.0001, warm_start=False,
                                    random_state=None, selection='cyclic')
                else:
                    print(" model types supported: ridge, lasso, elastic_net")
                
                regr.fit(X_train, y_train)
                #y_pred = regr.predict(X_test[:,nannot:]) 


            model1 = model1 + [regr]
            
        support = support + [np.array(support1)]
        model = model + [np.array(model1)]    
    return np.array(model), np.array(support)        

In [ ]:
def predict_classes_tree(model,X2,X2_cl,cli,nout=1,nmin = 20):
    
    ncl1 = np.max(X2_cl[:,cli[0]])+1 
    ncl2 = np.max(X2_cl[:,cli[1]])+1 
    y_pred = np.zeros((X2.shape[0],nout))
    for i1 in range(ncl1):
        #print(i1," f1")
        if len(model)>i1:
            #print(i1," f2")
            model1 = model[i1]
            for i2 in range(ncl2):

                is_in_cl = (X2_cl[:,cli[0]]==i1)&(X2_cl[:,cli[1]]==i2)

                support2 = np.sum(is_in_cl)
                
                #print(i1,i2," f3 ",support2)
                #regr = 0
                if support2>nmin:
                    #print(i1,i2," f3 ",support2)
                    if len(model1)>i2:
                        #print(i1,i2," f4")
                        regr = model1[i2]
                        #regr = sk.linear_model.MultiTaskElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=True,
                        #                    normalize=False,
                        #                    max_iter=1000, copy_X=True, tol=0.0001, warm_start=False,
                        #                    random_state=None, selection='cyclic')
                        #regr.fit(X_train, y_train)
                        if regr!=0:
                            X_test = X2[is_in_cl,:]
                            y_pred[is_in_cl] = regr.predict(X_test).reshape((-1,1)) 
    
    return y_pred       

In [ ]:
## Draw some interesting plots

In [ ]:
# Load important genes from iRF search

%matplotlib inline
import matplotlib.pyplot as plt
imp  = np.load('importance.npy')
imp = pd.DataFrame(imp)
imp = pd.concat([imp,pd.DataFrame(ge_columns_train )],axis=1)
imp.columns = ['importance', 'genes']
imp = imp.set_index('genes')

imp0  = np.load('importance0.npy')
imp0 = pd.DataFrame(imp0)
imp0 = pd.concat([imp0,pd.DataFrame(ge_columns_train )],axis=1)
imp0.columns = ['importance0', 'genes']
imp0 = imp0.set_index('genes')


# List of important genes : iRF
imp = pd.concat([imp,imp0],axis=1)

imp50 = imp.sort_values('importance',ascending=False).iloc[0:49,:]
plt.plot(imp.sort_values('importance',ascending=False).loc[:,['importance','importance0']].values)
imp50

In [ ]:
classes_columns_train

In [ ]:
# note: synapse type hierarhy in X(:,classes_columns_train) should be from modf!!!

from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import time
t1=time.time()

Dn = 5
cla_n = classes_columns_train #['ex_inh']

ge_n = imp50.index[0:50].tolist() #ge_columns_train #imp50.index[0:25].tolist()

#ge_n =ge_columns_train
X2 = X_train0.loc[:,ge_n ]

do_PCA=False
if do_PCA:
    # PCA transformation
    pca=PCA(n_components=30)
    X2 =pca.fit_transform(X_train0.loc[:,ge_n ].values)
    print(pca.explained_variance_ratio_)
    print(pca.singular_values_) 
    ge_n = ['pca'+str(s) for s in range(X2.shape[1])]
    X2=X2[0:X_train0.shape[0]:Dn,:]
else:
    #i_cl = np.nonzero(X2.columns.isin(cla_n))[0]
    X2=X2.iloc[0:X_train0.shape[0]:Dn,:].values

X2_cl = X_train0.loc[:,cla_n ]

X2_cl=X2_cl.iloc[0:X_train0.shape[0]:Dn,:].values
stp_n = ['A5_20Hz']
#  'A2_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A2_50Hz',
#  'A2_10Hz',
#  'A5_50Hz',
#  'A5_10Hz',
y2 =  X_train0.loc[:,stp_n].iloc[0:X_train0.shape[0]:Dn,:].values
##header = ge_n + cla_n + stp_n 
#header  = ge_n  + stp_n 

iy0=0
mod_index = modf.index[[14]] #[3,9,13,16] [3] # modf.index #[0, 3, 9, 11, 13, 15]

for ig,g in enumerate(ge_n):
    #f, ax =plt.subplots(figsize=(16, 10))
    ##f, ax = plt.figure()
    ##ax = f.add_axes()
    
    #plt.title(stp_columns[iy]+", model : "+mdn+", cv : "+str(icv))
    #plt.title(stp_columns[iy]+", model : "+mdn)
    
    #ax.set_title(stp_columns[i]+' '+', subclass out of bag: '+str(i0))
    #yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
    #plt.plot(y2.ravel(),'ob')
    #plt.plot(y_pred.ravel(),'xr')
    for i in mod_index:
        md = modf.loc[i,:]


        mdn = md['name']
        #print(mdn)

        cla_n = np.array(classes_columns_train)

        cli = np.nonzero(cla_n==mdn+'__pre')[0] + np.arange(2)

        mdcl2 = np.char.strip(np.array(str.split(md['classes_post'],',')))
        mdcl1 = np.char.strip(np.array(str.split(md['classes'],','))) 
        #print(mdcl1)
        #print(mdcl2)
        
        f, ax =plt.subplots(figsize=(16, 10))
        ##f, ax = plt.figure()
        ##ax = f.add_axes()
    
        #plt.title(stp_columns[iy]+", model : "+mdn+", cv : "+str(icv))
        plt.title(g+", model : "+mdn+" classes pre : "+str(mdcl1)+" classes post : "+str(mdcl2))
        
        plt.scatter(X2[:,ig],y2.ravel(),marker='o',c=X2_cl[:,cli[1]],s=100,alpha=0.5)
        plt.scatter(X2[:,ig],y2.ravel(),marker='o',c=X2_cl[:,cli[0]],s=10)
       

### Fit set of linear models for each level of cell types hierarhy

In [ ]:
# note: synapse type hierarhy in X(:,classes_columns_train) should be from modf!!!

import matplotlib.pyplot as plt
import time
t1=time.time()

Dn = 10
cla_n = classes_columns_train #['ex_inh']
ge_n = ge_columns_train #imp50.index[0:25].tolist()
X2 = X_train0.loc[:,ge_n ]
X2_cl = X_train0.loc[:,cla_n ]
#i_cl = np.nonzero(X2.columns.isin(cla_n))[0]
X2=X2.iloc[0:X_train0.shape[0]:Dn,:].values
X2_cl=X2_cl.iloc[0:X_train0.shape[0]:Dn,:].values
stp_n = ['A2_20Hz']
#  'A2_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A2_50Hz',
#  'A2_10Hz',
#  'A5_50Hz',
#  'A5_10Hz',
y2 =  X_train0.loc[:,stp_n].iloc[0:X_train0.shape[0]:Dn,:].values
##header = ge_n + cla_n + stp_n 
#header  = ge_n  + stp_n 

iy0=0
mod_index = modf.index #[3] # modf.index #[0, 3, 9, 11, 13, 15]
Y_pred = []
for i in mod_index:
    md = modf.loc[i,:]


    mdn = md['name']
    print(mdn)

    cla_n = np.array(classes_columns_train)

    cli = np.nonzero(cla_n==mdn+'__pre')[0] + np.arange(2)

    mdcl2 = np.char.strip(np.array(str.split(md['classes_post'],',')))
    mdcl1 = np.char.strip(np.array(str.split(md['classes'],','))) 
    print(mdcl1)
    print(mdcl2)

    y_pred = np.zeros((0,y2.shape[1]))
    
    ncv = 10
    n_samp_cv = np.rint(X2.shape[0]/ncv)
    samples_all = np.arange(X2.shape[0])
    r2cv = np.zeros(ncv)
    r4cv = np.zeros(ncv)
    for icv in range(ncv): #range(ncv):
        samples_test = (np.arange(n_samp_cv) + icv*n_samp_cv).astype(int)
        samples_train = np.delete(np.copy(samples_all),samples_test)
        X2train, y2train, X2train_cl = X2[samples_train,:], y2[samples_train,:], X2_cl[samples_train,:]
        X2test, y2test, X2test_cl = X2[samples_test,:], y2[samples_test,:], X2_cl[samples_test,:]
        
        model, support = fit_classes_tree(X2train,y2train,X2train_cl,cli,model_type='elastic_net',
                                          nmin = 20, alpha=0.2, l1_ratio=0.05)
        #model, support = fit_classes_tree(X2,y2,X2_cl,cli,model_type='ridge',nmin = 20, alpha=1, l1_ratio=0.5)
        y_pred_i = predict_classes_tree(model,X2test,X2test_cl,cli,nout=y2.shape[1],nmin = 2)
        y_pred = np.concatenate([y_pred, y_pred_i],axis=0)
        
        iy=iy0
        if np.var(y2[:,iy])!=0:
            nonz = y_pred_i[:,iy]!=0
            if np.sum(nonz)>0:
                r2cv[icv]=1 - np.mean((y2test[nonz,iy] - y_pred_i[nonz,iy])**2)/np.var(y2[:,iy])
                r4cv[icv]=1 - np.mean((y2test[nonz,iy] - y_pred_i[nonz,iy])**2)/np.var(y2test[nonz,iy])
    
    Y_pred = Y_pred + [y_pred]
    
    f, ax =plt.subplots(figsize=(16, 10))
    ##f, ax = plt.figure()
    ##ax = f.add_axes()
    
    #plt.title(stp_columns[iy]+", model : "+mdn+", cv : "+str(icv))
    plt.title(stp_columns_train[iy]+", model : "+mdn)
    
    #ax.set_title(stp_columns[i]+' '+', subclass out of bag: '+str(i0))
    #yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
    plt.plot(y2.ravel(),'ob')
    plt.plot(y_pred.ravel(),'xr')
    #plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)
    iy=iy0
    r2=0
    r4=0
    if np.var(y2[:,iy])!=0:
        r2=1 - np.mean((y2[:,iy] - y_pred[:,iy])**2)/np.var(y2[:,iy])
        r4=1 - np.var(y_pred[:,iy])/np.var(y2[:,iy])
        
    print("R**2 = ",r2,"  Part of Var = ",r4,"\n",
          "  R**2 cv mean = ", np.mean(r2cv),"\n  R**2 cv unnormed mean = ", np.mean(r4cv),
          "\n  R**2 cv = ", r2cv,"\n  R**2 cv unnormed = ", r4cv,"\n\n")
    
    
t2 =time.time()    
print("Elapsed time ",t2-t1)

## Crossvalidation - kernel SVR on scVI space, RF with distance spliting penalty? - nonlocal predictors vs local

In [ ]:
gs_name = '_gs1512' #'_gs15656' #'_gs1512' #'_gs15656' #'_gs1512'   #'_gs5188' #'_gs1512'

In [ ]:
do_laptop = True

In [ ]:
import pandas as pd
import numpy as np
columns  = pd.read_hdf('stp_to_ge_all_columns'+gs_name+'.hdf')
ge_columns_train      = columns.loc[:,'ge_columns_train'].values[0]
annot_columns_train   = columns.loc[:,'annot_columns_train'].values[0]
stp_columns_train     = columns.loc[:,'stp_columns_train'].values[0]
classes_columns_train = columns.loc[:,'classes_columns_train'].values[0]
#print(X_train.columns)
X_train0 = pd.read_hdf('stp_to_ge_all_data'+gs_name+'.hdf')

In [ ]:
X_train0.shape
print(X_train0.columns[0:20])
print(X_train0.columns[200:300])
X_train0.columns[300:]

In [ ]:
gs_name

In [ ]:
if (gs_name=='_gs5188')|(gs_name=='_gs1512')|(gs_name=='_gs15656'):
    nlf = 10   # 20 were in scvi model
elif (gs_name=='_gs219'):
    nlf = 10   # 10 were in scvi model
lf_names = np.arange(nlf).astype(str)
lf_names1 =  ['pre__'+s for s in lf_names.tolist()]
lf_names2 =  ['post__'+s for s in lf_names.tolist()]

lf_scvi_names = lf_names1 + lf_names2

ge_columns_train2 = list(set(ge_columns_train).difference(set(lf_scvi_names)))
lf_scvi_names

### tSNE : cell types, STP space



In [ ]:
d_aba

In [ ]:
if do_laptop==False:
    d_aba  ='/home/stepaniu/Documents/references/transcriptomes to STP/transcriptomes/ABA_2019/'
else:
    d_aba ='/Users/stepaniu/Documents/jan_2020/ABA_2019/'

df_aba_g = pd.read_csv(d_aba + 'medians.csv')
df_aba_c = pd.read_csv(d_aba + 'sample_annotations.csv')
df_aba_tsne = pd.read_csv(d_aba + '2d_coordinates.csv')
df_aba_tsne =df_aba_tsne.set_index('sample_name')
df_aba_tsne 

In [ ]:
#in_20190=df_aba_c.loc[:,'region_label'].isin(['VISp'])
#set(df_aba_c.loc[in_20190&df_aba_c.loc[:,'class_label'].isin(['GABAergic', 'Glutamatergic']),'subclass_label'])

In [ ]:
#set(df_aba_vis_c.loc[df_aba_vis_c.loc[:,'class'].isin(['GABAergic', 'Glutamatergic']),'subclass'])

In [ ]:
#do_laptop=0
if do_laptop:
    d4 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/'
else:
    d4='~/Documents/references/transcriptomes to STP/transcriptomes/ABA/mouse_VISp_gene_expression_matrices_2018-06-14/'

fn='mouse_VISp_2018-06-14_samples-columns.csv'
df_aba_vis_c=pd.read_csv(d4+fn)

In [ ]:
df_aba_vis_c

In [ ]:
import time



#
#
#  ALLIGNMET ABA 2018 and ABA 2019 samples
#
#


do_alignment=0
if do_alignment:
    t1=time.time()
    do_pre=0
    if do_pre:
        samps = sampr
    else:
        samps = sampo

    if do_laptop:
        d4 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/'
    else:
        d4='~/Documents/references/transcriptomes to STP/transcriptomes/ABA/mouse_VISp_gene_expression_matrices_2018-06-14/'

    fn='mouse_VISp_2018-06-14_samples-columns.csv'
    df_aba_vis_c=pd.read_csv(d4+fn)


    #sm = [ 'F2S4_160825_057_G01']
    #sm  =['F2S4_160331_001_B01']

    #in_2019 = df_aba_c.loc[:,'subclass_label'].isin([df_2018_donor['subclass']])

    in_20190=df_aba_c.loc[:,'region_label'].isin(['VISp'])

    ll = []
    nn = []
    # index 	sample_name 	cluster 	sample_name_19 	cluster_label
    col1819=['index','sample_name','cluster','sample_name_19','cluster_label']
    df_aba18_to_19 = pd.DataFrame([],columns = col1819)
    for sm in samps: #.iloc[1:1000]: 
        if (df_aba18_to_19.loc[:,'sample_name']==sm).sum()==0:
            df_2018_donor = df_aba_vis_c.loc[df_aba_vis_c.loc[:,'sample_name'].isin([sm]),['sample_name', 'donor','genotype','sex','subclass','class','cluster','brain_subregion']]
            #df_2018_donor
            #in_2019=in_2019&df_aba_c.loc[:,'cortical_layer_label'].isin(['L4'])
            
            #in_2019=in_2019&df_aba_c.loc[:,'external_donor_name_label'].isin(df_2018_donor['donor'])
            in_2019=n_20190&df_aba_c.loc[:,'cortical_layer_label'].isin(df_2018_donor['brain_subregion'])
            in_2019=in_2019&df_aba_c.loc[:,'full_genotype_label'].isin(df_2018_donor['genotype'])
            in_2019=in_2019&df_aba_c.loc[:,'donor_sex_label'].isin(df_2018_donor['sex'])
            in_2019=in_2019&df_aba_c.loc[:,'external_donor_name_label'].map(str).isin(df_2018_donor['donor'].map(str))
            in_2019 = in_2019&df_aba_c.loc[:,'subclass_label'].isin(df_2018_donor['subclass'])
            
            ##df_aba_c.loc[in_2019,'sample_name']
            col19 = ['sample_name', 'cortical_layer_label',
                                             'full_genotype_label','external_donor_name_label',
                                             'cluster_label','class_label','subclass_label',
                                             'cell_type_alt_alias_label','donor_sex_label',
                                             'cell_type_alias_label']
            df_out19 = df_aba_c.loc[in_2019,col19]
            col19[0] = 'sample_name_19'
            df_out19.columns=col19

            
            ##in_2019=in_2019&df_aba_c.loc[:,'external_donor_name_label'].isin(df_2018_donor['donor'])
            in_2018=df_aba_vis_c.loc[:,'brain_subregion'].isin(df_2018_donor['brain_subregion'])
            in_2018=in_2018&df_aba_vis_c.loc[:,'genotype'].isin(df_2018_donor['genotype'])
            in_2018=in_2018&df_aba_vis_c.loc[:,'sex'].isin(df_2018_donor['sex'])
            in_2018=in_2018&df_aba_vis_c.loc[:,'donor'].isin(df_2018_donor['donor'])
            in_2018 = in_2018&df_aba_vis_c.loc[:,'subclass'].isin(df_2018_donor['subclass'])
            ##df_aba_c.loc[in_2019,'sample_name']
            df_out18 = df_aba_vis_c.loc[in_2018,['sample_name', 'brain_subregion','genotype','donor','cluster','class','subclass','sex']]


            #if df_out18.shape[0]==df_out19.shape[0]:
            nmim = min([df_out18.shape[0],df_out19.shape[0]])
            df1819 = pd.concat([df_out18.iloc[:nmim].loc[:,['sample_name','cluster']].reset_index(),
                                df_out19.iloc[:nmim].loc[:,['sample_name_19','cluster_label']].reset_index(drop=True)],axis=1)
            df_aba18_to_19 = df_aba18_to_19.append(df1819)

            ll = ll + [df_out19.loc[:,'cortical_layer_label'].isin(df_2018_donor['brain_subregion']).sum()]
            nn = nn + [df_out19.shape[0]]
            #print(df_out18)
            #print(df_out19)
    t2 =time.time()
    print('elapsed time '+str(t2-t1))
    print(df_aba18_to_19.shape )
    if do_pre:
        df_aba18_to_19.to_excel('aba18_in_aba19_presynaptic.xlsx')
    else:
        df_aba18_to_19.to_excel('aba18_in_aba19_postsynaptic.xlsx')
        
    print(sampr.shape[0])    
    df_aba18_to_19 
    
### filter
df_aba18_to_19 = pd.read_excel('aba18_in_aba19_postsynaptic.xlsx')
df_aba18_to_19_2 = df_aba18_to_19.loc[df_aba18_to_19.loc[:,'sample_name'].isin(sampo),:].copy()

df_aba18_to_19 = pd.read_excel('aba18_in_aba19_presynaptic.xlsx')
df_aba18_to_19_3 = df_aba18_to_19.loc[df_aba18_to_19.loc[:,'sample_name'].isin(sampr),:].copy()

df_aba18_to_19_2 = df_aba18_to_19_2.append(df_aba18_to_19_3)


df_aba18_to_19_2.to_excel('aba18_in_aba19_all_filtered.xlsx')

In [ ]:
df_aba18_to_19_2 = pd.read_excel('aba18_in_aba19_all_filtered.xlsx')

In [ ]:
#[np.array([])]*cl18.shape[0]

In [ ]:
#to do !!! - clusters 18 to 19

cl18 = set(df_aba18_to_19_2.loc[:,'cluster'])
cl18=list(cl18)
# stpn_test=[s+'_test' for s in stp_n]
cl18_2 = np.array([cl18,[s+' counts' for s in cl18]])
cl18_2= np.reshape(cl18_2.transpose(),(-1,))
cl18=np.array(cl18)
#df_cl18_to_cl19 = pd.DataFrame(np.zeros((5,cl18.shape[0]*2)),columns=cl18_2)
df_cl18_to_cl19 = pd.DataFrame(['']*cl18.shape[0],index=cl18)
for c18 in cl18:
    c19=df_aba18_to_19_2.loc[df_aba18_to_19_2.loc[:,'cluster']==c18,'cluster_label'].value_counts()
    print('\n\n')
    print(c18)
    print(c19)
    #nmin=min([df_cl18_to_cl19.shape[0],c19.shape[0]])
    nmin=min([1,c19.shape[0]])
    
    #df_cl18_to_cl19.loc[:,c18] = df_cl18_to_cl19.loc[:,c18].astype(str)
    df_cl18_to_cl19.loc[c18,0] = c19.index[:nmin].tolist()[0] #.to_numpy()[0] #.tolist()
    #df_cl18_to_cl19.iloc[0:nmin,:].loc[:,c18+' counts'] = c19[:nmin].values/c19.values.sum()
    #df_cl18_to_cl19.loc[:nmin,c18] = c19[:nmin]/c19.sum()
df_cl18_to_cl19

In [ ]:
# Load STP types based on F/D
remove_st=[]

df_Stp_type_p = pd.read_excel('stp types probability cortex and hippocampus.xlsx')
#print('df_Stp_type_p.shape : ',df_Stp_type_p.shape, '  X_train0.shape : ', X_train0.shape[0]/N_bootstraps)
all_samples2 = np.ones(df_Stp_type_p.shape[0])
for st in remove_st: # remove bad samples
    all_samples2[st] = 0
df_Stp_type_p = df_Stp_type_p.iloc[all_samples2==1,:].reset_index()



df8 = df_Stp_type_p.loc[:,'median'].copy()  # stp type based on median of f/d
df8.name='type'
df9=df8.copy()
df9.loc[df8>2]=1
df9.loc[df8<0.4]=2
df9.loc[(df8>=0.4)&(df8<=2)]=3
df_Stp_type_p = pd.concat([df_Stp_type_p,df9],axis=1)

df_Stp_type_p.loc[:,'median'] = np.log(df_Stp_type_p.loc[:,'median'])


df_Stp_type_p_c=df_Stp_type_p.loc[0:53,:]
df_Stp_type_p_h=df_Stp_type_p.loc[53:,:]

print('df_Stp_type_p.shape : ',df_Stp_type_p.shape)
df_Stp_type_p.head()


In [ ]:
df_Stp_type_p_h.head()

In [ ]:
in_data22 = df_aba_c.loc[:,'subclass_label'].isin([scstpo])
tsne1tp2 = df_aba_tsne.loc[ df_aba_c.loc[in_data22, 'sample_name'], 'tsne_1']
tsne2tp2 = df_aba_tsne.loc[ df_aba_c.loc[in_data22, 'sample_name'], 'tsne_2']

In [ ]:
X_train0

In [ ]:
#
#   All synapses : cortex?
#
#

import matplotlib.pyplot as plt

gray = np.array([200, 200, 200, 1])/255
colas = np.array([[1,1,0],[0,0,1],[0,1,0]])
for fff in range(4):
    
    plot_gaba = (fff==0)|(fff==2)

    # in_data = df_aba_c.loc[:,'class_label'].isin(['GABAergic', 'Glutamatergic'])
    # in_data = in_data&df_aba_c.loc[:,'region_label'].isin(['VISp','ENTl','ENTm','HIP'])
    # tsne1 = df_aba_tsne.loc[ df_aba_c.loc[in_data, 'sample_name'], 'tsne_1']
    # tsne2 = df_aba_tsne.loc[ df_aba_c.loc[in_data, 'sample_name'], 'tsne_2']
    # c = df_aba_c.loc[in_data, 'subclass_color']

    fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
    # plt.scatter(tsne1.values,tsne2.values,
    #                 edgecolors='none', c=c, s=10, alpha=0.1)

    # sbcls = set(df_aba_c.loc[in_data, 'subclass_label'])
    # for tp in sbcls:
    #     in_data2 = in_data&df_aba_c.loc[:,'subclass_label'].isin([tp])
    #     tsne1tp = df_aba_tsne.loc[ df_aba_c.loc[in_data2, 'sample_name'], 'tsne_1']
    #     tsne2tp = df_aba_tsne.loc[ df_aba_c.loc[in_data2, 'sample_name'], 'tsne_2']
    #     plt.text(tsne1tp.median(),tsne2tp.median(),tp,fontsize=12)



    X_train1 = X_train0.loc[X_train0.loc[:,'samples_pre'].isna()==False,:]
    sampr = X_train1.loc[:,'samples_pre']

    ##df_aba18_to_19 = pd.read_excel('aba18_in_aba19_presynaptic.xlsx')
    #sampr19 = df_aba18_to_19_2.loc[df_aba18_to_19_2.loc[:,'sample_name'].isin(sampr),'sample_name_19']
    df_aba18_to_19_3 = df_aba18_to_19_2.set_index('sample_name')
    sampr19 = df_aba18_to_19_3.loc[sampr,'sample_name_19']
    sampr190 = sampr19.copy()
    sampr19 = sampr19.loc[sampr19.isna()==False ] 



    #sampr18 = df_aba18_to_19_2.loc[df_aba18_to_19_2.loc[:,'sample_name'].isin(sampr),'cluster']
    #sampr19 = df_cl18_to_cl19[sampr18]
    #sampr19 = 
    #### restart here!!!

    tsne1pr = df_aba_tsne.loc[ sampr19, 'tsne_1']
    tsne2pr = df_aba_tsne.loc[ sampr19, 'tsne_2']
    plt.scatter(tsne1pr.values,tsne2pr.values,
                    edgecolors='none', c='gray', s=15, alpha=0.4)
    #X_train0.loc[:,annot_columns_train + list(ge_npr) + cla_n]


    X_train2 = X_train0.loc[X_train0.loc[:,'samples_post'].isna()==False,:]
    sampo = X_train2.loc[:,'samples_post']


    ##df_aba18_to_19 = pd.read_excel('aba18_in_aba19_postsynaptic.xlsx')
    #sampo19 = df_aba18_to_19_2.loc[df_aba18_to_19_2.loc[:,'sample_name'].isin(sampo),'sample_name_19']
    sampo19 = df_aba18_to_19_3.loc[sampo,'sample_name_19']
    sampo190 = sampo19.copy()
    sampo19 = sampo19.loc[sampo19.isna()==False ] 

    tsne1po = df_aba_tsne.loc[ sampo19, 'tsne_1']
    tsne2po = df_aba_tsne.loc[ sampo19, 'tsne_2']
    plt.scatter(tsne1po.values,tsne2po.values,
                    edgecolors='none', c='gray', s=15, alpha=0.4)


    #### SYNAPSES
    do_show_all_synapses=0
    if do_show_all_synapses==1:
        x = pd.concat([tsne1pr,tsne1po]).T
        y = pd.concat([tsne2pr,tsne2po]).T
        plt.plot(x.values,y.values, color=gray, linewidth=0.2, alpha=0.5)

    # #### SYNAPSES2
    #do_laptop=0
    if do_laptop:
        d4 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/'
    else:
        d4='~/Documents/references/transcriptomes to STP/transcriptomes/ABA/mouse_VISp_gene_expression_matrices_2018-06-14/'

    fn='mouse_VISp_2018-06-14_samples-columns.csv'
    df_aba_vis_c=pd.read_csv(d4+fn)

    in_data = df_aba_vis_c.loc[:,'class'].isin(['GABAergic', 'Glutamatergic'])
    #in_data = in_data&df_aba_c.loc[:,'region_label'].isin(['VISp','ENTl','ENTm','HIP'])

    # df_aba_c2 = df_aba_c.loc[in_data,:].set_index('sample_name')
    # df_aba_c3 = df_aba_c2.loc[sampr19,:]

    # df_aba18_to_19_4 = df_aba18_to_19_2.set_index('sample_name_19')
    # X_train0pr = X_train0.set_index('samples_pre').loc[:,'samples_post']
    # X_train0pr = df_aba18_to_19_3.loc[X_train0pr,'sample_name_19']
    # X_train0pr = X_train0pr.loc[df_aba18_to_19_4.loc[sampr19,'sample_name']]
    # X_train0pr.index =  sampr19

    df_aba_vis_c2 = df_aba_vis_c.set_index('sample_name')
    df_aba_c2  =df_aba_c.set_index('cluster_label')
    #L5 IT_1
    #L5 IT ALM Cpa6 Gpr88


    clu1 = df_aba_vis_c2.loc[sampr, 'cluster']
    clu1 = clu1[clu1.isna()==False]

    dif = list(set(clu1).difference(set(df_cl18_to_cl19.index)))
    dif19 = [ 'L6 IT_5','L6 IT_5','Sst_16','Vip_8','Vip_9','Vip_11']
    ddd=pd.DataFrame([dif19,dif]).T.set_index(1)
    df_cl18_to_cl19_2 = pd.concat([df_cl18_to_cl19,ddd]) 
    df_cl18_to_cl19_2.loc['L5 IT ALM Cpa6 Gpr88',0]='L5 IT_1'
    df_cl18_to_cl19_2.loc['Vip Igfbp4 Mab21l1',0]='Vip_3'
    df_cl18_to_cl19_2.loc['Vip Igfbp6 Car10',0]='Vip_3'
    df_cl18_to_cl19_2.loc['Vip Rspo1 Itga4',0]='Vip_12'
    df_cl18_to_cl19_2.loc['L6 IT ALM Tgfb1',0]='L6 IT_5'
    df_cl18_to_cl19_2.loc['Vip Crispld2 Htr2c',0]='Vip_8'
    df_cl18_to_cl19_2.loc['Sst Crh 4930553C11Rik ',0]='Sst_20'
    df_cl18_to_cl19_2.loc['L6b Hsd17b2',0]='L6b_5'
    #Vip Crispld2 Kcne4
    #Sst Crh 4930553C11Rik 
    #L6b Hsd17b2
 



    clu1_19 = df_cl18_to_cl19_2.loc[clu1,:]


    sbcls1 = set(df_aba_c.loc[df_aba_c.loc[:,'cluster_label'].isin(clu1_19.loc[:,0]), 'subclass_label'])
    clu1_19_ =clu1_19.drop_duplicates()
    sbs119=df_aba_c2.loc[clu1_19_.loc[:,0], 'subclass_label'].reset_index().drop_duplicates(['cluster_label']).set_index('cluster_label')
    sbs119 = sbs119.loc[clu1_19.loc[:,0]]


    clu2 = df_aba_vis_c2.loc[sampo, 'cluster']
    clu2 = clu2[clu2.isna()==False]
    clu2_19 = df_cl18_to_cl19_2.loc[clu2,:]
    sbcls2 = set(df_aba_c.loc[df_aba_c.loc[:,'cluster_label'].isin(clu2_19.loc[:,0]), 'subclass_label'])


    clu2_19_ =clu2_19.drop_duplicates()
    sbs219=df_aba_c2.loc[clu2_19_.loc[:,0], 'subclass_label'].reset_index().drop_duplicates(['cluster_label']).set_index('cluster_label')
    sbs219 = sbs219.loc[clu2_19.loc[:,0]]

    c_sb_18_19 = pd.concat([sampr.loc[0:10599],sampo.loc[0:10599],clu1_19.reset_index(),clu2_19.reset_index(),
                            sbs119.reset_index(),sbs219.reset_index()],axis=1)
    c_sb_18_19.columns = ['samples_pre', 'samples_post' ,'cluster_pre','cluster_pre_19'
                          ,'cluster_post','cluster_post_19',
                          'cluster_label_', 'subclass_label_pre_19','cluster_label_p','subclass_label_post_19']

    # df_aba_c2 = df_aba_c.loc[in_data,:].set_index('sample_name')
    # df_aba_c3 = df_aba_c2.loc[sampr,:]
    # X_train0pr = X_train0.set_index('samples_pre')

    # in_data31 = in_data&df_aba_c.loc[:,'sample_name'].isin(sampr)
    # sbcls1 = set(df_aba_c.loc[in_data31, 'subclass_label'])

    # in_data32 = in_data&df_aba_c.loc[:,'sample_name'].isin(sampo)
    # sbcls2 = set(df_aba_c.loc[in_data32, 'subclass_label'])

    Nbootstrap = 200
    Nst = int(c_sb_18_19.shape[0]/Nbootstrap)
    for ii in range(Nst):
        stii = c_sb_18_19.loc[ii*Nbootstrap:(ii+1)*Nbootstrap-1,:]
        scstpr  = stii.loc[:,'subclass_label_pre_19'].value_counts().index[0]
        scstpo  = stii.loc[:,'subclass_label_post_19'].value_counts().index[0]

        in_data21 = df_aba_c.loc[:,'subclass_label'].isin([scstpr])
        tsne1tp1 = df_aba_tsne.loc[ df_aba_c.loc[in_data21, 'sample_name'], 'tsne_1']
        tsne2tp1 = df_aba_tsne.loc[ df_aba_c.loc[in_data21, 'sample_name'], 'tsne_2']

        in_data22 = df_aba_c.loc[:,'subclass_label'].isin([scstpo])
        tsne1tp2 = df_aba_tsne.loc[ df_aba_c.loc[in_data22, 'sample_name'], 'tsne_1']
        tsne2tp2 = df_aba_tsne.loc[ df_aba_c.loc[in_data22, 'sample_name'], 'tsne_2']

        if df_aba_c.loc[in_data21, 'class_label'].iloc[0]=='GABAergic':
            co1='k'
            lw1 = 5
            is_gaba = True
            
        else:
            co1='k'
            lw1 = 5
            is_gaba = False

        if fff>1:
            #col = colas[df_Stp_type_p_c.loc[ii,'type'].astype(int)-1,:]  
            stype=df_Stp_type_p_c.loc[ii,'type'] #.astype(int)
            if stype==1:
                co1='y'
            if stype==2:
                co1='b'
            if stype==3:
                col='g'
            #print(ii,df_Stp_type_p_c.loc[ii,:],col)
        
        x = np.array([tsne1tp1.median(),tsne1tp2.median()])
        y = np.array([tsne2tp1.median(),tsne2tp2.median()])
        if (is_gaba == plot_gaba):
            #print(col)
            plt.plot(x,y, color=co1, linewidth=lw1, alpha=0.2)
            #plt.plot(x,y, color=co1, linewidth=lw1)
            
         
        #plt.text(tsne1tp.median(),tsne2tp.median(),tp,fontsize=18)    


    # for tp1 in sbcls1:
    #     in_data21 = df_aba_c.loc[:,'subclass_label'].isin([tp1])
    #     tsne1tp1 = df_aba_tsne.loc[ df_aba_c.loc[in_data21, 'sample_name'], 'tsne_1']
    #     tsne2tp1 = df_aba_tsne.loc[ df_aba_c.loc[in_data21, 'sample_name'], 'tsne_2']
    #     n1 = tsne1tp1.shape[0]

    #     if df_aba_c.loc[in_data21, 'class_label'].iloc[0]=='GABAergic':
    #         co1='g'
    #         lw1 = 8
    #     else:
    #         co1='r'
    #         lw1 = 4

    #     df_aba_c31 = c_sb_18_19.loc[c_sb_18_19.loc[:,'subclass_label_pre_19'].isin([tp1]),:]

    #     #df_aba_c31po = df_aba_c31.loc[:,'samples_post']
    #     n30=df_aba_c31.shape[0]
    #     #print(tp1 + '  '+str([n30, n1]))
    #     if n30>0:
    #         for tp2 in sbcls2:
    #             in_data22 = df_aba_c.loc[:,'subclass_label'].isin([tp2])
    #             tsne1tp2 = df_aba_tsne.loc[ df_aba_c.loc[in_data22, 'sample_name'], 'tsne_1']
    #             tsne2tp2 = df_aba_tsne.loc[ df_aba_c.loc[in_data22, 'sample_name'], 'tsne_2']
    #             n2 = tsne1tp2.shape[0]



    #             #df_aba_c32 = df_aba_c2.loc[df_aba_c31po,:]
    #             df_aba_c32tp2 = df_aba_c31.loc[df_aba_c31.loc[:,'subclass_label_post_19'].isin([tp2]),:]
    #             n3 = df_aba_c32tp2.shape[0]
    #             #print(tp1 + '  '+tp2+'  ' +str([n3, n30, n1, n2]))
    #             if n3>0:
    #                 x = np.array([tsne1tp1.median(),tsne1tp2.median()])
    #                 y = np.array([tsne2tp1.median(),tsne2tp2.median()])
    #                 plt.plot(x,y, color=co1, linewidth=lw1, alpha=0.2)
    #                 #plt.text(tsne1tp.median(),tsne2tp.median(),tp,fontsize=18)


    #### SYNAPSES3
    #import plotly.graph_objects as go


    do_synapses3 = 1
    label =[]
    x = []
    y = []
    source = []
    target =[]
    value = []
    nl=0
    if do_synapses3==1:
        Nbootstrap = 200
        Nst = int(c_sb_18_19.shape[0]/200)
        for ii in range(Nst):
            stii = c_sb_18_19.loc[ii*Nbootstrap:(ii+1)*Nbootstrap-1,:]
            scstpr  = stii.loc[:,'subclass_label_pre_19'].value_counts().index[0]
            scstpo  = stii.loc[:,'subclass_label_post_19'].value_counts().index[0]

            in_data21 = df_aba_c.loc[:,'subclass_label'].isin([scstpr])
            tsne1tp1 = df_aba_tsne.loc[ df_aba_c.loc[in_data21, 'sample_name'], 'tsne_1']
            tsne2tp1 = df_aba_tsne.loc[ df_aba_c.loc[in_data21, 'sample_name'], 'tsne_2']

            in_data22 = df_aba_c.loc[:,'subclass_label'].isin([scstpo])
            tsne1tp2 = df_aba_tsne.loc[ df_aba_c.loc[in_data22, 'sample_name'], 'tsne_1']
            tsne2tp2 = df_aba_tsne.loc[ df_aba_c.loc[in_data22, 'sample_name'], 'tsne_2']

            if df_aba_c.loc[in_data21, 'class_label'].iloc[0]=='GABAergic':
                co1='k'
                lw1 = 8
                is_gaba = True
            else:
                co1='k'
                lw1 = 4
                is_gaba = False

            #x = np.array([tsne1tp1.median(),tsne1tp2.median()])
            #y = np.array([tsne2tp1.median(),tsne2tp2.median()])
            #plt.plot(x,y, color=co1, linewidth=lw1, alpha=0.4)
            ##plt.text(tsne1tp.median(),tsne2tp.median(),tp,fontsize=18)  

            tp1 = scstpr
            tp2 = scstpo
            if len(label)==0:

                fl1=True
            else:
                fl1 = sum(np.array(label)==tp1)==0

            if fl1:
                nl=nl+1
                nl1 = np.nonzero(np.array(label)==tp1)[0]
                label = label + [tp1]
                x = x + [tsne1tp1.median()]
                y = y + [tsne2tp1.median()]


            if sum(np.array(label)==tp2)==0:
                nl=nl+1
                nl2 = np.nonzero(np.array(label)==tp2)[0]
                label = label + [tp2]
                x = x + [ tsne1tp2.median()]
                y = y + [ tsne2tp2.median()]

            nl1 = np.nonzero(np.array(label)==tp1)[0]
            nl2 = np.nonzero(np.array(label)==tp2)[0]

            if x[nl1[0]]<=x[nl2[0]]:
                source = source + [nl1[0]]
                target = target + [nl2[0]]
            else:
                source = source + [nl2[0]]
                target = target + [nl1[0]]
            value = value + [0.5]




        x = np.array(x)
        x = (x - x.min())
        x = x/x.max()/1.1 + 0.01
        x=x.tolist()
        y = np.array(y)
        y = (y - y.min())
        y = y/y.max()/1.1 + 0.01
        y=y.tolist()

    #     fig = go.Figure()
    #     fig.add_trace(go.Sankey(
    #         arrangement = "snap",
    #         node = dict(
    #             label= label,
    #              x= x,
    #              y= y,
    #              pad = 0.1,
    #              thickness = 10,
    #              line = dict(color = "black", width = 0.1)),  # 10 Pixels 
    #         link = dict(
    #             source= source,
    #             target= target,
    #             value= value)))
    #     fig.update_layout( font_size=20)
    #     fig.show()







    #### TSNE 
    in_data = df_aba_c.loc[:,'class_label'].isin(['GABAergic', 'Glutamatergic'])
    in_data = in_data&df_aba_c.loc[:,'region_label'].isin(['VISp','ENTl','ENTm','HIP'])
    tsne1 = df_aba_tsne.loc[ df_aba_c.loc[in_data, 'sample_name'], 'tsne_1']
    tsne2 = df_aba_tsne.loc[ df_aba_c.loc[in_data, 'sample_name'], 'tsne_2']
    c = df_aba_c.loc[in_data, 'subclass_color']
    plt.scatter(tsne1.values,tsne2.values,
                    edgecolors='none', c=c, s=7, alpha=0.1)

    #### Subclasses names
    in_data3 = in_data&df_aba_c.loc[:,'sample_name'].isin(sampr19.append(sampo19))
    sbcls = set(df_aba_c.loc[in_data3, 'subclass_label'])
    for tp in sbcls:
        dy=0
        if tp=='Meis2':
            dy=-2
        dx=0
        if tp=='Pvalb':
            dx=+7
        if tp=='Sst':
            dx=+2   
        if tp=='L4 IT':
            dx=+4     
        in_data2 = in_data&df_aba_c.loc[:,'subclass_label'].isin([tp])
        tsne1tp = df_aba_tsne.loc[ df_aba_c.loc[in_data2, 'sample_name'], 'tsne_1']
        tsne2tp = df_aba_tsne.loc[ df_aba_c.loc[in_data2, 'sample_name'], 'tsne_2']
        plt.text(tsne1tp.median()+dx,tsne2tp.median()+dy,tp,fontsize=22,c=np.array([50, 50, 50])/255)




    plt.axis('off')
    #fig.set_tight_layout(True)


In [ ]:
# ABA 2020 10x from site

d_aba2020= '/Users/stepaniu/Documents/jan_2020/ABA_2020/'
df_tsne20 = pd.read_csv(d_aba2020+'tsne.csv')
df_meta20=pd.read_hdf('metadata_aba2020.hdf')
df_meta20

In [ ]:
#df_meta20.loc[:10,:].to_excel('meta_2020.xlsx')

In [ ]:
df_meta20.columns

In [ ]:
print(set(df_meta20.loc[:,'platform_label']))
print(df_meta20.shape)
print(set(df_meta20.loc[:,'cortical_layer_label']))
print(set(df_meta20.loc[:,'region_label']))

In [ ]:
# ABA 2020 10x from U19 Zeng site http://data.nemoarchive.org/biccn/lab/zeng/transcriptome/scell/10X/processed/YaoHippo2020/
# 1093036 cells CTX and HIPP without layers
d_aba2020= '/Users/stepaniu/Documents/jan_2020/ABA_2020/Zeng_Tasic_2020_10x_CTX_HIPP/'
df_tsne20x = pd.read_csv(d_aba2020+'CTX_Hip_umap.2d.10x.csv')
#df_meta20=pd.read_hdf('metadata_aba2020.hdf')
df_meta20x=pd.read_csv(d_aba2020 + 'CTX_Hip_anno_10x.v2.csv')
df_meta20x

In [ ]:
df_meta20x.columns

In [ ]:
print(set(df_meta20x.loc[:,'platform_label']))
print(df_meta20x.shape)
print(set(df_meta20x.loc[:,'cortical_layer_label']))
print(set(df_meta20x.loc[:,'region_label']))

In [ ]:
# ABA 2020 Smart_seq from AIBS site - 74973 cells CTX and HIPP
d_aba2020= '/Users/stepaniu/Documents/jan_2020/ABA_2020/Zeng_Tasic_YaoHippo2020_SmartSeq_CTX_HIPP/'
df_tsne20ss = pd.read_csv(d_aba2020+'CTX_Hip_umap.2d.SS.csv')
#df_meta20=pd.read_hdf('metadata_aba2020.hdf')
df_meta20ss=pd.read_csv(d_aba2020 + 'CTX_Hip_anno_SS.v2.csv')
df_meta20ss

In [ ]:
df_meta20ss.columns

In [ ]:
print(set(df_meta20ss.loc[:,'platform_label']))
print(df_meta20ss.shape)
print(set(df_meta20ss.loc[:,'cortical_layer_label']))
print(set(df_meta20ss.loc[:,'region_label']))

In [ ]:
#X_train0.iloc[0:-1:200,:].loc[:,annot_columns].to_excel('temp.xlsx')

In [ ]:
#
#   All synapses : hippocampus
#
#

import matplotlib.pyplot as plt

gray = np.array([200, 200, 200, 1])/255

for fff in range(4):
    
    plot_gaba = (fff==0)|(fff==2)

    fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')

    ### PRESYNAPTIC CELLS
    X_train1 = X_train0.loc[X_train0.loc[:,'samples_pre'].isna()==False,:]
    sampr = X_train1.loc[:,'samples_pre']

    #df_aba18_to_19 = pd.read_excel('aba18_in_aba19_presynaptic.xlsx')
    #sampr19 = df_aba18_to_19.loc[df_aba18_to_19.loc[:,'sample_name'].isin(sampr),'sample_name_19']

    tsne1pr = df_aba_tsne.loc[ sampr, 'tsne_1']
    tsne2pr = df_aba_tsne.loc[ sampr, 'tsne_2']
    plt.scatter(tsne1pr.values,tsne2pr.values,
                    edgecolors='none', c='gray', s=15, alpha=0.4)
    #X_train0.loc[:,annot_columns_train + list(ge_npr) + cla_n]

    ### POSTSYNAPTIC CELLS
    X_train2 = X_train0.loc[X_train0.loc[:,'samples_post'].isna()==False,:]
    sampo = X_train2.loc[:,'samples_post']

    #df_aba18_to_19 = pd.read_excel('aba18_in_aba19_postsynaptic.xlsx')
    #sampo19 = df_aba18_to_19.loc[df_aba18_to_19.loc[:,'sample_name'].isin(sampo),'sample_name_19']

    tsne1po = df_aba_tsne.loc[ sampo, 'tsne_1']
    tsne2po = df_aba_tsne.loc[ sampo, 'tsne_2']
    plt.scatter(tsne1po.values,tsne2po.values,
                    edgecolors='none', c='gray', s=4, alpha=0.4)


    #### SYNAPSES
    do_show_all_synapses=0
    if do_show_all_synapses==1:
        x = pd.concat([tsne1pr,tsne1po]).T
        y = pd.concat([tsne2pr,tsne2po]).T
        plt.plot(x.values,y.values, color=gray, linewidth=0.2, alpha=0.5)

    #### SYNAPSES2
    df_aba_c2 = df_aba_c.loc[in_data,:].set_index('sample_name')
    df_aba_c3 = df_aba_c2.loc[sampr,:]
    X_train0pr = X_train0.reset_index().set_index('samples_pre')

    in_data31 = in_data&df_aba_c.loc[:,'sample_name'].isin(sampr)
    sbcls1 = set(df_aba_c.loc[in_data31, 'subclass_label'])

    in_data32 = in_data&df_aba_c.loc[:,'sample_name'].isin(sampo)
    sbcls2 = set(df_aba_c.loc[in_data32, 'subclass_label'])
    do_synapses2=1
    if do_synapses2==1:
        for tp1 in sbcls1:
            in_data21 = in_data&df_aba_c.loc[:,'subclass_label'].isin([tp1])
            tsne1tp1 = df_aba_tsne.loc[ df_aba_c.loc[in_data21, 'sample_name'], 'tsne_1']
            tsne2tp1 = df_aba_tsne.loc[ df_aba_c.loc[in_data21, 'sample_name'], 'tsne_2']
            n1 = tsne1tp1.shape[0]

            if df_aba_c.loc[in_data21, 'class_label'].iloc[0]=='GABAergic':
                co1='k'
                lw1 = 5
                is_gaba=True
            else:
                co1='k'
                lw1 = 5
                is_gaba=False
                
    

            df_aba_c31 = df_aba_c3.loc[df_aba_c3.loc[:,'subclass_label'].isin([tp1]),:]
            df_aba_c31po = X_train0pr.loc[df_aba_c31.index,'samples_post']
            df_aba_c31poall = X_train0pr.loc[df_aba_c31.index,:]
            n30=df_aba_c31po.shape[0]
            #print(tp1 + '  '+str([n30, n1]))
            if n30>0:
                for tp2 in sbcls2:
                    in_data22 = in_data&df_aba_c.loc[:,'subclass_label'].isin([tp2])
                    tsne1tp2 = df_aba_tsne.loc[ df_aba_c.loc[in_data22, 'sample_name'], 'tsne_1']
                    tsne2tp2 = df_aba_tsne.loc[ df_aba_c.loc[in_data22, 'sample_name'], 'tsne_2']
                    n2 = tsne1tp2.shape[0]



                    df_aba_c32 = df_aba_c2.loc[df_aba_c31po,:]
                    df_aba_c32tp2 = df_aba_c32.loc[df_aba_c32.loc[:,'subclass_label'].isin([tp2]),:]
                    n3 = df_aba_c32tp2.shape[0]
                    #print(tp1 + '  '+tp2+'  ' +str([n3, n30, n1, n2]))
                    #df_aba_c32tp2.index - all tp2 postsynaptic samples
                    
                    
                    
                    if n3>0:
                        if fff>1:
                            idx=df_aba_c31poall.loc[df_aba_c31po.isin(df_aba_c32tp2.index),'index']
                            ii =int(idx.iloc[0]/Nbootstrap)
                            #col = colas[df_Stp_type_p_c.loc[ii,'type'].astype(int)-1,:]  
                            stype=df_Stp_type_p.loc[ii,'type'] #.astype(int)
                            if stype==1:
                                co1='y'
                            if stype==2:
                                co1='b'
                            if stype==3:
                                col='g'
                            print(ii,df_Stp_type_p.loc[ii,:],col,tp1,tp2)
                        
                        x = np.array([tsne1tp1.median(),tsne1tp2.median()])
                        y = np.array([tsne2tp1.median(),tsne2tp2.median()])
                        if is_gaba==plot_gaba:
                            plt.plot(x,y, color=co1, linewidth=lw1, alpha=0.2)
                            #plt.plot(x,y, color=co1, linewidth=lw1)

                        #plt.text(tsne1tp.median(),tsne2tp.median(),tp,fontsize=18)

    #### SYNAPSES3
    import plotly.graph_objects as go


    do_synapses3 = 1
    label =[]
    x = []
    y = []
    source = []
    target =[]
    value = []
    nl=0
    if do_synapses3==1:
        for tp1 in sbcls1:
            in_data21 = in_data&df_aba_c.loc[:,'subclass_label'].isin([tp1])
            tsne1tp1 = df_aba_tsne.loc[ df_aba_c.loc[in_data21, 'sample_name'], 'tsne_1']
            tsne2tp1 = df_aba_tsne.loc[ df_aba_c.loc[in_data21, 'sample_name'], 'tsne_2']
            n1 = tsne1tp1.shape[0]

            if df_aba_c.loc[in_data21, 'class_label'].iloc[0]=='GABAergic':
                co1='g'
                lw1 = 8
            else:
                co1='r'
                lw1 = 4
                
                

            df_aba_c31 = df_aba_c3.loc[df_aba_c3.loc[:,'subclass_label'].isin([tp1]),:]
            df_aba_c31po = X_train0pr.loc[df_aba_c31.index,'samples_post']
            n30=df_aba_c31po.shape[0]
            #print(tp1 + '  '+str([n30, n1]))
            if n30>0:
                for tp2 in sbcls2:
                    in_data22 = in_data&df_aba_c.loc[:,'subclass_label'].isin([tp2])
                    tsne1tp2 = df_aba_tsne.loc[ df_aba_c.loc[in_data22, 'sample_name'], 'tsne_1']
                    tsne2tp2 = df_aba_tsne.loc[ df_aba_c.loc[in_data22, 'sample_name'], 'tsne_2']
                    n2 = tsne1tp2.shape[0]



                    df_aba_c32 = df_aba_c2.loc[df_aba_c31po,:]
                    df_aba_c32tp2 = df_aba_c32.loc[df_aba_c32.loc[:,'subclass_label'].isin([tp2]),:]
                    n3 = df_aba_c32tp2.shape[0]
                    #print(tp1 + '  '+tp2+'  ' +str([n3, n30, n1, n2]))
                    if n3>0:


                        if len(label)==0:

                            fl1=True
                        else:
                            fl1 = sum(np.array(label)==tp1)==0

                        if fl1:
                            nl=nl+1
                            nl1 = np.nonzero(np.array(label)==tp1)[0]
                            label = label + [tp1]
                            x = x + [tsne1tp1.median()]
                            y = y + [tsne2tp1.median()]


                        if sum(np.array(label)==tp2)==0:
                            nl=nl+1
                            nl2 = np.nonzero(np.array(label)==tp2)[0]
                            label = label + [tp2]
                            x = x + [ tsne1tp2.median()]
                            y = y + [ tsne2tp2.median()]

                        nl1 = np.nonzero(np.array(label)==tp1)[0]
                        nl2 = np.nonzero(np.array(label)==tp2)[0]

                        if x[nl1[0]]<=x[nl2[0]]:
                            source = source + [nl1[0]]
                            target = target + [nl2[0]]
                        else:
                            source = source + [nl2[0]]
                            target = target + [nl1[0]]
                        value = value + [0.1]

                        #x = np.array([tsne1tp1.median(),tsne1tp2.median()])
                        #y = np.array([tsne2tp1.median(),tsne2tp2.median()])
                        #plt.plot(x,y, color=co1, linewidth=lw1, alpha=0.2)
                        ##plt.text(tsne1tp.median(),tsne2tp.median(),tp,fontsize=18)


        x = np.array(x)
        x = (x - x.min())
        x = x/x.max()/1.0 + 0.01
        x=x.tolist()
        y = np.array(y)
        y = (y - y.min())
        y = y/y.max()/1.0 + 0.01
        y=y.tolist()

    #     fig = go.Figure()
    #     fig.add_trace(go.Sankey(
    #         arrangement = "snap",
    #         node = dict(
    #             label= label,
    #              x= x,
    #              y= y,
    #              pad = 0.1,
    #              thickness = 10,
    #              line = dict(color = "black", width = 0.05)),  # 10 Pixels 
    #         link = dict(
    #             source= source,
    #             target= target,
    #             value= value)))
    #     fig.update_layout( font_size=20)
    #     fig.show()





    #### TSNE 
    in_data = df_aba_c.loc[:,'class_label'].isin(['GABAergic', 'Glutamatergic'])
    in_data = in_data&df_aba_c.loc[:,'region_label'].isin(['VISp','ENTl','ENTm','HIP'])
    tsne1 = df_aba_tsne.loc[ df_aba_c.loc[in_data, 'sample_name'], 'tsne_1']
    tsne2 = df_aba_tsne.loc[ df_aba_c.loc[in_data, 'sample_name'], 'tsne_2']
    c = df_aba_c.loc[in_data, 'subclass_color']
    plt.scatter(tsne1.values,tsne2.values,
                    edgecolors='none', c=c, s=7, alpha=0.1)

    #### Subclasses names
    in_data3 = in_data&df_aba_c.loc[:,'sample_name'].isin(sampr.append(sampo))
    sbcls = set(df_aba_c.loc[in_data3, 'subclass_label'])
    for tp in sbcls:
        in_data2 = in_data&df_aba_c.loc[:,'subclass_label'].isin([tp])
        tsne1tp = df_aba_tsne.loc[ df_aba_c.loc[in_data2, 'sample_name'], 'tsne_1']
        tsne2tp = df_aba_tsne.loc[ df_aba_c.loc[in_data2, 'sample_name'], 'tsne_2']
        plt.text(tsne1tp.median(),tsne2tp.median(),tp,fontsize=22)



    plt.axis('off')
    #fig.set_tight_layout(True)


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Sankey(
    arrangement = "snap",
    node = {
        "label": ["A", "B", "C", "D", "E", "F"],
        "x": [0.2, 0.1, 0.5, 0.7, 0.3, 0.5],
        "y": [0.7, 0.5, 0.2, 0.4, 0.2, 0.3],
        'pad':10},  # 10 Pixels 
    link = {
        "source": [0, 0, 1, 2, 5, 4, 3, 5],
        "target": [5, 3, 4, 3, 0, 2, 2, 3],
        "value": [1, 2, 1, 1, 1, 1, 1, 2]}))

fig.show()

In [ ]:
X_train0pr = X_train0.reset_index().set_index('samples_pre')

In [ ]:
X_train0pr

In [ ]:
 df_aba_c.loc[in_data22, 'class_label'].iloc[0]

In [ ]:
sbcls2

In [ ]:
yy=y_train[:,nannot:]
y_u = UMAP(spread=2).fit_transform(yy)

from sklearn.manifold import TSNE

y_u2 = TSNE(n_components=2).fit_transform(yy)

fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
plt.scatter(y_u[:, 0], y_u[:, 1],
                edgecolors='none', c="#ff4400", s=10)

plt.axis('off')
fig.set_tight_layout(True)

fig1 = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
plt.scatter(y_u2[:, 0], y_u2[:, 1],
                edgecolors='none', c="#ff4400", s=10)

plt.axis('off')
fig1.set_tight_layout(True)


In [ ]:
# to do:
#
#
# 1) visualize scVI latent space? - a. distances betwean synapses types b. paga? 
#
#
# 2) kernels - rbf will not be good?
# train SVR in scVI space - a) factors b) imputed ge
#
# 3) RF with distance penalty?
#
# 4) test other classifiers in scVI space

In [ ]:
# DISTANCES
from sklearn import metrics

##lyamba_post = 0.5
# ['all', 'ex_inh']
Dn = 5
Dn2 = int(100/Dn)
#cla_n = ['ex_in_ex_in__pre',
#         'ex_in_ex_in__post'] 

#cla_n = ['ex_in_ex_in__pre'] 
cla_n = ['vse__pre','vse__post']

#ge_n = imp50.index[0:25].tolist()
ge_n = lf_scvi_names #ge_columns_train2
ge_n2=np.array(ge_n)
ge_npr = ge_n2[np.char.rfind(ge_n2 , 'pre__')>-1]
ge_nps = ge_n2[np.char.rfind(ge_n2 , 'post__')>-1]

nannot = len(annot_columns_train)
lge_n  = len(ge_n)
lge_npr  = len(ge_npr)
lge_nps  = len(ge_nps)


X2pr = X_train0.loc[:,annot_columns_train + list(ge_npr) + cla_n]
X2ps = X_train0.loc[:,annot_columns_train + list(ge_nps) + cla_n]

if len(cla_n)>0:
    i_clpr = np.nonzero(X2pr.columns.isin(cla_n))[0]  - nannot
    i_clps = np.nonzero(X2pr.columns.isin(cla_n))[0]  - nannot
    i_cl2pr = np.nonzero(X2pr.columns.isin(cla_n))[0]  
    i_cl2ps = np.nonzero(X2ps.columns.isin(cla_n))[0]  

    #X2.loc[:,cla_n] = X2.loc[:,cla_n]+1

X2pr=X2pr.iloc[0:X_train0.shape[0]:Dn,:].values
X2ps=X2ps.iloc[0:X_train0.shape[0]:Dn,:].values


# select y
stp_n = ['A2_20Hz']

# 'A2_20Hz',
#  'A3_20Hz',
#  'A4_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A1_50Hz',
#  'A2_50Hz',
#  'A3_50Hz',
#  'A4_50Hz',
#  'A5_50Hz',
#  'A250_50Hz',
#  'A1000_50Hz',
#  'A1_10Hz',
#  'A2_10Hz',
#  'A3_10Hz',
#  'A4_10Hz',
#  'A5_10Hz',
#  'A250_10Hz',
#  'A1000_10Hz']
y2 =  X_train0.loc[:,annot_columns_train+stp_n].iloc[0:X_train0.shape[0]:Dn,:].values

i_Xpr = np.arange(nannot,nannot+lge_npr)
i_Xps = np.arange(nannot,nannot+lge_nps)

#[‘braycurtis’, ‘canberra’, ‘chebyshev’, ‘correlation’, ‘dice’, ‘hamming’, ‘jaccard’,
# ‘kulsinski’, ‘mahalanobis’, ‘minkowski’, ‘rogerstanimoto’, ‘russellrao’, ‘seuclidean’,
# ‘sokalmichener’, ‘sokalsneath’, ‘sqeuclidean’, ‘yule’] 

Dpr = metrics.pairwise_distances(X2pr[:,i_Xpr], metric='sqeuclidean', n_jobs=-1, force_all_finite=True)
Dps = metrics.pairwise_distances(X2ps[:,i_Xps], metric='sqeuclidean', n_jobs=-1, force_all_finite=True)

X2cl = X_train0.loc[:,cla_n]
X2cl = X2cl.iloc[0:X_train0.shape[0]:Dn,:].values
D_mask = np.ones(Dpr.shape)
#D_mask[]

alp=16
D = D_mask*np.sqrt(np.sqrt(alp*Dpr**4 + Dps**4))
D = pd.DataFrame(D, columns=X2pr[:,i_cl2pr], index = X2pr[:,i_cl2pr] )

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(15, 15))
im = ax.imshow(-D)

# We want to show all ticks...
xD = np.arange(0,len(D.columns),Dn2)
yD = np.arange(0,len(D.index),Dn2)
ax.set_xticks(xD)
ax.set_yticks(yD)
# ... and label them with the respective list entries
ax.set_xticklabels(D.columns[xD])
ax.set_yticklabels(D.index[yD])

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
#for i in range(len(hip2aba.index)):
#    for j in range(len(hip2aba.columns)):
#       text = ax.text(j, i, hip2aba[i, j],
#                       ha="center", va="center", color="w")

ax.set_title("scVI latent factors based distances between synapses types")
fig.tight_layout()
plt.show()


In [ ]:
### gs = '5188'

fig, ax = plt.subplots(figsize=(15, 7))
yD2 = np.arange(0,1600,400)
yD2n = D.columns[yD2]
plt.plot(np.arange(D.shape[0]), D.iloc[:,yD2],figure=fig)
# We want to show all ticks...
xD = np.arange(0,len(D.columns),Dn2)

ax.set_xticks(xD)

# ... and label them with the respective list entries
ax.set_xticklabels(D.columns[xD])


# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor")

ax.legend(yD2n)
plt.show()

In [ ]:
### gs = '1512'

fig, ax = plt.subplots(figsize=(15, 7))
yD2 = np.arange(0,1600,400)
yD2n = D.columns[yD2]
plt.plot(np.arange(D.shape[0]), D.iloc[:,yD2],figure=fig)
# We want to show all ticks...
xD = np.arange(0,len(D.columns),Dn2)

ax.set_xticks(xD)

# ... and label them with the respective list entries
ax.set_xticklabels(D.columns[xD])


# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor")

ax.legend(yD2n)
plt.show()

In [ ]:
### gs = '219'

fig, ax = plt.subplots(figsize=(15, 7))
yD2 = np.arange(0,1600,400)
yD2n = D.columns[yD2]
plt.plot(np.arange(D.shape[0]), D.iloc[:,yD2],figure=fig)
# We want to show all ticks...
xD = np.arange(0,len(D.columns),Dn2)

ax.set_xticks(xD)

# ... and label them with the respective list entries
ax.set_xticklabels(D.columns[xD])


# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor")

ax.legend(yD2n)
plt.show()

In [ ]:
import matplotlib
cols=matplotlib.colors.ColorConverter.to_rgba_array(['b','r','#467821','0.4','m','y','purple',
                                                'cyan','#B0E0E6','#8C0900','k','#30a2da','#6d904f','#ccebc4'])

In [ ]:
cols[2,:].reshape((1,-1))

In [ ]:
### gs='15656'

from sklearn.decomposition import PCA

# PCA transformation
pca=PCA(n_components=10)
#X2pr_pca =pca.fit_transform(X2pr[:,i_Xpr])

ge_n = lf_scvi_names  #lf_scvi_names #ge_columns_train2
ge_n2=np.array(ge_n)
ge_npr = ge_n2[np.char.rfind(ge_n2 , 'pre__')>-1]

X2 = X_train0.loc[:,annot_columns_train + list(ge_npr) + cla_n].values
i_X2 = np.arange(nannot,nannot+len(ge_npr))
X2_pca =pca.fit_transform(X2[:,i_X2])

print(pca.explained_variance_ratio_)
print(pca.singular_values_) 

fig, ax = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,1],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    


fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,2],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,3],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

In [ ]:
### gs='5188'

from sklearn.decomposition import PCA

# PCA transformation
pca=PCA(n_components=20)
#X2pr_pca =pca.fit_transform(X2pr[:,i_Xpr])

ge_n = lf_scvi_names  #lf_scvi_names #ge_columns_train2
ge_n2=np.array(ge_n)
ge_npr = ge_n2[np.char.rfind(ge_n2 , 'pre__')>-1]

X2 = X_train0.loc[:,annot_columns_train + list(ge_npr) + cla_n].values
i_X2 = np.arange(nannot,nannot+len(ge_npr))
X2_pca =pca.fit_transform(X2[:,i_X2])

print(pca.explained_variance_ratio_)
print(pca.singular_values_) 

fig, ax = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,1],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    


fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,2],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,3],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

In [ ]:
### gs='1512'

from sklearn.decomposition import PCA

# PCA transformation
pca=PCA(n_components=10)
#X2pr_pca =pca.fit_transform(X2pr[:,i_Xpr])

ge_n = lf_scvi_names  #lf_scvi_names #ge_columns_train2
ge_n2=np.array(ge_n)
ge_npr = ge_n2[np.char.rfind(ge_n2 , 'pre__')>-1]

X2 = X_train0.loc[:,annot_columns_train + list(ge_npr) + cla_n].values
i_X2 = np.arange(nannot,nannot+len(ge_npr))
X2_pca =pca.fit_transform(X2[:,i_X2])

print(pca.explained_variance_ratio_)
print(pca.singular_values_) 

fig, ax = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,1],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    


fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,2],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,3],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

In [ ]:
from sklearn.decomposition import PCA

# PCA transformation
pca=PCA(n_components=10)
#X2pr_pca =pca.fit_transform(X2pr[:,i_Xpr])

ge_n = lf_scvi_names  #lf_scvi_names #ge_columns_train2
ge_n2=np.array(ge_n)
ge_npr = ge_n2[np.char.rfind(ge_n2 , 'pre__')>-1]

X2 = X_train0.loc[:,annot_columns_train + list(ge_npr) + cla_n].values
i_X2 = np.arange(nannot,nannot+len(ge_npr))
X2_pca =pca.fit_transform(X2[:,i_X2])

print(pca.explained_variance_ratio_)
print(pca.singular_values_) 

fig, ax = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,1],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    


fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,2],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,3],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

In [ ]:
from sklearn.decomposition import PCA

# PCA transformation
pca=PCA(n_components=10)
#X2pr_pca =pca.fit_transform(X2pr[:,i_Xpr])

ge_n = lf_scvi_names  #lf_scvi_names #ge_columns_train2
ge_n2=np.array(ge_n)
ge_npr = ge_n2[np.char.rfind(ge_n2 , 'pre__')>-1]

X2 = X_train0.loc[:,annot_columns_train + list(ge_npr) + cla_n].values
i_X2 = np.arange(nannot,nannot+len(ge_npr))
X2_pca =pca.fit_transform(X2[:,i_X2])

print(pca.explained_variance_ratio_)
print(pca.singular_values_) 

fig, ax = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,1],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    


fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,2],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,-2]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,-2]==cii,0],X2_pca[X2[:,-2]==cii,3],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

In [ ]:
from sklearn.decomposition import PCA

# PCA transformation
pca=PCA(n_components=10)
#X2pr_pca =pca.fit_transform(X2pr[:,i_Xpr])

ge_n = lf_scvi_names  #lf_scvi_names #ge_columns_train2
ge_n2=np.array(ge_n)
ge_npr = ge_n2[np.char.rfind(ge_n2 , 'post__')>-1]

X2 = X_train0.loc[:,annot_columns_train + list(ge_npr) + cla_n].values
i_X2 = np.arange(nannot,nannot+len(ge_npr))
X2_pca =pca.fit_transform(X2[:,i_X2])

print(pca.explained_variance_ratio_)
print(pca.singular_values_) 

i_X_post = -1

fig, ax = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,i_X_post]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,i_X_post]==cii,0],X2_pca[X2[:,i_X_post]==cii,1],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    


fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,i_X_post]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,i_X_post]==cii,0],X2_pca[X2[:,i_X_post]==cii,2],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,i_X_post]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,i_X_post]==cii,0],X2_pca[X2[:,i_X_post]==cii,3],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

In [ ]:
from sklearn.decomposition import PCA

# PCA transformation
pca=PCA(n_components=10)
#X2pr_pca =pca.fit_transform(X2pr[:,i_Xpr])

ge_n = lf_scvi_names  #lf_scvi_names #ge_columns_train2
ge_n2=np.array(ge_n)
ge_npr = ge_n2[np.char.rfind(ge_n2 , 'post__')>-1]

X2 = X_train0.loc[:,annot_columns_train + list(ge_npr) + cla_n].values
i_X2 = np.arange(nannot,nannot+len(ge_npr))
X2_pca =pca.fit_transform(X2[:,i_X2])

print(pca.explained_variance_ratio_)
print(pca.singular_values_) 

i_X_post = -1

fig, ax = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,i_X_post]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,i_X_post]==cii,0],X2_pca[X2[:,i_X_post]==cii,1],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    


fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,i_X_post]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,i_X_post]==cii,0],X2_pca[X2[:,i_X_post]==cii,2],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

fig1, ax1 = plt.subplots(figsize=(15, 10))

allcl0 = list(set(X2[:,i_X_post]))
for ii,cii in enumerate(allcl0): 
    plt.scatter(X2_pca[X2[:,i_X_post]==cii,0],X2_pca[X2[:,i_X_post]==cii,3],c=cols[ii,:].reshape((1,-1)))
plt.legend(allcl0)    

# STOP HERE!!!

In [ ]:
import scanpy as sc

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, color_map='Greys')
sc.logging.print_versions()
results_file = 'X_scvi.h5ad'


In [ ]:
import anndata
adt = anndata.AnnData(X2[:,i_X2],obs = X2[:,i_X_post])

In [ ]:
#adt.__dict__

In [ ]:
ge_n = ge_columns_train2  #lf_scvi_names #ge_columns_train2
ge_n2=np.array(ge_n)
ge_npr1 = ge_n2[np.char.rfind(ge_n2 , 'post__')>-1]

X21 = X_train0.loc[:,annot_columns_train + list(ge_npr1) + cla_n].values
i_X21 = np.arange(nannot,nannot+len(ge_npr1))
#X2_pca =pca.fit_transform(X2[:,i_X2])


adt1 = anndata.AnnData(X21[:,i_X21],obs = X21[:,i_X_post])

#sc.pp.filter

In [ ]:
import scanpy.api as sca
sca.pp.filter(adt1,min_cells=3)
#sc.pp.filter(adt1,min_genes=10)


In [ ]:
sc.pp

In [ ]:
dfX2pr = pd.DataFrame(X2pr)
dfX2pr = dfX2pr.set_index([dfX2pr.columns[-1]])
dfX2pr.to_excel('clustering_scvi_pre.xlsx')
dfX2pr

In [ ]:
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    
    return  linkage_matrix







clust = AgglomerativeClustering(n_clusters=None, affinity='euclidean', memory=None, connectivity=None,
                                           compute_full_tree='auto', linkage='ward', distance_threshold=1)

clupr = clust.fit(X2pr[:,i_Xpr])
clups = clust.fit(X2ps[:,i_Xps])

fig, ax = plt.subplots(figsize=(15, 9))
plt.title('Hierarchical Clustering Dendrogram')
# plot the top three levels of the dendrogram
lima = plot_dendrogram(clupr, truncate_mode='level', p=6,ax=ax)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()


In [ ]:
lima.shape
clupr.children_ 
clupr.distances_

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(15, 15))
im = ax.imshow(-D)

# We want to show all ticks...
xD = np.arange(0,len(D.columns),Dn2)
yD = np.arange(0,len(D.index),Dn2)
ax.set_xticks(xD)
ax.set_yticks(yD)
# ... and label them with the respective list entries
ax.set_xticklabels(D.columns[xD])
ax.set_yticklabels(D.index[yD])

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
#for i in range(len(hip2aba.index)):
#    for j in range(len(hip2aba.columns)):
#       text = ax.text(j, i, hip2aba[i, j],
#                       ha="center", va="center", color="w")

ax.set_title("scVI latent factors based distances between synapses types")
fig.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
yD2 = np.arange(0,1600,400)
yD2n = D.columns[yD2]
plt.plot(np.arange(D.shape[0]), D.iloc[:,yD2],figure=fig)
# We want to show all ticks...
xD = np.arange(0,len(D.columns),Dn2)

ax.set_xticks(xD)

# ... and label them with the respective list entries
ax.set_xticklabels(D.columns[xD])


# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor")

ax.legend(yD2n)
plt.show()

# START HERE!!!

### test pyMC3 model

In [ ]:
from matplotlib.patches import Ellipse
import numpy as np
def plot_ellipse(ax,ms,ss):
    #Seaborn color palette
    colors = [(0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (1.0, 0.4980392156862745, 0.054901960784313725),
 (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
 (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
 (0.5803921568627451, 0.403921568627451, 0.7411764705882353),
 (0.5490196078431373, 0.33725490196078434, 0.29411764705882354),
 (0.8901960784313725, 0.4666666666666667, 0.7607843137254902),
 (0.4980392156862745, 0.4980392156862745, 0.4980392156862745),
 (0.7372549019607844, 0.7411764705882353, 0.13333333333333333),
 (0.09019607843137255, 0.7450980392156863, 0.8117647058823529)]
    k = ms.shape[0]
    for i in range(k):
        colr = colors[i]
        var, U = np.linalg.eig(ss[i])
        angle = 180. / np.pi * np.arccos(np.abs(U[0, 0])) 
        #Set the angle right
        if (var[0]>var[1] and ss[i][1,0]<0) or (var[1]>var[0] and ss[i][1,0]>0):
            angle = -angle
        e = Ellipse(ms[i,:], 2 * np.sqrt(5.991 * var[0]),
                    2 * np.sqrt(5.991 * var[1]),
                    angle=angle)
        e.set_alpha(0.25)
        e.set_facecolor(colr)
        e.set_zorder(10);
        ax.add_artist(e);

In [ ]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
from theano.tensor.nlinalg import matrix_inverse
#from ellipse import plot_ellipse

# Seed the rng for exact reproducible results
seed = 68492
np.random.seed(seed)

## Generate some data for three groups.
# Means, variances, covariances, and proportions
mus = np.array([[-4,2],[0,1],[6,-2]]) 
variance1   = [1,.4,1.5]
variance2   = [1,.8,5  ]
covariances = [.5,0, -1]
ps = np.array([0.2, 0.5, 0.3])
D = mus[0].shape[0]
# Total amount of data
N = 1000
# Number of groups
K = 3
# Form covariance matrix for each group
sigmas = [np.array([[variance1[i],covariances[i]],[covariances[i],variance2[i]]]) for i in range(K)]
# Form group assignments
zs = np.array([np.random.multinomial(1, ps) for _ in range(N)]).T
xs = [z[:, np.newaxis] * np.random.multivariate_normal(m, s, size=N)
      for z, m, s in zip(zs, mus, sigmas)]
# Stack data into single array
data = np.sum(np.dstack(xs), axis=2)

## Plot them nicely
# Prepare subplots
fig, ax = plt.subplots(figsize=(8, 6))
# First, scatter
plt.scatter(data[:, 0], data[:, 1], c='g', alpha=0.5)
plt.scatter(mus[0, 0], mus[0, 1], c='r', s=100)
plt.scatter(mus[1, 0], mus[1, 1], c='b', s=100)
plt.scatter(mus[2, 0], mus[2, 1], c='y', s=100)
# Then, ellipses
plot_ellipse(ax,mus,sigmas)
ax.axis('equal')
plt.show()


## Build model and sample
# Number of iterations for sampler
draws = 2000
# Prepare lists of starting points for mu to prevent label-switching problem
testvals = [[-2,-2],[0,0],[2,2]]
import time
t1=time.time()
# Model structure
with pm.Model() as mvgmm:


    
    # Prior over component weights
    p = pm.Dirichlet('p', a=np.array([1.]*K))

    # Prior over component means
    mus = [pm.MvNormal('mu_%d' % i,
                        mu=pm.floatX(np.zeros(D)),
                        tau=pm.floatX(0.1 * np.eye(D)),
                        shape=(D,),
                        testval=pm.floatX(testvals[i]))
           for i in range(K)]

    # Cholesky decomposed LKJ prior over component covariance matrices
    packed_L = [pm.LKJCholeskyCov('packed_L_%d' % i,
                                  n=D,
                                  eta=2.,
                                  sd_dist=pm.HalfCauchy.dist(1))
                for i in range(K)]

    # Unpack packed_L into full array
    L = [pm.expand_packed_triangular(D, packed_L[i])
         for i in range(K)]

    # Convert L to sigma and tau for convenience
    sigma = [pm.Deterministic('sigma_%d' % i ,L[i].dot(L[i].T))
             for i in range(K)]
    tau = [pm.Deterministic('tau_%d' % i,matrix_inverse(sigma[i]))
           for i in range(K)]

    print("# Specify the likelihood")
    # Specify the likelihood
    mvnl = [pm.MvNormal.dist(mu=mus[i],chol=L[i])  
           for i in range(K)]    
    Y_obs = pm.Mixture('Y_obs',w=p, comp_dists=mvnl,observed=data)  

    print("# Start the sampler!")
    # Start the sampler!
    trace = pm.sample(draws, step=pm.NUTS(), chains=4)
    
    
t2=time.time()
print('elapsed time ',str(t2-t1))

## Plot traces
pm.traceplot(trace, var_names=['p', 'mu_0','mu_1','mu_2','tau_0','tau_1','tau_2','sigma_0','sigma_1','sigma_2'])
plt.show()

## Posterior Predictive Checks  
# Obtain posterior samples
pp = pm.sample_posterior_predictive(model=mvgmm, trace=trace)



#f, ax =plt.subplots(figsize=(15, 5))
f, ax =plt.subplots()
# Plot original data
plt.hist(data[:,0], bins=20, alpha=0.5)
# Plot posterior predictives on top of that 
plt.hist(pp['Y_obs'][7000,:,0], bins=20, alpha=0.5)
# Add legend and axes labels
plt.legend(['Data','Predictions'])
plt.xlabel('Simulated values')
plt.ylabel('Frequencies')
plt.show() 

f1, ax1 =plt.subplots()
# Plot original data
plt.hist(data[:,1], bins=20, alpha=0.5)
# Plot posterior predictives on top of that 
plt.hist(pp['Y_obs'][7000,:,1], bins=20, alpha=0.5)
# Add legend and axes labels
plt.legend(['Data','Predictions'])
plt.xlabel('Simulated values')
plt.ylabel('Frequencies')
plt.show() 

In [ ]:
gs_name = '_gs1512' #'_gs15656' #'_gs1512' #'_gs15656' #'_gs1512'   #'_gs5188' #'_gs1512'



import pandas as pd
import numpy as np
columns  = pd.read_hdf('stp_to_ge_all_columns'+gs_name+'.hdf')
ge_columns_train      = columns.loc[:,'ge_columns_train'].values[0]
annot_columns_train   = columns.loc[:,'annot_columns_train'].values[0]
stp_columns_train     = columns.loc[:,'stp_columns_train'].values[0]
classes_columns_train = columns.loc[:,'classes_columns_train'].values[0]
#print(X_train.columns)
X_train0 = pd.read_hdf('stp_to_ge_all_data'+gs_name+'.hdf')

X_train0.shape
print(X_train0.columns[200:300])
print(X_train0.columns[300:])



print('gs_name - '+gs_name)

if (gs_name=='_gs5188')|(gs_name=='_gs1512')|(gs_name=='_gs15656'):
    nlf = 10   # 20 were in scvi model
elif (gs_name=='_gs219'):
    nlf = 10   # 10 were in scvi model
lf_names = np.arange(nlf).astype(str)
lf_names1 =  ['pre__'+s for s in lf_names.tolist()]
lf_names2 =  ['post__'+s for s in lf_names.tolist()]

lf_scvi_names = lf_names1 + lf_names2

ge_columns_train2 = list(set(ge_columns_train).difference(set(lf_scvi_names)))
lf_scvi_names

#
#
#  genes to STP: Hierarchical Linear Model
#
#

#### 4 models:
#    A. 0->subclass_pre->+class_post
#    B. 0->class_pre->subclass_pre->+x class_post
#    C. 0->class_pre->subclass_pre->+subclass_post 
#    D. 0->class_pre->+x class_post

H_Models = pd.DataFrame([           ['Ms1c2'  ,['subclass_pre', 'class_post']],
                                    ['Mc1s1c2',['class_pre', 'subclass_pre', 'class_post']],
                                    ['Mc1s1s2',['class_pre', 'subclass_pre', 'subclass_post']],
                                    ['Mc1c2'  ,['class_pre', 'class_post']]],
                                    columns = ['name','structure'])


#### Load Data
#### modify data: DO class_pre, class_post, subclass_pre, subclass_post columns
subcl_names_pre = ['ex_ctx__pre', 'ex_ec__pre','ex_hipp__pre','sst__pre','pvalb__pre','cge__pre']
subcl_names_post = ['ex_ctx__post', 'ex_ec__post','ex_hipp__post','sst__post','pvalb__post','cge__post']
new_syntype_columns = pd.DataFrame([['class_pre',['ex__pre', 'inh__pre'], ['Glutamatergic', 'GABAergic']],
                                    ['subclass_pre',subcl_names_pre, subcl_names_pre],
                                    ['class_post',['ex__post', 'inh__post'], ['Glutamatergic', 'GABAergic']],
                                    ['subclass_post',subcl_names_post, subcl_names_post]], columns = ['group','columns_X0','new_names'])

for g in new_syntype_columns.index:
    df_new = X_train0.loc[:,new_syntype_columns.loc[g,'columns_X0']].copy()
    g20 = new_syntype_columns.loc[g,'columns_X0'][0]
    for g2 in new_syntype_columns.loc[g,'columns_X0']:
        df_new.loc[df_new.loc[:,g2]!=0, g20] = g2
    df_new_g = df_new.loc[:,g20].copy()   
    df_new_g.name = new_syntype_columns.loc[g,'group']
    X_train0 = pd.concat([X_train0, df_new_g],axis=1)
    classes_columns_train = classes_columns_train + [df_new_g.name]

#X_train0.loc[:,classes_columns_train].to_excel('X_train0.xlsx')  



####

Dn = 50
N_bootstraps = 100
Dn2 = int(N_bootstraps/Dn) # 100 - should be a number of bootstraps per synapse type !!!
Dn3 = int(X_train0.shape[0]/N_bootstraps) # number of synapse_types
#cla_n = ['ex_in_ex_in__pre',
#         'ex_in_ex_in__post'] 
#cla_n = ['ex_in_ex_in__pre'] 
#cla_n = []

HM_name = 'Ms1c2'
HM = H_Models.loc[H_Models.loc[:,'name']==HM_name, :]
cla_n  = HM['structure'][0]

#ge_n = imp50.index[0:25].tolist()
ge_n = lf_scvi_names #ge_columns_train2

nannot = len(annot_columns_train)
lge_n  = len(ge_n)

X2 = X_train0.loc[:,annot_columns_train + ge_n + cla_n]
if len(cla_n)>0:
    i_cl = np.nonzero(X2.columns.isin(cla_n))[0] - nannot
    #X2.loc[:,cla_n] = X2.loc[:,cla_n]+1

X2=X2.iloc[0:X_train0.shape[0]:Dn,:]

In [ ]:
# Index each of the unique variable values
n_levels = len(cla_n)
cla_n2 = np.array(cla_n)
index = []
index_df = []
for lv in range(n_levels):
    lv_index = X2.groupby(cla_n[0:lv+1] ).all().reset_index().reset_index()[['index']+cla_n[0:lv+1]]
    index = index + [lv_index]
    
    lv_indexes_df = lv_index
    if lv==1:
        lv_indexes_df = pd.merge(index[lv-1], index[lv], how='inner',
                                 on=cla_n[0:lv], suffixes=('_l'+str(lv-1), '_l'+str(lv)))
    elif lv>1:
        lv_indexes_df = pd.merge(lv_indexes_df, index[lv], how='inner', on=cla_n[0:lv])
    index_df = index_df + [lv_indexes_df ]    
        
X3 = pd.merge(X2, lv_indexes_df, how='inner', on=cla_n[0:]).reset_index()

import pymc3 as pm
import theano.tensor as T

indexes = []
count = []
for lv in range(n_levels):
    if lv==0:
        index_nm = 'index'
    else:
        index_nm = 'index_l'+str(lv-1)
    lv_indexes = index_df[lv][index_nm].values
    indexes = indexes + [lv_indexes]
    count = count + [len(lv_indexes)]

# degree_indexes = degree_index['index'].values
# degree_count = len(degree_indexes)
# degree_state_indexes = degree_state_indexes_df['index_d'].values
# degree_state_count = len(degree_state_indexes)
# degree_state_county_indexes = degree_state_county_indexes_df['index_ds'].values
# degree_state_county_count = len(degree_state_county_indexes)

with pm.Model() as mvgmm:
    # Prior over component weights
    p = pm.Dirichlet('p', a=np.array([1.]*K))

    # Prior over component means
    mus = [pm.MvNormal('mu_%d' % i,
                        mu=pm.floatX(np.zeros(D)),
                        tau=pm.floatX(0.1 * np.eye(D)),
                        shape=(D,),
                        testval=pm.floatX(testvals[i]))
           for i in range(K)]

    # Cholesky decomposed LKJ prior over component covariance matrices
    packed_L = [pm.LKJCholeskyCov('packed_L_%d' % i,
                                  n=D,
                                  eta=2.,
                                  sd_dist=pm.HalfCauchy.dist(1))
                for i in range(K)]

    # Unpack packed_L into full array
    L = [pm.expand_packed_triangular(D, packed_L[i])
         for i in range(K)]

    # Convert L to sigma and tau for convenience
    sigma = [pm.Deterministic('sigma_%d' % i ,L[i].dot(L[i].T))
             for i in range(K)]
    tau = [pm.Deterministic('tau_%d' % i,matrix_inverse(sigma[i]))
           for i in range(K)]

    print("# Specify the likelihood")
    # Specify the likelihood
    mvnl = [pm.MvNormal.dist(mu=mus[i],chol=L[i])  
           for i in range(K)]    
    Y_obs = pm.Mixture('Y_obs',w=p, comp_dists=mvnl,observed=data)  

    print("# Start the sampler!")
    # Start the sampler!
    trace = pm.sample(draws, step=pm.NUTS(), chains=4)
    

with pm.Model() as model:
    lv=0 # global
    mu_0 = pm.MvNormal('mu_0',
                        mu=pm.floatX(np.zeros(D)),
                        sig=pm.floatX(2 * np.eye(D)),
                        shape=(D,),
                        testval=pm.floatX(testvals[i]))
    mu_sig_0 = pm.Uniform('mu_sig_0', lower=0, upper=4)
    b_0 = pm.MvNormal('b_0',
                        mu=pm.floatX(np.zeros(D)),
                        sig=pm.floatX(2 * np.eye(D)),
                        shape=(D,),
                        testval=pm.floatX(testvals[i]))
    b_sig_0 = pm.Uniform('b_sig_0', lower=0, upper=2)
    
            
    lv=1  #first level: e-e, e-i, i-i, i-e
    # Prior over component means
    mu_1 = pm.MvNormal('mu_1',
                        mu=mu_0,
                        sig=pm.floatX(2  * np.eye(D)) ,
                        shape=(D,count[0]))
    mu_sig_1 = pm.Uniform('mu_sig_1', lower=0, upper=3)
    b_1 = pm.MvNormal('b_1',
                        mu=pm.floatX(np.zeros(D)),
                        sig=pm.floatX(2  * np.eye(D)),
                        shape=(D,count[0]))
    b_sig_1 = pm.Uniform('b_sig_1', lower=0, upper=2)       


    
    lv=2
    # Prior over component means
    mu_2 = pm.MvNormal('mu_%d' % i,
                        mu=mu_1,
                        sig=pm.floatX(mu_sig_1  * np.eye(D)) ,
                        shape=(D,count[0]))
    
    # Cholesky decomposed LKJ prior over component covariance matrices
    packed_L = [pm.LKJCholeskyCov('packed_L_%d' % i,
                                  n=D,
                                  eta=2.,
                                  sd_dist=pm.HalfCauchy.dist(1))
                for i in range(K)]

    # Unpack packed_L into full array
    L = [pm.expand_packed_triangular(D, packed_L[i])
         for i in range(K)]

    
    # Convert L to sigma and tau for convenience
    sigma = [pm.Deterministic('sigma_%d' % i ,L[i].dot(L[i].T))
             for i in range(K)]
    tau = [pm.Deterministic('tau_%d' % i,matrix_inverse(sigma[i]))
           for i in range(K)]


    sigma_unexplained = pm.HalfNormal('sigma_unexplained', sd=0.2) # unexplained variability
    y_prediction = mu_2[indexed_salary_df['index'].values] * indexed_salary_df.month_index.values + degree_state_county_b[indexed_salary_df['index'].values]
    
    y_like = pm.Normal('y_likelihood', mu=y_prediction, sd=sigma_unexplained, observed=y3.loc[:,stp_n].values)
    
    # robust?
    #y_like = pm.StudentT('y_like', nu=1, mu=0, sd=sigma_unexplained, observed=y_prediction - y3.loc[:,stp_n].values)
    
    global_m = pm.Normal('global_m', mu=6500, sd=100**2)
    global_m_sd = pm.Uniform('global_m_sd', lower=0, upper=1000)
    global_b = pm.Normal('global_b', mu=1000, sd=100**2)
    global_b_sd = pm.Uniform('global_b_sd', lower=0, upper=1000)
        
    degree_m = pm.Normal('degree_m', mu=global_m, sd=global_m_sd, shape=degree_count)
    degree_m_sd = pm.Uniform('degree_m_sd', lower=0, upper=1000, shape=degree_count)
    degree_b = pm.Normal('degree_b', mu=global_b, sd=global_b_sd, shape=degree_count)
    degree_b_sd = pm.Uniform('degree_b_sd', lower=0, upper=1000, shape=degree_count)
    
    degree_state_m = pm.Normal('degree_state_m', mu=degree_m[degree_state_indexes], sd=degree_m_sd[degree_state_indexes], shape=degree_state_count)
    degree_state_m_sd = pm.Uniform('degree_state_m_sd', lower=0, upper=1000, shape=degree_state_count)
    degree_state_b = pm.Normal('degree_state_b', mu=degree_b[degree_state_indexes], sd=degree_b_sd[degree_state_indexes], shape=degree_state_count)
    degree_state_b_sd = pm.Uniform('degree_state_b_sd', lower=0, upper=1000, shape=degree_state_count)
    
    degree_state_county_m = pm.Normal('degree_state_county_m', mu=degree_state_m[degree_state_county_indexes], sd=degree_state_m_sd[degree_state_county_indexes], shape=degree_state_county_count)
    degree_state_county_b = pm.Normal('degree_state_county_b', mu=degree_state_b[degree_state_county_indexes], sd=degree_state_b_sd[degree_state_county_indexes], shape=degree_state_county_count)
    
    error = pm.Uniform('error', lower=0, upper=10000)
    
    y_prediction = degree_state_county_m[indexed_salary_df['index'].values] * indexed_salary_df.month_index.values + degree_state_county_b[indexed_salary_df['index'].values]
    
    pm.StudentT('y_like', nu=1, mu=0, sd=error, observed=y_prediction - indexed_salary_df.salary.values)
    
    #model_trace = pm.sample(5000)
    #trace = pm.sample(draws=DRAWS+BURNIN, chains=CHAINS, cores=CORES, tune=TUNE, random_seed=SEED, nuts_kwargs={'target_accept':0.95})


In [ ]:
#
#
#  genes to STP: Hierarchical Linear Model
#
#

#### 4 models:
#    A. 0->subclass_pre->+class_post
#    B. 0->class_pre->subclass_pre->+x class_post
#    C. 0->class_pre->subclass_pre->+subclass_post 
#    D. 0->class_pre->+x class_post

H_Models = pd.DataFrame([           ['Ms1c2'  ,['subclass_pre', 'class_post']],
                                    ['Mc1s1c2',['class_pre', 'subclass_pre', 'class_post']],
                                    ['Mc1s1s2',['class_pre', 'subclass_pre', 'subclass_post']],
                                    ['Mc1c2'  ,['class_pre', 'class_post']]],
                                    columns = ['name','structure'])


#### Load Data
#### modify data: DO class_pre, class_post, subclass_pre, subclass_post columns
subcl_names_pre = ['ex_ctx__pre', 'ex_ec__pre','ex_hipp__pre','sst__pre','pvalb__pre','cge__pre']
subcl_names_post = ['ex_ctx__post', 'ex_ec__post','ex_hipp__post','sst__post','pvalb__post','cge__post']
new_syntype_columns = pd.DataFrame([['class_pre',['ex__pre', 'inh__pre'], ['Glutamatergic', 'GABAergic']],
                                    ['subclass_pre',subcl_names_pre, subcl_names_pre],
                                    ['class_post',['ex__post', 'inh__post'], ['Glutamatergic', 'GABAergic']],
                                    ['subclass_post',subcl_names_post, subcl_names_post]], columns = ['group','columns_X0','new_names'])

for g in new_syntype_columns.index:
    df_new = X_train0.loc[:,new_syntype_columns.loc[g,'columns_X0']].copy()
    g20 = new_syntype_columns.loc[g,'columns_X0'][0]
    for g2 in new_syntype_columns.loc[g,'columns_X0']:
        df_new.loc[df_new.loc[:,g2]!=0, g20] = g2
    df_new_g = df_new.loc[:,g20].copy()   
    df_new_g.name = new_syntype_columns.loc[g,'group']
    X_train0 = pd.concat([X_train0, df_new_g],axis=1)
    classes_columns_train = classes_columns_train + [df_new_g.name]

#X_train0.loc[:,classes_columns_train].to_excel('X_train0.xlsx')  



####

Dn = 50
N_bootstraps = 100
Dn2 = int(N_bootstraps/Dn) # 100 - should be a number of bootstraps per synapse type !!!
Dn3 = int(X_train0.shape[0]/N_bootstraps) # number of synapse_types
#cla_n = ['ex_in_ex_in__pre',
#         'ex_in_ex_in__post'] 
#cla_n = ['ex_in_ex_in__pre'] 
#cla_n = []

HM_name = 'Ms1c2'
HM = H_Models.loc[H_Models.loc[:,'name']==HM_name, :]
cla_n  = HM['structure'][0]

#ge_n = imp50.index[0:25].tolist()
ge_n = lf_scvi_names #ge_columns_train2

nannot = len(annot_columns_train)
lge_n  = len(ge_n)

X2 = X_train0.loc[:,annot_columns_train + ge_n + cla_n]
if len(cla_n)>0:
    i_cl = np.nonzero(X2.columns.isin(cla_n))[0] - nannot
    #X2.loc[:,cla_n] = X2.loc[:,cla_n]+1

X2=X2.iloc[0:X_train0.shape[0]:Dn,:]

X2 =X2.values

# select y
stp_n = ['A2_20Hz'] #,'A5_20Hz','A250_20Hz']

# 'A2_20Hz',
#  'A3_20Hz',
#  'A4_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A1_50Hz',
#  'A2_50Hz',
#  'A3_50Hz',
#  'A4_50Hz',
#  'A5_50Hz',
#  'A250_50Hz',
#  'A1000_50Hz',
#  'A1_10Hz',
#  'A2_10Hz',
#  'A3_10Hz',
#  'A4_10Hz',
#  'A5_10Hz',
#  'A250_10Hz',
#  'A1000_10Hz']
y2 =  X_train0.loc[:,annot_columns_train+stp_n].iloc[0:X_train0.shape[0]:Dn,:].values

#header = ge_n + cla_n + stp_n 
header = ge_n  + stp_n 
annot_columns2 = annot_columns_train

### PyMC3 example : HLM 1D

In [ ]:
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline



# These are the "true" slopes and intercepts that I made up.
set_parameters = (
    (('no degree', 'CA', 'Riverside County'), (5000, 1250)),
    (('no degree', 'IL', 'Cook County'), (6500, 1150)),
    (('no degree', 'IL', 'Lake County'), (7000, 1350)),
    
    (('degree', 'CA', 'Riverside County'), (6000, 1250)),
    (('degree', 'IL', 'Cook County'), (7500, 1150)),
    (('degree', 'IL', 'Lake County'), (8000, 1350))
)

data_points = np.random.randint(1,24, size=len(set_parameters))

# Go through each definition above and generate a fake time series


rows = []
for set_parameter, N in zip(set_parameters, data_points):
  key, parameters = set_parameter
  population_rows = [{
      'degree': key[0],
      'state': key[1],
      'county': key[2],
      'month_index': i,
      'salary': i*parameters[0] + parameters[1] + np.random.normal(loc=0, scale=500)
  } for i in range(N)]
  rows += population_rows

salary_df = pd.DataFrame(rows)

#plt.scatter(salary_df.month_index, salary_df.salary, s=2);
salary_df.to_excel('temp2.xlsx')
#salary_df

# Index each of the unique variable values
degree_index = salary_df.groupby('degree').all().reset_index().reset_index()[['index', 'degree']]
degree_state_index = salary_df.groupby(['degree', 'state']).all().reset_index().reset_index()[['index', 'degree', 'state']]
degree_state_county_index = salary_df.groupby(['degree', 'state', 'county']).all().reset_index().reset_index()[['index', 'degree', 'state', 'county']]

degree_state_indexes_df = pd.merge(degree_index, degree_state_index, how='inner', on='degree', suffixes=('_d', '_ds'))
degree_state_county_indexes_df = pd.merge(degree_state_indexes_df, degree_state_county_index, how='inner', on=['degree', 'state'])
indexed_salary_df = pd.merge(salary_df, degree_state_county_indexes_df, how='inner', on=['degree', 'state', 'county']).reset_index()


import pymc3 as pm

degree_indexes = degree_index['index'].values
degree_count = len(degree_indexes)
degree_state_indexes = degree_state_indexes_df['index_d'].values
degree_state_count = len(degree_state_indexes)
degree_state_county_indexes = degree_state_county_indexes_df['index_ds'].values
degree_state_county_count = len(degree_state_county_indexes)

with pm.Model() as model:
    global_m = pm.Normal('global_m', mu=6500, sd=100**2)
    global_m_sd = pm.Uniform('global_m_sd', lower=0, upper=1000)
    global_b = pm.Normal('global_b', mu=1000, sd=100**2)
    global_b_sd = pm.Uniform('global_b_sd', lower=0, upper=1000)
    
    degree_m = pm.Normal('degree_m', mu=global_m, sd=global_m_sd, shape=degree_count)
    degree_m_sd = pm.Uniform('degree_m_sd', lower=0, upper=1000, shape=degree_count)
    degree_b = pm.Normal('degree_b', mu=global_b, sd=global_b_sd, shape=degree_count)
    degree_b_sd = pm.Uniform('degree_b_sd', lower=0, upper=1000, shape=degree_count)
    
    degree_state_m = pm.Normal('degree_state_m', mu=degree_m[degree_state_indexes], sd=degree_m_sd[degree_state_indexes], shape=degree_state_count)
    degree_state_m_sd = pm.Uniform('degree_state_m_sd', lower=0, upper=1000, shape=degree_state_count)
    degree_state_b = pm.Normal('degree_state_b', mu=degree_b[degree_state_indexes], sd=degree_b_sd[degree_state_indexes], shape=degree_state_count)
    degree_state_b_sd = pm.Uniform('degree_state_b_sd', lower=0, upper=1000, shape=degree_state_count)
    
    degree_state_county_m = pm.Normal('degree_state_county_m', mu=degree_state_m[degree_state_county_indexes], sd=degree_state_m_sd[degree_state_county_indexes], shape=degree_state_county_count)
    degree_state_county_b = pm.Normal('degree_state_county_b', mu=degree_state_b[degree_state_county_indexes], sd=degree_state_b_sd[degree_state_county_indexes], shape=degree_state_county_count)
    
    error = pm.Uniform('error', lower=0, upper=10000)
    
    y_prediction = degree_state_county_m[indexed_salary_df['index'].values] * indexed_salary_df.month_index.values + degree_state_county_b[indexed_salary_df['index'].values]
    
    pm.StudentT('y_like', nu=1, mu=0, sd=error, observed=y_prediction - indexed_salary_df.salary.values)
    
    model_trace = pm.sample(5000)

## CV of different type for all ML

In [ ]:

import sklearn as sk

def fit_classes_tree(X2,y2,X2_cl,cli,model_type='lasso',nmin = 20, alpha=1, l1_ratio=0.5):
    model = []
    support =[]
    ncl1 = np.max(X2_cl[:,cli[0]])+1 
    ncl2 = np.max(X2_cl[:,cli[1]])+1 
    for i1 in range(ncl1):
        model1 = []
        support1 = []
        for i2 in range(ncl2):

            is_in_cl = (X2_cl[:,cli[0]]==i1)&(X2_cl[:,cli[1]]==i2)
            #           support2 = [np.sum(is_in_cl),
            #           np.nonzero((X2_cl[:,cli[0]]==i1))[0],
            #           np.nonzero((X2_cl[:,cli[1]]==i2))[0]]
            support2 = np.sum(is_in_cl)
            
            support1 = support1 + [support2]
            regr = 0
            if support2>nmin:
                y_train = y2[is_in_cl,:]
                X_train = X2[is_in_cl,:]
                if model_type=='ridge':
                    regr = sk.linear_model.Ridge(alpha=0.50, fit_intercept=True, normalize=False,
                                                 copy_X=True, max_iter=None, tol=0.001, solver='auto',
                                                 random_state=None)
                
                elif model_type=='lasso':
                    regr = sk.linear_model.Lasso(alpha=alpha, fit_intercept=True, normalize=False,
                                                 precompute=False, copy_X=True,
                                                 max_iter=1000, tol=0.0001, warm_start=False, positive=False, 
                                                 random_state=None, selection='cyclic')
                elif model_type=='elastic_net':   
                    regr = sk.linear_model.ElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=True,
                                    normalize=False,
                                    max_iter=1000, copy_X=True, tol=0.0001, warm_start=False,
                                    random_state=None, selection='cyclic')
                else:
                    print(" model types supported: ridge, lasso, elastic_net")
                
                regr.fit(X_train, y_train)
                #y_pred = regr.predict(X_test[:,nannot:]) 


            model1 = model1 + [regr]
            
        support = support + [np.array(support1)]
        model = model + [np.array(model1)]    
    return np.array(model), np.array(support)        

In [ ]:
def predict_classes_tree(model,X2,X2_cl,cli,nout=1,nmin = 20):
    
    ncl1 = np.max(X2_cl[:,cli[0]])+1 
    ncl2 = np.max(X2_cl[:,cli[1]])+1 
    y_pred = np.zeros((X2.shape[0],nout))
    for i1 in range(ncl1):
        #print(i1," f1")
        if len(model)>i1:
            #print(i1," f2")
            model1 = model[i1]
            for i2 in range(ncl2):

                is_in_cl = (X2_cl[:,cli[0]]==i1)&(X2_cl[:,cli[1]]==i2)

                support2 = np.sum(is_in_cl)
                
                #print(i1,i2," f3 ",support2)
                #regr = 0
                if support2>nmin:
                    #print(i1,i2," f3 ",support2)
                    if len(model1)>i2:
                        #print(i1,i2," f4")
                        regr = model1[i2]
                        #regr = sk.linear_model.MultiTaskElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=True,
                        #                    normalize=False,
                        #                    max_iter=1000, copy_X=True, tol=0.0001, warm_start=False,
                        #                    random_state=None, selection='cyclic')
                        #regr.fit(X_train, y_train)
                        if regr!=0:
                            X_test = X2[is_in_cl,:]
                            y_pred[is_in_cl] = regr.predict(X_test).reshape((-1,1)) 
    
    return y_pred       

In [ ]:
def cv_random_groups(n_splits, groups_index, part ):
    #  part % random syn-types level2 cv 
    #part = 10 # % of removed samples
    #n_splits=10
    
    test_indexes = []
    names = []
    #cla_n = [cla_cell_type_l2]

    #groups_index = X_train0.loc[:,cla_n]
    #groups_index=groups_index.iloc[0:X_train0.shape[0]:Dn,:].values
    all_gr = set(groups_index.reshape((-1,)))
    all_gr = all_gr.difference(set([0]))
    all_gr = list(all_gr)

    #dn_syn_type = 100
    nall = groups_index.shape[0]
    all_index = np.arange(nall)
    Dn3_out = int(nall*part/100)
    idx0 = all_gr  #np.arange(len(all_gr))
    #idx1 = np.arange(Dn2)
    for icv in range(n_splits):
        idx0_out = np.random.choice(idx0, size=len(all_gr), replace=False, p=None)
        test_index = np.zeros((0,))
        for io in idx0_out:
            #train_index  = train_index + icv*dn + np.arange(dn)
            idx1 = all_index[np.any(groups_index==io,axis=1)]
            if len(idx1) + len(test_index)<Dn3_out:
                test_index  = np.append(test_index, idx1)
            else:
                idx1 = np.random.choice(idx1, size=Dn3_out-len(test_index), replace=False, p=None)
                test_index  = np.append(test_index, idx1)
                break

        test_indexes = test_indexes + [test_index]   
        names = names + ['syn_types_l0_'+str(icv)+'  '+np.array2string(idx0_out)]
        #df_cv.loc[icv,'train_index']=[train_index]
        #df_cv.loc[icv,'name']='syn_types_l0_'+str(icv)+'  '+np.array2string(idx0_out)

    df_cv1  = pd.DataFrame([], columns = ['test_index', 'name'])
    return df_cv

In [ ]:
gs_name = '_gs1512' #'_gs15656' #'_gs1512' #'_gs15656' #'_gs1512'   #'_gs5188' #'_gs1512'



import pandas as pd
import numpy as np
columns  = pd.read_hdf('stp_to_ge_all_columns'+gs_name+'.hdf')
ge_columns_train      = columns.loc[:,'ge_columns_train'].values[0]
annot_columns_train   = columns.loc[:,'annot_columns_train'].values[0]
stp_columns_train     = columns.loc[:,'stp_columns_train'].values[0]
classes_columns_train = columns.loc[:,'classes_columns_train'].values[0]
#print(X_train.columns)
X_train0 = pd.read_hdf('stp_to_ge_all_data'+gs_name+'.hdf')

X_train0.shape
print(X_train0.columns[200:300])
print(X_train0.columns[300:])



print('gs_name - '+gs_name)

if (gs_name=='_gs5188')|(gs_name=='_gs1512')|(gs_name=='_gs15656'):
    nlf = 10   # 20 were in scvi model
elif (gs_name=='_gs219'):
    nlf = 10   # 10 were in scvi model
lf_names = np.arange(nlf).astype(str)
lf_names1 =  ['pre__'+s for s in lf_names.tolist()]
lf_names2 =  ['post__'+s for s in lf_names.tolist()]

lf_scvi_names = lf_names1 + lf_names2

ge_columns_train2 = list(set(ge_columns_train).difference(set(lf_scvi_names)))
lf_scvi_names

#
#
#  genes to STP: Hierarchical Linear Model
#
#

#### 4 models:
#    A. 0->subclass_pre->+class_post
#    B. 0->class_pre->subclass_pre->+x class_post
#    C. 0->class_pre->subclass_pre->+subclass_post 
#    D. 0->class_pre->+x class_post

H_Models = pd.DataFrame([           ['Ms1c2'  ,['subclass_pre', 'class_post']],
                                    ['Mc1s1c2',['class_pre', 'subclass_pre', 'class_post']],
                                    ['Mc1s1s2',['class_pre', 'subclass_pre', 'subclass_post']],
                                    ['Mc1c2'  ,['class_pre', 'class_post']]],
                                    columns = ['name','structure'])


#### Load Data
#### modify data: DO class_pre, class_post, subclass_pre, subclass_post columns
subcl_names_pre = ['ex_ctx__pre', 'ex_ec__pre','ex_hipp__pre','sst__pre','pvalb__pre','cge__pre']
subcl_names_post = ['ex_ctx__post', 'ex_ec__post','ex_hipp__post','sst__post','pvalb__post','cge__post']
new_syntype_columns = pd.DataFrame([['class_pre',['ex__pre', 'inh__pre'], ['Glutamatergic', 'GABAergic']],
                                    ['subclass_pre',subcl_names_pre, subcl_names_pre],
                                    ['class_post',['ex__post', 'inh__post'], ['Glutamatergic', 'GABAergic']],
                                    ['subclass_post',subcl_names_post, subcl_names_post]], columns = ['group','columns_X0','new_names'])

for g in new_syntype_columns.index:
    df_new = X_train0.loc[:,new_syntype_columns.loc[g,'columns_X0']].copy()
    g20 = new_syntype_columns.loc[g,'columns_X0'][0]
    for g2 in new_syntype_columns.loc[g,'columns_X0']:
        df_new.loc[df_new.loc[:,g2]!=0, g20] = g2
    df_new_g = df_new.loc[:,g20].copy()   
    df_new_g.name = new_syntype_columns.loc[g,'group']
    X_train0 = pd.concat([X_train0, df_new_g],axis=1)
    classes_columns_train = classes_columns_train + [df_new_g.name]

#X_train0.loc[:,classes_columns_train].to_excel('X_train0.xlsx')  



####

# Dn = 50
# N_bootstraps = 100
# Dn2 = int(N_bootstraps/Dn) # 100 - should be a number of bootstraps per synapse type !!!
# Dn3 = int(X_train0.shape[0]/N_bootstraps) # number of synapse_types

# HM_name = 'Ms1c2'
# HM = H_Models.loc[H_Models.loc[:,'name']==HM_name, :]
# cla_n  = HM['structure'][0]

# #ge_n = imp50.index[0:25].tolist()
# ge_n = lf_scvi_names #ge_columns_train2

# nannot = len(annot_columns_train)
# lge_n  = len(ge_n)

# X2 = X_train0.loc[:,annot_columns_train + ge_n + cla_n]
# if len(cla_n)>0:
#     i_cl = np.nonzero(X2.columns.isin(cla_n))[0] - nannot
#     #X2.loc[:,cla_n] = X2.loc[:,cla_n]+1

# X2=X2.iloc[0:X_train0.shape[0]:Dn,:]

In [ ]:
# note: synapse type hierarhy in X(:,classes_columns_train) should be from H_Models!!!

import matplotlib.pyplot as plt
import time
t1=time.time()

Dn = 10
N_bootstraps = 100
Dn2 = int(N_bootstraps/Dn) # 100 - should be a number of bootstraps per synapse type !!!
Dn3 = int(X_train0.shape[0]/N_bootstraps) # number of synapse_types

####
####   SELECT X
####
cla_n = classes_columns_train #['ex_inh']
cla_n2 = pd.Series(cla_n)

#ge_n = imp50.index[0:25].tolist()
#ge_n = lf_scvi_names #ge_columns_train2
ge_n = ge_columns_train #imp50.index[0:25].tolist()


#X2 = X_train0.loc[:,annot_columns_train + ge_n + cla_n]
X2 = X_train0.loc[:,ge_n ]
X2_cl = X_train0.loc[:,cla_n ]
X2_an = X_train0.loc[:,annot_columns_train ]
#i_cl = np.nonzero(X2.columns.isin(cla_n))[0]
nannot = len(annot_columns_train)
lge_n  = len(ge_n)
if len(cla_n)>0:
    i_cl = np.nonzero(X2.columns.isin(cla_n))[0] - nannot
    #X2.loc[:,cla_n] = X2.loc[:,cla_n]+1
    
X2=X2.iloc[0:X_train0.shape[0]:Dn,:].values
X2_cl=X2_cl.iloc[0:X_train0.shape[0]:Dn,:].values
X2_an=X2_an.iloc[0:X_train0.shape[0]:Dn,:].values


stp_n = ['A2_20Hz']
#  'A2_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A2_50Hz',
#  'A2_10Hz',
#  'A5_50Hz',
#  'A5_10Hz',
y2 =  X_train0.loc[:,stp_n].iloc[0:X_train0.shape[0]:Dn,:].values
##header = ge_n + cla_n + stp_n 
#header  = ge_n  + stp_n 

iy0=0
mod_index = modf.index #[3] # modf.index #[0, 3, 9, 11, 13, 15]
Y_pred = []
for i,mdn in enumerate(H_Models.loc[:,'name']):
    
    # HM_name = 'Ms1c2'
    md = H_Models.loc[H_Models.loc[:,'name']==mdn, :]
    

    #md = modf.loc[i,:]


    #mdn = md['name']
    print(mdn)

    cli = cla_n2.index[cla_n2.isin(md['structure'].values[0])].values

    #mdcl2 = np.char.strip(np.array(str.split(md['classes_post'],',')))
    #mdcl1 = np.char.strip(np.array(str.split(md['classes'],','))) 
    print(set(X2_cl[:,cli[0]]))
    print(set(X2_cl[:,cli[1]]))
    
    X3_cl = np.copy(X2_cl)
    for cli2 in cli:
        n2n=pd.DataFrame(list(set(X2_cl[:,cli2]))).reset_index().set_index(0)
        X3_cl[:,cli2] = n2n.loc[X2_cl[:,cli2]].values.ravel()

    y_pred = np.zeros((0,y2.shape[1]))
    
    ncv = 10
    n_samp_cv = np.rint(X2.shape[0]/ncv)
    samples_all = np.arange(X2.shape[0])
    r2cv = np.zeros(ncv)
    r4cv = np.zeros(ncv)
    for icv in range(ncv): #range(ncv):
        samples_test = (np.arange(n_samp_cv) + icv*n_samp_cv).astype(int)
        samples_train = np.delete(np.copy(samples_all),samples_test)
        X2train, y2train, X2train_cl = X2[samples_train,:], y2[samples_train,:], X3_cl[samples_train,:]
        X2test, y2test, X2test_cl = X2[samples_test,:], y2[samples_test,:], X3_cl[samples_test,:]
        
        model, support = fit_classes_tree(X2train,y2train,X2train_cl,cli,model_type='elastic_net',
                                          nmin = 20, alpha=0.2, l1_ratio=0.05)
        #model, support = fit_classes_tree(X2,y2,X2_cl,cli,model_type='ridge',nmin = 20, alpha=1, l1_ratio=0.5)
        y_pred_i = predict_classes_tree(model,X2test,X2test_cl,cli,nout=y2.shape[1],nmin = 2)
        y_pred = np.concatenate([y_pred, y_pred_i],axis=0)
        
        iy=iy0
        if np.var(y2[:,iy])!=0:
            nonz = y_pred_i[:,iy]!=0
            if np.sum(nonz)>0:
                r2cv[icv]=1 - np.mean((y2test[nonz,iy] - y_pred_i[nonz,iy])**2)/np.var(y2[:,iy])
                r4cv[icv]=1 - np.mean((y2test[nonz,iy] - y_pred_i[nonz,iy])**2)/np.var(y2test[nonz,iy])
    
    Y_pred = Y_pred + [y_pred]
    
    f, ax =plt.subplots(figsize=(16, 10))
    ##f, ax = plt.figure()
    ##ax = f.add_axes()
    
    #plt.title(stp_columns[iy]+", model : "+mdn+", cv : "+str(icv))
    plt.title(stp_columns_train[iy]+", model : "+mdn)
    
    #ax.set_title(stp_columns[i]+' '+', subclass out of bag: '+str(i0))
    #yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
    plt.plot(y2.ravel(),'ob')
    plt.plot(y_pred.ravel(),'xr')
    #plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)
    iy=iy0
    r2=0
    r4=0
    if np.var(y2[:,iy])!=0:
        r2=1 - np.mean((y2[:,iy] - y_pred[:,iy])**2)/np.var(y2[:,iy])
        r4=1 - np.var(y_pred[:,iy])/np.var(y2[:,iy])
        
    print("R**2 = ",r2,
          "  Part of Var = ",r4,"\n",
          "  R**2 cv mean = ", np.mean(r2cv),
          "\n  R**2 cv unnormed mean = ", np.mean(r4cv),
          "\n  R**2 cv = ", r2cv,
          "\n  R**2 cv unnormed = ", r4cv,
          "\n\n")
    
    
t2 =time.time()    
print("Elapsed time ",t2-t1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

from sklearn import preprocessing


from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import sklearn as sk

import time
t1 = time.time()

do_oob_subclasses=0

#X2 = np.copy(X[range(0,X.shape[0],1)])
#y2 = np.copy(y[range(0,X.shape[0],1)])
#nsel = np.arange(1)
#y2=y2[:,0:nannot+nsel.shape[0]]

#import models.linear_regr as lir
#import models.mean_regr as mir

#from models.svm_regr import svm_regr
#from models.DT_sklearn_regr import DT_sklearn_regr

from importlib import reload
#mtree = reload(mtree)
#lir=reload(lir)
#mir=reload(mir)

# ====================
# Settings
# ====================
mode = "regr"  # "clf" / "regr"
save_model_tree = True  # save model tree?
save_model_tree_predictions = True  # save model tree predictions/explanations?
cross_validation = True  # cross-validate model tree?

#ML_METHOD = 1

# ====================
# Load data
# ====================


# ['all', 'ex_inh']
Dn = 5
Dn2 = int(100/Dn) # 100 - should be a number of bootstraps per synapse type !!!
Dn3 = int(X_train0.shape[0]/100) # number of synapse_types
#cla_n = ['ex_in_ex_in__pre',
#         'ex_in_ex_in__post'] 

#cla_n = ['ex_in_ex_in__pre'] 
cla_n = []

#ge_n = imp50.index[0:25].tolist()
ge_n = lf_scvi_names #ge_columns_train2

nannot = len(annot_columns_train)
lge_n  = len(ge_n)

X2 = X_train0.loc[:,annot_columns_train + ge_n + cla_n]

if len(cla_n)>0:
    i_cl = np.nonzero(X2.columns.isin(cla_n))[0] - nannot

    #X2.loc[:,cla_n] = X2.loc[:,cla_n]+1

X2=X2.iloc[0:X_train0.shape[0]:Dn,:].values

# select y
stp_n = ['A2_20Hz'] #,'A5_20Hz','A250_20Hz']

# 'A2_20Hz',
#  'A3_20Hz',
#  'A4_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A1_50Hz',
#  'A2_50Hz',
#  'A3_50Hz',
#  'A4_50Hz',
#  'A5_50Hz',
#  'A250_50Hz',
#  'A1000_50Hz',
#  'A1_10Hz',
#  'A2_10Hz',
#  'A3_10Hz',
#  'A4_10Hz',
#  'A5_10Hz',
#  'A250_10Hz',
#  'A1000_10Hz']
y2 =  X_train0.loc[:,annot_columns_train+stp_n].iloc[0:X_train0.shape[0]:Dn,:].values

#header = ge_n + cla_n + stp_n 
header = ge_n  + stp_n 
annot_columns2 = annot_columns_train
#
#
# CROSS-VALIDATION GROUPS
#
# cv groups
all_index = np.arange(X2.shape[0])

#
#
#
#
# 10-fold cv  : 10% of synapses in data collection order
#
#
test_indexes = []
names = []
n_splits=10
dn = int(X2.shape[0]/10)
#df_cv  = pd.DataFrame(np.zeros((n_splits,2)), columns = ['train_index', 'name'])
for icv in range(n_splits):
    if icv==n_splits:
        dn = X2.shape[0] - icv*dn
    test_index  = icv*dn + np.arange(dn)
    #df_cv.loc[icv,'train_index']=[train_index]
    #df_cv.loc[icv,'name']=str(icv)
    test_indexes = test_indexes + [test_index]
    names = names + [str(icv)]
    
df_cv  = pd.DataFrame([test_indexes, names]).T
df_cv.columns = ['test_index', 'name']


#
#
#
#
#  10% random  synaptic-types :  cre-lines+subregions (level1)  cv 
#
#
part = 10 # % of removed samples
n_splits=10
test_indexes = []
names = []
#dn_syn_type = 100
Dn3_out = int(Dn3*part/100)
idx0 = np.arange(Dn3)
idx1 = np.arange(Dn2)
for icv in range(n_splits):
    idx0_out = np.random.choice(idx0, size=Dn3_out, replace=False, p=None)
    test_index = np.zeros((0,))
    for io in idx0_out:
        #train_index  = train_index + icv*dn + np.arange(dn)
        test_index  = np.append(test_index, io*Dn2 + idx1)
    test_indexes = test_indexes + [test_index]
    names = names + ['syn_types_l0_'+str(icv)+'  '+np.array2string(idx0_out)]
    #df_cv.loc[icv,'train_index']=[train_index]
    #df_cv.loc[icv,'name']='syn_types_l0_'+str(icv)+'  '+np.array2string(idx0_out)

#df_cv1  = pd.DataFrame([train_indexes, names], columns = ['train_index', 'name'])  
df_cv1  = pd.DataFrame([test_indexes, names]).T
df_cv1.columns = ['test_index', 'name']
    
df_cv = pd.concat([df_cv, df_cv1],axis=0)    


#
#
#
#
#  10% random synaptic types : subclasses+regions (level2) cv 
#
#
part = 10 # % of removed samples
n_splits=10
cla_n = ['vse__pre', 'vse__post']
groups_index = X_train0.loc[:,cla_n]
groups_index=groups_index.iloc[0:X_train0.shape[0]:Dn,:].values

df_cv1 = cv_random_groups(n_splits, groups_index, part ) 

df_cv = pd.concat([df_cv, df_cv1],axis=0)  


#
#
#
#
#  10% random synaptic types : subclasses (level3) cv 
#
#
part = 10
n_splits=10

cla_n = ['ex_ctx__pre',
 'ex_ctx__post',
 'ex_ec__pre',
 'ex_ec__post',
 'ex_hipp__pre',
 'ex_hipp__post',
 'pvalb__pre',
 'pvalb__post',
 'sst__pre',
 'sst__post',
 'lamp5__pre',
 'lamp5__post',
 'vip__pre',
 'vip__post',
 'cck__pre',
 'cck__post']
#cla_n = ['cla_cell_type_l3_pre', 'cla_cell_type_l3_post']

groups_index = X_train0.loc[:,cla_n]
groups_index=groups_index.iloc[0:X_train0.shape[0]:Dn,:].values
#all_gr = list(set(groups_index))

df_cv1 = cv_random_groups(n_splits, groups_index, part ) 

df_cv = pd.concat([df_cv, df_cv1],axis=0) 

df_cv = df_cv.reset_index()


df_cv2 = df_cv.loc[10:18,:]   ## select for current calculations
#
#
# END CROSS-VALIDATION GROUPS
#
#

if do_oob_subclasses==0:
    ## select datasets for training and validation
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0) 

    #X, y = make_regression(n_features=4, n_informative=2,
    #                        random_state=0, shuffle=False)
    
    #n_splits=10
    #kf = KFold(n_splits=n_splits, shuffle=False)
    i0=0
    r6=[]
    r7=[]
    for ii in df_cv2.index:
        i0=i0+1
        #train_index, test_index in kf.split(X2):
        test_index = df_cv2.loc[ii,'test_index'].astype(int)
        train_index = np.delete(all_index.copy(), test_index)
        
        #i0=i0+1
        print('\n\n '+ str(i0)+'   '+df_cv2.loc[ii,'name'])
        #print("TRAIN:", train_index,"\n", "TEST:", test_index)
        
        X_train, X_test = X2[train_index], X2[test_index]
        y_train, y_test = y2[train_index], y2[test_index]
        
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0) 

        #
        #
        #  ML METHOD
        #
        #
        met=0 #ML_METHOD
        
        if (met<4):
            i_X = np.arange(nannot,nannot+lge_n)
        else: # use cell types information
            i_X = np.arange(nannot,X_train.shape[1])
            
            
        if met==0:
            regr = sk.neighbors.KNeighborsRegressor(n_neighbors=8, weights='uniform', 
                                       algorithm='auto', leaf_size=30, p=2,
                                       metric='minkowski', metric_params=None,
                                       n_jobs=-1)
        
        if met==1:
            regr = RandomForestRegressor(random_state=2026,max_depth=7,min_samples_leaf=5,
                                    n_estimators=200, oob_score=True, n_jobs=-1, max_samples=300)
                    # RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
            #           max_features='auto', max_leaf_nodes=None,
            #           min_impurity_decrease=0.0, min_impurity_split=None,
            #           min_samples_leaf=1, min_samples_split=2,
            #           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            #           oob_score=False, random_state=0, verbose=0, warm_start=False)
        elif met==2:
            if i0==1:
                regr = sk.linear_model.MultiTaskElasticNetCV(cv=5,n_jobs=-1)
                regr.fit(X[:,nannot:], y[:,nannot:])
                print("alpha ",regr.alpha_)
                print("l1_ratio ",regr.l1_ratio_)
        
            regr = sk.linear_model.MultiTaskElasticNet(alpha=regr.alpha_, l1_ratio=regr.l1_ratio_,
                                                       fit_intercept=True,
                                                       normalize=False,
                                 max_iter=1000, copy_X=True, tol=0.0001,
                                                       warm_start=False, positive=False, 
                                                       random_state=None, selection='cyclic')
        
        elif met==3:
            X_train[:,i_X] = preprocessing.scale(X_train[:,i_X])
            y_train[:,nannot:] = preprocessing.scale(y_train[:,nannot:])
            
            X_test[:,i_X] = preprocessing.scale(X_test[:,i_X])
            y_test[:,nannot:] = preprocessing.scale(y_test[:,nannot:])
            
            regr = sk.svm.SVR(kernel='poly', degree=3, gamma='scale',
                    coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, 
                              cache_size=200, verbose=False, max_iter=-1)
        
        
        elif met==4:
            # Choose model
            model = lir.linear_regr()
            model.lambda_classes  = 0.1
            model.class_index = i_cl

            # Build model tree
            model_tree = mtree.ModelTree(model, max_depth=4, min_samples_leaf=10,
                                   search_type="adaptive", n_search_grid=20)
            
        elif met==5:
            # Choose model
            #model = lir.linear_regr()
            model = mir.mean_regr()
            model.lambda_classes  = 0.0
            model.class_index = i_cl

            # Build model tree
            model_tree = mtree.RFRegressor(n_estimators=50, max_depth=4,
                 min_samples_split=2, min_samples_leaf=5, DTalgo='modeltree',
                 model_leaf = model, n_search_grid = 25, n_search_type = "adaptive",  
                 n_bootstrap=93*5, bootstrap_type='sequential', bootstrap_classes=None,n_bootstrap_groups=93)



        if met==3:
            #regr.fit(X_train[:,6:], y_train[:,0:].ravel())  
            regr.fit(X_train[:,i_X], y_train[:,nannot:].ravel())  
            y_pred = regr.predict(X_test[:,i_X]) 
            y_pred0 = regr.predict(X_train[:,i_X]) 
        elif met<4:
            regr.fit(X_train[:,i_X], y_train[:,nannot:])
            y_pred = regr.predict(X_test[:,i_X]) 
            y_pred0 = regr.predict(X_train[:,i_X]) 
        elif met==4:
            # ====================
            # Train model tree
            # ====================
            print("Training model tree with '{}'...".format(model.__class__.__name__))
            model_tree.fit(X_train[:,i_X], y_train[:,nannot:], verbose=True)
            y_pred = model_tree.predict(X_test[:,i_X])
            y_pred0 = model_tree.predict(X_train[:,i_X])
        elif met==5:    
            print("Training forest of model trees with '{}'...".format(model.__class__.__name__))
            model_tree.fit(X_train[:,i_X], y_train[:,nannot:], verbose=False)
            y_pred = model_tree.predict(X_test[:,i_X]) 
            y_pred0 = model_tree.predict(X_train[:,i_X]) 
        
        if (met==3)|(met==1)|(met==4)|(met==5)|(met==0):
            y_pred = y_pred.reshape((-1,1))
            y_pred0 = y_pred0.reshape((-1,1))
            
        
                
        stpn_test=[s+'_test' for s in stp_n]
        stpn_pred=[s+'_pred' for s in stp_n]
        
        yy = np.concatenate((y_test, y_pred),axis=1)
        #yy = pd.DataFrame(yy,columns=annot_columns_train +stpn_test[0:len(nsel)] +stpn_pred[0:len(nsel)])
        yy = pd.DataFrame(yy,columns=annot_columns_train +stpn_test +stpn_pred)
        yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
        yy1 = yy.set_index(annot_columns2)
        ###yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
        yy1=yy1.astype(float).groupby(annot_columns2).mean()
        
        yy00 = np.concatenate((y_train, y_pred0),axis=1)
        yy00 = pd.DataFrame(yy00,columns=annot_columns_train +stpn_test +stpn_pred)
        yy00.loc[:,annot_columns2] = yy00.loc[:,annot_columns2].astype(str)
        yy100 = yy00.set_index(annot_columns2)
        ###yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
        yy100=yy100.astype(float).groupby(annot_columns2).mean()
        
        #y_test2 = y_test[:,nannot:]
        #y_train2 = y_train[:,nannot:]
        r3=[]
        r5=[]
        r300=[]
        r500=[]
        for i in range(y_pred.shape[1]):
            if 1: #(i==np.array([0,2,4])).sum()>0:
                f, ax =plt.subplots(figsize=(15, 5))
                ##f, ax = plt.figure()
                ##ax = f.add_axes()
                ##plt.title(stp_columns[i])
                ax.set_title(stp_columns_train[i]+' '+', subclass out of bag: '+str(i0))
                yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
                plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)
                
                
                f0, ax0 =plt.subplots(figsize=(15, 5))
                ##f, ax = plt.figure()
                ##ax = f.add_axes()
                ##plt.title(stp_columns[i])
                ax.set_title(stp_columns_train[i]+' '+', training erros: '+str(i0))
                yy100.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax0)
                plt.xticks(np.arange(len(yy100.index)), yy100.index, rotation=90)
                
                f1, ax1 =plt.subplots(figsize=(15, 5))
                ax.set_title(stp_columns_train[0]+' '+', subclass out of bag: '+str(i0))
                plt.plot(np.concatenate([y_test[:,[nannot]],y_pred.reshape((-1,1))],axis=1))

                f2, ax2 =plt.subplots(figsize=(15, 5))
                ax.set_title(stp_columns_train[0]+' '+', subclass out of bag: '+str(i0))
                plt.plot(np.concatenate([y_train[:,[nannot]],y_pred0.reshape((-1,1))],axis=1))


            #ytest= yy1.loc[:,stpn_test[i]]  
            #ypred=yy1.loc[:,stpn_pred[i]]
            
            ##r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
            i2 = nannot+i
            if np.var(y_test[:,i2])!=0:
                r2=1 - np.mean((y_test[:,i2] - y_pred[:,i])**2)/np.var(y_test[:,i2])
            else:
                r2=0
            r4=1 - np.mean((y_test[:,i2] - y_pred[:,i])**2)/np.var(y2[:,i2])
            
            r200=1 - np.mean((y_train[:,i2] - y_pred0[:,i])**2)/np.var(y_train[:,i2])
            r400=1 - np.mean((y_train[:,i2] - y_pred0[:,i])**2)/np.var(y2[:,i2])    
           # print(ytest.iloc[istr:].var())
           # print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
           # print(str(i) +' R^2 = '+str(r2))
            r3 = r3 + [r2]
            r5 = r5 + [r4]
            r300 = r300 + [r200]
            r500 = r500 + [r400]
        print('subclass out of bag: '+str(i0))
        
        err =  y_test[:,[nannot]]-y_pred.reshape((-1,1))
        print('mean error_test**2 = '+str(np.mean(err**2)))
        err2 =  y_train[:,[nannot]]-y_pred0.reshape((-1,1))
        print('mean error_train**2 = '+str(np.mean(err2**2)))
        print('var all = '+str(np.mean(y2[:,[nannot]]**2) -np.mean(y2[:,[nannot]])**2))

        
        print('mean R^2 total = '+str(np.array(r5).mean()))
        print('all R^2 total = '+str(np.array(r5)))
        
        print('mean R^2 train total = '+str(np.array(r500).mean()))
        print('all R^2 train total = '+str(np.array(r500)))
        
        print('mean fitted var = '+str(np.array(r3).mean()))
        print('all fitted var = '+str(np.array(r3)))

        
        if met==5:
            tr=model_tree.models[0]
            df = tree_to_df(tr.tree)
            
            X3, y3 = df_tree_to_data(df)
            X3 = pd.DataFrame(X3,columns = ge_n + cla_n + ['leaves'])
            y3 = pd.DataFrame(y3,columns = stp_n)
            #X3.to_excel('X3.xlsx')
            X4 = count_subclasses_in_leaves(X3,cla_n)
            print(X4)

        r6 = r6 + [np.array(r3).mean()]
        r7 = r7 + [np.array(r5).mean()]
        
        if met==1:
            print('out of bag R^2: '+str(regr.oob_score_))

    rr = np.array([r6,r7])
    rr = pd.DataFrame(rr)


    
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))
#print(regr.predict([[0, 0, 0, 0]]))



In [ ]:
f1, ax1 =plt.subplots(figsize=(15, 5))
ax.set_title(stp_columns_train[0]+' '+', subclass out of bag: '+str(i0))
plt.plot(np.concatenate([y_test[:,[nannot]],y_pred.reshape((-1,1))],axis=1))


#plt.plot(err,color='g')
err =  y_test[:,[nannot]]-y_pred.reshape((-1,1))
print('mean error_test**2 = '+str(np.mean(err**2)))
err2 =  y_train[:,[nannot]]-y_pred0.reshape((-1,1))
print('mean error_train**2 = '+str(np.mean(err2**2)))
print('var all = '+str(np.mean(y2[:,[nannot]]**2) -np.mean(y2[:,[nannot]])**2))

f2, ax2 =plt.subplots(figsize=(15, 5))
ax.set_title(stp_columns_train[0]+' '+', subclass out of bag: '+str(i0))
plt.plot(np.concatenate([y_train[:,[nannot]],y_pred0.reshape((-1,1))],axis=1))

## Approach for testing leaves linearity for each gene aka Pavilidis 2019

In [ ]:
plt.plot(y_pred_i,'ro')

In [ ]:
# F-test of regression models with and without subclass splitting (Pavilidis 2019)

import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.graphics.api import interaction_plot, abline_plot
from statsmodels.stats.anova import anova_lm

formula = "A2_20Hz ~ C(ex_in)  +  pre__Cplx1"
lm = ols(formula, data=X_train0[0:X_train0.shape[0]:10]).fit()
#print(lm.summary())

In [ ]:
ii=np.nonzero(np.array(ge_columns_train)=='pre__Cplx1')[0]
ge_columns_train[ii[0]]

In [ ]:
np.array(ge_columns_train)

In [ ]:
#print(lm.__doc__)

In [ ]:
# Load important genes from iRF search

%matplotlib inline
import matplotlib.pyplot as plt
imp  = np.load('importance.npy')
imp = pd.DataFrame(imp)
imp = pd.concat([imp,pd.DataFrame(ge_columns_train)],axis=1)
imp.columns = ['importance', 'genes']
imp = imp.set_index('genes')

imp0  = np.load('importance0.npy')
imp0 = pd.DataFrame(imp0)
imp0 = pd.concat([imp0,pd.DataFrame(ge_columns_train)],axis=1)
imp0.columns = ['importance0', 'genes']
imp0 = imp0.set_index('genes')


# List of important genes : iRF
imp = pd.concat([imp,imp0],axis=1)

imp50 = imp.sort_values('importance',ascending=False).iloc[0:49,:]
plt.plot(imp.sort_values('importance',ascending=False).loc[:,['importance','importance0']].values)
imp50

In [ ]:
imp50.index[0:15].tolist()

In [ ]:
#import pixiedust

### train and test a model tree with linear leaves

In [ ]:
#%%pixie_debugger

import time

import numpy as np
import pandas as pd


d_mtree = "/home/stepaniu/LearningX/advanced_ML/model_tree"

import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, d_mtree)

#from models.linear_regr import linear_regr
import models.linear_regr as lir

from models.svm_regr import svm_regr
from models.DT_sklearn_regr import DT_sklearn_regr

from models.modal_clf import modal_clf
from models.DT_sklearn_clf import DT_sklearn_clf

import os, pickle, csv

import src.ModelTree as mtree
from src.utils import load_csv_data, cross_validate


from importlib import reload
mtree = reload(mtree)
lir=reload(lir)



# ====================
# Settings
# ====================
mode = "regr"  # "clf" / "regr"
save_model_tree = True  # save model tree?
save_model_tree_predictions = True  # save model tree predictions/explanations?
cross_validation = True  # cross-validate model tree?

# ====================
# Load data
# ====================
#data_csv_data_filename = os.path.join(d_mtree,"data", "data_clf.csv")
#X, y, header = load_csv_data(data_csv_data_filename, mode=mode, verbose=True)

#X2 = np.copy(X[range(0,X.shape[0],10)])
#y2 = np.copy(y[range(0,X.shape[0],10)])
#nsel = np.arange(1)
#y2=y2[:,0:nannot+nsel.shape[0]]


# add unspliting class
#cl1 = np.zeros((X.shape[0],1))
#cl1[0:200]=1
#cl1[1000:1200]=2


#X2 = np.concatenate([X2,cl1],axis=1)
#i_cl = [X2.shape[1]-1]

# ['all', 'ex_inh']
Dn = 10
cla_n = ['ex_in_pre','ex_in_post']


ge_n = imp50.index[0:25].tolist()
X2 = X_train0.loc[:,ge_n + cla_n]
i_cl = np.nonzero(X2.columns.isin(cla_n))[0]
X2=X2.iloc[0:X_train0.shape[0]:Dn,:].values
stp_n = ['A2_20Hz']
#  'A2_20Hz',
#  'A3_20Hz',
#  'A4_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A1_50Hz',
#  'A2_50Hz',
#  'A3_50Hz',
#  'A4_50Hz',
#  'A5_50Hz',
#  'A250_50Hz',
#  'A1000_50Hz',
#  'A1_10Hz',
#  'A2_10Hz',
#  'A3_10Hz',
#  'A4_10Hz',
#  'A5_10Hz',
#  'A250_10Hz',
#  'A1000_10Hz']
y2 =  X_train0.loc[:,stp_n].iloc[0:X_train0.shape[0]:Dn,:].values
#header = ge_n + cla_n + stp_n 
header = ge_n  + stp_n 

t1=time.time()

# Choose model
model = lir.linear_regr()
model.lambda_classes  = 0.1
model.class_index = i_cl

# Build model tree
model_tree = mtree.ModelTree(model, max_depth=3, min_samples_leaf=10,
                       search_type="adaptive", n_search_grid=25)

#model_tree = mtree.ModelTree(model, max_depth=4, min_samples_leaf=10,
#                       search_type="greedy", n_search_grid=25)

# ====================
# Train model tree
# ====================
print("Training model tree with '{}'...".format(model.__class__.__name__))
model_tree.fit(X2, y2, verbose=True)

t2=time.time() 

y2_pred = model_tree.predict(X2)

t3=time.time() 

explanations = model_tree.explain(X2, header)
loss = model_tree.loss(X2, y2, y2_pred)
print(" -> loss_train={:.6f}\n".format(loss))
model_tree.export_graphviz(os.path.join("output", "model_tree"), header,
                           export_png=True, export_pdf=False)

print("elapsed time : ",[t2-t1,t3-t2])

# ====================
# Save model tree results
# ====================
if save_model_tree:
    model_tree_filename = os.path.join("output", "model_tree.p")
    print("Saving model tree to '{}'...".format(model_tree_filename))
    pickle.dump(model, open(model_tree_filename, 'wb'))

if save_model_tree_predictions:
    predictions_csv_filename = os.path.join("output", "model_tree_pred.csv")
    print("Saving mode tree predictions to '{}'".format(predictions_csv_filename))
    with open(predictions_csv_filename, "w") as f:
        writer = csv.writer(f)
        field_names = ["x2", "y2", "y2_pred", "explanation"]
        writer.writerow(field_names)
        for (x_i, y_i, y_pred_i, exp_i) in zip(X2, y2, y2_pred, explanations):
            field_values = [x_i, y_i, y_pred_i, exp_i]
            writer.writerow(field_values)

# ====================
# Cross-validate model tree
# ====================
cross_validation = 0
if cross_validation:
    t5 = time.time()
    cross_validate(model_tree, X2, y2, kfold=5, seed=1)
    t6 = time.time()
    print("time for cv : ", t6-t5)

### Test and train an RF of model_trees

In [ ]:
X2.shape

In [ ]:
#%%pixie_debugger

import time

import numpy as np
import pandas as pd


d_mtree = "/home/stepaniu/LearningX/advanced_ML/model_tree"

import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, d_mtree)

#from models.linear_regr import linear_regr
import models.linear_regr as lir

from models.svm_regr import svm_regr
from models.DT_sklearn_regr import DT_sklearn_regr

from models.modal_clf import modal_clf
from models.DT_sklearn_clf import DT_sklearn_clf

import os, pickle, csv

import src.ModelTree as mtree
from src.utils import load_csv_data, cross_validate


from importlib import reload
mtree = reload(mtree)
lir=reload(lir)



# ====================
# Settings
# ====================
mode = "regr"  # "clf" / "regr"
save_model_tree = True  # save model tree?
save_model_tree_predictions = True  # save model tree predictions/explanations?
cross_validation = True  # cross-validate model tree?

# ====================
# Load data
# ====================
#data_csv_data_filename = os.path.join(d_mtree,"data", "data_clf.csv")
#X, y, header = load_csv_data(data_csv_data_filename, mode=mode, verbose=True)

#X2 = np.copy(X[range(0,X.shape[0],10)])
#y2 = np.copy(y[range(0,X.shape[0],10)])
#nsel = np.arange(1)
#y2=y2[:,0:nannot+nsel.shape[0]]


# add unspliting class
#cl1 = np.zeros((X.shape[0],1))
#cl1[0:200]=1
#cl1[1000:1200]=2


#X2 = np.concatenate([X2,cl1],axis=1)
#i_cl = [X2.shape[1]-1]

# ['all', 'ex_inh']
Dn = 10
cla_n = ['ex_in_pre','ex_in_post']
cla_n = decode_classes2(cla_n,modf3)

ge_n = imp50.index[0:50].tolist()
X2 = X_train0.loc[:,ge_n + cla_n]
i_cl = np.nonzero(X2.columns.isin(cla_n))[0]
X2=X2.iloc[0:X_train0.shape[0]:Dn,:].values
stp_n = ['A2_20Hz']
#  'A2_20Hz',
#  'A3_20Hz',
#  'A4_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A1_50Hz',
#  'A2_50Hz',
#  'A3_50Hz',
#  'A4_50Hz',
#  'A5_50Hz',
#  'A250_50Hz',
#  'A1000_50Hz',
#  'A1_10Hz',
#  'A2_10Hz',
#  'A3_10Hz',
#  'A4_10Hz',
#  'A5_10Hz',
#  'A250_10Hz',
#  'A1000_10Hz']
y2 =  X_train0.loc[:,stp_n].iloc[0:X_train0.shape[0]:Dn,:].values
#header = ge_n + cla_n + stp_n 
header = ge_n  + stp_n 

t1=time.time()

# Choose model
model = lir.linear_regr()
model.lambda_classes  = 0.1
model.class_index = i_cl

# Build model tree RF
#model_tree = mtree.ModelTree(model, max_depth=4, min_samples_leaf=10,
#                       search_type="greedy", n_search_grid=100)
model_tree = mtree.RFRegressor(n_estimators=50, max_depth=4,
                 min_samples_split=2, min_samples_leaf=10, DTalgo='modeltree',
                 model_leaf = model, n_search_grid = 25, n_search_type = "adaptive",  
                 n_bootstrap=100, bootstrap_type='random', bootstrap_classes=None)

# ====================
# Train model tree
# ====================
print("Training model tree with '{}'...".format(model.__class__.__name__))
model_tree.fit(X2, y2, verbose=False)

t2=time.time() 

y2_pred = y2 #model_tree.predict(X2)

t3=time.time() 

#explanations = model_tree.explain(X2, header)
loss = 0 #model_tree.loss(X2, y2, y2_pred)
print(" -> loss_train={:.6f}\n".format(loss))


#model_tree.export_graphviz(os.path.join("output", "model_tree"), header,
#                           export_png=True, export_pdf=False)

print("elapsed time : ",[t2-t1,t3-t2])

# ====================
# Save model tree results
# ====================
if save_model_tree:
    model_tree_filename = os.path.join("output", "model_tree.p")
    print("Saving model tree to '{}'...".format(model_tree_filename))
    pickle.dump(model, open(model_tree_filename, 'wb'))

if save_model_tree_predictions:
    predictions_csv_filename = os.path.join("output", "model_tree_pred.csv")
    print("Saving mode tree predictions to '{}'".format(predictions_csv_filename))
    with open(predictions_csv_filename, "w") as f:
        writer = csv.writer(f)
        field_names = ["x2", "y2", "y2_pred", "explanation"]
        writer.writerow(field_names)
        
        # for (x_i, y_i, y_pred_i, exp_i) in zip(X2, y2, y2_pred, explanations):
        for (x_i, y_i, y_pred_i, exp_i) in zip(X2, y2, y2_pred, y2_pred):
            field_values = [x_i, y_i, y_pred_i, exp_i]
            writer.writerow(field_values)

# ====================
# Cross-validate model tree
# ====================
cross_validation = 0
if cross_validation:
    t5 = time.time()
    cross_validate(model_tree, X2, y2, kfold=5, seed=1)
    t6 = time.time()
    print("time for cv : ", t6-t5)

In [ ]:
cross_validation = 0
if cross_validation:
    t5 = time.time()
    cross_validate(model_tree, X2, y2, kfold=5, seed=1)
    t6 = time.time()
    print("time for cv : ", t6-t5)

In [ ]:
np.histogram([1,1,2,2,2,3,3,4,9])

In [ ]:
model_tree.models[0].__dict__

In [ ]:
def tree_to_df(tr,node_code='0'):
    columns = ['node_code','j_feature','threshold',
               'n_samples','y','X','loss']
    df = pd.DataFrame(np.zeros((1,7)),columns = columns)
    
    #if 'tree' in tr:
    #        tr=tr.tree
    df.loc[0,'loss'] = tr['loss']
    df.loc[0,'j_feature'] = tr['j_feature']
    df.loc[0,'threshold'] = tr['threshold']
    df.loc[0,'n_samples'] = tr['n_samples']
    df.loc[0,'node_code'] = node_code
    if 'data' in tr:  
        #print(tr['data'])
        df.loc[0,'X'] = [tr['data'][0]]
        df.loc[0,'y'] = [tr['data'][1]]
    else:
        dfl = tree_to_df(tr['children']['left'], node_code+'0')
        dfr = tree_to_df(tr['children']['right'],node_code+'1')

        df = pd.concat([df,dfl,dfr],axis=0).reset_index(drop=True)
        
    return df        

In [ ]:
tr=model_tree.models[0]
df = tree_to_df(tr.tree)
df

In [ ]:
def df_tree_to_data(df):
    k=0
    isX3=0
    y3=0
    for i in df.index:
        if (np.isnan(df.loc[i,'j_feature'])):
            Xi = df.loc[i,'X']
            #print(Xi.shape)
            y3i = df.loc[i,'y']
            X3i  = np.concatenate([Xi,np.ones((Xi.shape[0],1))*i],axis=1)
            if isX3==0:
                X3=X3i
                y3=y3i
                isX3=1
            else: 
                X3 = np.concatenate([X3,X3i],axis=0)
                y3 = np.concatenate([y3,y3i],axis=0)

    return X3, y3

In [ ]:
def  count_subclasses_in_leaves(X3,cla_n):
    lv=list(set(X3.loc[:,'leaves']))
    nx3 = X3.shape[0]
    ncl = int(X3.loc[:,cla_n].max().max()+1)
    nclan=len(cla_n)

    X4 = np.zeros((ncl*nclan,len(lv)))
    nx4 = ncl*nclan
    classes4 = (np.arange(nx4)/nx4*nclan).astype(int).reshape((nx4,1))
    subclasses4 = np.mod(np.arange(nx4),nclan).reshape((nx4,1))
    X4 = np.concatenate([classes4, subclasses4, X4],axis=1)
    X4 = pd.DataFrame(X4, columns=['classes','subclasses']+lv)

    for lvi in lv:
        X3i = X3.loc[X3.loc[:,'leaves']==lvi,cla_n]
        for icl,cla in enumerate(cla_n):
            for cl in range(ncl):
                X4.loc[cl+ncl*icl,lvi]=(X3i.loc[:,cla]==cl).sum()
                
    return X4            

In [ ]:
def gini_impurity(y):
    p2 = 0.0
    y_classes = list(set(y))
    for c in y_classes:
        p2 += (np.sum(y == c) / len(y))**2
    loss = p2
    return loss

In [ ]:
def loss(self, X, y, y_pred):
    if self.lambda_classes==0:
        l = mean_squared_error(y, y_pred)
    else:
        X_cl = X[:,self.class_index]
        gi=0
        for i in range(len(self.class_index)):
            x_cl = X_cl[:,i]
            gi = gi + 1-gini_impurity(x_cl[x_cl>0])  # x_cl==0 - should be subclasses not included in class i

        gi=gi*np.sum(x_cl>0)/x_cl.shape[0]
        l = mean_squared_error(y, y_pred) + self.lambda_classes*gi
    return l

In [ ]:
X3, y3 = df_tree_to_data(df)
X3 = pd.DataFrame(X3,columns = ge_n + cla_n + ['leaves'])
y3 = pd.DataFrame(y3,columns = stp_n)

X4 = count_subclasses_in_leaves(X3,cla_n)
    
X3.to_excel('X3.xlsx')
X4

In [ ]:
# compare  X_train  with  X3

## Crossvalidation - RF model_trees

In [ ]:
import pandas as pd
import numpy as np
columns  = pd.read_hdf('stp_to_ge_all_columns.hdf')
ge_columns_train      = columns.loc[:,'ge_columns_train'].values[0]
annot_columns_train   = columns.loc[:,'annot_columns_train'].values[0]
stp_columns_train     = columns.loc[:,'stp_columns_train'].values[0]
classes_columns_train = columns.loc[:,'classes_columns_train'].values[0]
#print(X_train.columns)
X_train0 = pd.read_hdf('stp_to_ge_all_data.hdf')

In [ ]:
classes_columns_train

In [ ]:
X_train0.columns[316:]

In [ ]:
#X2 = X_train0.loc[:,annot_columns_train + ge_n + cla_n]
X2 = X_train0.loc[:, annot_columns_train + cla_n ]
X2.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression


from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import sklearn as sk

import time
t1 = time.time()

do_oob_subclasses=0

#X2 = np.copy(X[range(0,X.shape[0],1)])
#y2 = np.copy(y[range(0,X.shape[0],1)])
#nsel = np.arange(1)
#y2=y2[:,0:nannot+nsel.shape[0]]

import models.linear_regr as lir
import models.mean_regr as mir

from models.svm_regr import svm_regr
from models.DT_sklearn_regr import DT_sklearn_regr

from importlib import reload
mtree = reload(mtree)
lir=reload(lir)
mir=reload(mir)

# ====================
# Settings
# ====================
mode = "regr"  # "clf" / "regr"
save_model_tree = True  # save model tree?
save_model_tree_predictions = True  # save model tree predictions/explanations?
cross_validation = True  # cross-validate model tree?

# ====================
# Load data
# ====================
nannot = len(annot_columns_train)
lge_n  = len(ge_n)

# ['all', 'ex_inh']
Dn = 1
#cla_n = ['ex_in_ex_in__pre',
#         'ex_in_ex_in__post'] 
cla_n = ['ex_in_ex_in__pre'] 

ge_n = imp50.index[0:25].tolist()
X2 = X_train0.loc[:,annot_columns_train + ge_n + cla_n]
i_cl = np.nonzero(X2.columns.isin(cla_n))[0] - nannot

X2.loc[:,cla_n] = X2.loc[:,cla_n]+1

X2=X2.iloc[0:X_train0.shape[0]:Dn,:].values
stp_n = ['A2_20Hz']

# 'A2_20Hz',
#  'A3_20Hz',
#  'A4_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A1_50Hz',
#  'A2_50Hz',
#  'A3_50Hz',
#  'A4_50Hz',
#  'A5_50Hz',
#  'A250_50Hz',
#  'A1000_50Hz',
#  'A1_10Hz',
#  'A2_10Hz',
#  'A3_10Hz',
#  'A4_10Hz',
#  'A5_10Hz',
#  'A250_10Hz',
#  'A1000_10Hz']
y2 =  X_train0.loc[:,annot_columns_train+stp_n].iloc[0:X_train0.shape[0]:Dn,:].values
#header = ge_n + cla_n + stp_n 
header = ge_n  + stp_n 
annot_columns2 = annot_columns_train
 

if do_oob_subclasses==0:
    ## select datasets for training and validation
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0) 

    #X, y = make_regression(n_features=4, n_informative=2,
    #                        random_state=0, shuffle=False)

    kf = KFold(n_splits=10, shuffle=False)
    i0=0
    r6=[]
    r7=[]
    for train_index, test_index in kf.split(X2):
        i0=i0+1
        print('\n\n '+ str(i0))
        #print("TRAIN:", train_index,"\n", "TEST:", test_index)
        
        X_train, X_test = X2[train_index], X2[test_index]
        y_train, y_test = y2[train_index], y2[test_index]
        
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0) 

        #
        #
        #  ML METHOD
        #
        #
        met=5
        
        if (met<4):
            i_X = np.arange(nannot,nannot+lge_n)
        else:
            i_X = np.arange(nannot,X_train.shape[1])
        
        if met==1:
            regr = RandomForestRegressor(random_state=2026,max_depth=4,min_samples_leaf=10,
                                    n_estimators=50, oob_score=True, n_jobs=-1, max_samples=200)
                    # RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
            #           max_features='auto', max_leaf_nodes=None,
            #           min_impurity_decrease=0.0, min_impurity_split=None,
            #           min_samples_leaf=1, min_samples_split=2,
            #           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            #           oob_score=False, random_state=0, verbose=0, warm_start=False)
        elif met==2:
            if i0==1:
                regr = sk.linear_model.MultiTaskElasticNetCV(cv=5,n_jobs=-1)
                regr.fit(X[:,nannot:], y[:,nannot:])
                print("alpha ",regr.alpha_)
                print("l1_ratio ",regr.l1_ratio_)
        
            regr = sk.linear_model.MultiTaskElasticNet(alpha=regr.alpha_, l1_ratio=regr.l1_ratio_,
                                                       fit_intercept=True,
                                                       normalize=False,
                                 max_iter=1000, copy_X=True, tol=0.0001,
                                                       warm_start=False, positive=False, 
                                                       random_state=None, selection='cyclic')
        
        elif met==3:
            regr = sk.svm.SVR(kernel='rbf', degree=3, gamma='scale',
                    coef0=0.0, tol=0.001, C=3.0, epsilon=0.1, shrinking=True, 
                              cache_size=200, verbose=False, max_iter=-1)
        
        
        elif met==4:
            # Choose model
            model = lir.linear_regr()
            model.lambda_classes  = 0.1
            model.class_index = i_cl

            # Build model tree
            model_tree = mtree.ModelTree(model, max_depth=4, min_samples_leaf=10,
                                   search_type="adaptive", n_search_grid=20)
            
        elif met==5:
            # Choose model
            #model = lir.linear_regr()
            model = mir.mean_regr()
            model.lambda_classes  = 0.0
            model.class_index = i_cl

            # Build model tree
            model_tree = mtree.RFRegressor(n_estimators=50, max_depth=4,
                 min_samples_split=2, min_samples_leaf=5, DTalgo='modeltree',
                 model_leaf = model, n_search_grid = 25, n_search_type = "adaptive",  
                 n_bootstrap=93*5, bootstrap_type='sequential', bootstrap_classes=None,n_bootstrap_groups=93)



        if met==3:
            #regr.fit(X_train[:,6:], y_train[:,0:].ravel())  
            regr.fit(X_train[:,i_X], y_train[:,nannot:].ravel())  
            y_pred = regr.predict(X_test[:,i_X]) 
        elif met<4:
            regr.fit(X_train[:,i_X], y_train[:,nannot:])
            y_pred = regr.predict(X_test[:,i_X]) 
        elif met==4:
            # ====================
            # Train model tree
            # ====================
            print("Training model tree with '{}'...".format(model.__class__.__name__))
            model_tree.fit(X_train[:,i_X], y_train[:,nannot:], verbose=True)
            y_pred = model_tree.predict(X_test[:,i_X])
        elif met==5:    
            print("Training forest of model trees with '{}'...".format(model.__class__.__name__))
            model_tree.fit(X_train[:,i_X], y_train[:,nannot:], verbose=False)
            y_pred = model_tree.predict(X_test[:,i_X]) 
        
        if (met==3)|(met==1)|(met==4)|(met==5):
            y_pred = y_pred.reshape((-1,1))
        y_test2 = y_test[:,nannot:]
        
        
        yy = np.concatenate((y_test, y_pred),axis=1)
        stpn_test=[s+'_test' for s in stp_n]
        stpn_pred=[s+'_pred' for s in stp_n]
        #yy = pd.DataFrame(yy,columns=annot_columns_train +stpn_test[0:len(nsel)] +stpn_pred[0:len(nsel)])
        
        yy = pd.DataFrame(yy,columns=annot_columns_train +stpn_test +stpn_pred)
        
        yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
        yy1 = yy.set_index(annot_columns2)

        ###yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
        yy1=yy1.astype(float).groupby(annot_columns2).mean()
        r3=[]
        r5=[]
        for i in range(y_pred.shape[1]):
            if (i==np.array([0,2,4])).sum()>0:
                f, ax =plt.subplots(figsize=(16, 10))
                ##f, ax = plt.figure()
                ##ax = f.add_axes()
                ##plt.title(stp_columns[i])
                ax.set_title(stp_columns_train[i]+' '+', subclass out of bag: '+str(i0))
                yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
                plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)


            #ytest= yy1.loc[:,stpn_test[i]]  
            #ypred=yy1.loc[:,stpn_pred[i]]
            
            ##r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
            if np.var(y_test2[:,i])!=0:
                r2=1 - np.mean((y_test2[:,i] - y_pred[:,i])**2)/np.var(y_test2[:,i])
                r4=1 - np.mean((y_test2[:,i] - y_pred[:,i])**2)/np.var(y2[:,nannot+i])
           # print(ytest.iloc[istr:].var())
           # print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
           # print(str(i) +' R^2 = '+str(r2))
            r3 = r3 + [r2]
            r5 = r5 + [r4]
        print('subclass out of bag: '+str(i0))
        print('mean R^2 = '+str(np.array(r3).mean()))
        print('all R^2 = '+str(np.array(r3)))
        print('mean R^2 total = '+str(np.array(r5).mean()))
        print('all R^2 total = '+str(np.array(r5)))
        
        if met==5:
            tr=model_tree.models[0]
            df = tree_to_df(tr.tree)
            
            X3, y3 = df_tree_to_data(df)
            X3 = pd.DataFrame(X3,columns = ge_n + cla_n + ['leaves'])
            y3 = pd.DataFrame(y3,columns = stp_n)
            #X3.to_excel('X3.xlsx')
            X4 = count_subclasses_in_leaves(X3,cla_n)
            print(X4)

        r6 = r6 + [np.array(r3).mean()]
        r7 = r7 + [np.array(r5).mean()]
        
        if met==1:
            print('out of bag R^2: '+str(regr.oob_score_))

    rr = np.array([r6,r7])
    rr = pd.DataFrame(rr)


    
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))
#print(regr.predict([[0, 0, 0, 0]]))



In [ ]:
import matplotlib

%matplotlib widget

import matplotlib.pyplot as plt


plt.rcParams["figure.figsize"] = (20,10)
#plt.plot([1,2,3])

A=plt.imread('output/model_tree.png')
fig, ax = plt.subplots() #(figsize=(5,10))
im = ax.imshow(A)
plt.show()

In [ ]:
# 
sb0 = np.arange(len(classes))
mods  = []
mods2  = []
for i in range(classes.shape[1]):
    sbi = classes.loc['subclasses',i]
    print(sbi)
        
    subclasses3 = subclasses2[sbi]
    X_train, y_train =  make_data(X,y,ge_data_subcl,[subclasses3,[]])   
    regr = sk.linear_model.MultiTaskElasticNet(alpha=0.1, l1_ratio=0.5, fit_intercept=True,
                                                   normalize=False,max_iter=1000, copy_X=True, tol=0.0001, 
                                        warm_start=False, random_state=None, selection='cyclic')
    regr.fit(X_train[:,nannot:], y_train[:,nannot:])
    
    
    X_train2, y_train2 =  make_data(X,y,ge_data_subcl,[[],subclasses3]) 
    regr2 = sk.linear_model.MultiTaskElasticNet(alpha=0.1, l1_ratio=0.5, fit_intercept=True,
                                                   normalize=False,max_iter=1000, copy_X=True, tol=0.0001, 
                                        warm_start=False, random_state=None, selection='cyclic')
    regr2.fit(X_train2[:,nannot:], y_train2[:,nannot:])
    
    mods = mods + [regr]
    mods2 = mods2 + [regr2]
    #X_test = X[in_subcl,:]
    #y_test = y[in_subcl,:]

In [ ]:
print(X.shape)
y.shape

In [ ]:
y_pred.shape
y_train.shape

In [ ]:
rr # elnet a=0.1

In [ ]:
rr # elastic net a=1

In [ ]:
rr # cv 300

In [ ]:
rr # cv 1000

In [ ]:
rr # cv 8000

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression


from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import sklearn as sk

import time
t1 = time.time()

do_oob_subclasses=0

X2 = np.copy(X[range(0,X.shape[0],1)])
y2 = np.copy(y[range(0,X.shape[0],1)])
nsel = np.arange(1)
y2=y2[:,0:nannot+nsel.shape[0]]

if do_oob_subclasses==0:
    ## select datasets for training and validation
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0) 

    #X, y = make_regression(n_features=4, n_informative=2,
    #                        random_state=0, shuffle=False)

    kf = KFold(n_splits=10, shuffle=False)
    i0=0
    r6=[]
    r7=[]
    for train_index, test_index in kf.split(X2):
        i0=i0+1
        print('\n\n '+ str(i0))
        #print("TRAIN:", train_index,"\n", "TEST:", test_index)
        
        X_train, X_test = X2[train_index], X2[test_index]
        y_train, y_test = y2[train_index], y2[test_index]
        
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0) 


        met=1
        
        if met==1:
            regr = RandomForestRegressor(random_state=2026,max_depth=8,min_samples_leaf=1,
                                    n_estimators=2000, oob_score=True, n_jobs=-1, max_samples=500)
                    # RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
            #           max_features='auto', max_leaf_nodes=None,
            #           min_impurity_decrease=0.0, min_impurity_split=None,
            #           min_samples_leaf=1, min_samples_split=2,
            #           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            #           oob_score=False, random_state=0, verbose=0, warm_start=False)
        elif met==2:
            if i0==1:
                regr = sk.linear_model.MultiTaskElasticNetCV(cv=5,n_jobs=-1)
                regr.fit(X[:,nannot:], y[:,nannot:])
                print("alpha ",regr.alpha_)
                print("l1_ratio ",regr.l1_ratio_)
        
            regr = sk.linear_model.MultiTaskElasticNet(alpha=regr.alpha_, l1_ratio=regr.l1_ratio_, fit_intercept=True,
                                                       normalize=False,
                                 max_iter=1000, copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')
        
        elif met==3:
            regr = sk.svm.SVR(kernel='rbf', degree=3, gamma='scale',
                    coef0=0.0, tol=0.001, C=3.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
        
        if met==3:
            #regr.fit(X_train[:,6:], y_train[:,0:].ravel())  
            regr.fit(X_train[:,nannot:], y_train[:,nannot:].ravel())  
        else:
            regr.fit(X_train[:,nannot:], y_train[:,nannot:])


        y_pred = regr.predict(X_test[:,nannot:]) 
        if (met==3)|(met==1):
            y_pred = y_pred.reshape((-1,1))
        y_test2 = y_test[:,nannot:]
        
        
        yy = np.concatenate((y_test, y_pred),axis=1)
        stpn_test=[s+'_test' for s in stp_columns]
        stpn_pred=[s+'_pred' for s in stp_columns]
        yy = pd.DataFrame(yy,columns=annot_columns +stpn_test[0:len(nsel)] +stpn_pred[0:len(nsel)])
        yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
        yy1 = yy.set_index(annot_columns2)

        ###yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
        yy1=yy1.astype(float).groupby(annot_columns2).mean()
        r3=[]
        r5=[]
        for i in range(y_pred.shape[1]):
            if (i==np.array([0,2,4])).sum()>0:
                f, ax =plt.subplots(figsize=(16, 10))
                ##f, ax = plt.figure()
                ##ax = f.add_axes()
                ##plt.title(stp_columns[i])
                ax.set_title(stp_columns[i]+' '+', subclass out of bag: '+str(i0))
                yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
                plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)


            #ytest= yy1.loc[:,stpn_test[i]]  
            #ypred=yy1.loc[:,stpn_pred[i]]
            
            ##r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
            if np.var(y_test2[:,i])!=0:
                r2=1 - np.mean((y_test2[:,i] - y_pred[:,i])**2)/np.var(y_test2[:,i])
                r4=1 - np.mean((y_test2[:,i] - y_pred[:,i])**2)/np.var(y[:,nannot+i])
           # print(ytest.iloc[istr:].var())
           # print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
           # print(str(i) +' R^2 = '+str(r2))
            r3 = r3 + [r2]
            r5 = r5 + [r4]
        print('subclass out of bag: '+str(i0))
        print('mean R^2 = '+str(np.array(r3).mean()))
        print('all R^2 = '+str(np.array(r3)))
        print('mean R^2 total = '+str(np.array(r5).mean()))
        print('all R^2 total = '+str(np.array(r5)))

        r6 = r6 + [np.array(r3).mean()]
        r7 = r7 + [np.array(r5).mean()]
        
        if met==1:
            print('out of bag R^2: '+str(regr.oob_score_))

    rr = np.array([r6,r7])
    rr = pd.DataFrame(rr)
else:
    
    oobs = []
    errors = []
    annot_columns2 = annot_columns
    
    r6=[]
    r7=[]
    for i0 in range(len(subclasses2)):
        in_subcl = ge_data_subcl.loc[:,'subclass_pre']==subclasses2[i0]
        in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_post']==subclasses2[i0])
        in_subcl = np.nonzero(in_subcl.values)[0]

        X_train = np.delete(X, in_subcl,axis=0)
        y_train = np.delete(y, in_subcl,axis=0)
        #X_train = X[:,:]
        #y_train = y[:,:]
        
        X_test = X[in_subcl,:]
        y_test = y[in_subcl,:]
        
        do_this = 1

        X_train, X_test0, y_train, y_test0 = train_test_split(X, y, test_size=0.25, random_state=0) 
        
        print(X_train.shape)
        print(X_test.shape)
        
        if do_this==1:
            #if i0=0:
            regr = RandomForestRegressor(random_state=2026,
                                    n_estimators=500, oob_score=True, n_jobs=-1, max_samples=300)
                
        else:
            regr = RandomForestRegressor(random_state=2026,
                                    n_estimators=500, oob_score=True, n_jobs=-1)
        
        regr.fit(X_train[:,nannot:], y_train[:,nannot:])  
        
        y_pred = regr.predict(X_test[:,nannot:]) 
        y_test2 = y_test[:,nannot:]
        
        
        yy = np.concatenate((y_test, y_pred),axis=1)
        stpn_test=[s+'_test' for s in stp_columns]
        stpn_pred=[s+'_pred' for s in stp_columns]
        yy = pd.DataFrame(yy,columns=annot_columns +stpn_test +stpn_pred)
        yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
        yy1 = yy.set_index(annot_columns2)

        ###yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
        yy1=yy1.astype(float).groupby(annot_columns2).mean()
        r3=[]
        r5=[]
        for i in range(y_pred.shape[1]):
            if (i==np.array([0,1,9])).sum()>0:
                f, ax =plt.subplots(figsize=(16, 10))
                ##f, ax = plt.figure()
                ##ax = f.add_axes()
                ##plt.title(stp_columns[i])
                ax.set_title(stp_columns[i]+' '+', subclass out of bag: '+subclasses2[i0])
                yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
                plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)


            #ytest= yy1.loc[:,stpn_test[i]]  
            #ypred=yy1.loc[:,stpn_pred[i]]
            
            ##r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
            if np.var(y_test2[:,i])!=0:
                r2=1 - np.mean((y_test2[:,i] - y_pred[:,i])**2)/np.var(y_test2[:,i])
                r4=1 - np.mean((y_test2[:,i] - y_pred[:,i])**2)/np.var(y[:,nannot+i])
           # print(ytest.iloc[istr:].var())
           # print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
           # print(str(i) +' R^2 = '+str(r2))
            r3 = r3 + [r2]
            r5 = r5 + [r4]
        print('subclass out of bag: '+subclasses2[i0])
        print('mean R^2 = '+str(np.array(r3).mean()))
        print('all R^2 = '+str(np.array(r3)))
        print('mean R^2 total = '+str(np.array(r5).mean()))
        print('all R^2 total = '+str(np.array(r5)))

        r6 = r6 + [np.array(r3).mean()]
        r7 = r7 + [np.array(r5).mean()]
        oobs = oobs + [regr.oob_score_]
    print('out of bag R^2: '+str(oobs))
    
    rr = np.array([subclasses2,r6,r7])
    rr = pd.DataFrame(rr)
    
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))
#print(regr.predict([[0, 0, 0, 0]]))



In [ ]:
TO DO : CHECK R^2 for ca and dg, ee cases!!!

In [ ]:
yy

In [ ]:
print(regr.alpha_)
print(regr.__dict__)

In [ ]:
rr # out of 10 cross val

In [ ]:
rr # all data, 300 bs

In [ ]:
rr # all data, 1000 bs

In [ ]:
rr # cross val, 1000 bs

In [ ]:
rr # cross val, 300 bs

In [ ]:
rr # cross val, all bs

In [ ]:
rr  # cross val, 93 classes bs

In [ ]:
rr  # all data, classes bs

In [ ]:
ca1=1.6
ca2=2
k=2.79
# k=1.09

cc1=(k**4 + ca1**4)/ca1**4*ca2**4/(k**4+ca2**4)

k=1.09
cc2=(k**4 + ca1**4)/ca1**4*ca2**4/(k**4+ca2**4)

print(str([cc1,cc2,0.5*(cc1+cc2)]))

## Linear regression model

In [ ]:
# Linear regression

t1 = time.time()

#regr = RandomForestRegressor(max_depth=15, random_state=0,
#                            n_estimators=100)
#regr = RandomForestRegressor(random_state=2026,
#                            n_estimators=500)
import matplotlib.pyplot as plt 
from sklearn import linear_model
regr2 = linear_model.Lasso(alpha=0.01)
regr2.fit(X_train[:,nannot:], y_train[:,nannot:])  
#Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
#   normalize=False, positive=False, precompute=False, random_state=None,
#   selection='cyclic', tol=0.0001, warm_start=False)


#regr.fit(X_train[:,6:], y_train[:,0:].ravel())  
#regr.fit(X_train[:,nannot:], y_train[:,nannot:])  
# RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
#           max_features='auto', max_leaf_nodes=None,
#           min_impurity_decrease=0.0, min_impurity_split=None,
#           min_samples_leaf=1, min_samples_split=2,
#           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
#           oob_score=False, random_state=0, verbose=0, warm_start=False)
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))
#print(regr.predict([[0, 0, 0, 0]]))

# some analysis of results
y_predl = regr2.predict(X_test[:,nannot:])  
if len(y_predl.shape)==1:
    y_predl=y_predl.reshape((len(y_predl),1))
    
yyl = np.concatenate((y_test, y_predl),axis=1)
stpn_test=[s+'_test' for s in stp_columns]
stpn_pred=[s+'_pred' for s in stp_columns]
yyl = pd.DataFrame(yyl,columns=annot_columns +stpn_test +stpn_pred)




# make input and output stp labels dataset with errors
stpn_error = [s+'_error' for s in stp_columns]
yel = pd.DataFrame(yyl.loc[:,stpn_pred].values - yyl.loc[:,stpn_test].values, columns=stpn_error)
yyl = pd.concat([yyl, yel.abs()], axis=1)

#yy.set_index(['pre','post']).groupby()
#yy.set_index(['pre','post']).groupby(['pre','post']).mean()

# plot errors
annot_columns2 = annot_columns

fig, ax = plt.subplots(figsize=(16, 10))

#yy1l=yyl.set_index(annot_columns2)
yyl.loc[:,annot_columns2] = yyl.loc[:,annot_columns2].astype(str)
yy1l = yyl.set_index(annot_columns2)
yy1l.loc[:,stpn_error]=yy1l.loc[:,stpn_error].values/(yy1l.loc[:,stpn_test].values+1e-15)*100
yy1l=yy1l.astype(float).groupby(annot_columns2).mean()
yy1l.loc[:,stpn_error].plot(ax=ax)
plt.xticks(np.arange(len(yy1l.index)), yy1l.index, rotation=90)
plt.title('errors of An/A1 predicted by lasso linear regrassion, %')
plt.ylim((0,100))    


## plot test vs train STP data

1. plot all parameters
2. additional cross validation? - better estimate of generalization error?
3. check dependence on rf parameters
4. dependence on % of training data
5. lists of most significant genes
6. check gs assignment - plot some figures
7. project on hipp-ds -> predict stp, compare
8. an:a1 vs TM-par
## 
## new (sep 2019):
9. compare with linear fit
10. iRFcompare small gs predictions
11. compare with factors predicting neuron types, predict n-types using small ds
12. R^2
13. compare with list of STP molecular factors
14. relative errors
15. some graphs for particular factors?
16. iRF




In [ ]:
#annot_columns +stpn_test +stpn_pred


In [ ]:
print(X_train.shape)
print(y_train.shape)
y_test.shape

In [ ]:
# RF case
y_pred = regr.predict(X_test[:,nannot:]) 
if len(y_pred.shape)==1:
    y_pred=y_pred.reshape((len(y_pred),1))





from sklearn import metrics

#print('Mean Absolute Error:', metrics.mean_absolute_error(y_test[:,2:].ravel(), y_pred))  
#print('Mean Squared Error:', metrics.mean_squared_error(y_test[:,2:].ravel(), y_pred))  
#print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test[:,2:].ravel(), y_pred))) 

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test[:,nannot:], y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test[:,nannot:], y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test[:,nannot:], y_pred))) 


#y_pred2 = y_pred.reshape((len(y_pred),nstp))
yy = np.concatenate((y_test, y_pred),axis=1)
stpn_test=[s+'_test' for s in stp_columns]
stpn_pred=[s+'_pred' for s in stp_columns]
yy = pd.DataFrame(yy,columns=annot_columns +stpn_test +stpn_pred)




# make input and output stp labels dataset with errors
stpn_error = [s+'_error' for s in stp_columns]
ye = pd.DataFrame(yy.loc[:,stpn_pred].values - yy.loc[:,stpn_test].values, columns=stpn_error)
yy = pd.concat([yy, ye.abs()], axis=1)

#yy.set_index(['pre','post']).groupby()
#yy.set_index(['pre','post']).groupby(['pre','post']).mean()

# plot errors
annot_columns2 = annot_columns

fig, ax = plt.subplots(figsize=(16, 10))

#yy1=yy.set_index(annot_columns2)
yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
yy1 = yy.set_index(annot_columns2)
yy1.loc[:,stpn_error]=yy1.loc[:,stpn_error].values/(yy1.loc[:,stpn_test].values+1e-15)*100
yy1=yy1.astype(float).groupby(annot_columns2).mean()
yy1.loc[:,stpn_error].plot(ax=ax)
plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)
plt.title('errors of An/A1 predicted by RF, %')
plt.ylim((0,100))

In [ ]:
yy.to_excel('all_tested_rf_280gs_with_PPR.xlsx')
yy.head()

In [ ]:
# Linear model case

#plot predicted and real (test) stp data
#plt.figure()
#yy1l=yyl.set_index(annot_columns2)

#yyl.loc[:,annot_columns2] = yyl.loc[:,annot_columns2].astype(str)
yy1l = yyl.set_index(annot_columns2)
#yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
yy1l=yy1l.astype(float).groupby(annot_columns2).mean()

r3=[]
for i in range(nstp):
    f, ax =plt.subplots(figsize=(16, 10))
    #f, ax = plt.figure()
    #ax = f.add_axes()
    #plt.title(stp_columns[i])
    ax.set_title(stp_columns[i])
    yy1l.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
    plt.xticks(np.arange(len(yy1l.index)), yy1l.index, rotation=90)
    if i==0:
        istr=0
    else:
        istr=0 #11 - if ppr included
    
    ytest= yy1l.loc[:,stpn_test[i]]  
    ypred=yy1l.loc[:,stpn_pred[i]]
    
    if ytest.iloc[istr:].var()!=0:
        r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/(ytest.iloc[istr:].var() )
    print(str(i) +' R^2 = '+str(r2))
    r3 = r3 + [r2]

print('mean R^2 = '+str(np.array(r3).mean()))
#print(yy1.mean())
yy1l.loc[:,stpn_test +stpn_pred]

In [ ]:
# RF model case

#plot predicted and real (test) stp data
#plt.figure()

#yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
#yy1 = yy.set_index(annot_columns2)

##yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
yy1=yy1.astype(float).groupby(annot_columns2).mean()
r3=[]
for i in range(0,nstp):
    f, ax =plt.subplots(figsize=(16, 10))
    #f, ax = plt.figure()
    #ax = f.add_axes()
    #plt.title(stp_columns[i])
    ax.set_title(stp_columns[i])
    yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
    plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)
    if i==0:
        istr=0
    else:
        istr=0 # 11 - if ppr data excluded
        
    ytest= yy1.loc[:,stpn_test[i]]  
    ypred=yy1.loc[:,stpn_pred[i]]
    #r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
    if ytest.iloc[istr:].var()!=0:
        r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).var()/ytest.iloc[istr:].var()
    print(ytest.iloc[istr:].var())
    print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
    print(str(i) +' R^2 = '+str(r2))
    r3 = r3 + [r2]

print('mean R^2 = '+str(np.array(r3).mean()))

#f, ax =plt.subplots(figsize=(10, 10))
#yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)

#print(yy1.mean())
yy1.loc[:,stpn_test +stpn_pred]

In [ ]:
#yy1.to_excel('results_RF_10gs_without_PPR.xlsx')
yy1.to_excel('results_RF_10gs_with_PPR.xlsx')

## Look at most important genes

In [ ]:
#linear model case

# set of most significant genes
 #fil = regr2.feature_importances_

#plt.plot(fi)
fil = regr2.coef_
fil = np.absolute(fil)
#fil = np.sum(fil,axis=0).transpose()
fil = fil[0,:].transpose()
fil.shape

ifi = np.nonzero(fil)[0]
fi2l = fil[ifi]
ifi2 = np.argsort(fi2l)
l=len(fi2l)
#print(np.concatenate((ifi2.reshape((l,1)),fi2[ifi2].reshape((l,1))),axis=1))

print(len(ge_columns))
print(len(ge_data.columns))
print(fil.shape)
print(ge_columns[0:6])

fi2l = pd.DataFrame(fil,columns=['importance'],index = ge_columns[6:])
fi2l = pd.concat([fi2l, pd.DataFrame(np.repeat('post',len(fi2l.index)),columns = ['compartment'],index=fi2l.index)],axis=1)
#fi2.loc[np.arange(len(fi2.index))<len(pregs),'compartment']='pre'
fi2l.loc[fi2l.index.str.contains('pre'),'compartment']='pre'

fi2l.index =  [id[5:] for id in fi2l.index ]

fi2l=fi2l.sort_values('importance',ascending=False)
best10 = fi2l.iloc[0:10,:]
best20 = fi2l.iloc[0:20,:]
best50 = fi2l.iloc[0:50,:]


## linear case
#fi2l.iloc[0:50,:] # A1:5, without averaging, 500 trees, with Yuste, 384gs

In [ ]:
fi2l.iloc[0:50,:] 

In [ ]:
fi2l.iloc[0:50,:] # A1:5, without averaging, 500 trees,  50 best genes found from 280 gs

In [ ]:
fi2l.iloc[0:50,:] # A1:5, without averaging, 500 trees, with Yuste, 20 best genes found from 280 gs

In [ ]:
# RF model case

# set of most significant genes
fi = regr.feature_importances_
#plt.plot(fi)

ifi = np.nonzero(fi)[0]
fi2 = fi[ifi]
ifi2 = np.argsort(fi2)
l=len(fi2)
#print(np.concatenate((ifi2.reshape((l,1)),fi2[ifi2].reshape((l,1))),axis=1))

print(len(ge_columns))
print(len(ge_data.columns))
print(fi.shape)
print(ge_columns[0:6])

fi2 = pd.DataFrame(fi,columns=['importance'],index = ge_columns[6:])
fi2 = pd.concat([fi2, pd.DataFrame(np.repeat('post',len(fi2.index)),columns = ['compartment'],index=fi2.index)],axis=1)
#fi2.loc[np.arange(len(fi2.index))<len(pregs),'compartment']='pre'
fi2.loc[fi2.index.str.contains('pre'),'compartment']='pre'

fi2.index =  [id[5:] for id in fi2.index ]

fi2=fi2.sort_values('importance',ascending=False)
best10 = fi2.iloc[0:10,:]
best20 = fi2.iloc[0:20,:]
best50 = fi2.iloc[0:50,:]

In [ ]:
fi2.iloc[0:50,:] 

# STOP!!!

In [ ]:
fi2.to_excel('best_genes_hipp.xlsx')

## Compare with VISp trained model

In [ ]:
import pickle
RF_file = open('RF_290_hipp.pickle', mode='wb')
pickle.dump(regr,RF_file)

In [ ]:
import os
os.getcwd()

In [ ]:
RF_file = open('RF_290_visp.pickle', mode='rb')
regr_visp = pickle.load(RF_file)

In [ ]:
# RF case
y_pred = regr_visp.predict(X_test[:,nannot:]) 
if len(y_pred.shape)==1:
    y_pred=y_pred.reshape((len(y_pred),1))





from sklearn import metrics

#print('Mean Absolute Error:', metrics.mean_absolute_error(y_test[:,2:].ravel(), y_pred))  
#print('Mean Squared Error:', metrics.mean_squared_error(y_test[:,2:].ravel(), y_pred))  
#print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test[:,2:].ravel(), y_pred))) 

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test[:,nannot:], y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test[:,nannot:], y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test[:,nannot:], y_pred))) 


#y_pred2 = y_pred.reshape((len(y_pred),nstp))
yy = np.concatenate((y_test, y_pred),axis=1)
stpn_test=[s+'_test' for s in stp_columns]
stpn_pred=[s+'_pred' for s in stp_columns]
yy = pd.DataFrame(yy,columns=annot_columns +stpn_test +stpn_pred)




# make input and output stp labels dataset with errors
stpn_error = [s+'_error' for s in stp_columns]
ye = pd.DataFrame(yy.loc[:,stpn_pred].values - yy.loc[:,stpn_test].values, columns=stpn_error)
yy = pd.concat([yy, ye.abs()], axis=1)

#yy.set_index(['pre','post']).groupby()
#yy.set_index(['pre','post']).groupby(['pre','post']).mean()

# plot errors
annot_columns2 = annot_columns

fig, ax = plt.subplots(figsize=(12, 18))

yy1=yy.set_index(annot_columns2)
yy1.loc[:,stpn_error]=yy1.loc[:,stpn_error].values/(yy1.loc[:,stpn_test].values+1e-15)*100
yy1=yy1.astype(float).groupby(annot_columns2).mean()
yy1.loc[:,stpn_error].plot(ax=ax)
plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)


In [ ]:
# RF model case

#plot predicted and real (test) stp data
#plt.figure()
yy1=yy.set_index(annot_columns2)
#yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
yy1=yy1.astype(float).groupby(annot_columns2).mean()
r3=[]
for i in range(nstp):
    f, ax =plt.subplots(figsize=(10, 10))
    #f, ax = plt.figure()
    #ax = f.add_axes()
    #plt.title(stp_columns[i])
    ax.set_title(stp_columns[i])
    yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
    plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)
    if i==0:
        istr=0
    else:
        istr=0 # 11 - if ppr data excluded
        
    ytest= yy1.loc[:,stpn_test[i]]  
    ypred=yy1.loc[:,stpn_pred[i]]
    #r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
    r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).var()/ytest.iloc[istr:].var()
    print(ytest.iloc[istr:].var())
    print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
    print(str(i) +' R^2 = '+str(r2))
    r3 = r3 + [r2]

print('mean R^2 = '+str(np.array(r3).mean()))

f, ax =plt.subplots(figsize=(10, 10))
yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)

#print(yy1.mean())
yy1.loc[:,stpn_test +stpn_pred]

In [ ]:
fi2.iloc[0:50,:] # A1:5, without averaging, 500 trees,  10 from 280 gs R^2 = 0.86

In [ ]:
#fi2.to_excel('10_selected_from_280_gs.xlsx')

# START!
## iterative RF - search for stable interations of features

## run python version of iRF :

## # PYTHON iRF WORKS ONLY WITH BYNARY CLASSIFICATION DATA!!! - USE R iRF2.0 for multiclasses (or 1D regression)!!!

In [ ]:
#import numpy as np
from irf import irf_utils

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import matplotlib.pyplot as plt
rng = np.random.RandomState(10)  # deterministic random data
a = np.hstack((rng.normal(size=1000),  rng.normal(loc=5, scale=2, size=1000)))
print(a.shape)
plt.hist(a, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

bins =[0.45, 0.75,1.0, 1.5, 5]
y_train[:,nannot]
in_bins =  np.histogram(y_train[:,nannot],10)

In [ ]:
import umap
from sklearn.datasets import load_digits

digits = load_digits()

# embedding = umap.UMAP().fit_transform(digits.data)



embedding = umap.UMAP(n_neighbors=5,
                      min_dist=0.3,
                      metric='correlation').fit_transform(digits.data)

In [ ]:
#import louvain
import pyclustering

In [ ]:
from pyclustering.cluster import cluster_visualizer

In [ ]:
yy=y_train[:,nannot:]
y_u = UMAP(spread=2).fit_transform(yy)

from sklearn.manifold import TSNE

y_u2 = TSNE(n_components=2).fit_transform(yy)

fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
plt.scatter(y_u[:, 0], y_u[:, 1],
                edgecolors='none', c="#ff4400", s=10)

plt.axis('off')
fig.set_tight_layout(True)

fig1 = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
plt.scatter(y_u2[:, 0], y_u2[:, 1],
                edgecolors='none', c="#ff4400", s=10)

plt.axis('off')
fig1.set_tight_layout(True)

In [ ]:
colors = ["#991f1f", "#ff9999", "#ff4400", "#ff8800", "#664014", "#665c52",
          "#cca300", "#f1ff33", "#b4cca3", "#0e6600", "#33ff4e", "#00ccbe",
          "#0088ff", "#7aa6cc", "#293966", "#0000ff", "#9352cc", "#cca3c9", "#cc2996"]

fig = plt.figure(figsize=(15, 15), facecolor='w', edgecolor='k')
for i, k in enumerate(list(set(df_aba_vis_in_c_s_vis.loc[:,'cluster_label'] ))):
    i_clu = df_aba_vis_in_c_s_vis.index[df_aba_vis_in_c_s_vis.loc[:,'cluster_label'].isin([k])]
    c =df_aba_vis_in_c_s_vis.loc[i_clu,'cluster_color'].iloc[0]
    
    plt.scatter(latent_u_vis[i_clu, 0], latent_u_vis[i_clu, 1], label=k,
                edgecolors='none', c=c, s=15)
    plt.legend(loc=9, borderaxespad=0, fontsize='small', markerscale=3)

plt.axis('off')
fig.set_tight_layout(True)

## run R iRF in notebook

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
np.save('X.npy',X[:,nannot:].astype(float))
np.save('y.npy',y[:,nannot:].astype(float))

np.save('X_train.npy',X_train[:,nannot:].astype(float))
np.save('X_test.npy',X_test[:,nannot:].astype(float))
np.save('y_train.npy',y_train[:,nannot:].astype(float))
np.save('y_test.npy',y_test[:,nannot:].astype(float))

#np.savetxt('X_train.csv',X_train[:,nannot:].astype(),fmt='%10.5f', delimiter=', ')

In [ ]:
from rpy2.robjects import r, pandas2ri
import rpy2.robjects.numpy2ri

In [ ]:
from rpy2.robjects.packages import importr
irfr = importr('iRF')
bs = importr('base')

In [ ]:
#iRFr=r('iRF')
iRFr = irfr.iRF
X_test_r=X_test[:,nannot:]
X_train_r=X_train[:,nannot:]
y_train_r = y_train[:,nannot:]
y_test_r = y_test[:,nannot:]

import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

X_test_r2 = ro.r.matrix(X_test_r, nrow=X_test_r.shape[0], ncol=X_test_r.shape[1])
X_train_r2 = ro.r.matrix(X_train_r, nrow=X_train_r.shape[0], ncol=X_train_r.shape[1])
y_test_r2 = ro.r.matrix(y_test_r, nrow=y_test_r.shape[0], ncol=y_test_r.shape[1])
y_train_r2 = ro.r.matrix(y_train_r, nrow=y_train_r.shape[0], ncol=y_train_r.shape[1])
ro.r.assign("X_test_r", X_test_r2 )
ro.r.assign("X_train_r", X_train_r2 )

In [ ]:
fit = irfr.iRF(x=X_train_r2, y=y_train_r2, xtest=X_test_r2, ytest=y_test_r2 )

In [ ]:
X_test_r

In [ ]:
# PYTHON iRF WORKS ONLY WITH BYNARY CLASSIFICATION DATA!!! - USE R iRF2.0 for multiclasses (or 1D regression)!!!

from irf import irf_utils
import time
t1 = time.time()

all_rf_weights, all_K_iter_rf_data, \
    all_rf_bootstrap_output, all_rit_bootstrap_output, \
    stability_score = irf_utils.run_iRF(X_train=X_train[:,nannot:],
                                        X_test=X_test[:,nannot:],
                                        y_train=y_train[:,-1],
                                        y_test=y_test[:,-1],
                                        K=5,                          # number of iteration
                                        n_estimators=20,              # number of trees in the forest
                                        B=30,
                                        random_state_classifier=2026, # random seed
                                        propn_n_samples=.2,
                                        bin_class_type=1,
                                        M=20,
                                        max_depth=5,
                                        noisy_split=False,
                                        num_splits=2,
                                        n_estimators_bootstrap=5)


t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))

In [ ]:
fit = iRFr(x=X_train_r, 
         y=y_train_r, 
         xtest=X_test_r, 
         ytest=y_test_r, 
         n.iter=5, 
         n.core=8,
         select.iter = TRUE,
         n.bootstrap=10
        )

## run iRF in R studio

In [ ]:

np.save('X.npy',X[:,nannot:].astype(float))
np.save('y.npy',y[:,nannot:].astype(float))

In [ ]:
### HERE clustering by xmeans_STP.ipynb  should be done

In [ ]:
yc  = np.load('y_clusters.npy')

X_train, X_test, y_train, y_test = train_test_split(X, yc, test_size=0.25, random_state=0)
print(X_train.shape)
print(X_test.shape)
print(set(yc[:,-1]))
print( len(list( set(yc[:,-1]) )))
print(yc.shape)

np.save('Xtrain.npy',X_train[:,nannot:].astype(float))
np.save('yctrain.npy',np.rint(y_train[:,-1]).astype(float))
np.save('Xtest.npy',X_test[:,nannot:].astype(float))
np.save('yctest.npy',np.rint(y_test[:,-1]).astype(float))

In [ ]:
### HERE do iRF.Rmd comutations

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
imp  = np.load('importance.npy')
imp = pd.DataFrame(imp)
imp = pd.concat([imp,pd.DataFrame(ge_columns[6:])],axis=1)
imp.columns = ['importance', 'genes']
imp = imp.set_index('genes')

imp0  = np.load('importance0.npy')
imp0 = pd.DataFrame(imp0)
imp0 = pd.concat([imp0,pd.DataFrame(ge_columns[6:])],axis=1)
imp0.columns = ['importance0', 'genes']
imp0 = imp0.set_index('genes')


# List of important genes : iRF
imp = pd.concat([imp,imp0],axis=1)

imp50 = imp.sort_values('importance',ascending=False).iloc[0:49,:]
plt.plot(imp.sort_values('importance',ascending=False).loc[:,['importance','importance0']].values)
imp50

In [ ]:
#annot_columns + ge_columns[6:]
imp500 = imp.sort_values('importance0',ascending=False).iloc[0:50,:]
imp500

In [ ]:
ge_columns[6:]

In [ ]:
### # train subclasses with best iRF subsets:

In [ ]:
ge_data_subcl

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

from sklearn.model_selection import train_test_split
import time
t1 = time.time()

ge_columns_x = ge_columns[6:]
ge_columns_x = pd.DataFrame(ge_columns_x)

nsel=25
print(imp50.index[0:nsel-1])
selected_columns_x = ge_columns_x.isin(imp50.index[0:nsel-1])
selected_columns_x =ge_columns_x[selected_columns_x.values].index.values 
print(selected_columns_x)
selected_columns_x2 = np.concatenate([np.arange(nannot),selected_columns_x+ nannot])
X_selected = X[:,selected_columns_x2]

do_oob_subclasses=1

subclasses2 = pd.unique(subclasses2 )
print(subclasses2) 

if do_oob_subclasses==1:

    
    oobs = []
    errors = []
    annot_columns2 = annot_columns
    
    r6=[]
    r7=[]
    for i0 in range(len(subclasses2)):
        in_subcl = ge_data_subcl.loc[:,'subclass_pre']==subclasses2[i0]
        in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_post']==subclasses2[i0])
        in_subcl = np.nonzero(in_subcl.values)[0]

        X_train = np.delete(X_selected, in_subcl,axis=0)
        y_train = np.delete(y, in_subcl,axis=0)
        #X_train = X[:,:]
        #y_train = y[:,:]
        
        X_test = X_selected[in_subcl,:]
        y_test = y[in_subcl,:]
        
        

        X_train, X_test0, y_train, y_test0 = train_test_split(X_selected, y, test_size=0.25, random_state=0) 
        
        print(X_train.shape)
        print(X_test.shape)
        
        regr = RandomForestRegressor(random_state=2026,
                                    n_estimators=2500, oob_score=True, n_jobs=-1, max_samples=93)
        regr.fit(X_train[:,nannot:], y_train[:,nannot:])  
        
        y_pred = regr.predict(X_test[:,nannot:]) 
        y_test2 = y_test[:,nannot:]
        
        
        yy = np.concatenate((y_test, y_pred),axis=1)
        stpn_test=[s+'_test' for s in stp_columns]
        stpn_pred=[s+'_pred' for s in stp_columns]
        yy = pd.DataFrame(yy,columns=annot_columns +stpn_test +stpn_pred)
        yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
        yy1 = yy.set_index(annot_columns2)

        ###yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
        yy1=yy1.astype(float).groupby(annot_columns2).mean()
        r3=[]
        r5=[]
        for i in range(y_pred.shape[1]):
            if (i==np.array([0,1,9])).sum()>0:
                f, ax =plt.subplots(figsize=(16, 10))
                ##f, ax = plt.figure()
                ##ax = f.add_axes()
                ##plt.title(stp_columns[i])
                ax.set_title(stp_columns[i]+' '+', subclass out of bag: '+subclasses2[i0])
                yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
                plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)


            #ytest= yy1.loc[:,stpn_test[i]]  
            #ypred=yy1.loc[:,stpn_pred[i]]
            
            ##r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
            if np.var(y_test2[:,i])!=0:
                r2=1 - np.var(y_test2[:,i] - y_pred[:,i])/np.var(y_test2[:,i])
                r4=1 - np.var(y_test2[:,i] - y_pred[:,i])/np.var(y[:,nannot+i])
           # print(ytest.iloc[istr:].var())
           # print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
           # print(str(i) +' R^2 = '+str(r2))
            r3 = r3 + [r2]
            r5 = r5 + [r4]
        print('subclass out of bag: '+subclasses2[i0])
        print('mean R^2 = '+str(np.array(r3).mean()))
        print('all R^2 = '+str(np.array(r3)))
        print('mean R^2 total = '+str(np.array(r5).mean()))
        print('all R^2 total = '+str(np.array(r5)))

        r6 = r6 + [np.array(r3).mean()]
        r7 = r7 + [np.array(r5).mean()]
        oobs = oobs + [regr.oob_score_]
    print('out of bag R^2: '+str(oobs))
    
    rr = np.array([subclasses2,r6,r7])
    rr = pd.DataFrame(rr)
    
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))
#print(regr.predict([[0, 0, 0, 0]]))





In [ ]:
rr

In [ ]:
rr

## Utils

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

def rfr_model(X, y):
    # Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    
    rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],                               random_state=False, verbose=False)
    # Perform K-Fold CV
    scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')

    return scores